In [1]:
import numpy as np
from scipy.stats import truncnorm
from transitions import Machine

# simulation/calculation step size, make sure smaller than any relevant values
# all frequency values will be rounded to step size
STEP_SIZE = 5   # in GHz

# Note: cooling is positive in GHz and heating is negative in GHz
# Laser parameters
NUM_OF_LASERS = 8   # number of lasers and/or MRMs
LASER_FREQ_SPACING = 400  # in GHz
LASER_FREQ_UNCERTAINTY = 10  # in GHz, per stdev
# MRM parameters
MRM_FSR = 1800  # in GHz
MRM_FREQ_SPACING = -200  # in GHz, negative freq is longer wavelength
MRM_FREQ_UNCERTAINTY = 50  # in GHz, per stdev
# CRR parameters
CRR_FSR = 1800  # in GHz
CRR_FREQ_SPACING = -200  # in GHz, negative freq is longer wavelength
CRR_FREQ_UNCERTAINTY = 50  # in GHz , per stdev

# MRM, CRR, and laser local uncertainty/variations will be sampled from
# truncated Gaussian distribution (`truncnorm`). The value `TRUNCATION` below
# refers to how many stdev should we truncate the Gaussian.
# `corners` in the code below will evaluate at this `TRUNCATION` * stdev value
TRUNCATION = 5  # * stdev

# will track the resonances -5*FSR over, ..., 0*FSR over, ... 4*FSR over
# for both MRMs and CRRs.
WHICH_RESONANCES = list(range(-5, 5, 1))
BARREL_ROLL_SEQ = [0, 5, 1, 6, 2, 7, 3, 4]


class Lasers(object):

    """Lasers
    Initialize the lasers. Flags:
    - variations (bool): whether to enable laser wavelength uncertainty
    - corners (bool): if True, only evaluate the corners of the wavelength
        uncertainties. Won't do anything if `variations` is `False`.
    - reverse (bool): if True, go in decreasing frequency.
        (higher frequnency first in the list).
    - offest (int): specify the offset from 0. Otherwise, the lowest frequency
        will always be 0 GHz.

    The object has two important properties:
    - `laser_lines` (list): gives us the frequency of the lasers.
    - `brightness` (list): will say `bright` if that laser is uncaptured.
        Capture a laser line with the function `capture_laser_line`.
        That specific laser brightness will change to the MRM/CRR ring idx.
    """

    def __init__(self,
                 variations=False,
                 corners=False,
                 reverse=False,
                 offset=0) -> None:
        self._delta = None

        # start by having all the laser lines 'bright'
        # will be replaced by the MRM number that captures that laser
        # Note: need to considerq upstream vs. downstream
        self._brightness = ['bright'] * NUM_OF_LASERS

        # If reverse, the first laser line will have the highest frequency
        self._reverse = reverse

        # offset specifies the additive frequency offset from 0
        self._offset = offset

        self.populate_laser_lines(variations, corners)

    def populate_laser_lines(self, variations, corners):
        # the first laser line is always at 0 GHz
        laser_lines = np.arange(
            NUM_OF_LASERS, dtype=int) * LASER_FREQ_SPACING

        if self._reverse:
            laser_lines = np.flip(laser_lines)

        laser_lines += self._offset

        if variations:
            if corners:  # only evaluate the corners
                delta = np.random.randint(2, size=laser_lines.size)
                delta = 2*delta - 1  # convert to [-1, +1] random numbers
                delta *= LASER_FREQ_UNCERTAINTY * TRUNCATION
            else:
                delta = truncnorm.rvs(a=-1*TRUNCATION, b=TRUNCATION,
                                      scale=LASER_FREQ_UNCERTAINTY,
                                      size=laser_lines.size)
            # round to the nearest STEP_SIZE
            delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE
            delta = delta.astype(int)

            self._delta = delta
            laser_lines += delta

        self._laser_lines = laser_lines

    def capture_laser_line(self, which_laser, which_ring, type='mrm'):
        if type == 'mrm':
            assert self._brightness[which_laser] == 'bright', \
                f"Laser line {which_laser} (at {self._laser_lines[which_laser]} GHz) should be bright to be captured"
        else:  # 'crr'
            assert self._brightness[which_laser] == which_ring, \
                f"Laser line {which_laser} (at {self._laser_lines[which_laser]} GHz) should be numbered {which_ring} to be captured"
        # which_ring will modulate this laser
        self._brightness[which_laser] = which_ring

    @property
    def laser_lines(self):
        return self._laser_lines

    @property
    def brightness(self):
        return self._brightness

    def __str__(self) -> str:
        return f"Lines: {self.laser_lines}, Brightness: {self.brightness}"

    def __repr__(self) -> str:
        return self.__str__()


class RingFSM(object):
    """RingFSM
    Base class for all rings: MRMs and CRRs.
    However, they will be modified below, use MRM_FSR and CRR_FSR instead.
    """

    states = ['idle', 'cold', 'search', 'lock']
    INITIAL_COOLDOWN = 200  # in GHz
    # Using this default value because it should clear laser uncertainty and
    # the ring resonance uncertainty at ± 100 GHz.
    DEFAULT_FSR = 1800  # in GHz

    def __init__(self,
                 resonances,
                 lasers,
                 initial_loc=0,
                 this_ring=0,  # this ring's index in the bank
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz
                 step_size=STEP_SIZE,
                 heating_threshold=-1*DEFAULT_FSR,
                 cooling_threshold=INITIAL_COOLDOWN,
                 ring_fsr=DEFAULT_FSR,
                 ) -> None:
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self.loc = initial_loc  # location of the resonance

        self.locked_laser_idx = None

        self._initial_loc = initial_loc
        self._this_ring = this_ring
        self._previous_ring = previous_ring
        self._initial_resonances = resonances
        self._lasers = lasers

        self._cool_down_by = cool_down_by
        self._step_size = step_size
        self._ring_fsr = abs(ring_fsr)

        assert heating_threshold >= -1*self._ring_fsr and heating_threshold <= 0
        self._heating_threshold = heating_threshold
        assert cooling_threshold <= self._ring_fsr and cooling_threshold >= 0
        self._cooling_threshold = cooling_threshold

        self._found_all_lines = False

        self.machine = Machine(
            model=self, states=RingFSM.states, initial='idle')

        # 'start' starts the search algorithm
        self.machine.add_transition(trigger='start',
                                    source='idle',
                                    dest='cold',
                                    after='cool_step')

        # once the ring has cooled, start searching
        self.machine.add_transition(trigger='keep_searching',
                                    source='cold',
                                    dest='search',
                                    after='heat_step')

        # 'reset' puts the ring back to idle from any state
        self.machine.add_transition(trigger='reset',
                                    source='*',
                                    dest='idle',
                                    after='reset_resonances')

        # will 'keep_searching' until we find all the laser lines
        self.machine.add_transition(
            trigger='keep_searching',
            source='search',
            dest='search',
            after='heat_step')

        # lock ring once the laser lines have been found
        self.machine.add_transition(
            trigger='lock_to_line',
            source='search',
            dest='lock',
            after='lock_to_laser',
            conditions=['found_all_lines'])

    def reset_resonances(self):
        # reset everything
        self.loc = self._initial_loc
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self._found_all_lines = False

    def cool(self, amount):
        self.loc += amount

    def heat(self, amount):
        self.loc -= amount

    def cool_step(self):
        # cool the resonances by the cool down by amount
        self.cool(self._cool_down_by)

    def heat_step(self):
        # check if there's a laser signal to be found
        found = self._check_laser_signal()
        # found is tuple (bool, idx | None)
        # if found: (True, laser idx), and if not found: (False, None)
        if found[0]:
            self.laser_locs_seen.append(self.loc)
            self.laser_brightness_seen.append(
                self._lasers.brightness[found[1]])
            self.laser_indices_seen.append(found[1])

        # check if we have found all lasers
        self._check_exit_search_condition()

        # move the resonance location by specified step_size
        self.heat(self._step_size)

    def lock_to_laser(self):

        # Correct the range of numbers to between [-1*FSR, cooling_threshold]
        candidate_loc = self.laser_locs_seen[-1]
        # bring any number beyond 0 and -1*fsr to this range
        candidate_loc = candidate_loc % (-1*self._ring_fsr)
        # bring any number between -1*fsr and the cooling limit to a positive value (cold)
        cooling_limit = self._cooling_threshold - self._ring_fsr
        if candidate_loc > (-1*self._ring_fsr) and candidate_loc <= cooling_limit:
            candidate_loc += self._ring_fsr

        # jump to the laser signal we want to capture
        self.loc = candidate_loc
        # check and capture that laser signal
        self._check_laser_signal(capture=True)

    def _capture_line(self, idx, ring_number):
        return self._lasers.capture_laser_line(idx, ring_number, type='mrm')

    def _check_laser_signal(self, capture=False):
        # check if there's any laser signal at this location
        # and, also decide to capture.
        found_laser = False
        idx = None
        for r in self.resonances:
            if (r in self._lasers.laser_lines):
                # get the index
                idx = np.where(self._lasers.laser_lines == r)
                assert len(idx) == 1
                idx = idx[0][0]
                if capture:
                    self._capture_line(idx, self._this_ring)
                    self.locked_laser_idx = idx
                found_laser = True
        # if there's a laser there, found_laser will return that laser's index
        return (found_laser, idx)

    def _check_exit_search_condition(self):
        # This is the crux of the search algorithm.
        # In this case, we consider the signal-based algo.
        # if we have seen one signal modulated by the previous and one bright, we have found all lines
        if self._previous_ring is None:  # the first ring in the bank
            if len(self.laser_locs_seen) >= 2:
                self._found_all_lines = True
        else:  # all the other rings
            if len(self.laser_locs_seen) >= 2:
                # NB: this line is the only significant change in this signal-based algorithm
                if self.laser_brightness_seen[-2] == self._previous_ring \
                        and self.laser_brightness_seen[-1] == 'bright':  # we'll capture this one
                    self._found_all_lines = True

    @property
    def found_all_lines(self):
        return self._found_all_lines

    @property
    def resonances(self):
        return self.loc + self._initial_resonances

    @property
    def within_threshold(self):
        # if the number is beyond the cooling and heating threshold, then it's outside of threshold
        # otherwise, it's within threshold
        return self.loc >= self._heating_threshold and self.loc <= self._cooling_threshold


class MRM_FSM(RingFSM):

    INITIAL_COOLDOWN = 200  # in GHz

    def __init__(self,
                 resonances,  # a list of this ring's resonances
                 lasers,  # laser object
                 initial_loc=0,  # initial heating/cooling in GHz
                 this_ring=0,  # this ring's index in the bank
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz before searching
                 step_size=STEP_SIZE,
                 heating_threshold=-1*MRM_FSR,
                 cooling_threshold=INITIAL_COOLDOWN
                 ) -> None:

        super().__init__(resonances, lasers, initial_loc, this_ring,
                         previous_ring, cool_down_by, step_size, heating_threshold, cooling_threshold, MRM_FSR)


class CRR_FSM(RingFSM):

    INITIAL_COOLDOWN = 200  # in GHz

    def __init__(self,
                 resonances,  # a list of this ring's resonances
                 lasers,  # laser object
                 initial_loc=0,  # initial heating/cooling in GHz
                 this_ring=0,  # this ring's index in the bank
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz before searching
                 step_size=STEP_SIZE,
                 heating_threshold=-1*CRR_FSR,
                 cooling_threshold=INITIAL_COOLDOWN
                 ) -> None:

        super().__init__(resonances, lasers, initial_loc, this_ring,
                         previous_ring, cool_down_by, step_size, heating_threshold, cooling_threshold, CRR_FSR)

    def _check_exit_search_condition(self):
        # This is the crux of the search algorithm.
        if len(self.laser_locs_seen) >= 1:
            # CRR needs to capture laser used by an MRM of the same idx
            if self.laser_brightness_seen[-1] == self._this_ring:
                self._found_all_lines = True

    def _capture_line(self, idx, ring_number):
        return self._lasers.capture_laser_line(idx, ring_number, type='crr')

In [2]:
from collections import deque
import copy


def populate_resonances(freq_spacing,
                        freq_uncertainty,
                        fsr,
                        global_variations=False,
                        variations=False,
                        corners=False):
    """populate_resonances (base function)
    USE THE FUNCTIONS `populate_mrm_resonances` and `populate_crr_resonances`
    BELOW INSTEAD.

    Initialize the resonances of the rings. Flags:
    - freq_spacing (int): frequency spacing from one ring to the other.
    - freq_uncertainty (int): local resonance frequency uncertainty will be
        sampled from a uniform dist between [-1*freq_uncertainty, freq_uncertainty].
    - fsr (int): the FSR of each ring.
    - global_variations (bool): whether to enable global resonance frequency
        uncertainty with a uniform distribution of [-1*MRM_FSR, MRM_FSR]
    - variations (bool): whether to enable local resonance frequency uncertainty
    - corners (bool): if True, only evaluate the corners of the frequency
        uncertainties. Won't do anything if `variations` is `False`.

    Returns:
    - lines (np.ndarray): A numpy 2D-array, organized where the first dim
        corresponds to which ring in the array and the second dim
        corresponds to which resonance frequency of that ring.

        For example:
        lines[3] will return the resonances of the 4th ring in the bank.
        lines[3][0] will return the first resonance (lowest value in freq)
            of the 4th ring in the bank.
    """

    lines = np.arange(NUM_OF_LASERS, dtype=np.int64) * freq_spacing
    lines = np.vstack([lines] * len(WHICH_RESONANCES)).T

    fsr_shifts = np.array(WHICH_RESONANCES) * fsr
    lines += fsr_shifts

    if variations:  # assume we keep the FSR to be the same
        if corners:  # only evaluate the corners
            delta = np.random.randint(2, size=NUM_OF_LASERS)
            delta = 2*delta - 1  # convert to [-1, +1] random numbers
            delta *= freq_uncertainty * TRUNCATION
        else:
            delta = truncnorm.rvs(a=-1*TRUNCATION, b=TRUNCATION,
                                  scale=freq_uncertainty, size=NUM_OF_LASERS)
        # round to the nearest STEP_SIZE
        delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE

        delta = delta.astype(int)
        delta = delta[:, np.newaxis]
        lines += delta

    if global_variations:  # the global variation can be as much as ±1 FSR
        global_delta = np.random.uniform(-.5*fsr, .5*fsr)
        global_delta = np.round(global_delta/STEP_SIZE, decimals=0) * STEP_SIZE
        global_delta = global_delta.astype(int)
        lines += global_delta

    return lines


def populate_mrm_resonances(global_variations=False,
                            variations=False,
                            corners=False):
    return populate_resonances(MRM_FREQ_SPACING, MRM_FREQ_UNCERTAINTY, MRM_FSR,
                               global_variations, variations, corners)


def populate_crr_resonances(global_variations=False,
                            variations=False,
                            corners=False):
    return populate_resonances(CRR_FREQ_SPACING, CRR_FREQ_UNCERTAINTY, CRR_FSR,
                               global_variations, variations, corners)


# HELPER FUNCTIONS
def check_barrel_roll(list1, list2):
    # check for barrel roll in sequence or reverse sequence
    assert len(list1) == len(list2)

    def check_equivalent(list1, list2):
        d1 = deque(list1)
        d2 = deque(list2)

        equivalent = d1 == d2

        for _ in range(len(list1)):
            d1.rotate(1)
            if d1 == d2:
                equivalent = True

        return equivalent

    list1_ = copy.deepcopy(list1)
    equivalent = check_equivalent(list1_, list2)
    list1_.reverse()
    equivalent = equivalent or check_equivalent(list1_, list2)
    return equivalent


def calculate_maxima(amt_heat_applied):
    colder = [n for n in amt_heat_applied if n >= 0]
    hotter = [n for n in amt_heat_applied if n <= 0]

    max_cool = max(colder) if len(colder) > 0 else 0
    max_heat = min(hotter) if len(hotter) > 0 else 0
    max_diff = max_cool - max_heat
    return max_cool, max_heat, max_diff

In [3]:
# Try out the algorithm with a single try
lasers = Lasers(reverse=True, offset=-4*LASER_FREQ_SPACING, variations=True, corners=False)
all_mrm_resonances_ = populate_mrm_resonances(variations=False, corners=False)
all_mrm_resonances = all_mrm_resonances_
all_mrm_resonances[2] = all_mrm_resonances_[6]
all_mrm_resonances[6] = all_mrm_resonances_[2]

all_mrms = []

# Lock all the MRMs
for idx in range(NUM_OF_LASERS):
    previous_ring = None if idx == 0 else (idx-1)

    mrm = MRM_FSM(resonances=all_mrm_resonances[idx],
                  lasers=lasers,
                  this_ring=idx,
                  previous_ring=previous_ring)
    mrm.start()

    while not mrm.found_all_lines:
        mrm.keep_searching()

    mrm.lock_to_line()
    all_mrms.append(mrm)

    # print(f"Locked to laser # {mrm.locked_laser_idx}")
    # print(lasers.brightness)

all_crr_resonances = populate_crr_resonances(variations=True, corners=False)
all_crrs = []

# Now lock all the CRRs
for idx in range(NUM_OF_LASERS):
    import copy
    lasers_ = copy.deepcopy(lasers)
    previous_ring = None if idx == 0 else (idx-1)
    crr = CRR_FSM(resonances=all_crr_resonances[idx],
                  lasers=lasers_,
                  this_ring=idx,
                  previous_ring=previous_ring)
    crr.start()

    while not crr.found_all_lines:
        crr.keep_searching()

    crr.lock_to_line()
    all_crrs.append(crr)


mrm_lasers_locked_to = [int(m.locked_laser_idx) for m in all_mrms]
crr_lasers_locked_to = [int(m.locked_laser_idx) for m in all_crrs]

# check for barrel roll in sequence or reverse sequence
print("Barrel roll check?", check_barrel_roll(mrm_lasers_locked_to, BARREL_ROLL_SEQ))

print(f"MRM and CRR lasers: {mrm_lasers_locked_to} & {crr_lasers_locked_to}")

amt_heat_applied = [m.loc for m in all_mrms]
print(f"Amount of heat applied for MRM: {amt_heat_applied}")
amt_heat_applied = [m.loc for m in all_crrs]
print(f"Amount of heat applied for CRR: {amt_heat_applied}")

Barrel roll check? True
MRM and CRR lasers: [4, 0, 5, 1, 6, 2, 7, 3] & [4, 0, 5, 1, 6, 2, 7, 3]
Amount of heat applied for MRM: [-400, -410, -1405, -405, -390, -410, -385, -415]
Amount of heat applied for CRR: [-450, -520, -440, -430, -445, -410, -380, -475]


In [4]:
# Run Monte Carlo and get distribution

import pandas as pd
import copy
from tqdm.notebook import tqdm

eval_corners = False
global_variations = True  # main difficulty
local_variations = True
laser_variations = True

cycling_threshold_mrm = 4/9  # ratio of FSR
cycling_threshold_crr = 6/9  # ratio of FSR
cooling_threshold_mrm = 2/9  # ratio of FSR
cooling_threshold_crr = 2/9  # ratio of FSR


# CYCLING: uncomment one of the two below
# Option A: with cycling:
cycling_freqs = [-k * 100 for k in range(18)]
# Option B: without cycling
# cycling_freqs = [0]

num_trials = 1000
num_trials = 2001
trial_results_mrm = []
trial_results_crr = []

failed_trials = []

for _ in tqdm(range(num_trials)):
    lasers = Lasers(reverse=True, offset=-4*LASER_FREQ_SPACING,
                    variations=laser_variations, corners=eval_corners)
    all_mrm_resonances = populate_mrm_resonances(global_variations=global_variations,
                                                 variations=local_variations,
                                                 corners=eval_corners)
    all_crr_resonances = populate_crr_resonances(global_variations=global_variations,
                                                 variations=local_variations,
                                                 corners=eval_corners)

    for ctry, cfreq in enumerate(cycling_freqs):
        # Get all the MRMs to lock properly first
        lasers_ = copy.deepcopy(lasers)
        all_mrms = []
        for idx in range(NUM_OF_LASERS):
            previous_ring = None if idx == 0 else (idx-1)
            mrm = MRM_FSM(resonances=all_mrm_resonances[idx],
                            lasers=lasers_,
                            heating_threshold=-cycling_threshold_mrm * MRM_FSR,
                            cooling_threshold=cooling_threshold_mrm * MRM_FSR,
                            cool_down_by=cooling_threshold_mrm * MRM_FSR,
                            initial_loc=cfreq,
                            this_ring=idx,
                            previous_ring=previous_ring)
                            
            # Run the search algorithm for each MRM
            mrm.start()
            while not mrm.found_all_lines:
                mrm.keep_searching()
            mrm.lock_to_line()
            all_mrms.append(mrm)

        # Save all the MRM information here
        mrm_lasers_locked_to = [int(m.locked_laser_idx) for m in all_mrms]
        amt_heat_applied_mrm = [m.loc for m in all_mrms]
        max_cool_mrm, max_heat_mrm, max_diff_mrm = \
            calculate_maxima(amt_heat_applied_mrm)

        # Lock all the CRRs to the corresponding MRMs
        all_crrs = []
        for idx in range(NUM_OF_LASERS):
            lasers__ = copy.deepcopy(lasers_)
            previous_ring = None if idx == 0 else (idx-1)
            crr = CRR_FSM(resonances=all_crr_resonances[idx],
                            lasers=lasers__,
                            this_ring=idx,
                            previous_ring=idx-1,
                            heating_threshold=-cycling_threshold_crr * MRM_FSR,
                            cooling_threshold=cooling_threshold_crr * MRM_FSR,
                            cool_down_by=cooling_threshold_crr * MRM_FSR)
            crr.start()
            while not crr.found_all_lines:
                crr.keep_searching()
            crr.lock_to_line()
            all_crrs.append(crr)

        # Save all the CRR information here
        crr_lasers_locked_to = [int(m.locked_laser_idx) for m in all_crrs]
        amt_heat_applied_crr = [m.loc for m in all_crrs]
        max_cool_crr, max_heat_crr, max_diff_crr = calculate_maxima(
            amt_heat_applied_crr)

        # Check if cycling is needed
        mrm_cycling_condition = bool(np.array([m.within_threshold for m in all_mrms]).all())
        crr_cycling_condition = bool(np.array([m.within_threshold for m in all_crrs]).all())
        # if the above are true, then we're within threshold--we don't cycle.
        cycling_condition = not(mrm_cycling_condition and crr_cycling_condition)

        # Previous cycling condition below
        # mrm_cycling_condition = abs(max_heat_mrm) > (MRM_FSR * cycling_threshold_mrm)
        # crr_cycling_condition = abs(max_heat_crr) > (MRM_FSR * cycling_threshold_crr)
        # cycling_condition = mrm_cycling_condition or crr_cycling_condition

        # if `cycling_condition == True`: cycle, otherwise break the loop
        if not cycling_condition:
            break

    mrm_result = {'rings_are_locked_to': mrm_lasers_locked_to,
                  # check if barrel roll equivalent
                  'barrel_roll': check_barrel_roll(mrm_lasers_locked_to, BARREL_ROLL_SEQ),
                  'max_cool': max_cool_mrm,
                  'max_heat': max_heat_mrm,
                  'max_diff': max_diff_mrm}

    mrm_numbers = list(range(NUM_OF_LASERS))
    # (-) value is heat and (+) value is cool
    mrm_heat_result = dict(zip(mrm_numbers, amt_heat_applied_mrm))
    mrm_result.update(mrm_heat_result)

    # check if we were able to lock within the threshold
    mrm_threshold = [m.within_threshold for m in all_mrms]
    crr_threshold = [m.within_threshold for m in all_crrs]
    successful_lock = mrm_threshold + crr_threshold
    successful_lock = bool(np.array(successful_lock).all())

    overall_result = {'cycling_trial': ctry, 'successful_lock': successful_lock}
    mrm_result.update(overall_result)

    crr_result = {'rings_are_locked_to': crr_lasers_locked_to,
                  'mrm_crr_equivalent': mrm_lasers_locked_to == crr_lasers_locked_to,
                  'max_cool': max_cool_crr,
                  'max_heat': max_heat_crr,
                  'max_diff': max_diff_crr}

    crr_numbers = list(range(NUM_OF_LASERS))
    # (-) value is heat and (+) value is cool
    heat_result = dict(zip(crr_numbers, amt_heat_applied_crr))
    crr_result.update(heat_result)

    trial_results_mrm.append(mrm_result)
    trial_results_crr.append(crr_result)

    if not successful_lock:  # these are the failed results
       laser_numbers = [f"laser_{idx}" for idx in range(NUM_OF_LASERS)]
       middle_idx = WHICH_RESONANCES.index(0)
       mrm_numbers = [f"mrm_{idx}" for idx in range(NUM_OF_LASERS)]
       mrm_resonance = [int(r[middle_idx]) for r in all_mrm_resonances]
       crr_numbers = [f"crr_{idx}" for idx in range(NUM_OF_LASERS)]
       crr_resonance = [int(r[middle_idx]) for r in all_crr_resonances]

       failed_result = dict(zip(laser_numbers, list(lasers.laser_lines)))
       M = dict(zip(mrm_numbers, mrm_resonance))
       failed_result.update(M)
       C = dict(zip(crr_numbers, crr_resonance))
       failed_result.update(C)
       failed_result.update({'rings_are_locked_to': mrm_lasers_locked_to})

       failed_trials.append(failed_result)
       

trial_results_mrm = pd.DataFrame(trial_results_mrm)
trial_results_crr = pd.DataFrame(trial_results_crr)

# output failed trials to an excel file
failed_trials = pd.DataFrame(failed_trials)
failed_trials.to_excel("failed_trials.xlsx")

trial_results_mrm.to_csv('trial_results_mrm_signal_algo.csv')
trial_results_crr.to_csv('trial_results_crr_signal_algo.csv')

  0%|          | 0/2001 [00:00<?, ?it/s]

In [5]:
trial_results_mrm.describe()

,max_cool,max_heat,max_diff,0,1,2,3,4,5,6,7,cycling_trial
count,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000
mean,94.995002,-239.135432,334.130435,-18.928036,-41.649175,-69.432784,-88.315842,-108.833083,-132.258871,-155.937031,-191.751624,0.954023
std,93.218963,182.138895,130.906812,190.059432,206.631747,207.507086,209.277417,205.366745,201.163043,195.908930,187.784831,2.478081
min,-0.000000,-800.000000,65.000000,-770.000000,-765.000000,-785.000000,-775.000000,-800.000000,-795.000000,-785.000000,-785.000000,0.000000
25%,0.000000,-325.000000,255.000000,-125.000000,-165.000000,-180.000000,-195.000000,-210.000000,-225.000000,-235.000000,-290.000000,0.000000
50%,80.000000,-195.000000,305.000000,35.000000,5.000000,-40.000000,-60.000000,-85.000000,-110.000000,-120.000000,-150.000000,0.000000
75%,165.000000,-110.000000,370.000000,120.000000,115.000000,80.000000,65.000000,30.000000,5.000000,-30.000000,-55.000000,0.000000
max,390.000000,0.000000,800.000000,315.000000,380.000000,365.000000,390.000000,360.000000,380.000000,350.000000,190.000000,17.000000


In [6]:
trial_results_crr.describe()

,max_cool,max_heat,max_diff,0,1,2,3,4,5,6,7
count,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000
mean,111.971514,-420.467266,532.438781,-186.914043,-211.339330,-234.392804,-256.214393,-276.829085,-299.227886,-318.980510,-359.430285
std,140.343057,397.368493,301.647683,419.824915,417.021169,417.505651,416.176975,419.612622,420.375268,418.034371,421.070392
min,-0.000000,-1255.000000,65.000000,-1195.000000,-1195.000000,-1190.000000,-1200.000000,-1190.000000,-1255.000000,-1200.000000,-1195.000000
25%,0.000000,-775.000000,305.000000,-545.000000,-570.000000,-595.000000,-610.000000,-640.000000,-670.000000,-685.000000,-730.000000
50%,0.000000,-315.000000,385.000000,-95.000000,-135.000000,-155.000000,-180.000000,-195.000000,-215.000000,-230.000000,-280.000000
75%,245.000000,-30.000000,775.000000,200.000000,170.000000,145.000000,110.000000,85.000000,65.000000,40.000000,15.000000
max,400.000000,0.000000,1570.000000,400.000000,400.000000,400.000000,400.000000,400.000000,395.000000,385.000000,370.000000


In [7]:
# Now plot the results of the Monte Carlo runs
import plotly.express as px

# Plot MRM
# Heat and cool amount
fig = px.histogram(np.array(trial_results_mrm[range(NUM_OF_LASERS)]).reshape(-1))  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat (-) & Cool (+) amount (GHz)", yaxis_title="Count", title="MRM heating & cooling", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(-MRM_FSR, MRM_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"mrm_heating_{MRM_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

# Plot the maximum differences
fig = px.histogram(trial_results_mrm, x="max_diff")  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat amount difference between rings (GHz)", yaxis_title="Count", title="MRM max diff between rings", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, 2*MRM_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"mrm_diff_{MRM_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

# Plot CRR
# Heat and cool amount
fig = px.histogram(np.array(trial_results_crr[range(NUM_OF_LASERS)]).reshape(-1))  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat (-) & Cool (+) amount (GHz)", yaxis_title="Count", title="CRR heating & cooling", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(-CRR_FSR, CRR_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"crr_heating_{CRR_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

# Plot the maximum differences
fig = px.histogram(trial_results_crr, x="max_diff")  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat amount difference between rings (GHz)", yaxis_title="Count", title="CRR max diff between rings", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, 2*CRR_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"crr_diff_{CRR_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

In [8]:
# check the results: barrel-roll and meeting our threshold requirements
print("All in barrel-roll sequence? ", trial_results_mrm['barrel_roll'].all())
print("All locked within the heating and cooling limits?", trial_results_mrm['successful_lock'].all())
percent_success = np.count_nonzero(trial_results_mrm['successful_lock'])/len(trial_results_mrm['successful_lock'])
print(f"Percentage of successful lock: {percent_success * 100}%")

# display all the trial results in a markdown table
from IPython.display import display, Markdown
display(Markdown("# MRM results"))
display(Markdown(trial_results_mrm.to_markdown()))

display(Markdown("# CRR results"))
display(Markdown(trial_results_crr.to_markdown()))

All in barrel-roll sequence?  True
All locked within the heating and cooling limits? False
Percentage of successful lock: 99.95002498750625%


# MRM results

|      | rings_are_locked_to      | barrel_roll   |   max_cool |   max_heat |   max_diff |    0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |   cycling_trial | successful_lock   |
|-----:|:-------------------------|:--------------|-----------:|-----------:|-----------:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|----------------:|:------------------|
|    0 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        320 |          0 |        320 |  145 |  240 |  225 |  255 |  230 |  320 |   95 |   40 |               0 | True              |
|    1 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -290 |        290 |  -10 |  -15 | -175 |  -55 |  -25 | -220 | -125 | -290 |               1 | True              |
|    2 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        125 |       -235 |        360 |   80 |   30 |   -5 |  125 | -125 | -235 | -110 |  -30 |               0 | True              |
|    3 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |       -240 |        325 |  -25 |   85 |  -20 |   -5 |   -0 |  -55 |  -55 | -240 |               0 | True              |
|    4 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        150 |       -135 |        285 |  150 |  -25 |  -45 |  -55 |   60 |  -40 | -105 | -135 |               0 | True              |
|    5 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        265 |        -10 |        275 |  175 |  240 |  265 |  150 |  155 |   -5 |  -10 |   30 |               0 | True              |
|    6 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -340 |        340 | -125 | -185 | -215 | -185 | -160 | -175 | -190 | -340 |               0 | True              |
|    7 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         35 |       -245 |        280 |   35 |  -95 |  -15 |  -70 |   -0 |   -0 | -240 | -245 |               0 | True              |
|    8 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |       -305 |        355 |   50 |  -20 | -195 | -165 | -125 | -115 | -255 | -305 |               0 | True              |
|    9 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -745 |        745 | -435 | -490 | -635 | -645 | -710 | -650 | -745 | -650 |               5 | True              |
|   10 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        285 |          0 |        285 |  180 |  285 |  125 |  115 |   10 |   15 |  130 |  105 |               0 | True              |
|   11 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         95 |       -155 |        250 |   55 |   80 |   25 |   20 |  -50 |   95 |   25 | -155 |               0 | True              |
|   12 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        260 |        -80 |        340 |  205 |  170 |  150 |  175 |  260 |  -15 |  -50 |  -80 |               0 | True              |
|   13 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        135 |       -190 |        325 |  120 |   45 |   -5 |   15 |  135 |   25 |  -10 | -190 |               0 | True              |
|   14 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        300 |          0 |        300 |  190 |  300 |  255 |  105 |   25 |  120 |  125 |  110 |               0 | True              |
|   15 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        100 |       -180 |        280 |   25 |   85 |   95 |   80 |  100 |   50 | -140 | -180 |               0 | True              |
|   16 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        210 |        -15 |        225 |  145 |  210 |  130 |  175 |  125 |  115 |  180 |  -15 |               0 | True              |
|   17 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |       -135 |        305 |   75 |   65 |   60 |  160 |  170 |   90 |  170 | -135 |               0 | True              |
|   18 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        210 |       -160 |        370 |  205 |  150 |   -0 |  210 |  -65 | -160 |  -80 |   15 |              16 | True              |
|   19 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        215 |       -140 |        355 |  190 |  170 |  215 |  200 |   85 |   15 |  145 | -140 |               0 | True              |
|   20 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        240 |          0 |        240 |  170 |  145 |  240 |  225 |  205 |  225 |  240 |   75 |               0 | True              |
|   21 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         80 |       -175 |        255 |  -25 | -170 |   80 |  -70 | -125 |   10 |  -30 | -175 |               0 | True              |
|   22 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        115 |       -295 |        410 |  115 |  110 |   35 |   40 |   75 | -295 | -140 | -100 |               0 | True              |
|   23 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -390 |        390 |  -35 | -110 |  -75 | -185 | -130 | -390 | -290 | -300 |               1 | True              |
|   24 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        190 |        -95 |        285 |   90 |   80 |  125 |  190 |   65 |  190 |  170 |  -95 |               0 | True              |
|   25 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        155 |       -160 |        315 |  130 |   90 |   55 |  155 | -135 | -160 |  -45 |  -90 |               0 | True              |
|   26 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -350 |        350 | -130 | -295 | -350 | -330 | -320 | -340 | -250 | -350 |               2 | True              |
|   27 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -330 |        330 | -180 | -175 |  -95 | -170 | -120 | -110 | -210 | -330 |               0 | True              |
|   28 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -380 |        380 | -200 | -145 | -330 | -350 | -320 | -335 | -310 | -380 |               3 | True              |
|   29 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -565 |        565 | -265 | -315 | -395 | -325 | -335 | -345 | -565 | -525 |               3 | True              |
|   30 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -205 |        350 |  120 |  100 |   85 |   35 |  145 |   45 | -145 | -205 |               0 | True              |
|   31 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         75 |       -225 |        300 |   75 | -175 | -205 | -185 | -120 | -110 | -225 | -175 |               0 | True              |
|   32 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          5 |       -190 |        195 |  -40 | -110 |  -90 |    5 |  -30 |   -0 |  -40 | -190 |               0 | True              |
|   33 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -340 |        340 |  -25 |  -60 |  -25 | -205 |  -65 | -150 | -110 | -340 |               0 | True              |
|   34 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        110 |       -145 |        255 |   90 |  110 |   15 |  -85 |  -65 | -110 | -145 |  -85 |               0 | True              |
|   35 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        390 |          0 |        390 |  175 |  205 |   80 |  390 |  135 |  195 |   40 |  140 |               0 | True              |
|   36 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        110 |       -270 |        380 |   20 |  110 | -270 | -155 | -195 | -220 | -140 | -165 |               0 | True              |
|   37 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         10 |       -245 |        255 |  -20 |  -80 |   -0 |   10 | -245 | -190 | -215 |  -65 |               1 | True              |
|   38 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         55 |       -245 |        300 |   55 | -130 |  -45 |   50 | -215 | -200 | -245 | -235 |               0 | True              |
|   39 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        140 |       -230 |        370 |  125 |   85 |  -25 |   40 |   65 |   10 |  140 | -230 |               0 | True              |
|   40 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -400 |        400 | -295 | -370 | -375 | -340 | -385 | -400 | -335 | -350 |               2 | True              |
|   41 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |       -280 |        330 |    5 |   50 | -280 | -210 | -250 | -170 | -165 | -165 |               0 | True              |
|   42 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         70 |       -180 |        250 |  -85 |   70 |  -80 | -120 | -170 | -115 | -180 |  -90 |               1 | True              |
|   43 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -190 |        190 |  -25 |  -15 | -155 |  -65 |  -95 | -175 | -100 | -190 |               0 | True              |
|   44 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        240 |       -165 |        405 |  140 |  240 |  135 |  200 |  165 |   60 |  -55 | -165 |               0 | True              |
|   45 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -425 |        425 | -145 | -105 | -195 | -190 | -160 | -105 | -160 | -425 |               0 | True              |
|   46 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -110 |        245 |  135 |   85 |  -75 | -110 |  -20 | -110 |  -60 |  -20 |               0 | True              |
|   47 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        260 |       -105 |        365 |  160 |   75 |  260 |  120 |  170 |  -95 |   25 | -105 |               0 | True              |
|   48 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         45 |       -325 |        370 |  -10 |   -5 |   20 |  -60 | -100 |  -65 |   45 | -325 |               0 | True              |
|   49 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -310 |        310 | -225 | -165 | -215 | -150 | -200 | -310 | -250 | -150 |               1 | True              |
|   50 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        145 |        -95 |        240 |   70 |   55 |   60 |  145 |  135 |   25 |   35 |  -95 |               0 | True              |
|   51 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         80 |       -325 |        405 |   80 | -120 | -175 | -195 | -210 | -230 | -325 | -165 |               0 | True              |
|   52 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -385 |        385 |  -70 | -270 | -120 | -350 | -310 | -300 | -370 | -385 |               1 | True              |
|   53 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        225 |          0 |        225 |  130 |  220 |  180 |  105 |  225 |  185 |  160 |  110 |               0 | True              |
|   54 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        300 |        -45 |        345 |  165 |  165 |  270 |  300 |   65 |   -5 |    5 |  -45 |               0 | True              |
|   55 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -300 |        300 |   -5 | -300 | -255 | -220 | -205 | -150 | -255 | -220 |               0 | True              |
|   56 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        150 |       -155 |        305 |  150 |  150 | -140 | -110 |   10 |  -60 |  -70 | -155 |               0 | True              |
|   57 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        105 |       -280 |        385 |  105 |  -40 |  -55 |   -0 |  -60 | -280 | -235 | -160 |               0 | True              |
|   58 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -470 |        470 | -190 |  -85 | -215 | -385 | -470 | -340 | -365 | -390 |               2 | True              |
|   59 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |       -160 |        330 |  125 |  115 |   60 |  170 |  155 |   85 | -160 |  -45 |               0 | True              |
|   60 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         10 |       -270 |        280 |  -35 |   10 | -170 | -220 | -150 | -245 | -245 | -270 |               1 | True              |
|   61 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -110 |        225 |   90 |  115 | -100 |  -55 | -100 |  -60 | -105 | -110 |               0 | True              |
|   62 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -685 |        685 | -455 | -385 | -395 | -620 | -580 | -685 | -665 | -630 |               5 | True              |
|   63 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        130 |       -140 |        270 |  -15 |  130 |  -20 |   20 |   30 |  -45 | -140 | -135 |               1 | True              |
|   64 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        125 |       -240 |        365 |  125 |   55 | -205 | -125 | -175 | -240 | -150 |  -95 |               0 | True              |
|   65 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         40 |       -340 |        380 |   10 |   40 |  -85 |  -10 | -315 | -265 | -340 | -305 |               0 | True              |
|   66 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         25 |       -215 |        240 |   25 |  -15 | -145 | -160 | -155 | -190 | -185 | -215 |               0 | True              |
|   67 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |          0 |        245 |  130 |  175 |  205 |  245 |   35 |   25 |   10 |   45 |               0 | True              |
|   68 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        380 |          0 |        380 |  285 |  380 |   50 |  280 |  115 |  190 |  170 |   10 |              16 | True              |
|   69 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        165 |       -135 |        300 |   65 |   80 |  165 |   35 | -135 |   -0 | -115 |  -35 |               0 | True              |
|   70 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        160 |       -155 |        315 |  160 |   65 | -155 | -145 |  -85 | -130 | -110 | -155 |               0 | True              |
|   71 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        130 |       -295 |        425 |  130 |   30 |  115 |   -5 |   -0 | -175 | -180 | -295 |               0 | True              |
|   72 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -450 |        450 | -230 | -330 | -300 | -315 | -290 | -225 | -285 | -450 |               1 | True              |
|   73 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -260 |        320 |   60 |   35 | -170 | -205 | -100 | -260 | -210 | -145 |               0 | True              |
|   74 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        140 |       -170 |        310 |   70 |   60 |   40 |  140 |   80 | -100 | -170 |   40 |               0 | True              |
|   75 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        110 |       -145 |        255 |   55 |   75 |  110 |   60 |  -40 | -145 | -130 |  -55 |               0 | True              |
|   76 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -315 |        315 | -130 |  -65 |  -60 |  -60 |  -90 |  -45 |  -70 | -315 |               0 | True              |
|   77 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -545 |        545 | -320 | -380 | -335 | -385 | -320 | -350 | -545 | -495 |               4 | True              |
|   78 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         75 |       -270 |        345 |   75 |  -10 | -100 |  -90 | -160 | -235 | -270 | -210 |               0 | True              |
|   79 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -390 |        390 |  -80 | -360 | -315 | -370 | -265 | -335 | -390 | -325 |               2 | True              |
|   80 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -305 |        305 | -140 |  -55 |  -85 |  -90 | -145 |  -75 | -115 | -305 |               0 | True              |
|   81 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        125 |       -215 |        340 |   90 |   20 |   -0 |  125 |   75 |   35 |   70 | -215 |               0 | True              |
|   82 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -290 |        350 |   60 |   30 |   10 |  -20 |   10 |  -75 |  -10 | -290 |               0 | True              |
|   83 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        135 |       -220 |        355 |   60 |   50 |  135 |  130 |   85 |  -50 | -110 | -220 |               0 | True              |
|   84 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -355 |        355 | -150 | -190 | -165 | -195 | -165 | -100 |  -70 | -355 |               0 | True              |
|   85 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -335 |        335 | -200 | -115 | -145 | -180 |  -65 | -320 | -290 | -335 |               3 | True              |
|   86 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        205 |        -70 |        275 |  155 |  205 |   60 |   25 |  -70 |  -30 |   15 |  -10 |               0 | True              |
|   87 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -430 |        430 | -225 |  -30 | -200 | -165 | -230 | -320 | -430 | -370 |               3 | True              |
|   88 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -130 |        130 | -130 | -105 | -120 |  -85 | -105 | -125 |  -90 |  -25 |               0 | True              |
|   89 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -175 |        175 |  -80 | -125 | -150 | -155 |  -65 | -175 | -110 | -160 |               0 | True              |
|   90 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        220 |          0 |        220 |  170 |  220 |   15 |   35 |   30 |   45 |  115 |   45 |               0 | True              |
|   91 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        100 |       -195 |        295 |   -5 |  100 |   85 | -120 | -135 | -195 | -185 | -160 |               0 | True              |
|   92 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |       -335 |        365 |   30 | -275 | -190 | -225 | -190 | -335 | -185 | -255 |               0 | True              |
|   93 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         25 |        -85 |        110 |  -85 |   20 |   25 |  -55 |  -75 |  -60 |  -15 |  -30 |               0 | True              |
|   94 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |       -130 |        325 |   25 |  145 |   35 |  160 |   85 |  195 | -130 | -130 |               0 | True              |
|   95 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        135 |       -190 |        325 |   40 |   20 |   10 |   45 |  135 | -175 | -190 | -165 |               0 | True              |
|   96 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -190 |        325 |  110 |  135 |   45 | -170 | -140 | -165 | -190 | -165 |               0 | True              |
|   97 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        100 |        -80 |        180 |  -30 |  -80 |  100 |   50 |    5 |   95 |   95 |   40 |               0 | True              |
|   98 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        205 |        -55 |        260 |  165 |  205 |  205 |  180 |  205 |   65 |    5 |  -55 |               0 | True              |
|   99 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -325 |        325 |  -10 |  -30 |  -55 |  -70 |  -70 |  -45 |  -95 | -325 |               0 | True              |
|  100 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        125 |       -130 |        255 |   70 |   20 |   60 |  115 |   90 |  125 |   60 | -130 |               0 | True              |
|  101 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        120 |       -125 |        245 |   85 |  -20 |  110 |  120 |   90 | -105 |  -80 | -125 |               0 | True              |
|  102 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        240 |       -105 |        345 |  175 |  175 |  130 |  240 |  190 |  235 | -105 |   55 |               0 | True              |
|  103 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        240 |          0 |        240 |  175 |  240 |  160 |  195 |  195 |  135 |  120 |   15 |               0 | True              |
|  104 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         10 |       -370 |        380 |   10 |    5 | -105 | -285 | -250 | -210 | -310 | -370 |               0 | True              |
|  105 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -475 |        475 | -290 | -290 | -170 | -325 | -170 | -215 | -235 | -475 |               1 | True              |
|  106 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         70 |       -190 |        260 |   70 |  -80 | -190 | -185 | -125 | -160 | -155 | -155 |               0 | True              |
|  107 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        225 |        -95 |        320 |  140 |   95 |   55 |  215 |  225 |  135 |  -60 |  -95 |               0 | True              |
|  108 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        110 |       -130 |        240 |  100 |  110 |   90 |  -75 |  -90 | -120 | -130 |  -80 |               0 | True              |
|  109 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |       -120 |        315 |  100 |  195 |  170 |   30 |  -90 | -100 | -120 | -120 |               0 | True              |
|  110 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        135 |       -135 |        270 |  135 | -125 | -105 | -135 |  -15 |  -85 | -115 |  -95 |               0 | True              |
|  111 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         65 |       -240 |        305 |  -55 |  -60 |  -45 |  -65 |  -55 |   65 | -105 | -240 |               0 | True              |
|  112 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        135 |       -185 |        320 |   30 |  135 |  115 |   40 | -130 | -130 | -185 |  -35 |               0 | True              |
|  113 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -185 |        365 |  180 |  160 |   75 |  115 |   90 |  -65 | -185 | -135 |               0 | True              |
|  114 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        225 |          0 |        225 |  205 |  225 |  130 |  225 |  225 |  160 |  190 |   30 |               0 | True              |
|  115 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -390 |        390 | -270 | -175 | -125 | -345 | -375 | -380 | -390 | -290 |               3 | True              |
|  116 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -475 |        475 | -340 | -430 | -440 | -455 | -395 | -435 | -475 | -460 |               2 | True              |
|  117 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        120 |        -85 |        205 |   30 |  110 |   75 |  105 |  120 |   20 |  -85 |  -80 |               0 | True              |
|  118 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -390 |        390 |  -80 | -390 | -290 | -355 | -385 | -285 | -285 | -320 |               1 | True              |
|  119 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -610 |        610 | -335 | -335 | -490 | -610 | -570 | -495 | -525 | -500 |               4 | True              |
|  120 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -230 |        315 |   85 | -230 | -100 |  -60 | -125 | -140 |  -15 | -150 |               0 | True              |
|  121 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         55 |       -325 |        380 |   55 |    5 | -290 | -325 | -190 | -265 | -175 | -205 |               0 | True              |
|  122 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         10 |       -205 |        215 | -110 |  -55 |  -40 |   10 |  -85 |  -25 |  -50 | -205 |               0 | True              |
|  123 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |       -235 |        295 |   60 | -225 |  -85 | -150 | -160 | -120 | -220 | -235 |               0 | True              |
|  124 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 | -185 | -190 | -115 | -110 | -205 | -170 |  -20 | -150 |               0 | True              |
|  125 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -575 |        575 | -480 | -515 | -435 | -575 | -480 | -495 | -420 | -490 |               3 | True              |
|  126 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -555 |        555 | -365 | -360 | -425 | -555 | -510 | -535 | -510 | -435 |               4 | True              |
|  127 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         95 |       -275 |        370 |   95 |   10 | -165 | -150 | -220 | -160 | -275 | -180 |               0 | True              |
|  128 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -280 |        280 |  -25 |  -40 | -120 |  -45 |  -70 | -120 |  -45 | -280 |               0 | True              |
|  129 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -260 |        260 | -190 | -175 | -140 | -190 | -260 | -145 | -100 |  -85 |               0 | True              |
|  130 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -220 |        280 |   60 |  -30 | -215 | -175 | -220 | -205 | -220 | -160 |               0 | True              |
|  131 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        150 |       -160 |        310 |  150 |   10 |   65 | -160 |  -55 | -110 |  -75 |  -45 |               0 | True              |
|  132 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -265 |        340 |   75 |   60 |    5 | -130 | -100 | -175 | -185 | -265 |               0 | True              |
|  133 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -305 |        325 |  -80 | -155 |   20 | -100 | -145 | -100 |  -40 | -305 |               0 | True              |
|  134 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         75 |       -240 |        315 |  -30 |   70 |   -5 |   75 |   20 |   50 | -225 | -240 |               1 | True              |
|  135 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        235 |          0 |        235 |  220 |  200 |  170 |  140 |  210 |  235 |    5 |   10 |               0 | True              |
|  136 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -450 |        450 |  -65 | -230 | -330 | -340 | -325 | -390 | -450 | -315 |               1 | True              |
|  137 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -395 |        395 | -100 | -105 |  -90 | -110 | -105 | -165 | -330 | -395 |               1 | True              |
|  138 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         60 |       -265 |        325 |   25 |   60 |    5 |   20 | -125 | -265 | -195 |  -10 |               0 | True              |
|  139 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        225 |       -120 |        345 |  205 |  225 |   90 |  -70 | -120 |  -30 |   55 |  -40 |               0 | True              |
|  140 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        145 |        -25 |        170 |  115 |  105 |  105 |   75 |  135 |  145 |  115 |  -25 |               0 | True              |
|  141 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        155 |          0 |        155 |  110 |  140 |   55 |  140 |   45 |   90 |  155 |  105 |              16 | True              |
|  142 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        180 |       -115 |        295 |  180 |  -10 |   -5 |   25 |  -50 |   -5 | -115 |   30 |               0 | True              |
|  143 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         70 |       -165 |        235 |   70 |   55 |   35 |  -65 | -105 | -165 | -120 |  -40 |               0 | True              |
|  144 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         70 |       -195 |        265 |   20 |   60 |   70 |   15 | -170 |  -85 | -195 | -120 |               0 | True              |
|  145 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -400 |        400 | -395 | -380 | -265 | -300 | -400 | -290 | -320 | -365 |               2 | True              |
|  146 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -640 |        640 | -375 | -425 | -520 | -630 | -635 | -610 | -560 | -640 |               4 | True              |
|  147 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        215 |        -60 |        275 |   90 |  215 |  145 |  155 |  -15 |  -60 |  -45 |  -35 |               0 | True              |
|  148 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        275 |          0 |        275 |  165 |  245 |  275 |   10 |   80 |  140 |   55 |   55 |               0 | True              |
|  149 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        140 |        -60 |        200 |  140 |  -45 |   70 |  -20 |   20 |   65 |  -10 |  -60 |               0 | True              |
|  150 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         65 |       -265 |        330 |   65 |  -75 |  -80 |   20 |   15 |    5 | -265 | -180 |               0 | True              |
|  151 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -305 |        305 | -145 | -190 | -305 | -235 | -275 | -150 | -230 | -225 |               0 | True              |
|  152 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        240 |        -65 |        305 |  185 |  100 |  240 |  100 |  -65 |  -20 |  -20 |  -45 |               0 | True              |
|  153 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        150 |       -175 |        325 |   15 |  150 |  -30 |  -65 | -115 |  -15 | -145 | -175 |               0 | True              |
|  154 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 | -150 | -110 | -205 | -110 | -135 | -110 | -115 |  -10 |               0 | True              |
|  155 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        175 |       -170 |        345 |  175 |  150 |  115 | -105 |  -90 | -170 | -115 | -170 |               0 | True              |
|  156 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        245 |          0 |        245 |  195 |  175 |  195 |  245 |  215 |  235 |  230 |   60 |               0 | True              |
|  157 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        280 |        -20 |        300 |  215 |  270 |  280 |  190 |  215 |   50 |  -10 |  -20 |               0 | True              |
|  158 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -135 |        135 |  -80 | -100 |  -70 | -120 |  -55 | -125 |  -90 | -135 |               0 | True              |
|  159 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -280 |        280 |  -10 | -280 | -225 | -230 | -165 | -245 | -220 | -205 |               0 | True              |
|  160 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         45 |       -200 |        245 |   30 |   45 |  -45 |  -70 | -100 |  -35 |   -0 | -200 |               0 | True              |
|  161 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -415 |        415 | -325 | -345 | -325 | -325 | -270 | -240 | -305 | -415 |               2 | True              |
|  162 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -425 |        425 | -125 | -415 | -370 | -395 | -425 | -365 | -405 | -400 |               2 | True              |
|  163 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        165 |       -110 |        275 |  145 |  165 | -110 |   -5 |  -35 |   40 |   25 |   55 |               0 | True              |
|  164 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        175 |       -190 |        365 |  115 |   45 |  175 |   70 |   55 |  145 |  -65 | -190 |               0 | True              |
|  165 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |       -265 |        315 |   20 |   50 | -225 | -185 | -155 | -195 | -250 | -265 |               0 | True              |
|  166 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         25 |       -380 |        405 |  -75 |  -75 |  -25 |   25 | -135 | -380 | -335 | -235 |               1 | True              |
|  167 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -140 |        140 |  -45 |  -90 |  -30 |  -15 |  -70 |  -70 | -140 |  -25 |               0 | True              |
|  168 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         30 |       -295 |        325 |   30 |  -45 | -135 | -100 | -210 | -225 | -295 | -165 |               0 | True              |
|  169 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |       -205 |        235 | -115 |  -40 |  -40 |  -70 |  -25 |   30 |  -75 | -205 |               0 | True              |
|  170 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        300 |        -60 |        360 |   90 |  210 |   90 |  300 |  165 |  110 |  -60 |  -50 |               0 | True              |
|  171 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -535 |        535 | -270 | -255 | -295 | -275 | -190 | -320 | -535 | -500 |               3 | True              |
|  172 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        225 |          0 |        225 |  210 |  180 |   90 |  140 |  170 |  180 |  225 |   35 |               0 | True              |
|  173 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -445 |        445 | -445 | -310 | -440 | -335 | -395 | -400 | -310 | -330 |               3 | True              |
|  174 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         50 |       -195 |        245 |   45 |   50 |   15 |  -65 |   -0 |    5 | -110 | -195 |               0 | True              |
|  175 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        185 |        -15 |        200 |  175 |  185 |   65 |  -15 |   -0 |   60 |   45 |   15 |               0 | True              |
|  176 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -220 |        400 |  180 |   70 |  165 |   60 |  110 | -105 |  -70 | -220 |               0 | True              |
|  177 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |        -90 |        260 |    5 |  170 |   70 |  135 |  155 |  165 |  -90 |  -70 |               0 | True              |
|  178 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -705 |        705 | -605 | -540 | -580 | -670 | -585 | -650 | -705 | -705 |               7 | True              |
|  179 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        125 |       -115 |        240 |  125 |  110 |  110 |   70 |   60 |  -95 |  -85 | -115 |               0 | True              |
|  180 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -370 |        370 | -150 | -225 |  -60 |  -65 |  -80 | -160 | -370 | -305 |               2 | True              |
|  181 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        105 |       -215 |        320 |  105 |   20 | -175 |  -55 | -215 | -150 |  -60 | -115 |               0 | True              |
|  182 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -225 |        365 |  140 |   15 |  -75 | -155 | -160 | -180 |  -80 | -225 |               0 | True              |
|  183 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |        -50 |        295 |  205 |  245 |  245 |  195 |  125 |   -0 |   85 |  -50 |               0 | True              |
|  184 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        250 |          0 |        250 |  150 |  250 |  145 |  155 |  170 |  220 |  230 |   25 |               0 | True              |
|  185 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -145 |        165 |   20 | -100 | -145 | -110 | -145 | -110 |  -95 | -110 |               0 | True              |
|  186 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        175 |       -120 |        295 |  175 |   80 |  -60 |  -30 |  -50 |  -95 | -120 | -115 |               0 | True              |
|  187 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        170 |        -90 |        260 |  110 |   55 |  170 |  130 |  -65 |  -90 |  -10 |  -30 |               0 | True              |
|  188 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        110 |       -200 |        310 |   -5 |   -0 |   25 |  110 |   80 | -200 | -155 | -180 |               0 | True              |
|  189 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -775 |        775 | -560 | -620 | -525 | -530 | -535 | -625 | -755 | -775 |               6 | True              |
|  190 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -330 |        330 |  -80 | -105 | -130 |   -5 |  -45 | -115 | -120 | -330 |               0 | True              |
|  191 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -380 |        380 | -205 | -150 | -120 | -175 | -120 | -110 | -380 | -345 |               3 | True              |
|  192 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -595 |        595 | -320 | -360 | -235 | -365 | -595 | -570 | -535 | -535 |               4 | True              |
|  193 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |        -75 |        155 |  -35 |   35 |   10 |   40 |  -55 |   80 |  -75 |   75 |              17 | True              |
|  194 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -260 |        260 | -205 | -195 | -260 | -225 | -185 | -245 | -135 | -125 |               1 | True              |
|  195 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -555 |        555 | -350 | -305 | -405 | -295 | -350 | -265 | -275 | -555 |               2 | True              |
|  196 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         50 |       -310 |        360 |   40 |  -10 |    5 | -100 |   50 |  -55 | -205 | -310 |               0 | True              |
|  197 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        370 |        -55 |        425 |  215 |  150 |  275 |  370 |  245 |  285 |  110 |  -55 |               0 | True              |
|  198 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        230 |          0 |        230 |  180 |  230 |  165 |   75 |   25 |   65 |   90 |   65 |               0 | True              |
|  199 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         75 |        -55 |        130 |   -0 |   10 |   25 |   75 |   30 |   30 |   40 |  -55 |               0 | True              |
|  200 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |        -40 |        235 |  115 |  125 |  115 |   35 |  195 |  130 |  -30 |  -40 |               0 | True              |
|  201 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        190 |       -140 |        330 |  130 |  190 |   85 |   85 |  -95 |  -55 |  -20 | -140 |               0 | True              |
|  202 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -265 |        265 |  -50 |  -35 |  -50 |  -80 | -135 |  -65 |  -65 | -265 |               0 | True              |
|  203 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        105 |       -200 |        305 |  105 |   10 |   35 |  -55 |   -5 |  -70 | -200 | -170 |               0 | True              |
|  204 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -390 |        390 | -230 | -225 | -300 | -235 | -265 | -335 | -190 | -390 |               1 | True              |
|  205 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        255 |        -95 |        350 |  145 |  255 |   85 |  130 |   80 |  195 |  155 |  -95 |               0 | True              |
|  206 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -155 |        295 |  140 |   95 |  -85 | -155 |  -30 |  -90 |  -55 |  -80 |               0 | True              |
|  207 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        240 |        -30 |        270 |  180 |  240 |  170 |  215 |   50 |   75 |   45 |  -30 |               0 | True              |
|  208 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         70 |       -175 |        245 |   35 |   70 |   45 | -100 |  -75 | -160 | -175 | -130 |               0 | True              |
|  209 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |       -210 |        360 |   35 |   35 |   90 |  140 |   20 |  150 | -140 | -210 |               0 | True              |
|  210 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         45 |       -180 |        225 |   45 | -155 | -120 |  -40 | -170 | -180 | -105 |  -30 |               0 | True              |
|  211 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         40 |       -210 |        250 |   40 |   30 | -210 | -135 | -100 | -105 |  -80 | -140 |               0 | True              |
|  212 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        110 |       -130 |        240 |  -15 | -105 |   30 |   20 |   15 |  110 |   65 | -130 |               0 | True              |
|  213 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        110 |       -100 |        210 |   55 |  110 |   55 |   30 |  -70 |  -55 |  -60 | -100 |               0 | True              |
|  214 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        240 |          0 |        240 |  100 |  185 |  240 |  145 |  210 |  200 |  185 |   30 |               0 | True              |
|  215 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         85 |       -210 |        295 |   55 |   25 |   30 |   -5 |   85 | -210 | -200 | -205 |               0 | True              |
|  216 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        145 |       -140 |        285 |  120 |   55 |   75 |  115 |  125 |  145 |   40 | -140 |               0 | True              |
|  217 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -160 |        160 |  -90 |  -85 | -130 | -155 | -140 |  -45 |  -30 | -160 |               0 | True              |
|  218 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        110 |       -145 |        255 |   60 |  110 |  110 |  -95 | -145 | -120 | -120 | -130 |               0 | True              |
|  219 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        145 |       -110 |        255 |  120 |  145 |  110 |   95 |   80 | -110 | -105 |  -40 |               0 | True              |
|  220 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |        -85 |        225 |  115 |  140 |  -85 |  -65 |  -60 |   95 |  -50 |   30 |               0 | True              |
|  221 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        115 |       -220 |        335 |   55 |  115 |  -40 |  -35 | -180 | -200 | -220 | -215 |               0 | True              |
|  222 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |       -155 |        335 |  160 |  105 |  150 |  165 |  180 |  170 | -155 | -130 |               0 | True              |
|  223 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         -0 |       -195 |        195 | -155 |   -5 | -115 |   -0 | -115 | -160 |  -70 | -195 |               0 | True              |
|  224 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        175 |         -5 |        180 |  175 |   45 |   95 |   -5 |   35 |   20 |   20 |   60 |               0 | True              |
|  225 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |       -325 |        355 |   30 | -300 | -325 | -205 | -225 | -240 | -195 | -245 |               0 | True              |
|  226 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        100 |       -235 |        335 |  100 |   10 |   30 | -115 | -105 | -230 | -235 | -190 |               0 | True              |
|  227 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -305 |        305 | -100 | -155 | -135 |  -65 | -140 | -115 |  -75 | -305 |               0 | True              |
|  228 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         55 |       -290 |        345 |   25 |   -0 |   55 |  -70 | -290 | -230 | -265 | -240 |               0 | True              |
|  229 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |        -30 |        165 |  110 |  135 |  -30 |   15 |  -25 |   60 |   -5 |   20 |               0 | True              |
|  230 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         65 |       -115 |        180 |  -45 |  -75 | -115 |  -35 |   65 |  -20 |  -70 |  -40 |               0 | True              |
|  231 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -615 |        615 | -330 | -380 | -480 | -580 | -545 | -615 | -550 | -530 |               4 | True              |
|  232 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         10 |       -225 |        235 |  -15 |  -40 |  -70 |   10 | -105 |  -45 | -220 | -225 |               1 | True              |
|  233 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |       -100 |        265 |  165 |  -45 |   30 |  -15 |   70 |  125 |   75 | -100 |               0 | True              |
|  234 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -790 |        790 | -760 | -685 | -705 | -635 | -715 | -790 | -700 | -690 |               6 | True              |
|  235 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -260 |        260 | -140 | -135 | -200 | -150 | -260 | -130 | -175 | -195 |               0 | True              |
|  236 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -455 |        455 | -165 | -105 | -360 | -435 | -420 | -320 | -455 | -435 |               2 | True              |
|  237 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -535 |        535 | -330 | -210 | -235 | -415 | -400 | -455 | -500 | -535 |               4 | True              |
|  238 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        100 |       -190 |        290 |   95 |   55 |  100 | -165 | -115 | -190 | -125 | -115 |               0 | True              |
|  239 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        170 |        -90 |        260 |  155 |  140 |  125 |  170 |  125 |  -60 |  -90 |  -30 |               0 | True              |
|  240 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        195 |       -140 |        335 |  130 |  195 |   70 |  130 |  170 | -140 |  -55 |  -10 |               0 | True              |
|  241 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         -0 |       -350 |        350 |   -0 |  -50 | -100 | -350 | -240 | -245 | -215 | -165 |               0 | True              |
|  242 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        295 |          0 |        295 |  295 |  290 |  240 |  250 |  130 |  210 |   25 |   25 |              16 | True              |
|  243 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        260 |        -30 |        290 |  165 |  180 |  260 |  215 |  225 |  -30 |   80 |   25 |               0 | True              |
|  244 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         65 |       -265 |        330 |   65 |  -50 | -100 |  -40 |  -45 |  -50 |  -55 | -265 |               0 | True              |
|  245 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -325 |        355 |   30 |   25 | -185 | -155 | -230 | -325 | -190 | -180 |               0 | True              |
|  246 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         95 |       -220 |        315 |   45 |   45 |  -25 |  -50 |   95 |   35 |   50 | -220 |               0 | True              |
|  247 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        200 |        -70 |        270 |   75 |  200 |   10 |   40 |  -35 |  -70 |  -15 |   35 |               0 | True              |
|  248 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -175 |        205 |   30 |   20 |   10 |  -90 |  -75 | -175 |  -65 | -175 |               0 | True              |
|  249 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        225 |        -25 |        250 |  145 |  220 |   80 |  225 |  180 |  210 |  115 |  -25 |               0 | True              |
|  250 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -300 |        300 | -130 |  -55 |  -70 | -150 |  -85 |  -50 | -250 | -300 |               0 | True              |
|  251 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -320 |        320 |  -30 | -175 | -185 | -100 | -105 | -180 | -120 | -320 |               0 | True              |
|  252 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -545 |        545 | -315 | -485 | -485 | -475 | -545 | -500 | -540 | -480 |               4 | True              |
|  253 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -435 |        435 | -190 | -340 | -320 | -310 | -435 | -370 | -305 | -360 |               3 | True              |
|  254 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         30 |       -120 |        150 |   15 |  -55 |  -25 | -120 |  -50 |  -25 |   30 |  -65 |              16 | True              |
|  255 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -170 |        200 |   30 |   25 |  -85 | -170 | -140 |  -70 |  -45 | -120 |               0 | True              |
|  256 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |          0 |        245 |  180 |  220 |  245 |  205 |  125 |   95 |  125 |   60 |               0 | True              |
|  257 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        125 |       -170 |        295 |   40 |   95 |  125 |   50 |  100 |   95 |   85 | -170 |               0 | True              |
|  258 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         80 |       -270 |        350 |   80 |    5 | -185 | -175 | -135 | -270 | -255 | -220 |               0 | True              |
|  259 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        125 |       -225 |        350 |  125 |  105 |  115 |  -20 | -190 | -160 | -225 |  -40 |               0 | True              |
|  260 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        170 |       -140 |        310 |  170 |  125 |  -40 |  -10 | -140 |  -35 |  -15 | -100 |               0 | True              |
|  261 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |        -75 |        115 |  -55 |  -15 |  -25 |  -40 |  -10 |   40 |  -75 |   15 |               0 | True              |
|  262 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         95 |       -155 |        250 |   95 |   50 |  -95 | -110 | -155 | -150 | -120 | -150 |               0 | True              |
|  263 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -290 |        290 | -235 | -290 | -185 | -270 | -170 | -210 | -270 | -285 |               1 | True              |
|  264 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -195 |        260 |   65 |   45 |   55 |  -40 | -165 | -195 | -165 | -180 |               0 | True              |
|  265 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |        -10 |        190 |  180 |  120 |   15 |   45 |   10 |   40 |  -10 |   35 |               0 | True              |
|  266 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        165 |       -145 |        310 |   10 |   60 |   80 |  165 |  -90 |  -20 | -135 | -145 |               0 | True              |
|  267 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        110 |       -140 |        250 |  110 |  100 |   75 |  -25 |  -90 |   10 |  -90 | -140 |               0 | True              |
|  268 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        125 |       -230 |        355 |  125 |   80 |   10 |  110 |   20 | -190 | -205 | -230 |               0 | True              |
|  269 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         35 |       -185 |        220 |    5 |   30 |   -5 | -170 |   35 |  -10 | -185 | -155 |               0 | True              |
|  270 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -415 |        415 | -225 | -190 | -200 | -275 | -165 | -340 | -415 | -355 |               3 | True              |
|  271 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -455 |        455 | -395 | -445 | -355 | -420 | -225 | -310 | -245 | -455 |               2 | True              |
|  272 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -135 |        135 | -135 |  -65 | -125 |  -55 |  -55 |  -55 |  -95 |  -80 |               0 | True              |
|  273 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |       -210 |        360 |  150 |  100 |   80 |  -15 | -130 | -210 |  -65 | -155 |               0 | True              |
|  274 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        110 |       -150 |        260 |  110 |   95 |   40 |   85 | -150 |  -30 | -125 |  -90 |               0 | True              |
|  275 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |       -170 |        325 |  155 | -135 |   10 |   -0 | -100 | -120 |  -40 | -170 |               0 | True              |
|  276 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -240 |        240 | -115 | -170 | -220 | -200 | -235 | -225 | -240 | -185 |               0 | True              |
|  277 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         10 |       -235 |        245 |   10 | -180 | -215 | -235 | -205 | -200 | -160 | -185 |               0 | True              |
|  278 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        175 |       -130 |        305 |   65 |  175 |   65 |  130 |  125 |   20 |  -70 | -130 |               0 | True              |
|  279 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |        -75 |        235 |   80 |  160 |   60 |  145 |  135 |  105 |  -70 |  -75 |               0 | True              |
|  280 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        105 |       -170 |        275 |   65 |  105 | -170 | -135 |   -0 |  -30 | -170 |  -90 |               0 | True              |
|  281 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        250 |       -110 |        360 |  185 |  110 |  250 | -110 |  -40 |  -30 |  -70 |   75 |               0 | True              |
|  282 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -330 |        330 | -125 | -145 | -150 |  -35 |  -95 | -155 | -135 | -330 |               0 | True              |
|  283 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |        -75 |        225 |   15 |  150 |  105 |   15 |   45 |   30 |  -45 |  -75 |               0 | True              |
|  284 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |       -165 |        325 |  155 |   75 |  160 |  100 |  150 |  155 | -165 |  -15 |               0 | True              |
|  285 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -590 |        590 | -250 | -450 | -590 | -520 | -530 | -555 | -460 | -430 |               3 | True              |
|  286 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -430 |        430 | -220 | -245 | -160 | -355 | -430 | -345 | -390 | -410 |               3 | True              |
|  287 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -440 |        440 | -200 | -160 | -215 | -250 | -240 | -245 | -245 | -440 |               0 | True              |
|  288 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -230 |        345 |  115 |   65 | -170 | -125 | -140 | -230 | -130 | -215 |               0 | True              |
|  289 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |       -220 |        325 |   40 |   50 |  105 |  -40 |   80 |  -15 |  -10 | -220 |               0 | True              |
|  290 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        175 |       -155 |        330 |  135 |  125 |  155 |  100 |  175 |  165 |   35 | -155 |               0 | True              |
|  291 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |       -115 |        270 |  155 |   10 | -115 |  -55 |  -25 |  -20 |  -75 |   35 |               0 | True              |
|  292 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        340 |        -80 |        420 |  190 |  340 |  125 |  210 |   25 |   90 |  -80 |   30 |               0 | True              |
|  293 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        150 |       -185 |        335 |   55 |  150 |   95 | -110 | -130 | -185 | -140 | -145 |               0 | True              |
|  294 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -610 |        610 | -350 | -485 | -380 | -610 | -610 | -470 | -505 | -580 |               4 | True              |
|  295 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        150 |       -100 |        250 |  150 |  135 |  105 |  -45 |   10 |  -85 | -100 |  -55 |               0 | True              |
|  296 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        205 |       -165 |        370 |   75 |   75 |  205 |   65 |  125 |  -90 | -165 |  -35 |               0 | True              |
|  297 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        185 |        -80 |        265 |   65 |  100 |   15 |  185 |  135 |  120 |  155 |  -80 |               0 | True              |
|  298 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        190 |       -125 |        315 |  120 |  190 | -125 |  -75 |  -30 |  -95 |   10 |   25 |               0 | True              |
|  299 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -760 |        760 | -615 | -650 | -700 | -705 | -760 | -725 | -585 | -665 |               5 | True              |
|  300 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         55 |       -210 |        265 |   55 |   55 | -145 | -105 | -210 |  -50 |  -55 | -155 |               0 | True              |
|  301 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -665 |        665 | -360 | -570 | -555 | -515 | -625 | -640 | -545 | -665 |               4 | True              |
|  302 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        230 |        -70 |        300 |  150 |  215 |  230 |  175 |   70 |  225 |  220 |  -70 |               0 | True              |
|  303 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        115 |        -90 |        205 |   -5 |  -90 |  -15 |  115 |  -30 |  105 |  -20 |   25 |               0 | True              |
|  304 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        330 |          0 |        330 |  135 |  160 |  200 |  330 |  210 |  230 |  180 |   80 |               0 | True              |
|  305 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -335 |        335 | -230 | -335 | -235 | -155 | -220 | -220 | -330 | -265 |               1 | True              |
|  306 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |        -70 |        190 |  120 |  -10 |  -55 |  -70 |  -20 |  -40 |   -5 |  -10 |               0 | True              |
|  307 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -260 |        260 |  -40 | -125 |  -65 | -115 | -140 | -135 | -130 | -260 |               0 | True              |
|  308 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -245 |        310 |   65 | -245 | -210 | -235 |  -70 | -135 | -155 | -185 |               0 | True              |
|  309 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        285 |        -60 |        345 |  190 |  215 |  235 |  285 |   80 |  -60 |   60 |   25 |               0 | True              |
|  310 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        175 |       -170 |        345 |   50 |  165 |  175 |  140 |   50 |  115 |   95 | -170 |               0 | True              |
|  311 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |        -60 |        165 |  105 |   50 |   30 |   60 |   45 |   50 |  -45 |  -60 |               0 | True              |
|  312 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -365 |        365 | -180 | -325 | -345 | -295 | -260 | -230 | -365 | -295 |               2 | True              |
|  313 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        125 |       -205 |        330 |   85 |  125 |  115 |  100 |   55 | -205 |  -20 | -170 |               0 | True              |
|  314 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         50 |       -360 |        410 |   10 |   10 |   50 |  -15 | -110 | -360 | -160 | -185 |               0 | True              |
|  315 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        130 |       -145 |        275 |   40 |   60 |  130 |  -70 | -100 | -145 |  -95 |  -45 |               0 | True              |
|  316 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -770 |        770 | -770 | -685 | -735 | -615 | -605 | -655 | -730 | -695 |               6 | True              |
|  317 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         55 |       -105 |        160 |   40 |   55 |   40 |   35 |   25 |   30 |   15 | -105 |               0 | True              |
|  318 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -140 |        285 |  110 |  145 |  110 |  105 |  145 |   55 | -140 |   -5 |               0 | True              |
|  319 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -205 |        270 |   45 |   65 |   25 | -180 | -125 | -175 | -205 |  -85 |               0 | True              |
|  320 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -425 |        425 | -280 | -230 | -260 | -425 | -220 | -285 | -405 | -300 |               1 | True              |
|  321 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        190 |       -165 |        355 |  115 |  120 |  115 |   70 |   55 |  190 |  100 | -165 |               0 | True              |
|  322 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -575 |        575 | -325 | -340 | -345 | -335 | -575 | -525 | -540 | -460 |               4 | True              |
|  323 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -715 |        715 | -590 | -610 | -645 | -650 | -715 | -605 | -590 | -650 |               4 | True              |
|  324 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |        -70 |        150 |   70 |   80 |   35 |   10 |   10 |    5 |  -25 |  -70 |              16 | True              |
|  325 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        210 |       -155 |        365 |  150 |  110 |  210 |  110 |  110 |   25 | -155 |  -75 |               0 | True              |
|  326 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -405 |        405 | -120 | -365 | -405 | -385 | -205 | -370 | -380 | -360 |               2 | True              |
|  327 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -200 |        345 |   25 |  145 |  -10 |  100 |   50 |   40 | -200 |  -80 |               0 | True              |
|  328 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -345 |        345 | -100 | -120 | -120 | -120 |  -70 | -105 | -100 | -345 |               0 | True              |
|  329 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -235 |        235 | -180 | -190 | -205 | -235 | -175 | -130 | -140 |  -75 |               0 | True              |
|  330 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        155 |       -130 |        285 |  155 |  150 |   40 |  130 |   65 | -100 |  -90 | -130 |               0 | True              |
|  331 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        295 |         -5 |        300 |  215 |  295 |  260 |  195 |  240 |  170 |  115 |   -5 |              16 | True              |
|  332 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -280 |        300 |  -95 |    5 |   20 |   -5 |   10 |   15 |  -60 | -280 |               0 | True              |
|  333 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -275 |        275 |  -95 |  -85 |  -20 |  -70 |  -70 | -120 |  -75 | -275 |               0 | True              |
|  334 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -395 |        395 | -110 |  -65 | -175 | -320 | -330 | -395 | -360 | -325 |               2 | True              |
|  335 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         50 |       -225 |        275 |   50 | -210 | -180 | -160 |  -85 | -145 | -225 | -195 |               0 | True              |
|  336 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -260 |        400 |   25 |  140 | -140 | -125 | -180 | -190 | -150 | -260 |               0 | True              |
|  337 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         25 |       -265 |        290 |  -90 |   25 |  -35 |   10 |    5 |  -95 | -245 | -265 |               2 | True              |
|  338 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -480 |        480 | -150 | -265 | -165 | -180 | -130 | -155 | -240 | -480 |               0 | True              |
|  339 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -350 |        350 | -115 |  -75 |  -15 |  -15 | -280 | -215 | -340 | -350 |               2 | True              |
|  340 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        250 |          0 |        250 |  185 |  200 |  225 |  250 |  115 |  100 |  110 |   55 |               0 | True              |
|  341 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -570 |        570 | -305 | -325 | -540 | -500 | -560 | -555 | -545 | -570 |               4 | True              |
|  342 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        165 |       -145 |        310 |   30 |  165 |   65 |   25 |  -70 |  -90 | -145 | -130 |               0 | True              |
|  343 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         80 |       -235 |        315 |   80 |   50 |   65 |   80 | -235 | -150 | -130 | -100 |               0 | True              |
|  344 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -785 |        785 | -555 | -615 | -540 | -635 | -765 | -750 | -785 | -785 |               6 | True              |
|  345 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        120 |       -255 |        375 |   80 |   45 |  120 |   30 |   60 |   -0 |  -90 | -255 |               0 | True              |
|  346 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        110 |       -245 |        355 |  110 | -130 | -245 | -150 | -170 | -230 | -140 | -190 |               0 | True              |
|  347 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        170 |       -155 |        325 |  140 |  155 |   85 |  170 |  -45 | -130 | -155 | -115 |               0 | True              |
|  348 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        170 |       -100 |        270 |  170 |   25 |  -30 |  -15 |  -35 | -100 |  -10 |  -25 |               0 | True              |
|  349 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -180 |        180 | -180 |  -80 |  -90 |  -50 |  -85 | -120 |  -80 |  -50 |               0 | True              |
|  350 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |        -95 |        290 |  100 |  155 |  195 |   85 |  175 |   90 |  -95 |   10 |               0 | True              |
|  351 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -360 |        360 |  -80 | -175 |  -65 | -185 | -105 | -105 | -210 | -360 |               0 | True              |
|  352 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        125 |       -195 |        320 |   35 |  125 | -100 | -130 | -145 |  -90 |  -40 | -195 |               0 | True              |
|  353 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -210 |        325 |  115 |   55 | -205 |  -55 | -175 | -210 | -105 |  -80 |               0 | True              |
|  354 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         90 |       -205 |        295 |   90 |   35 | -175 |  -50 | -160 |  -75 | -125 | -205 |               0 | True              |
|  355 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        190 |        -85 |        275 |  190 |  -15 |   60 |  -85 |   -0 |  -70 |   35 |   25 |               0 | True              |
|  356 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        210 |        -80 |        290 |   40 |  100 |  175 |  210 |   95 |  155 |  -80 |  -80 |               0 | True              |
|  357 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -375 |        375 | -230 | -140 | -375 | -315 | -355 | -330 | -355 | -275 |               3 | True              |
|  358 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        130 |       -120 |        250 |  130 | -120 |  -35 |  -80 |  -10 |  -15 |  -25 |  -40 |               0 | True              |
|  359 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -585 |        585 | -280 | -215 | -300 | -470 | -505 | -560 | -525 | -585 |               3 | True              |
|  360 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -560 |        560 | -500 | -525 | -560 | -520 | -445 | -525 | -530 | -430 |               3 | True              |
|  361 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -420 |        420 | -130 | -180 | -175 | -420 | -365 | -345 | -300 | -330 |               2 | True              |
|  362 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -355 |        355 |  -70 | -110 |  -40 |  -65 |  -55 |   -5 |  -55 | -355 |               0 | True              |
|  363 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -265 |        265 |   -5 |  -30 |  -75 | -195 | -235 | -265 | -170 | -255 |               0 | True              |
|  364 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -235 |        235 | -155 | -105 | -185 | -160 |  -70 | -235 | -205 |  -80 |               0 | True              |
|  365 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -645 |        645 | -540 | -500 | -555 | -645 | -540 | -555 | -435 | -480 |               4 | True              |
|  366 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        125 |       -375 |        500 |  125 |   -0 |  -20 |   -5 |  -30 | -375 | -210 | -135 |               0 | True              |
|  367 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         40 |       -265 |        305 |  -20 |  -15 |  -30 |  -15 |   35 |  -10 |   40 | -265 |               0 | True              |
|  368 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        120 |       -250 |        370 |   55 |  120 |    5 |  -80 |  -85 |  -10 | -190 | -250 |               0 | True              |
|  369 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -770 |        770 | -455 | -580 | -550 | -585 | -525 | -770 | -685 | -735 |               5 | True              |
|  370 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |        -35 |        235 |  150 |  185 |  180 |  155 |  175 |  200 |  -35 |   55 |               0 | True              |
|  371 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -285 |        385 |  -10 |  -25 |   50 |  -10 |  100 | -285 | -220 | -210 |               1 | True              |
|  372 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -215 |        290 |   75 |  -20 |   10 | -195 | -215 | -110 |  -90 | -135 |               0 | True              |
|  373 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -380 |        380 |  -35 |  -60 | -130 | -160 | -315 | -240 | -265 | -380 |               1 | True              |
|  374 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         40 |       -235 |        275 |   35 |  -30 |   15 |   40 |   -0 | -120 | -190 | -235 |               0 | True              |
|  375 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        120 |       -220 |        340 |   10 |  -80 |  120 |   80 |   65 | -160 | -220 | -200 |               1 | True              |
|  376 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        240 |        -40 |        280 |  240 |  -20 |    5 |   80 |   30 |   15 |  -40 |   20 |              16 | True              |
|  377 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        155 |       -130 |        285 |  155 |  110 |   35 |  135 |  -45 |  -55 | -130 |  -15 |               0 | True              |
|  378 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         80 |       -220 |        300 |  -65 |   80 |  -90 |   65 | -105 |  -80 | -125 | -220 |               0 | True              |
|  379 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -570 |        570 | -370 | -530 | -550 | -440 | -545 | -510 | -570 | -520 |               4 | True              |
|  380 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |       -155 |        305 |   95 |  150 |  100 |  120 |  150 |   50 | -155 |  -15 |               0 | True              |
|  381 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        115 |       -275 |        390 |  115 | -135 |  -35 |   20 | -155 |  -40 | -275 | -230 |               0 | True              |
|  382 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -775 |        775 | -515 | -565 | -485 | -605 | -775 | -740 | -770 | -655 |               6 | True              |
|  383 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -105 |        205 |  100 |   20 |  -40 |   50 |   50 | -105 |  -30 |  -20 |               0 | True              |
|  384 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        100 |       -195 |        295 |  -30 |   45 |   45 |    5 |   30 |  100 |   60 | -195 |               0 | True              |
|  385 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        280 |       -105 |        385 |  135 |  160 |  125 |  280 |   35 |    5 |  -70 | -105 |               0 | True              |
|  386 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -670 |        670 | -570 | -580 | -625 | -530 | -510 | -535 | -670 | -600 |               4 | True              |
|  387 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -135 |        135 | -110 | -135 |  -20 |  -85 |  -35 | -135 | -130 |  -20 |               0 | True              |
|  388 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         55 |        -80 |        135 |   55 |   10 |   55 |   15 |   15 |  -10 |  -45 |  -80 |              16 | True              |
|  389 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        135 |       -165 |        300 |  125 |   15 |   50 |   75 |  135 | -135 | -150 | -165 |               0 | True              |
|  390 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -260 |        260 |  -10 |  -60 |   -5 |  -50 |  -70 |  -90 |   -5 | -260 |               0 | True              |
|  391 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         65 |       -125 |        190 |  -85 | -105 |  -15 |   45 | -125 |   65 |  -15 |  -65 |               0 | True              |
|  392 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -380 |        380 | -115 | -325 | -185 | -170 | -225 | -255 | -195 | -380 |               0 | True              |
|  393 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -195 |        225 |   30 |  -15 | -190 | -195 |  -95 | -125 |  -80 | -140 |               0 | True              |
|  394 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -380 |        380 |  -90 | -105 | -160 |  -70 |  -75 | -105 | -380 | -365 |               1 | True              |
|  395 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        235 |        -25 |        260 |  185 |  235 |  150 |  195 |  -10 |   -5 |  -25 |   90 |               0 | True              |
|  396 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         60 |       -290 |        350 |   60 |    5 |   -5 | -235 | -245 | -220 | -290 | -285 |               0 | True              |
|  397 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -250 |        250 |  -80 |  -55 |  -30 |  -95 | -100 |  -85 |  -25 | -250 |               0 | True              |
|  398 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -200 |        275 |   55 |   15 |   75 | -125 | -100 | -200 | -155 |  -50 |               0 | True              |
|  399 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        145 |       -130 |        275 |  145 |   30 |   -0 |  -90 |  -80 | -110 | -110 | -130 |               0 | True              |
|  400 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        200 |        -60 |        260 |  190 |  200 |   -0 |  -60 |   95 |   50 |   -0 |   15 |               0 | True              |
|  401 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |       -305 |        390 |  -40 |   -0 |  -70 |  -25 |   50 |  -55 |   85 | -305 |               0 | True              |
|  402 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        170 |        -40 |        210 |  170 |  -40 |  100 |   55 |   75 |  150 |  100 |  -10 |               0 | True              |
|  403 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -285 |        285 |  -10 |  -85 |  -80 | -130 | -190 | -210 | -180 | -285 |               0 | True              |
|  404 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        210 |       -150 |        360 |  210 |  150 |  170 |  180 | -110 | -150 |  -35 |  -20 |               0 | True              |
|  405 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -115 |        215 |  100 |  -50 |  -80 |  -65 |   35 | -115 | -100 |  -90 |               0 | True              |
|  406 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |        -75 |        105 |   30 |   15 |  -25 |  -35 |   -5 |  -25 |  -65 |  -75 |               0 | True              |
|  407 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         95 |       -190 |        285 |   30 |   35 |  -40 |   70 |   95 |   85 | -190 | -145 |               0 | True              |
|  408 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         35 |       -205 |        240 |   20 |   10 |   35 | -190 | -175 | -105 | -190 | -205 |               0 | True              |
|  409 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -575 |        575 | -290 | -295 | -540 | -575 | -450 | -525 | -560 | -435 |               4 | True              |
|  410 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        300 |       -115 |        415 |  140 |  145 |  150 |  300 |   -0 | -115 |  -70 |  -45 |               0 | True              |
|  411 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         70 |       -195 |        265 |   20 |   25 |  -65 |   -0 |   70 | -195 | -185 | -125 |               0 | True              |
|  412 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        215 |       -205 |        420 |  100 |  215 |  150 |  200 |  205 |  140 | -205 |  -45 |               0 | True              |
|  413 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -505 |        505 | -320 | -305 | -285 | -220 | -355 | -250 | -505 | -495 |               4 | True              |
|  414 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -620 |        620 | -475 | -500 | -620 | -590 | -510 | -530 | -555 | -525 |               3 | True              |
|  415 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -330 |        330 |  -85 |  -80 | -100 | -110 |  -90 |  -15 | -100 | -330 |               0 | True              |
|  416 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |       -160 |        200 |  -85 | -155 | -130 | -160 |  -20 |   -5 |   40 |  -30 |               0 | True              |
|  417 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        235 |       -135 |        370 |  165 |  115 |  160 |  235 | -135 |  -40 |   -0 |  -50 |               0 | True              |
|  418 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        110 |        -70 |        180 |   20 |   55 |   85 |   45 |  105 |  105 |  110 |  -70 |               0 | True              |
|  419 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         70 |       -130 |        200 |   70 |   60 | -130 | -110 | -115 |   35 | -110 |  -40 |               0 | True              |
|  420 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        160 |       -150 |        310 |   90 |   75 |  160 |   -0 |  105 | -145 | -100 | -150 |               0 | True              |
|  421 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         65 |       -255 |        320 |   50 |   65 | -255 | -115 |  -70 | -130 | -105 | -140 |               0 | True              |
|  422 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         85 |       -315 |        400 |   85 | -155 |  -75 |  -60 |  -45 | -100 | -315 | -295 |               0 | True              |
|  423 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         -0 |       -385 |        385 | -180 | -120 |  -85 |   -0 |  -80 | -130 | -105 | -385 |               0 | True              |
|  424 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |       -140 |        300 |   95 |  105 |  160 |  -40 |   -0 | -110 | -105 | -140 |               0 | True              |
|  425 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -235 |        235 | -155 | -100 | -210 | -205 | -130 | -135 | -235 | -170 |               0 | True              |
|  426 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         80 |       -225 |        305 |   20 |   80 |   20 | -180 | -130 | -225 | -205 | -180 |               0 | True              |
|  427 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        135 |       -200 |        335 |   85 |  135 |   75 |  100 |  -65 | -180 | -145 | -200 |               0 | True              |
|  428 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        275 |        -70 |        345 |  175 |  130 |  205 |  130 |  275 |  -50 |  -70 |   -5 |               0 | True              |
|  429 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -215 |        215 | -105 |  -35 | -115 |  -45 |  -50 |  -90 |   -5 | -215 |               0 | True              |
|  430 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -380 |        380 | -100 | -300 | -180 | -240 | -125 | -225 | -380 | -315 |               2 | True              |
|  431 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        200 |        -20 |        220 |  145 |  110 |  150 |   90 |  145 |  100 |  200 |  -20 |              15 | True              |
|  432 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -225 |        225 | -185 |  -85 |  -50 | -220 |  -40 | -220 | -225 | -185 |               0 | True              |
|  433 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |       -145 |        305 |  160 |  150 |   50 |  -55 |   10 | -135 | -145 | -115 |               0 | True              |
|  434 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |       -185 |        290 |   35 |   80 |  105 |   65 |   80 |   50 |   45 | -185 |               0 | True              |
|  435 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        100 |       -200 |        300 |   55 |   20 |  -30 |  -70 |  100 |  -40 |   -0 | -200 |               0 | True              |
|  436 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         15 |       -160 |        175 | -100 |  -25 |  -10 |   15 |   15 |  -65 |  -65 | -160 |               0 | True              |
|  437 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |       -305 |        335 |  -60 |   20 |  -35 |  -20 |  -65 |   30 |  -70 | -305 |               0 | True              |
|  438 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 |  -95 | -165 | -105 | -195 | -205 | -245 | -230 | -210 |               0 | True              |
|  439 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        160 |        -25 |        185 |   80 |  125 |  160 |  120 |  160 |   80 |   55 |  -25 |               0 | True              |
|  440 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -285 |        285 | -165 | -130 | -285 |  -60 | -215 |  -70 | -200 | -285 |               0 | True              |
|  441 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -165 |        300 |  115 |  135 |  -30 |  -10 |   35 | -165 |  -90 |  -30 |               0 | True              |
|  442 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        240 |       -100 |        340 |   35 |  150 |  195 |  135 |  175 |   80 |  240 | -100 |               0 | True              |
|  443 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        150 |        -25 |        175 |  120 |  150 |  145 |   45 |  -25 |  -25 |  -25 |  -25 |               0 | True              |
|  444 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        305 |          0 |        305 |  175 |  305 |  300 |  235 |   60 |   75 |   15 |   45 |               0 | True              |
|  445 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        105 |       -245 |        350 |   15 |  -85 |   -0 |  -30 |  105 |   35 | -190 | -245 |               0 | True              |
|  446 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -540 |        540 | -220 | -390 | -540 | -485 | -460 | -370 | -480 | -485 |               3 | True              |
|  447 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -655 |        655 | -370 | -310 | -370 | -575 | -545 | -655 | -600 | -595 |               4 | True              |
|  448 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -550 |        550 | -510 | -515 | -550 | -470 | -525 | -455 | -535 | -485 |               4 | True              |
|  449 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         75 |       -260 |        335 |   65 |   40 |   30 |   75 | -245 | -155 | -195 | -260 |               0 | True              |
|  450 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |       -185 |        355 |   75 |   45 |  130 |   70 |  170 |   80 |  -40 | -185 |               0 | True              |
|  451 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -605 |        605 | -305 | -300 | -420 | -375 | -515 | -390 | -375 | -605 |               1 | True              |
|  452 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -680 |        680 | -380 | -430 | -415 | -365 | -400 | -420 | -445 | -680 |               3 | True              |
|  453 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -280 |        280 | -215 | -195 | -165 | -180 | -280 | -195 | -215 | -100 |               0 | True              |
|  454 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -415 |        415 | -390 | -335 | -230 | -295 | -415 | -275 | -400 | -350 |               3 | True              |
|  455 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         25 |       -260 |        285 |   25 |  -65 | -175 | -140 | -260 | -110 |  -30 | -200 |               0 | True              |
|  456 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        170 |        -65 |        235 |  170 |   60 |  -65 |  -45 |  -25 |  -30 |  -35 |   -0 |               0 | True              |
|  457 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -415 |        415 |  -90 | -180 | -130 | -130 | -195 | -150 | -195 | -415 |               0 | True              |
|  458 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -635 |        635 | -400 | -480 | -405 | -480 | -395 | -340 | -635 | -595 |               5 | True              |
|  459 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -415 |        415 | -385 | -310 | -415 | -255 | -320 | -250 | -165 | -315 |               2 | True              |
|  460 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 | -165 | -135 | -185 | -150 | -230 | -125 | -255 | -175 |               0 | True              |
|  461 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |       -140 |        305 |  165 |  -40 |  -70 |  -15 |  -90 |    5 | -140 |  -55 |               0 | True              |
|  462 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |       -180 |        220 |  -90 |   40 |  -70 |  -90 | -180 |    5 |  -85 |   -0 |               0 | True              |
|  463 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        120 |       -150 |        270 |   40 |   40 |  120 |  -70 |   10 |   10 |  -20 | -150 |               0 | True              |
|  464 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |       -115 |        230 |  115 |  -30 | -115 |  -95 |  -55 |   75 |  -30 |   15 |               0 | True              |
|  465 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -305 |        305 | -140 | -150 | -115 |  -90 | -160 | -185 | -215 | -305 |               0 | True              |
|  466 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -345 |        345 | -150 | -260 | -145 | -225 | -250 | -180 |  -75 | -345 |               0 | True              |
|  467 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -665 |        665 | -415 | -475 | -470 | -515 | -470 | -615 | -665 | -655 |               5 | True              |
|  468 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         15 |       -285 |        300 |   15 | -210 |  -50 | -245 | -260 | -270 | -265 | -285 |               0 | True              |
|  469 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        140 |       -210 |        350 |   -0 |  140 |   -0 |  -25 |   20 | -210 | -175 | -195 |               0 | True              |
|  470 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |       -130 |        280 |   55 |   45 |   -0 |  150 |   65 |   60 |   50 | -130 |               0 | True              |
|  471 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        155 |       -215 |        370 |   45 |   15 |  -15 |  -30 |  155 | -165 | -180 | -215 |               0 | True              |
|  472 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        185 |       -105 |        290 |   40 |  155 |  185 |  170 | -100 |   15 |   30 | -105 |               0 | True              |
|  473 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        140 |       -155 |        295 |   65 |  100 |   70 |   25 |  140 |   40 | -155 | -140 |               0 | True              |
|  474 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -410 |        410 | -100 | -100 | -255 | -410 | -385 | -290 | -385 | -350 |               2 | True              |
|  475 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -120 |        185 |   40 |   15 |   65 |  -90 |  -25 |  -45 | -120 | -110 |               0 | True              |
|  476 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |       -150 |        310 |   35 |   40 |  160 |   15 | -150 |  -80 | -145 | -135 |               0 | True              |
|  477 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         75 |       -225 |        300 |   10 |   10 |  -20 |   75 |   15 |  -55 | -175 | -225 |               0 | True              |
|  478 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |        -95 |        275 |  180 |  160 |  -30 |   25 |  -75 |  -95 |  -30 |   -5 |               0 | True              |
|  479 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -225 |        225 | -155 | -165 | -220 | -225 | -180 | -195 | -145 | -210 |               0 | True              |
|  480 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -350 |        350 | -180 | -215 | -300 | -350 | -275 | -185 | -275 | -350 |               0 | True              |
|  481 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         35 |       -235 |        270 |   35 |  -45 | -235 | -195 | -185 | -100 | -125 | -180 |               0 | True              |
|  482 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |       -240 |        440 |   25 |   75 |  120 |  200 |  -30 | -240 | -110 | -125 |               0 | True              |
|  483 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -490 |        490 | -305 | -195 | -445 | -460 | -490 | -490 | -470 | -435 |               4 | True              |
|  484 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        115 |       -235 |        350 |  115 |   75 |  -15 |   10 |   35 | -130 |  -65 | -235 |               0 | True              |
|  485 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -650 |        650 | -305 | -375 | -360 | -335 | -350 | -390 | -545 | -650 |               3 | True              |
|  486 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         55 |       -325 |        380 |  -80 |   25 |  -40 |   55 |   20 |   25 |  -75 | -325 |               0 | True              |
|  487 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -395 |        395 | -210 | -190 | -180 | -170 | -225 | -240 | -205 | -395 |               0 | True              |
|  488 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |        -55 |        225 |   20 |   80 |   75 |   -0 |   85 |   90 |  170 |  -55 |               0 | True              |
|  489 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        200 |       -110 |        310 |   90 |  200 |   85 |  -90 |  -70 | -110 | -100 |  -30 |               0 | True              |
|  490 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        115 |       -145 |        260 |  115 |   60 |  115 |   70 |  115 |   75 |   65 | -145 |               0 | True              |
|  491 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         95 |       -255 |        350 |   70 |  -70 |   65 |   30 |   95 |   10 |   60 | -255 |               0 | True              |
|  492 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -285 |        285 |  -25 | -175 | -225 | -210 | -285 | -135 | -260 | -195 |               1 | True              |
|  493 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        215 |        -30 |        245 |  190 |  205 |  155 |  190 |  215 |  -25 |  -30 |   -5 |               0 | True              |
|  494 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -540 |        540 | -280 | -340 | -365 | -265 | -295 | -350 | -375 | -540 |               1 | True              |
|  495 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        210 |        -90 |        300 |   70 |  210 |  150 |  135 |  180 |  145 |  -90 |  -40 |               0 | True              |
|  496 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        205 |        -60 |        265 |  175 |  205 |  -50 |  -25 |  -60 |  -25 |   35 |  -15 |               0 | True              |
|  497 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -475 |        475 | -215 | -100 | -380 | -420 | -475 | -370 | -360 | -350 |               2 | True              |
|  498 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -340 |        340 |  -90 |  -35 |  -65 | -110 | -230 | -290 | -340 | -255 |               1 | True              |
|  499 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |        -95 |        265 |  110 |  -20 |   85 |   40 |  170 |   60 |   -5 |  -95 |               0 | True              |
|  500 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        315 |        -25 |        340 |  180 |  195 |  315 |   60 |   85 |   85 |  -15 |  -25 |               0 | True              |
|  501 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        190 |        -70 |        260 |  190 |  175 |   10 |   -5 |  -65 |  -70 |   40 |  -65 |               0 | True              |
|  502 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -775 |        775 | -530 | -760 | -775 | -735 | -730 | -690 | -765 | -735 |               6 | True              |
|  503 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -335 |        335 | -275 | -215 | -310 | -330 | -335 | -330 | -265 | -285 |               1 | True              |
|  504 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -485 |        485 | -195 | -220 | -275 | -250 | -225 | -125 | -255 | -485 |               1 | True              |
|  505 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |       -145 |        340 |  175 |  130 |   90 |  195 | -145 |  -45 |   60 |  -20 |               0 | True              |
|  506 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        215 |        -80 |        295 |  215 |  130 |  195 |  200 |   15 |  -15 |  -80 |   25 |               0 | True              |
|  507 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        255 |        -80 |        335 |  180 |  135 |  255 |   80 |  -65 |  -25 |   25 |  -80 |               0 | True              |
|  508 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        245 |        -45 |        290 |  215 |  105 |  245 |  -45 |  -30 |   55 |   30 |   75 |               0 | True              |
|  509 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -230 |        230 | -115 | -145 |  -35 | -180 | -130 |  -75 |  -85 | -230 |               0 | True              |
|  510 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        140 |       -140 |        280 |  140 |   80 |   80 |   90 |   75 |   70 | -140 |  -85 |               0 | True              |
|  511 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -145 | -190 | -155 | -255 | -265 | -255 | -100 | -250 |               0 | True              |
|  512 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -385 |        385 | -200 | -185 | -255 |  -50 | -185 | -190 | -180 | -385 |               0 | True              |
|  513 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 | -200 | -245 | -205 | -235 | -180 | -130 | -190 | -225 |               0 | True              |
|  514 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        120 |       -185 |        305 |   50 |  120 |  -90 |   65 |   -5 |  110 | -185 | -175 |               0 | True              |
|  515 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        200 |        -95 |        295 |  200 |   80 |  -65 |  -95 |   10 |  -30 |  -10 |   25 |               0 | True              |
|  516 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        210 |       -105 |        315 |  210 |  110 |  200 |  -10 |  -60 |  -55 |  -65 | -105 |               0 | True              |
|  517 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        105 |       -160 |        265 |  105 | -130 |  -70 | -160 |  -80 | -100 |  -35 |  -95 |               0 | True              |
|  518 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -240 |        240 | -190 | -210 | -140 | -180 | -240 | -185 | -215 | -155 |               0 | True              |
|  519 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -495 |        495 | -245 | -185 | -205 | -235 | -255 | -385 | -495 | -420 |               3 | True              |
|  520 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |       -135 |        255 |  120 |  -45 |  -80 |  -25 |  -60 | -135 |    5 |  -55 |               0 | True              |
|  521 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         35 |       -205 |        240 |   35 |  -95 |  -60 | -100 | -205 | -165 | -165 | -155 |               0 | True              |
|  522 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -460 |        460 | -240 | -420 | -460 | -410 | -375 | -445 | -455 | -410 |               3 | True              |
|  523 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -260 |        260 |  -90 | -220 | -225 | -130 | -260 | -165 | -145 | -180 |               0 | True              |
|  524 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        200 |        -85 |        285 |  185 |  125 |  200 |  -85 |  -45 |  -35 |  -55 |  -15 |               0 | True              |
|  525 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -400 |        400 | -190 | -275 | -215 | -360 | -240 | -400 | -225 | -320 |               0 | True              |
|  526 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        195 |       -150 |        345 |  125 |  130 |  145 |  195 |  110 |  -85 | -150 |  -15 |               0 | True              |
|  527 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        185 |        -40 |        225 |  185 |  -40 |  -30 |   -0 |    5 |   45 |    5 |    5 |               0 | True              |
|  528 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        205 |       -160 |        365 |  205 |  165 |  -35 | -120 |  -95 |  -80 |  -95 | -160 |               0 | True              |
|  529 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        215 |        -35 |        250 |  210 |  200 |  215 |   50 |  -35 |    5 |  -35 |   -0 |               0 | True              |
|  530 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -660 |        660 | -590 | -620 | -625 | -605 | -635 | -575 | -640 | -660 |               4 | True              |
|  531 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 | -165 |  -85 |  -85 | -160 | -230 | -160 | -245 | -135 |               0 | True              |
|  532 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        225 |        -65 |        290 |  175 |  215 |  125 |  225 |  165 |  180 |   50 |  -65 |               0 | True              |
|  533 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -325 |        325 | -155 | -200 |  -85 |  -85 | -135 |  -80 | -105 | -325 |               0 | True              |
|  534 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         -0 |       -280 |        280 |   -0 |   -5 |  -30 |  -60 |  -15 |  -60 |  -55 | -280 |               0 | True              |
|  535 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -330 |        330 |  -90 | -170 |  -40 |  -70 |  -60 |  -80 |  -35 | -330 |               0 | True              |
|  536 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         80 |       -185 |        265 |   80 |   -0 |  -45 |   -5 |   65 |   45 | -110 | -185 |               0 | True              |
|  537 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -235 |        415 |  100 |  180 |   65 |  105 |    5 | -210 | -115 | -235 |               0 | True              |
|  538 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -135 |        135 | -135 |  -30 |  -75 |  -60 |  -95 |  -45 |  -75 |  -95 |               0 | True              |
|  539 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        155 |       -105 |        260 |  155 |   85 |  145 |   70 |   -0 |  100 | -105 |  -65 |               0 | True              |
|  540 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -355 |        355 | -160 | -200 | -160 | -350 | -315 | -200 | -295 | -355 |               2 | True              |
|  541 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |        -70 |        260 |  145 |   65 |  190 |   55 |  155 |  -60 |  -55 |  -70 |               0 | True              |
|  542 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -520 |        520 | -410 | -450 | -455 | -510 | -450 | -395 | -375 | -520 |               3 | True              |
|  543 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |       -105 |        280 |  135 |  175 |   75 |  110 |  115 | -105 |  -45 | -105 |               0 | True              |
|  544 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -690 |        690 | -445 | -465 | -435 | -450 | -395 | -395 | -655 | -690 |               5 | True              |
|  545 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        180 |        -90 |        270 |  135 |  155 |  130 |  180 |  -90 |   -5 |  -30 |  -80 |               0 | True              |
|  546 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        155 |       -180 |        335 |  150 |  155 |   65 |   55 |   20 |  -75 | -180 | -105 |               0 | True              |
|  547 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -450 |        450 | -180 | -365 | -415 | -390 | -345 | -340 | -420 | -450 |               2 | True              |
|  548 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        200 |        -75 |        275 |  130 |   65 |   10 |  200 |   80 |  150 |  125 |  -75 |               0 | True              |
|  549 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        230 |        -40 |        270 |  160 |  190 |  115 |  230 |  130 |  -10 |  -40 |  -20 |               0 | True              |
|  550 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         45 |       -310 |        355 |   45 |  -70 | -220 | -235 | -305 | -310 | -155 | -200 |               0 | True              |
|  551 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        190 |       -120 |        310 |  145 |  115 |  190 |   90 |  -90 | -120 | -115 |  -30 |               0 | True              |
|  552 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        310 |          0 |        310 |  140 |  310 |  295 |  295 |   90 |  155 |   90 |   85 |               0 | True              |
|  553 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -775 |        775 | -350 | -460 | -550 | -775 | -685 | -740 | -710 | -750 |               4 | True              |
|  554 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         45 |       -200 |        245 |   40 |   45 | -130 | -200 | -135 | -110 | -115 | -180 |               0 | True              |
|  555 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        110 |        -10 |        120 |  -10 |   80 |   60 |   65 |  110 |   35 |   15 |   35 |               0 | True              |
|  556 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        300 |        -50 |        350 |  145 |  200 |  300 |   50 |   55 |  -50 |   20 |   35 |               0 | True              |
|  557 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         60 |       -260 |        320 |   60 |  -35 |  -70 | -140 | -260 | -140 | -225 | -225 |               0 | True              |
|  558 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        260 |          0 |        260 |  135 |  260 |  245 |  220 |  145 |  235 |  100 |   75 |               0 | True              |
|  559 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -150 |        270 |  120 |  120 | -150 | -105 |  -65 | -130 | -115 | -140 |               0 | True              |
|  560 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        205 |        -55 |        260 |  185 |  205 |   25 |  -55 |  -35 |  -55 |  -25 |  -15 |               0 | True              |
|  561 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         -0 |       -275 |        275 |  -70 |   -0 | -145 | -110 | -120 |  -80 |  -20 | -275 |               0 | True              |
|  562 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -405 |        405 | -180 | -130 | -195 | -125 | -185 | -135 | -105 | -405 |               0 | True              |
|  563 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        240 |          0 |        240 |  190 |  210 |  200 |  240 |   10 |   80 |   35 |   75 |               0 | True              |
|  564 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        115 |       -100 |        215 |   40 |  100 |  115 |    5 |  -20 |   30 | -100 | -100 |               0 | True              |
|  565 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        115 |       -135 |        250 |  -45 |  115 |   70 |   80 |  105 |   90 | -120 | -135 |               1 | True              |
|  566 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -645 |        645 | -355 | -395 | -405 | -335 | -405 | -315 | -645 | -600 |               4 | True              |
|  567 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         80 |       -210 |        290 |   80 |   65 |   -5 | -125 | -175 | -160 | -210 |  -45 |               0 | True              |
|  568 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -700 |        700 | -400 | -425 | -455 | -675 | -655 | -645 | -700 | -615 |               4 | True              |
|  569 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -555 |        555 | -230 | -185 | -275 | -400 | -370 | -555 | -505 | -400 |               3 | True              |
|  570 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        190 |       -150 |        340 |  190 | -100 |  -45 |  -60 |  -30 |  -75 | -150 |   40 |               0 | True              |
|  571 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        160 |        -35 |        195 |  160 |  -20 |   20 |  -25 |  -35 |   70 |   50 |   10 |               0 | True              |
|  572 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -640 |        640 | -405 | -355 | -400 | -370 | -550 | -565 | -600 | -640 |               5 | True              |
|  573 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 | -115 | -255 | -180 | -240 | -190 |  -90 | -185 | -170 |               0 | True              |
|  574 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        190 |       -105 |        295 |  150 |  105 |  135 |  190 |  115 |  165 | -105 |  -55 |               0 | True              |
|  575 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -335 |        335 |  -65 | -150 | -195 | -140 | -175 |  -75 | -125 | -335 |               0 | True              |
|  576 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -310 |        310 |  -10 |  -55 |  -55 | -140 | -280 | -310 | -215 | -190 |               1 | True              |
|  577 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -335 |        335 |  -30 | -210 | -270 | -305 | -315 | -140 | -335 | -240 |               1 | True              |
|  578 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -305 |        305 |  -15 |   -5 | -100 | -155 | -105 |  -80 |  -35 | -305 |               0 | True              |
|  579 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -645 |        645 | -340 | -410 | -525 | -540 | -645 | -635 | -455 | -555 |               4 | True              |
|  580 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -510 |        510 | -300 | -330 | -415 | -380 | -330 | -510 | -460 | -510 |               4 | True              |
|  581 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        180 |       -180 |        360 |  140 |  135 |  180 | -180 |  -25 |  -10 |   -5 |  -35 |               0 | True              |
|  582 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -145 |        325 |  180 |  120 |  175 |   30 |  115 | -145 | -120 | -105 |               0 | True              |
|  583 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        275 |        -80 |        355 |  185 |  275 |  265 |  245 |  190 |    5 |  -80 |   40 |               0 | True              |
|  584 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         50 |       -195 |        245 |   15 |   50 | -155 |  -50 |    5 | -180 | -195 | -145 |               0 | True              |
|  585 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -150 |        285 |  110 |  135 |  -90 | -105 | -120 |  -20 | -150 | -110 |               0 | True              |
|  586 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -455 |        455 | -155 | -155 |  -45 | -220 | -165 | -150 | -455 | -340 |               2 | True              |
|  587 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -705 |        705 | -400 | -310 | -440 | -470 | -530 | -475 | -340 | -705 |               2 | True              |
|  588 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -570 |        570 | -285 | -330 | -385 | -395 | -395 | -435 | -570 | -490 |               4 | True              |
|  589 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        360 |          0 |        360 |  195 |  315 |  285 |  225 |  360 |  285 |  135 |  105 |               0 | True              |
|  590 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        125 |       -145 |        270 |  120 |  100 |   60 |  125 |   85 |  -40 | -145 |  -70 |               0 | True              |
|  591 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |        -85 |        255 |  165 |  170 |  125 |  170 |  155 |  150 |  -85 |  -60 |               0 | True              |
|  592 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        155 |       -190 |        345 |   35 |  155 | -105 | -190 | -125 | -135 | -115 | -105 |               0 | True              |
|  593 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         40 |       -330 |        370 |  -15 |  -70 |  -75 |  -20 |   40 | -150 | -280 | -330 |               1 | True              |
|  594 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        210 |       -130 |        340 |  125 |   15 |  -10 |   50 |  210 |  -70 | -100 | -130 |               0 | True              |
|  595 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         75 |       -275 |        350 |   75 | -130 | -180 |  -55 | -150 | -155 | -275 | -155 |               0 | True              |
|  596 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        265 |        -35 |        300 |  155 |  130 |  265 |  150 |  215 |  -35 |   10 |  -15 |               0 | True              |
|  597 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        200 |        -75 |        275 |  200 |   20 |  -35 |   45 |  105 |   60 |   95 |  -75 |               0 | True              |
|  598 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         90 |       -180 |        270 |   90 |  -45 |  -75 |  -60 |  -20 |  -45 |   10 | -180 |               0 | True              |
|  599 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |       -105 |        295 |  135 |  185 |  160 |  190 |  175 |  -70 | -105 |   20 |               0 | True              |
|  600 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -730 |        730 | -495 | -535 | -570 | -730 | -655 | -525 | -730 | -665 |               6 | True              |
|  601 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        300 |        -55 |        355 |  130 |  300 |  -40 |  -20 |    5 |   20 |  -55 |  -15 |               0 | True              |
|  602 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -180 |        180 | -165 |  -55 | -180 | -125 |  -45 |  -95 | -145 | -120 |               0 | True              |
|  603 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -415 |        415 | -135 | -105 | -145 | -135 | -355 | -330 | -375 | -415 |               2 | True              |
|  604 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -315 |        315 | -170 | -315 | -310 | -205 | -255 | -250 | -235 | -250 |               0 | True              |
|  605 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         80 |       -135 |        215 |   80 | -135 | -135 |  -90 |  -35 | -110 | -130 | -130 |               0 | True              |
|  606 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         15 |       -315 |        330 |   15 |  -25 | -185 | -315 | -310 | -155 | -215 | -240 |               0 | True              |
|  607 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -330 |        330 |  -25 |  -45 |  -85 | -240 | -235 | -225 | -330 | -220 |               1 | True              |
|  608 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        230 |        -80 |        310 |  140 |  230 |  210 |  185 |  230 |  180 |   10 |  -80 |               0 | True              |
|  609 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        305 |         -0 |        305 |  195 |  180 |  185 |  250 |  305 |  290 |   45 |   -0 |               0 | True              |
|  610 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         60 |       -185 |        245 |   40 |   30 |  -45 |   60 | -185 |  -80 | -125 | -105 |               0 | True              |
|  611 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        190 |        -25 |        215 |  160 |  130 |   95 |  190 |  125 |  175 |   95 |  -25 |               0 | True              |
|  612 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -450 |        450 | -450 | -310 | -330 | -295 | -365 | -295 | -320 | -355 |               3 | True              |
|  613 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        100 |       -110 |        210 |   10 |   15 |  -30 |   90 |  100 |  -10 |   60 | -110 |               0 | True              |
|  614 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -140 |        140 |  -50 |  -40 |  -65 | -140 | -115 |  -95 |  -20 |  -60 |               0 | True              |
|  615 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         35 |       -270 |        305 | -100 | -125 | -105 | -105 |  -65 |   35 |  -85 | -270 |               0 | True              |
|  616 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -225 |        360 |  135 |   15 |   70 | -205 |  -65 | -120 | -145 | -225 |               0 | True              |
|  617 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -140 |        240 |  100 |  -75 |  -95 |  -20 |   -5 | -140 |  -95 |  -10 |               0 | True              |
|  618 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        115 |       -140 |        255 |   75 |  115 |   65 |   45 |   15 |   90 | -130 | -140 |               0 | True              |
|  619 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         80 |       -115 |        195 |   40 |   15 |   40 |   80 |   30 |   15 | -115 |  -65 |               0 | True              |
|  620 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        165 |       -150 |        315 |  125 |  165 |  115 | -125 |   10 |   45 |  -50 | -150 |               0 | True              |
|  621 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        245 |        -75 |        320 |  155 |  200 |  110 |  180 |  245 |  195 |  240 |  -75 |               0 | True              |
|  622 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -645 |        645 | -405 | -595 | -555 | -580 | -640 | -605 | -495 | -645 |               4 | True              |
|  623 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        190 |        -45 |        235 |  190 |   40 |    5 |   60 |  -40 |  -30 |   20 |  -45 |               0 | True              |
|  624 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         75 |       -155 |        230 |   -0 |  -90 |  -70 |  -90 |  -40 |  -20 | -155 |   75 |               0 | True              |
|  625 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        225 |        -40 |        265 |  135 |  225 |  100 |  165 |  180 |  145 |  -40 |   55 |               0 | True              |
|  626 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        245 |        -60 |        305 |  110 |  170 |  245 |  170 |  210 |  205 |  -60 |   15 |               0 | True              |
|  627 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -335 |        335 | -125 | -185 |  -65 |  -30 |  -45 | -120 | -335 | -180 |               2 | True              |
|  628 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -135 |        135 |  -25 | -135 |  -70 | -115 | -100 |  -25 |  -20 |  -75 |               0 | True              |
|  629 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -340 |        340 | -235 | -290 | -305 | -265 | -245 | -315 | -260 | -340 |               1 | True              |
|  630 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        165 |       -150 |        315 |  165 |   35 |  155 |  -80 |  -80 | -150 | -105 |  -95 |               0 | True              |
|  631 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         40 |       -200 |        240 |   20 |   -5 |  -10 | -115 |  -35 |   40 |  -65 | -200 |               0 | True              |
|  632 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        205 |        -30 |        235 |  205 |  205 |  -30 |   25 |   40 |   55 |   40 |   50 |               0 | True              |
|  633 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -340 |        340 | -105 | -170 | -160 | -160 | -130 | -130 | -205 | -340 |               0 | True              |
|  634 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        220 |       -135 |        355 |  170 |  220 | -100 | -130 | -100 | -135 |  -50 | -135 |               0 | True              |
|  635 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        130 |        -90 |        220 |  130 |   50 |  -90 |  -65 |  -90 |   50 |  -40 |  -25 |               0 | True              |
|  636 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        185 |        -65 |        250 |   70 |  185 |   95 |   30 |  125 |   90 |   50 |  -65 |               0 | True              |
|  637 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        185 |       -135 |        320 |  100 |  185 |   90 | -100 |   20 | -110 |  -50 | -135 |               0 | True              |
|  638 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        260 |        -80 |        340 |  190 |  255 |  200 |  260 |  -80 |   20 |   50 |   45 |               0 | True              |
|  639 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        250 |          0 |        250 |  180 |  215 |  210 |  250 |   25 |   20 |   30 |    5 |               0 | True              |
|  640 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -365 |        365 | -180 | -165 | -190 | -245 | -150 | -110 | -140 | -365 |               0 | True              |
|  641 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -160 |        295 |  115 |  135 |  110 |  -70 | -160 |  -45 |  -60 |  -65 |               0 | True              |
|  642 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -600 |        600 | -395 | -385 | -445 | -435 | -390 | -495 | -325 | -600 |               3 | True              |
|  643 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        210 |        -40 |        250 |  210 |  -30 |   25 |    5 |   -0 |   -5 |   25 |  -40 |               0 | True              |
|  644 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |       -155 |        310 |  155 |  -45 |  -50 | -155 |  -40 |  -80 |  -80 |  -30 |               0 | True              |
|  645 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -775 |        775 | -570 | -580 | -475 | -515 | -475 | -550 | -775 | -715 |               6 | True              |
|  646 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         15 |       -205 |        220 |  -40 |   15 |  -70 |  -20 |  -40 |  -55 |  -90 | -205 |               0 | True              |
|  647 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        130 |        -95 |        225 |   85 |   55 |   20 |  130 |  115 |   55 |  -45 |  -95 |               0 | True              |
|  648 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -160 |        160 |  -15 | -160 |  -90 |  -90 | -110 | -125 | -105 | -125 |               1 | True              |
|  649 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -465 |        465 | -155 | -245 | -230 | -230 | -200 | -145 | -150 | -465 |               0 | True              |
|  650 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -765 |        765 | -355 | -405 | -375 | -425 | -400 | -390 | -765 | -605 |               4 | True              |
|  651 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         75 |       -170 |        245 |   75 | -110 | -170 |   -5 |  -85 |  -75 |  -60 |  -85 |               0 | True              |
|  652 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -450 |        450 | -260 | -330 | -290 | -360 | -450 | -370 | -315 | -250 |               1 | True              |
|  653 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        190 |        -30 |        220 |  130 |  190 |  -10 |   -5 |  -30 |   -5 |   20 |   30 |               0 | True              |
|  654 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        190 |       -145 |        335 |  125 |  190 |  120 |   85 |  -45 |  -45 |  -75 | -145 |               0 | True              |
|  655 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        170 |        -60 |        230 |  170 |  130 |  125 |  -20 |   -5 |  -60 |   75 |  -55 |               0 | True              |
|  656 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        240 |       -115 |        355 |  215 |  240 |  155 |  130 |  140 |   30 | -115 |   65 |               0 | True              |
|  657 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -500 |        500 | -325 | -385 | -300 | -285 | -350 | -325 | -500 | -500 |               4 | True              |
|  658 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -230 |        230 | -210 | -155 | -150 | -230 | -160 | -195 | -155 | -180 |               0 | True              |
|  659 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -400 |        400 | -125 | -225 | -230 | -180 | -125 | -195 | -210 | -400 |               0 | True              |
|  660 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         50 |       -180 |        230 |  -50 |  -40 |  -20 |   30 |  -20 |   50 | -125 | -180 |               0 | True              |
|  661 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 | -185 | -205 | -190 | -120 | -110 | -145 | -175 | -130 |               0 | True              |
|  662 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 | -140 |  -30 |  -75 | -150 | -115 | -205 |  -45 |  -90 |               0 | True              |
|  663 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        200 |       -110 |        310 |  200 |   60 |   90 |  -60 |  -65 | -100 |  -15 | -110 |               0 | True              |
|  664 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 | -135 | -245 | -150 |  -65 | -105 | -115 | -150 |  -90 |               0 | True              |
|  665 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -555 |        555 | -305 | -350 | -350 | -335 | -285 | -290 | -555 | -485 |               4 | True              |
|  666 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -265 |        285 |  -55 |  -30 |   -0 |  -55 |   20 | -100 |  -65 | -265 |               0 | True              |
|  667 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        305 |        -45 |        350 |  280 |  285 |  170 |  160 |  305 |  300 |  -45 |  -10 |              16 | True              |
|  668 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        125 |       -205 |        330 |   55 |   75 |  125 | -165 |  -90 | -155 | -205 | -105 |               0 | True              |
|  669 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        320 |        -35 |        355 |  185 |  320 |   50 |  -35 |  115 |   75 |   75 |  140 |               0 | True              |
|  670 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         45 |       -220 |        265 |   45 |   -0 |  -35 |  -80 |  -65 | -205 | -200 | -220 |               0 | True              |
|  671 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -225 |        350 |  125 |  100 |  -35 |   10 |  -25 |   85 | -225 | -160 |               0 | True              |
|  672 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -600 |        600 | -300 | -395 | -600 | -525 | -560 | -570 | -570 | -525 |               4 | True              |
|  673 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        125 |        -75 |        200 |  -10 |  -75 |    5 |   85 |   90 |  125 |   -5 |   -0 |               0 | True              |
|  674 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -125 |        200 |   10 |   50 |   75 |  -45 |  -60 |  -70 | -125 |  -60 |               0 | True              |
|  675 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        205 |       -125 |        330 |   50 |   75 |  130 |  205 |  145 |   95 | -125 |  -80 |               0 | True              |
|  676 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        210 |        -90 |        300 |  195 |  210 |  140 |  190 |  185 |  -70 |  -65 |  -90 |               0 | True              |
|  677 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -450 |        450 | -305 | -365 | -450 | -430 | -370 | -350 | -305 | -430 |               3 | True              |
|  678 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -765 |        765 | -535 | -565 | -500 | -460 | -520 | -765 | -660 | -720 |               6 | True              |
|  679 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -390 |        390 | -175 | -255 | -205 | -130 | -155 | -170 | -210 | -390 |               0 | True              |
|  680 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        150 |        -85 |        235 |  150 |  135 |   55 |  -50 |   -5 |  -85 |   40 |   15 |               0 | True              |
|  681 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -295 |        325 |  -45 |   30 | -180 | -295 | -200 | -210 | -235 | -240 |               1 | True              |
|  682 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         35 |       -255 |        290 |   35 | -160 | -160 | -255 | -115 | -120 | -175 | -165 |               0 | True              |
|  683 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -355 |        355 |  -80 |  -65 | -185 | -110 | -105 |  -55 | -145 | -355 |               0 | True              |
|  684 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         60 |       -215 |        275 |   60 |   30 |  -70 | -135 | -145 | -155 | -180 | -215 |               0 | True              |
|  685 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        100 |       -235 |        335 |   70 |  100 | -235 | -105 | -125 | -110 | -105 | -175 |               0 | True              |
|  686 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |        -60 |        260 |   80 |   40 |   60 |  100 |  200 |  125 |   20 |  -60 |               0 | True              |
|  687 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -445 |        445 | -185 | -445 | -425 | -365 | -415 | -435 | -360 | -395 |               2 | True              |
|  688 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        130 |       -110 |        240 |   60 |  130 |  100 |   95 |  -30 |  -40 | -110 |  -80 |               0 | True              |
|  689 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        260 |        -30 |        290 |  135 |  240 |  180 |  260 |  125 |   40 |   50 |  -30 |               0 | True              |
|  690 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        175 |       -125 |        300 |  175 |  135 |   35 |  100 | -125 |  -30 |  -30 |   20 |               0 | True              |
|  691 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -415 |        415 | -155 |  -25 | -155 |  -85 | -115 | -120 | -145 | -415 |               0 | True              |
|  692 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -755 |        755 | -430 | -455 | -450 | -685 | -715 | -620 | -755 | -615 |               5 | True              |
|  693 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        275 |        -25 |        300 |  150 |   85 |  100 |  180 |  275 |   95 |  200 |  -25 |               0 | True              |
|  694 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        175 |       -120 |        295 |   25 |  150 |   80 |  175 |   70 |  135 | -120 |  -85 |               0 | True              |
|  695 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -365 |        365 | -120 | -165 |  -90 | -205 | -190 | -330 | -340 | -365 |               2 | True              |
|  696 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         40 |       -220 |        260 |   40 | -170 | -200 | -215 | -220 | -175 | -160 | -200 |               0 | True              |
|  697 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |        -10 |        135 |  120 |  105 |  125 |  105 |   65 |  120 |  -10 |   -0 |               0 | True              |
|  698 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -195 |        195 | -125 | -195 | -175 | -170 | -135 |  -60 | -150 | -185 |               0 | True              |
|  699 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        180 |       -195 |        375 |  180 |  -60 |  -95 |  -55 | -195 |  -95 |  -50 |  -55 |               0 | True              |
|  700 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         85 |       -170 |        255 |   55 |   85 | -170 | -125 | -145 | -115 |  -70 |  -75 |               0 | True              |
|  701 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        220 |        -40 |        260 |  160 |  155 |  145 |  160 |  220 |  -40 |  -20 |  -30 |               0 | True              |
|  702 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -130 |        215 |   85 |  -10 | -130 |  -45 |   15 |  -25 |  -25 | -125 |               0 | True              |
|  703 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         35 |       -240 |        275 |   35 |   20 | -115 | -135 |  -80 | -140 | -240 |  -90 |               0 | True              |
|  704 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         -0 |       -390 |        390 |  -90 |  -85 | -140 |   -0 | -295 | -390 | -280 | -270 |               1 | True              |
|  705 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        190 |       -110 |        300 |  180 |  190 |   90 | -110 |  -90 |   10 |  -95 |  -75 |               0 | True              |
|  706 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 | -150 |  -60 | -110 | -130 | -245 |  -35 | -145 | -135 |               0 | True              |
|  707 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -730 |        730 | -470 | -525 | -535 | -525 | -495 | -510 | -700 | -730 |               5 | True              |
|  708 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         65 |       -125 |        190 |   25 |   45 |   -0 |   65 |  -50 |  -50 | -125 |  -35 |              16 | True              |
|  709 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         15 |       -305 |        320 |   15 | -225 | -130 | -255 | -255 | -165 | -305 | -250 |               0 | True              |
|  710 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |       -100 |        295 |  195 |  140 |  145 |  170 | -100 |  -60 |  -45 |  -85 |               0 | True              |
|  711 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |       -180 |        360 |  180 |  135 |  -75 |  -20 | -130 | -180 | -130 |  -55 |               0 | True              |
|  712 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        235 |         -5 |        240 |  190 |  190 |  230 |  170 |  235 |  135 |   -5 |   10 |               0 | True              |
|  713 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -200 |        275 |   75 |   30 |  -50 | -125 | -200 | -185 | -200 | -180 |               0 | True              |
|  714 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        145 |       -150 |        295 |   80 |  140 |   85 |  145 |   65 | -150 | -115 |  -60 |               0 | True              |
|  715 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        225 |        -30 |        255 |  130 |  225 |  155 |   -0 |  -30 |   10 |   70 |   45 |               0 | True              |
|  716 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        190 |        -30 |        220 |  190 |  180 |  190 |  100 |   -5 |   -5 |  -30 |  -10 |               0 | True              |
|  717 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        110 |       -140 |        250 |   65 |   80 |  110 |  -55 |  -30 | -130 |  -60 | -140 |               0 | True              |
|  718 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -540 |        540 | -345 | -275 | -310 | -385 | -505 | -535 | -540 | -435 |               4 | True              |
|  719 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         85 |       -205 |        290 |   20 |   85 |  -10 | -205 |  -60 | -185 |  -80 | -140 |               0 | True              |
|  720 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        160 |         -0 |        160 |  155 |  160 |   95 |  110 |  140 |   60 |  150 |   -0 |               0 | True              |
|  721 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         40 |       -205 |        245 |   40 |  -95 |  -60 |  -25 |  -10 |  -75 | -205 | -160 |               0 | True              |
|  722 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         40 |       -245 |        285 |   40 |   -0 |   40 |   15 | -110 | -210 | -185 | -245 |               0 | True              |
|  723 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -590 |        590 | -495 | -440 | -485 | -540 | -460 | -590 | -535 | -525 |               4 | True              |
|  724 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        205 |        -75 |        280 |  170 |  205 |  205 |  205 |   60 |  -75 |   10 |   20 |               0 | True              |
|  725 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        215 |       -115 |        330 |  215 |   -5 | -105 |  -45 |  -70 | -115 |  -60 |  -70 |               0 | True              |
|  726 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        240 |        -80 |        320 |  205 |  240 |  120 |  110 |  225 |  160 |  155 |  -80 |               0 | True              |
|  727 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        260 |        -25 |        285 |  155 |  145 |  180 |  175 |  260 |  205 |  120 |  -25 |               0 | True              |
|  728 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -370 |        370 | -140 | -155 | -225 | -170 | -155 | -115 | -100 | -370 |               0 | True              |
|  729 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        180 |        -30 |        210 |  -30 |   85 |   95 |  110 |  180 |  125 |   60 |   30 |               0 | True              |
|  730 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         10 |       -205 |        215 |  -95 |  -45 |   10 | -205 |  -60 | -115 | -125 |  -15 |               0 | True              |
|  731 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        300 |        -25 |        325 |  125 |  225 |  155 |  200 |   75 |  300 |  -20 |  -25 |               0 | True              |
|  732 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         60 |       -275 |        335 |   60 |  -45 |    5 |  -15 |  -40 | -275 | -165 | -175 |               0 | True              |
|  733 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -145 |        285 |  140 |   85 |  -45 | -145 |  -95 |  -40 | -115 |  -40 |               0 | True              |
|  734 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         75 |       -295 |        370 |  -35 |  -25 |   60 |  -65 |   75 |  -35 | -205 | -295 |               0 | True              |
|  735 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         45 |       -190 |        235 |  -55 |  -35 |  -45 |  -40 |   45 | -165 | -135 | -190 |               0 | True              |
|  736 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -650 |        650 | -350 | -355 | -650 | -555 | -550 | -520 | -620 | -625 |               4 | True              |
|  737 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -685 |        685 | -370 | -380 | -610 | -610 | -600 | -585 | -570 | -685 |               4 | True              |
|  738 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -210 |        210 |  -20 | -180 |  -55 |  -90 | -150 |  -20 |  -90 | -210 |               0 | True              |
|  739 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        115 |       -180 |        295 |  115 |   40 |  105 |  -55 | -180 | -160 | -145 | -160 |               0 | True              |
|  740 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -615 |        615 | -310 | -405 | -560 | -540 | -540 | -525 | -460 | -615 |               4 | True              |
|  741 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -480 |        480 | -450 | -400 | -410 | -415 | -480 | -400 | -430 | -455 |               3 | True              |
|  742 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        235 |       -110 |        345 |  190 |  140 |  235 |  130 | -110 |   -0 |  -55 |  -30 |               0 | True              |
|  743 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -610 |        610 | -320 | -310 | -345 | -415 | -405 | -580 | -610 | -580 |               4 | True              |
|  744 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |        -55 |        200 |   45 |   40 |  110 |  125 |  145 |  130 |  -15 |  -55 |               0 | True              |
|  745 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         45 |       -140 |        185 |  -75 | -125 |   30 |  -75 |   45 | -140 |  -25 |  -35 |               0 | True              |
|  746 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -580 |        580 | -340 | -405 | -290 | -340 | -280 | -580 | -535 | -545 |               4 | True              |
|  747 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        170 |       -105 |        275 |  165 |   30 |   35 |   35 |  170 |  -90 |  -70 | -105 |               0 | True              |
|  748 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        170 |       -105 |        275 |   95 |   75 |  115 |  170 |  145 |  -35 | -105 |  -25 |               0 | True              |
|  749 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        105 |        -15 |        120 |  -10 |    5 |   65 |  -15 |   60 |   90 |  105 |  -15 |               0 | True              |
|  750 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        155 |        -75 |        230 |  150 |  110 |  135 |  135 |  110 |  155 |  -75 |  -55 |               0 | True              |
|  751 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |        -65 |        215 |   70 |  120 |   80 |  125 |   90 |  150 |  115 |  -65 |               0 | True              |
|  752 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -365 |        365 | -115 |  -80 | -195 | -100 | -100 |  -90 | -160 | -365 |               0 | True              |
|  753 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -370 |        370 |  -80 |  -55 | -190 |  -15 |  -75 |  -75 | -370 | -330 |               1 | True              |
|  754 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        155 |       -100 |        255 |  155 |  135 |  125 |   35 | -100 |   -5 |   15 |  -35 |               0 | True              |
|  755 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        105 |       -215 |        320 |   85 |  105 |    5 | -125 | -185 | -215 | -185 |  -80 |               0 | True              |
|  756 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -320 |        320 | -135 | -130 |  -40 | -315 | -300 | -235 | -300 | -320 |               2 | True              |
|  757 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        145 |        -90 |        235 |  145 |   40 |  -45 |  105 |   30 |   20 |  -90 |  -35 |               0 | True              |
|  758 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -470 |        470 | -385 | -395 | -470 | -405 | -340 | -395 | -265 | -385 |               2 | True              |
|  759 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |       -135 |        175 |  -50 |  -40 |  -75 |   40 |  -10 |   20 | -135 |   25 |               0 | True              |
|  760 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -790 |        790 | -505 | -615 | -550 | -525 | -535 | -790 | -695 | -650 |               6 | True              |
|  761 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         50 |       -165 |        215 |   50 |  -95 | -135 |  -45 | -165 | -110 | -160 | -140 |               0 | True              |
|  762 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -260 |        260 |  -55 |  -45 |  -60 |  -75 |  -45 |  -60 |  -30 | -260 |               0 | True              |
|  763 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -410 |        410 | -395 | -335 | -410 | -370 | -355 | -305 | -310 | -260 |               3 | True              |
|  764 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -670 |        670 | -360 | -300 | -375 | -470 | -585 | -585 | -670 | -590 |               4 | True              |
|  765 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |       -250 |        405 |  155 | -250 | -130 | -100 |  -95 | -160 | -190 |  -65 |               0 | True              |
|  766 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -350 |        350 | -350 | -300 | -205 | -320 | -350 | -315 | -280 | -285 |               2 | True              |
|  767 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -315 |        315 |  -25 | -100 |  -10 |  -40 |  -45 | -100 | -315 | -300 |               1 | True              |
|  768 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        100 |       -135 |        235 |   55 |   60 |   65 |  100 |   40 |   95 |  -45 | -135 |               0 | True              |
|  769 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |       -260 |        375 |  115 | -155 | -200 | -180 | -260 | -200 |  -55 | -195 |               0 | True              |
|  770 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        215 |       -245 |        460 |  125 |  100 |  215 |   85 |   60 |   55 | -245 | -145 |               0 | True              |
|  771 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -315 |        315 | -185 | -315 | -160 | -250 | -130 | -215 |  -85 | -125 |               0 | True              |
|  772 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        110 |        -90 |        200 |  110 |   55 |  -30 |  -60 |  -90 |  -90 |  -30 |  -85 |               0 | True              |
|  773 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         45 |       -225 |        270 |   45 |   25 | -115 | -190 | -140 | -130 | -170 | -225 |               0 | True              |
|  774 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         20 |        -95 |        115 |  -40 |  -70 |  -95 |  -15 |  -90 |   15 |   20 |  -55 |               0 | True              |
|  775 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         15 |       -110 |        125 |  -85 |  -10 |  -20 |   -5 |  -40 | -110 |  -15 |   15 |               0 | True              |
|  776 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         35 |       -235 |        270 |   35 | -145 | -145 | -120 | -235 |  -95 | -120 | -195 |               0 | True              |
|  777 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         90 |       -295 |        385 |   90 |  -15 |   45 |   25 | -160 | -180 | -295 | -165 |               0 | True              |
|  778 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         70 |       -230 |        300 |   70 |   55 |   -0 |    5 | -185 | -230 | -155 | -155 |               0 | True              |
|  779 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        185 |        -20 |        205 |  -15 |   45 |   65 |  185 |   -5 |   20 |  -20 |  105 |               0 | True              |
|  780 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -540 |        540 | -225 | -245 | -215 | -230 | -320 | -275 | -460 | -540 |               3 | True              |
|  781 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -605 |        605 | -335 | -285 | -570 | -510 | -565 | -540 | -605 | -585 |               4 | True              |
|  782 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         90 |       -190 |        280 |   60 |   40 |   90 |  -75 | -140 | -110 | -180 | -190 |               0 | True              |
|  783 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |       -135 |        330 |   45 |  195 |   95 |   70 | -115 |  -40 | -135 | -100 |               0 | True              |
|  784 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        275 |        -25 |        300 |  190 |  275 |  265 |  215 |  180 |   35 |   15 |  -25 |               0 | True              |
|  785 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        225 |        -45 |        270 |  145 |   65 |    5 |  225 |  195 |  120 |  105 |  -45 |               0 | True              |
|  786 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -665 |        665 | -510 | -510 | -490 | -490 | -580 | -660 | -665 | -665 |               6 | True              |
|  787 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        180 |       -135 |        315 |   75 |   45 |  150 |  180 |  -90 | -135 | -120 | -115 |               0 | True              |
|  788 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -165 |        250 |   85 |  -30 |  -95 | -100 | -165 | -140 | -165 |  -55 |               0 | True              |
|  789 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          5 |       -225 |        230 |    5 |  -60 | -225 | -165 | -130 | -125 | -220 | -145 |               0 | True              |
|  790 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |        -70 |        185 |  115 |   55 |  -45 |  -15 |  -25 |  -45 |   25 |  -70 |               0 | True              |
|  791 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        150 |        -70 |        220 |  120 |  150 |   45 |  125 |  130 |  -50 |  -70 |   40 |               0 | True              |
|  792 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        120 |       -205 |        325 |   30 |   40 |   60 |   55 |   50 |  120 | -140 | -205 |               0 | True              |
|  793 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         80 |       -125 |        205 |   15 |  -15 |   80 |  -80 |   25 |  -40 | -100 | -125 |               0 | True              |
|  794 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        135 |       -115 |        250 |   95 |  100 |   80 |   65 |  135 |   65 | -115 |  -80 |               0 | True              |
|  795 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        165 |       -190 |        355 |  125 |  165 |  -30 | -135 |  -30 |  -90 | -190 |  -85 |               0 | True              |
|  796 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         15 |       -170 |        185 |   15 | -145 | -135 | -140 | -170 | -160 | -145 | -135 |               0 | True              |
|  797 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         -0 |       -175 |        175 |   -0 | -100 | -110 |  -65 |  -75 | -175 |  -15 | -130 |               0 | True              |
|  798 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |       -245 |        305 |   60 | -155 | -225 | -190 | -245 | -175 |  -65 | -155 |               0 | True              |
|  799 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        230 |       -130 |        360 |   90 |  -35 |  135 |  175 |  230 |   50 |  170 | -130 |               0 | True              |
|  800 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -525 |        525 | -290 | -255 | -390 | -355 | -360 | -395 | -525 | -520 |               3 | True              |
|  801 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        285 |        -65 |        350 |  145 |  285 |  260 |   40 |  -30 |  -60 |  -65 |  -20 |               0 | True              |
|  802 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -600 |        600 | -535 | -505 | -520 | -575 | -435 | -525 | -525 | -600 |               4 | True              |
|  803 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        115 |       -160 |        275 |   30 |  115 |  110 |   65 |   95 | -115 | -160 | -100 |               0 | True              |
|  804 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |        -35 |        280 |  180 |  170 |  230 |  245 |   40 |  140 |  -35 |   80 |               0 | True              |
|  805 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         75 |       -155 |        230 |   70 |   25 |  -10 |   75 |   20 | -155 |  -85 | -115 |               0 | True              |
|  806 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -330 |        330 | -180 | -295 | -155 | -330 | -255 | -280 | -265 | -140 |               0 | True              |
|  807 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -195 |        195 | -105 | -195 | -115 | -140 | -135 | -115 | -160 | -120 |               0 | True              |
|  808 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        215 |       -190 |        405 |  160 |  190 |  215 |  105 | -135 |  -45 | -100 | -190 |               0 | True              |
|  809 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -640 |        640 | -360 | -405 | -605 | -585 | -640 | -635 | -575 | -605 |               4 | True              |
|  810 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        210 |        -75 |        285 |  140 |  180 |  155 |  210 |  185 |  210 |  210 |  -75 |               0 | True              |
|  811 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        220 |       -170 |        390 |  115 |   65 |   75 |  100 |  220 | -140 | -170 |  -30 |               0 | True              |
|  812 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -285 |        285 | -205 | -220 | -190 | -180 | -195 | -285 | -190 | -210 |               1 | True              |
|  813 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -480 |        480 | -200 | -150 | -195 | -120 |  -60 | -360 | -480 | -415 |               3 | True              |
|  814 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         45 |       -250 |        295 |   45 |  -10 |  -55 | -175 | -250 | -235 | -125 | -145 |               0 | True              |
|  815 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |       -190 |        370 |  180 |  180 | -190 | -105 |   65 |   35 |  -35 |  -55 |               0 | True              |
|  816 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -425 |        425 | -410 | -400 | -260 | -425 | -365 | -380 | -415 | -365 |               3 | True              |
|  817 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        185 |        -85 |        270 |  185 |   55 |  130 |  125 |  105 |  135 |  -85 |  -85 |               0 | True              |
|  818 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          5 |       -270 |        275 |    5 |  -90 |  -50 | -225 | -150 | -200 | -140 | -270 |               0 | True              |
|  819 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -570 |        570 | -410 | -400 | -400 | -570 | -435 | -555 | -440 | -420 |               3 | True              |
|  820 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -390 |        390 | -105 | -170 | -390 | -340 | -320 | -390 | -345 | -300 |               2 | True              |
|  821 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         40 |       -310 |        350 |   40 |   30 |   15 |  -20 |   10 | -310 | -190 | -140 |               0 | True              |
|  822 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         10 |       -370 |        380 |   10 |  -95 |  -95 | -105 | -105 | -270 | -370 | -305 |               0 | True              |
|  823 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -545 |        545 | -345 | -410 | -270 | -545 | -510 | -510 | -500 | -515 |               4 | True              |
|  824 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        210 |        -85 |        295 |  210 |  195 |   35 |  -25 |  -40 |   -0 |  -15 |  -85 |               0 | True              |
|  825 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         70 |       -245 |        315 |   -5 |  -50 |   15 |  -50 |  -40 |   70 |   10 | -245 |               0 | True              |
|  826 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -435 |        435 | -115 | -155 | -395 | -360 | -435 | -395 | -425 | -305 |               2 | True              |
|  827 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -475 |        475 | -280 | -295 | -140 | -110 | -430 | -450 | -435 | -475 |               3 | True              |
|  828 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -695 |        695 | -530 | -380 | -490 | -485 | -325 | -635 | -695 | -690 |               6 | True              |
|  829 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        260 |        -50 |        310 |  205 |  100 |  260 |  -15 |   75 |  -40 |  -40 |  -50 |               0 | True              |
|  830 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        175 |       -150 |        325 |  115 |  175 | -150 | -115 | -105 | -150 | -130 |  -85 |               0 | True              |
|  831 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -715 |        715 | -485 | -435 | -435 | -530 | -480 | -635 | -685 | -715 |               5 | True              |
|  832 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -615 |        615 | -340 | -400 | -390 | -270 | -290 | -265 | -510 | -615 |               4 | True              |
|  833 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -450 |        450 |  -75 | -125 | -155 | -105 | -145 | -110 | -175 | -450 |               0 | True              |
|  834 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -330 |        330 | -330 | -175 | -255 | -195 | -140 | -205 | -240 | -175 |               2 | True              |
|  835 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -505 |        505 | -230 | -250 | -480 | -460 | -505 | -445 | -465 | -490 |               3 | True              |
|  836 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        105 |       -155 |        260 |  105 | -105 |  -90 | -155 | -125 |  -80 | -110 | -130 |               0 | True              |
|  837 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -640 |        640 | -335 | -400 | -320 | -455 | -320 | -435 | -640 | -625 |               4 | True              |
|  838 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -435 |        435 | -165 | -380 | -240 | -300 | -175 | -265 | -220 | -435 |               0 | True              |
|  839 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -580 |        580 | -380 | -335 | -450 | -490 | -440 | -580 | -530 | -440 |               4 | True              |
|  840 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         25 |       -305 |        330 | -105 |   10 |   25 |  -40 | -305 | -275 | -255 | -280 |               1 | True              |
|  841 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        220 |          0 |        220 |  200 |  180 |  220 |  145 |  195 |    5 |   50 |   80 |              16 | True              |
|  842 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -480 |        480 | -190 | -190 | -160 | -260 | -195 | -285 | -225 | -480 |               0 | True              |
|  843 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -210 |        210 |  -85 | -100 | -210 | -160 | -110 | -200 | -120 | -105 |               0 | True              |
|  844 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         85 |       -185 |        270 |   80 |   25 |   70 |   20 |   -0 |   40 |   85 | -185 |               0 | True              |
|  845 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -525 |        525 | -380 | -320 | -350 | -320 | -240 | -455 | -525 | -510 |               5 | True              |
|  846 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |       -110 |        310 |  200 |  175 |   35 |   70 |  200 |  135 |  -95 | -110 |               0 | True              |
|  847 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -145 |        145 |  -75 |  -25 |  -50 | -145 | -105 |  -35 | -115 | -145 |               0 | True              |
|  848 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        220 |       -165 |        385 |  140 |   95 |  220 |   20 |  130 | -110 | -165 | -100 |               0 | True              |
|  849 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        200 |        -45 |        245 |  200 |  145 |  170 |  190 |  165 |  175 |  160 |  -45 |               0 | True              |
|  850 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 |  -90 | -205 | -135 | -115 | -150 |  -50 | -185 | -185 |               0 | True              |
|  851 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |       -110 |        355 |  215 |  220 |  245 |  225 |   35 | -110 |   15 |   -5 |               0 | True              |
|  852 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        165 |        -60 |        225 |  115 |  145 |  165 |   85 |  120 |   90 |  -60 |  -60 |               0 | True              |
|  853 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         75 |        -30 |        105 |  -20 |   75 |  -15 |   -5 |   45 |  -30 |   20 |   20 |               0 | True              |
|  854 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         55 |       -195 |        250 |   55 |    5 |   25 |  -75 | -195 | -120 | -105 | -180 |               0 | True              |
|  855 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |       -195 |        365 |   10 |  170 |   50 |  130 |   20 |  110 |   70 | -195 |               0 | True              |
|  856 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        115 |        -30 |        145 |  -30 |   25 |   -0 |  -10 |  115 |   75 |   -0 |   60 |               0 | True              |
|  857 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -265 |        350 |   85 | -190 | -190 | -145 | -265 | -175 | -145 | -225 |               0 | True              |
|  858 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |        -70 |        250 |  180 |   85 |  100 |  135 |  125 |   20 |  -70 |  -60 |               0 | True              |
|  859 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        325 |        -35 |        360 |  200 |  325 |  195 |  310 |  -35 |   45 |  100 |   40 |               0 | True              |
|  860 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         80 |       -180 |        260 |   80 | -110 |  -65 |  -80 |  -85 |  -90 | -180 | -125 |               0 | True              |
|  861 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -300 |        300 | -150 | -140 |  -40 | -220 | -275 | -220 | -300 | -205 |               2 | True              |
|  862 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -405 |        405 | -200 |  -85 | -405 | -380 | -360 | -325 | -350 | -360 |               2 | True              |
|  863 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |       -130 |        310 |  180 |   75 |  -35 | -130 |  -65 |  -35 |  -90 |  -95 |               0 | True              |
|  864 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |       -270 |        320 |   50 |  -50 | -260 | -230 | -270 | -225 | -235 | -210 |               0 | True              |
|  865 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        135 |       -160 |        295 |   60 |   30 |   30 |   50 |   -5 |   -0 |  135 | -160 |               0 | True              |
|  866 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |        -90 |        260 |  145 |  135 |   15 |   75 |  170 |  125 |  -80 |  -90 |               0 | True              |
|  867 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -445 |        445 | -320 | -220 | -245 | -340 | -280 | -290 | -410 | -445 |               4 | True              |
|  868 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -210 |        210 |  -65 |  -65 | -165 | -185 | -120 | -115 |  -90 | -210 |               0 | True              |
|  869 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        245 |        -45 |        290 |  145 |  125 |  165 |  135 |  165 |  245 |  180 |  -45 |               0 | True              |
|  870 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        185 |       -125 |        310 |   70 |  185 |  120 |  105 |  110 |   55 |   80 | -125 |               0 | True              |
|  871 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        155 |       -180 |        335 |  155 |  100 |  -55 |  -70 | -180 |  -70 | -170 | -160 |               0 | True              |
|  872 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |       -130 |        280 |  150 |   30 |  120 |   15 |  -25 | -130 |  -80 |   15 |               0 | True              |
|  873 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        335 |          0 |        335 |  205 |  320 |  205 |  335 |  110 |   50 |   15 |   70 |               0 | True              |
|  874 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -690 |        690 | -390 | -690 | -590 | -530 | -460 | -580 | -660 | -550 |               4 | True              |
|  875 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |        -95 |        275 |  180 |  155 |  -50 |  -35 |  -35 |  -95 |  -25 |  -95 |               0 | True              |
|  876 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -645 |        645 | -590 | -645 | -610 | -520 | -540 | -415 | -585 | -615 |               4 | True              |
|  877 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -410 |        440 |    5 |   30 | -170 | -215 | -410 | -325 | -180 | -225 |               0 | True              |
|  878 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         80 |       -175 |        255 |   60 |   80 | -120 |  -95 | -150 |  -80 | -175 | -135 |               0 | True              |
|  879 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         20 |       -150 |        170 | -115 | -100 |   20 |  -90 |  -60 | -110 |  -60 | -150 |               0 | True              |
|  880 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -510 |        510 | -205 | -225 | -280 | -210 | -250 | -320 | -285 | -510 |               0 | True              |
|  881 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        105 |       -150 |        255 |  105 |  -55 |  -15 | -100 |  -85 | -150 | -130 |  -95 |               0 | True              |
|  882 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |       -120 |        295 |   30 |  155 |   90 |   25 |  175 |  -60 | -120 |  -40 |               0 | True              |
|  883 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        155 |        -90 |        245 |  135 |  130 |  155 |  135 |   20 |  -70 |  -20 |  -90 |               0 | True              |
|  884 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         65 |       -140 |        205 |   60 |   65 |   20 |  -15 |   50 |   60 | -140 | -105 |               0 | True              |
|  885 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -655 |        655 | -405 | -450 | -510 | -605 | -465 | -605 | -655 | -515 |               5 | True              |
|  886 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -435 |        435 | -185 | -250 | -300 | -275 | -315 | -225 | -295 | -435 |               0 | True              |
|  887 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        360 |          0 |        360 |  280 |  315 |  360 |  300 |  265 |  180 |   10 |  120 |              16 | True              |
|  888 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -380 |        380 | -175 | -230 | -250 | -270 | -235 | -290 | -380 | -280 |               0 | True              |
|  889 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -380 |        380 |  -75 |  -85 | -100 | -280 | -225 | -315 | -180 | -380 |               1 | True              |
|  890 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |        -95 |        195 |   50 |   20 |   -5 |  100 |   90 |  -85 |  -95 |  -75 |               0 | True              |
|  891 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         40 |       -225 |        265 |   40 |  -65 | -150 | -115 | -180 | -185 | -225 | -145 |               0 | True              |
|  892 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -250 |        350 |  100 |   10 |   20 |   80 |   15 | -250 | -195 | -175 |               0 | True              |
|  893 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -740 |        740 | -520 | -620 | -560 | -590 | -505 | -490 | -550 | -740 |               4 | True              |
|  894 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        245 |        -25 |        270 |  190 |  210 |  180 |  230 |  245 |   20 |   70 |  -25 |               0 | True              |
|  895 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         95 |       -235 |        330 |   35 |   95 | -190 | -165 | -165 | -115 | -210 | -235 |               0 | True              |
|  896 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         80 |       -150 |        230 |   80 |  -35 |   65 |   65 |   35 |   60 | -150 |  -50 |               0 | True              |
|  897 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        170 |       -185 |        355 |  170 |  150 |  125 |   60 | -150 | -185 |  -45 |  -55 |               0 | True              |
|  898 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |       -185 |        305 |  120 | -185 | -120 | -125 | -140 | -105 |  -45 | -135 |               0 | True              |
|  899 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        225 |        -60 |        285 |  165 |  140 |  220 |  160 |  155 |  225 |  -35 |  -60 |               0 | True              |
|  900 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -255 |        355 |  100 | -235 | -180 | -125 | -135 | -225 | -255 | -170 |               0 | True              |
|  901 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        185 |       -120 |        305 |  110 |  185 |  140 | -100 | -120 |  -40 | -120 | -110 |               0 | True              |
|  902 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        115 |       -225 |        340 |  115 |  105 | -165 | -105 | -145 | -165 | -225 | -220 |               0 | True              |
|  903 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -535 |        535 | -340 | -305 | -435 | -405 | -355 | -430 | -385 | -535 |               2 | True              |
|  904 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        270 |        -15 |        285 |  185 |  250 |  250 |  250 |  270 |   75 |   45 |  -15 |               0 | True              |
|  905 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 | -160 | -130 | -245 | -165 | -220 | -145 |  -95 | -200 |               0 | True              |
|  906 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        105 |        -95 |        200 |  -80 |  -15 |  105 |  -95 |   20 |   50 |   40 |   -0 |               0 | True              |
|  907 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -345 |        345 | -190 | -120 | -120 | -135 |  -80 | -295 | -345 | -280 |               2 | True              |
|  908 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        155 |       -210 |        365 |   95 |  155 |   95 |   20 |  110 | -135 |  -60 | -210 |               0 | True              |
|  909 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        180 |       -140 |        320 |  180 |  130 |   30 |   70 |  -15 | -140 |  -40 |   15 |               0 | True              |
|  910 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        105 |       -260 |        365 |  105 |  -40 | -235 | -255 | -260 | -110 | -185 | -165 |               0 | True              |
|  911 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        185 |       -140 |        325 |   80 |  120 |  110 |  185 |   50 |   30 | -140 |  -50 |               0 | True              |
|  912 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -255 |        255 | -180 | -155 | -140 | -100 |  -95 |  -85 | -110 | -255 |               0 | True              |
|  913 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -315 |        315 | -110 | -280 | -235 | -270 | -295 | -315 | -215 | -280 |               2 | True              |
|  914 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        145 |       -140 |        285 |  120 |  145 |  -10 | -140 | -105 |   30 |  -95 |  -40 |               0 | True              |
|  915 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         25 |       -270 |        295 |  -55 |  -35 | -105 |  -20 |   25 | -220 | -175 | -270 |               1 | True              |
|  916 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -525 |        525 | -375 | -360 | -345 | -475 | -475 | -490 | -525 | -460 |               4 | True              |
|  917 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        115 |        -25 |        140 |  -25 |   15 |   80 |   -0 |  115 |   35 |    5 |  -10 |               0 | True              |
|  918 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        180 |        -80 |        260 |  -35 |   35 |   35 |  100 |  180 |   60 |   35 |  -80 |               0 | True              |
|  919 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -650 |        650 | -470 | -460 | -485 | -480 | -390 | -515 | -425 | -650 |               3 | True              |
|  920 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        140 |        -95 |        235 |  110 |  140 |  110 |  -55 |  -95 |   -5 |  -20 |  -85 |               0 | True              |
|  921 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        150 |       -140 |        290 |  130 |  150 | -115 |  -35 | -115 | -140 |  -50 |  -60 |               0 | True              |
|  922 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        340 |          0 |        340 |  180 |  340 |  220 |   45 |   40 |   45 |   65 |   95 |               0 | True              |
|  923 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        135 |       -150 |        285 |  135 |  -40 |  -40 |  -85 |   30 |  -75 | -115 | -150 |               0 | True              |
|  924 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |       -130 |        325 |  170 |  155 |  195 |  175 | -125 |    5 |  -20 | -130 |               0 | True              |
|  925 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        175 |       -155 |        330 |   80 |  175 |  -55 | -150 | -155 |  -80 |  -95 | -140 |               0 | True              |
|  926 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        145 |       -195 |        340 |  145 | -120 | -105 | -195 |  -70 | -155 | -110 | -170 |               0 | True              |
|  927 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        150 |       -105 |        255 |  150 |  -40 |    5 |   10 | -105 |  -30 |   -0 |  -65 |               0 | True              |
|  928 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -800 |        800 | -510 | -510 | -785 | -685 | -800 | -750 | -730 | -765 |               6 | True              |
|  929 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -605 |        605 | -530 | -420 | -505 | -585 | -605 | -550 | -545 | -575 |               4 | True              |
|  930 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        140 |       -100 |        240 |   -5 |  140 |  115 |  110 |   -0 |   40 |  115 | -100 |               0 | True              |
|  931 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         65 |       -155 |        220 |   65 |   -5 |  -25 |  -15 |  -75 | -135 | -150 | -155 |               0 | True              |
|  932 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        150 |       -120 |        270 |  150 |  -25 |  -75 |    5 |  -65 | -120 |   85 |   45 |               0 | True              |
|  933 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         80 |       -250 |        330 |   80 |  -45 | -115 |   65 |  -30 | -250 | -155 | -190 |               0 | True              |
|  934 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |        -70 |        250 |  175 |  180 |  -70 |   40 |   85 |   75 |  -45 |  110 |               0 | True              |
|  935 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -720 |        720 | -445 | -520 | -295 | -505 | -680 | -720 | -685 | -640 |               5 | True              |
|  936 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -315 |        315 | -120 | -315 | -190 | -220 | -260 |  -95 | -155 | -225 |               0 | True              |
|  937 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        155 |       -140 |        295 |   95 |  155 | -125 | -120 | -140 | -125 |  -45 | -100 |               0 | True              |
|  938 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        130 |       -210 |        340 |   55 |  130 | -135 | -180 | -145 | -140 | -210 | -155 |               0 | True              |
|  939 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -275 |        375 |  100 | -110 | -160 |  -95 |  -55 |  -40 |  -85 | -275 |               0 | True              |
|  940 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         60 |       -225 |        285 |   60 |  -65 | -110 |   -5 |   35 |   -0 | -225 | -150 |               0 | True              |
|  941 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -230 |        230 | -150 | -230 | -160 | -220 | -170 | -125 | -180 | -110 |               0 | True              |
|  942 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         10 |       -305 |        315 | -130 |   10 |  -80 |  -10 | -130 | -120 |  -90 | -305 |               0 | True              |
|  943 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -550 |        550 | -260 | -240 | -310 | -405 | -435 | -445 | -500 | -550 |               3 | True              |
|  944 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        205 |        -55 |        260 |  205 |   80 |  130 |  205 |  180 |  -55 |   -0 |   35 |               0 | True              |
|  945 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -340 |        340 |  -40 | -105 | -145 | -145 | -165 | -120 | -170 | -340 |               0 | True              |
|  946 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        175 |       -130 |        305 |  145 |   45 |  110 |  175 |  -65 | -130 |  -50 | -105 |               0 | True              |
|  947 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        220 |        -90 |        310 |  105 |  110 |  220 |  170 |  205 |  -15 |  -45 |  -90 |               0 | True              |
|  948 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |       -215 |        320 |    5 |   50 |  105 |   30 |   90 |   65 |   70 | -215 |               0 | True              |
|  949 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -620 |        620 | -340 | -355 | -475 | -445 | -350 | -530 | -620 | -600 |               4 | True              |
|  950 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         55 |       -220 |        275 |   55 |  -25 |   25 |  -25 |  -25 |  -75 | -170 | -220 |               0 | True              |
|  951 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        290 |        -75 |        365 |   90 |  235 |  290 |  -70 |  -75 |    5 |  -55 |    5 |               0 | True              |
|  952 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |        -60 |        165 |   25 |   55 |   65 |   75 |   20 |   50 |  105 |  -60 |               0 | True              |
|  953 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         70 |       -155 |        225 |   10 |   -5 |   20 | -110 |  -40 |   -0 |   70 | -155 |               0 | True              |
|  954 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        185 |       -160 |        345 |   40 |  185 |   95 |   45 |  -70 |  -80 | -160 |   15 |               0 | True              |
|  955 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         70 |       -235 |        305 |   70 |  -20 |  -65 |    5 |  -20 |  -30 |   -0 | -235 |               0 | True              |
|  956 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -235 |        235 |  -10 | -230 | -225 | -160 | -165 | -175 | -235 | -150 |               1 | True              |
|  957 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        135 |       -220 |        355 |   40 |   85 |   20 |  135 | -130 | -220 | -195 | -115 |               0 | True              |
|  958 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        195 |       -100 |        295 |  130 |  110 |  145 |   70 |   40 |  195 |  140 | -100 |               0 | True              |
|  959 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -145 |        145 |  -60 | -135 | -145 |  -80 | -135 |  -80 |  -90 | -145 |               0 | True              |
|  960 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          5 |       -395 |        400 |    5 | -250 | -395 | -305 | -250 | -240 | -255 | -310 |               0 | True              |
|  961 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         70 |       -175 |        245 |   50 |  -10 |   70 | -175 | -135 |  -55 | -150 | -125 |               0 | True              |
|  962 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         70 |       -230 |        300 |   60 |   70 |   60 |  -25 |   65 | -140 | -230 | -115 |               0 | True              |
|  963 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -270 |        270 | -260 | -270 |  -85 | -145 | -210 | -175 | -185 | -130 |               1 | True              |
|  964 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        215 |       -150 |        365 |  215 |  115 | -150 |  -50 | -150 |  -10 |  -65 | -110 |               0 | True              |
|  965 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -395 |        395 | -115 | -325 | -350 | -395 | -310 | -360 | -315 | -285 |               2 | True              |
|  966 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -475 |        475 | -215 | -175 | -225 | -245 | -190 | -360 | -475 | -435 |               3 | True              |
|  967 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -645 |        645 | -370 | -400 | -395 | -380 | -405 | -360 | -400 | -645 |               2 | True              |
|  968 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -420 |        420 | -345 | -420 | -340 | -250 | -340 | -285 | -270 | -285 |               2 | True              |
|  969 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         75 |       -245 |        320 |   15 |   75 |  -10 |   50 | -215 | -235 | -245 | -225 |               0 | True              |
|  970 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |       -215 |        335 |  120 |  -75 | -145 |  -60 | -135 | -135 | -215 |  -75 |               0 | True              |
|  971 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        135 |       -155 |        290 |   95 |  120 |  135 |  120 |   75 | -155 | -110 |  -95 |               0 | True              |
|  972 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        200 |        -60 |        260 |  160 |  185 |  190 |  200 |  -60 |  -60 |  -60 |  -40 |               0 | True              |
|  973 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        205 |        -45 |        250 |  205 |   70 |  -45 |   65 |   30 |   95 |   -0 |  -10 |               0 | True              |
|  974 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         55 |       -190 |        245 |  -20 |  -40 |    5 |   10 |   55 |   -0 | -180 | -190 |               1 | True              |
|  975 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -320 |        320 |  -55 |  -90 | -105 | -315 | -320 | -245 | -225 | -270 |               1 | True              |
|  976 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        145 |        -65 |        210 |  105 |  145 |   50 |   85 |  140 |  135 |   55 |  -65 |               0 | True              |
|  977 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -670 |        670 | -480 | -525 | -515 | -530 | -510 | -510 | -470 | -670 |               3 | True              |
|  978 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -285 |        285 | -170 | -240 | -180 | -125 | -110 | -195 | -150 | -285 |               0 | True              |
|  979 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         70 |       -230 |        300 |   70 |  -80 | -140 |  -85 |  -95 | -230 |  -60 | -200 |               0 | True              |
|  980 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         10 |       -140 |        150 | -140 |  -80 |  -25 | -100 |  -15 |  -25 |   10 | -115 |               0 | True              |
|  981 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -625 |        625 | -350 | -530 | -555 | -530 | -610 | -625 | -540 | -560 |               4 | True              |
|  982 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -180 |        200 |   20 | -175 | -160 | -180 | -180 | -115 | -140 | -130 |               0 | True              |
|  983 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |       -240 |        360 |  120 | -140 | -100 |  -30 | -145 | -240 | -190 | -170 |               0 | True              |
|  984 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         90 |       -295 |        385 |   90 | -190 | -190 | -220 | -295 | -250 | -180 | -195 |               0 | True              |
|  985 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -325 |        325 | -185 | -155 | -215 | -285 | -220 | -315 | -265 | -325 |               1 | True              |
|  986 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        105 |       -180 |        285 |  105 |   70 |   10 | -110 |  -60 | -140 | -180 | -145 |               0 | True              |
|  987 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -650 |        650 | -610 | -490 | -520 | -650 | -585 | -560 | -595 | -550 |               5 | True              |
|  988 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        300 |        -40 |        340 |  170 |  300 |  200 |  150 |  165 |  195 |   25 |  -40 |               0 | True              |
|  989 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -385 |        385 | -150 | -170 | -220 | -150 | -250 | -135 | -265 | -385 |               0 | True              |
|  990 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -615 |        615 | -295 | -285 | -240 | -520 | -495 | -525 | -520 | -615 |               4 | True              |
|  991 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        200 |          0 |        200 |  110 |   80 |  200 |  135 |  115 |  125 |  160 |   40 |               0 | True              |
|  992 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |       -115 |        305 |  160 |  135 |  190 |  190 |  135 | -115 |  -60 |   40 |               0 | True              |
|  993 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         70 |       -175 |        245 |   30 |  -15 |   70 | -150 | -175 | -115 | -125 |  -75 |               0 | True              |
|  994 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -415 |        415 | -285 | -290 | -300 | -270 | -415 | -275 | -290 | -350 |               1 | True              |
|  995 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        160 |       -175 |        335 |  115 |  160 |   25 |  125 |  115 | -175 | -115 |  -65 |               0 | True              |
|  996 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        140 |        -65 |        205 |   15 |  140 |  -15 |   20 |   80 |   65 |   -0 |  -65 |              16 | True              |
|  997 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -485 |        485 | -155 | -165 | -140 | -125 | -215 | -260 | -485 | -365 |               2 | True              |
|  998 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         45 |       -395 |        440 |   45 |   45 |  -20 |  -80 | -265 | -395 | -290 | -210 |               0 | True              |
|  999 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        150 |       -195 |        345 |  150 | -155 |  -45 |  -80 |  -45 | -195 | -115 | -115 |               0 | True              |
| 1000 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -440 |        440 | -110 | -330 | -345 | -440 | -360 | -295 | -320 | -285 |               2 | True              |
| 1001 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        175 |       -110 |        285 |  175 |  120 |  -15 | -110 |  -20 |   20 |  -35 | -105 |               0 | True              |
| 1002 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -420 |        420 | -240 |  -85 | -200 | -220 | -180 | -195 | -195 | -420 |               1 | True              |
| 1003 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        235 |       -195 |        430 |  170 |  235 |   45 |  -50 | -160 | -190 | -145 | -195 |               0 | True              |
| 1004 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        145 |       -230 |        375 |   10 |   95 |  -65 |   25 |   55 |   25 |  145 | -230 |               0 | True              |
| 1005 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        165 |       -130 |        295 |   70 |  120 |   90 |  125 |   85 |  165 | -130 |  -10 |               0 | True              |
| 1006 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -200 | -200 | -200 | -260 | -205 | -265 | -240 | -175 |               0 | True              |
| 1007 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         15 |       -175 |        190 |  -10 |   -0 |  -45 |   10 |   15 |    5 |   -0 | -175 |               0 | True              |
| 1008 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        100 |       -250 |        350 | -115 |  100 |  -45 | -250 | -225 | -220 | -225 | -225 |               2 | True              |
| 1009 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        200 |        -90 |        290 |  200 |  -85 |  -25 |  -90 |  -20 |  -80 |  -60 |  -35 |               0 | True              |
| 1010 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        185 |       -145 |        330 |   80 |  105 |  185 |  -90 |   -0 | -120 |  -20 | -145 |               0 | True              |
| 1011 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        160 |       -175 |        335 |  160 |   40 | -160 | -100 | -145 | -175 |  -35 |  -85 |               0 | True              |
| 1012 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |        -25 |         65 |   -0 |   15 |   40 |  -25 |    5 |   -5 |   35 |   15 |               0 | True              |
| 1013 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         70 |       -300 |        370 |   70 |   35 | -110 | -115 | -165 | -200 | -300 | -135 |               0 | True              |
| 1014 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         75 |       -215 |        290 |   25 |   75 |   65 |  -50 | -215 | -140 | -170 | -185 |               0 | True              |
| 1015 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         50 |       -195 |        245 |   50 |  -20 |   10 | -175 | -185 | -125 | -105 | -195 |               0 | True              |
| 1016 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -380 |        380 | -190 | -140 | -335 | -380 | -375 | -355 | -370 | -345 |               2 | True              |
| 1017 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         95 |       -105 |        200 |  -65 |   -5 |  -30 |  -80 |  -35 | -105 |   95 |  -40 |               0 | True              |
| 1018 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         30 |       -270 |        300 |   30 |  -55 |  -90 | -225 | -225 | -185 | -215 | -270 |               0 | True              |
| 1019 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -240 |        365 |   65 |  -70 |   75 |   90 |  125 |  -10 | -145 | -240 |               0 | True              |
| 1020 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         55 |       -105 |        160 |   30 |   -5 |   15 |  -70 |    5 |   55 |  -20 | -105 |              16 | True              |
| 1021 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        135 |       -155 |        290 |   55 |  -35 |   75 |   85 |  135 | -110 | -155 | -150 |               0 | True              |
| 1022 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -525 |        525 | -435 | -370 | -485 | -375 | -385 | -380 | -525 | -370 |               3 | True              |
| 1023 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        135 |       -150 |        285 |  135 |  135 |   75 |   85 |  -35 |  -90 | -115 | -150 |               0 | True              |
| 1024 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -170 |        170 |  -65 |  -15 |  -85 | -125 |  -15 | -120 |  -60 | -170 |               0 | True              |
| 1025 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         60 |       -165 |        225 |   -0 |   60 |  -55 |   35 |  -20 |   25 | -165 | -105 |               0 | True              |
| 1026 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        250 |        -45 |        295 |  145 |   90 |  250 |   20 |   85 |  -45 |   60 |   -5 |               0 | True              |
| 1027 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         45 |       -335 |        380 |  -50 |   20 |   45 |  -75 |  -70 |  -25 |  -40 | -335 |               0 | True              |
| 1028 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -330 |        330 | -105 | -170 |  -70 | -235 | -260 | -275 | -330 | -235 |               1 | True              |
| 1029 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |       -120 |        295 |   75 |  120 |  155 |  175 |  175 | -120 |  -65 |  -55 |               0 | True              |
| 1030 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        195 |       -140 |        335 |  105 |  145 |  195 |  -35 |   35 |    5 | -140 |  -40 |               0 | True              |
| 1031 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -795 |        795 | -525 | -505 | -705 | -710 | -685 | -795 | -755 | -750 |               5 | True              |
| 1032 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        330 |          0 |        330 |  140 |  330 |  260 |  305 |   80 |  120 |  145 |  145 |               0 | True              |
| 1033 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -500 |        500 | -260 | -255 | -325 | -375 | -500 | -375 | -495 | -405 |               3 | True              |
| 1034 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -460 |        460 | -315 | -275 | -345 | -270 | -370 | -305 | -360 | -460 |               2 | True              |
| 1035 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |       -125 |        205 |  -85 |  -70 |  -40 |   80 | -125 |   25 |   10 |   65 |               0 | True              |
| 1036 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        255 |         -5 |        260 |  180 |  240 |  255 |   -5 |   20 |   90 |   40 |   60 |               0 | True              |
| 1037 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        140 |       -130 |        270 |  140 |  105 |   30 |   55 |   75 |  125 | -130 |  -55 |               0 | True              |
| 1038 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -720 |        720 | -385 | -350 | -590 | -575 | -485 | -560 | -720 | -590 |               4 | True              |
| 1039 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        215 |         -0 |        215 |  205 |   45 |  215 |  180 |  155 |  160 |   -0 |   10 |               0 | True              |
| 1040 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         35 |       -285 |        320 |   15 |  -45 |   35 |  -45 |  -50 |   25 |  -25 | -285 |               0 | True              |
| 1041 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        155 |        -80 |        235 |  155 |   35 |   45 |  -50 |   60 |  -60 |  -80 |  -45 |               0 | True              |
| 1042 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        175 |       -165 |        340 |  175 |   10 |    5 |  -35 |  -55 |   20 |   10 | -165 |               0 | True              |
| 1043 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -230 |        230 |  -90 | -115 | -130 | -145 | -170 | -230 | -190 | -135 |               0 | True              |
| 1044 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        175 |        -90 |        265 |  175 |  -75 |  -25 |   65 |  -40 |  -90 |  -10 |   65 |               0 | True              |
| 1045 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -140 |        320 |   85 |  180 |  155 |  115 |  165 | -140 | -120 |  -95 |               0 | True              |
| 1046 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        260 |        -60 |        320 |   80 |  260 |  155 |  165 |  145 |  115 |  -60 |  -45 |               0 | True              |
| 1047 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        115 |       -100 |        215 |   75 |  115 |   90 |  -50 | -100 |  -35 |  -20 |   10 |               0 | True              |
| 1048 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         70 |       -215 |        285 |   70 | -185 | -205 | -215 |  -35 | -195 | -200 | -180 |               0 | True              |
| 1049 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         65 |       -140 |        205 |   30 |   60 |   10 |   55 |   65 | -110 |  -85 | -140 |               0 | True              |
| 1050 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        110 |       -210 |        320 |  110 |   90 |  -85 | -155 | -195 | -115 | -145 | -210 |               0 | True              |
| 1051 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        175 |        -45 |        220 |  145 |  175 |  170 |  -35 |   30 |  -45 |   40 |  -25 |               0 | True              |
| 1052 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        140 |        -25 |        165 |  140 |   80 |   15 |  -20 |  -25 |  140 |   75 |   20 |               0 | True              |
| 1053 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        130 |       -155 |        285 |   40 |  130 |  120 | -105 |  -50 | -155 | -140 | -150 |               0 | True              |
| 1054 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        120 |       -100 |        220 |   60 |  120 |  110 |   10 |   15 |  -90 | -100 |  -40 |               0 | True              |
| 1055 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        260 |        -25 |        285 |   50 |  190 |  185 |  145 |  260 |  135 |  -25 |   35 |               0 | True              |
| 1056 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        195 |        -55 |        250 |  195 |   60 |   30 |   75 |   -5 |   65 |   20 |  -55 |               0 | True              |
| 1057 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -145 |        170 |  -95 |  -15 |  -30 |  -80 |   25 |  -85 |  -65 | -145 |               0 | True              |
| 1058 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         40 |       -270 |        310 |   40 |  -50 |   35 |  -30 | -100 | -195 | -270 | -150 |               0 | True              |
| 1059 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        125 |       -240 |        365 |  125 | -165 | -130 | -240 |  -90 | -200 | -165 | -230 |               0 | True              |
| 1060 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |        -90 |        190 |  100 |   30 |  -10 |   50 |  -15 |  -20 |  100 |  -90 |               0 | True              |
| 1061 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -435 |        435 | -385 | -330 | -305 | -245 | -435 | -395 | -355 | -425 |               2 | True              |
| 1062 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        195 |        -35 |        230 |  195 |  100 |   90 |  -35 |   95 |  110 |   90 |   15 |               0 | True              |
| 1063 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         50 |       -175 |        225 |   25 |  -15 |  -30 |   50 |   50 |   20 | -175 | -150 |               0 | True              |
| 1064 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         60 |       -290 |        350 |   60 |    5 |  -10 | -100 |   20 | -270 | -270 | -290 |               0 | True              |
| 1065 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         10 |       -100 |        110 | -100 |   10 |  -25 |  -15 |  -30 |    5 |  -35 |  -70 |               0 | True              |
| 1066 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          5 |       -205 |        210 |  -65 | -205 | -145 |    5 | -135 |  -35 |  -85 | -145 |               0 | True              |
| 1067 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -285 |        285 | -145 | -115 | -170 | -180 | -155 | -285 | -190 | -185 |               0 | True              |
| 1068 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        365 |        -10 |        375 |  195 |   80 |  365 |  230 |  305 |  185 |  100 |  -10 |              15 | True              |
| 1069 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         50 |       -355 |        405 |   -5 | -135 |  -25 |   50 |  -35 | -355 | -150 | -305 |               0 | True              |
| 1070 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -500 |        500 | -355 | -335 | -240 | -335 | -260 | -500 | -485 | -470 |               4 | True              |
| 1071 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -280 |        280 |  -15 | -280 | -250 | -100 | -190 | -185 | -270 | -200 |               1 | True              |
| 1072 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         55 |       -125 |        180 |   -0 | -125 |  -95 |   55 |  -15 |   30 |  -65 |  -75 |               0 | True              |
| 1073 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        145 |       -115 |        260 |  145 |  120 |   30 |   35 | -110 | -115 |  -50 |  -75 |               0 | True              |
| 1074 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        225 |        -75 |        300 |  110 |   50 |  195 |  180 |  225 |  125 |   95 |  -75 |               0 | True              |
| 1075 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        170 |       -120 |        290 |  160 |  150 |   10 |  170 |  -55 |  -45 | -120 |   30 |               0 | True              |
| 1076 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        175 |          0 |        175 |  175 |   65 |   55 |   35 |   60 |   85 |   60 |   10 |               0 | True              |
| 1077 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        250 |        -55 |        305 |  160 |  205 |  200 |  250 |   10 |  -50 |   30 |  -55 |               0 | True              |
| 1078 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -300 |        300 | -165 |  -45 | -300 | -155 | -240 | -175 | -180 | -190 |               0 | True              |
| 1079 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -335 |        335 | -170 | -210 | -225 | -270 | -225 | -275 | -335 | -300 |               0 | True              |
| 1080 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        120 |       -225 |        345 |   90 |  120 |   95 |  110 |   20 |  -35 |  -85 | -225 |               0 | True              |
| 1081 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -745 |        745 | -395 | -670 | -745 | -685 | -695 | -655 | -660 | -740 |               5 | True              |
| 1082 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -530 |        530 | -345 | -450 | -530 | -485 | -460 | -490 | -485 | -510 |               4 | True              |
| 1083 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -530 |        530 | -300 | -270 | -230 | -225 | -285 | -520 | -495 | -530 |               3 | True              |
| 1084 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        185 |        -55 |        240 |  110 |  165 |   95 |  185 |  165 |   10 |  140 |  -55 |               0 | True              |
| 1085 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -585 |        585 | -530 | -575 | -585 | -570 | -585 | -540 | -560 | -520 |               4 | True              |
| 1086 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -325 |        325 |  -75 | -150 | -105 |  -25 |  -35 | -100 | -170 | -325 |               0 | True              |
| 1087 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         95 |       -140 |        235 |   40 |  -30 |   95 |  -25 |   65 | -105 | -140 |  -30 |               0 | True              |
| 1088 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -300 |        300 |  -60 | -100 | -120 | -135 | -145 |  -90 |  -85 | -300 |               0 | True              |
| 1089 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -640 |        640 | -375 | -425 | -415 | -420 | -450 | -430 | -590 | -640 |               4 | True              |
| 1090 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -380 |        380 | -180 | -215 | -200 | -265 | -255 | -170 | -215 | -380 |               0 | True              |
| 1091 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -510 |        510 | -195 | -250 | -250 | -435 | -420 | -400 | -410 | -510 |               2 | True              |
| 1092 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        155 |       -135 |        290 |   30 |  155 | -135 |  -15 |  -20 | -105 | -130 |  -45 |               0 | True              |
| 1093 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        150 |       -105 |        255 |  140 |  150 |   -0 |   60 | -105 |   25 |  -65 |  -35 |               0 | True              |
| 1094 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -735 |        735 | -475 | -370 | -440 | -735 | -695 | -695 | -675 | -675 |               5 | True              |
| 1095 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -530 |        530 | -395 | -395 | -305 | -330 | -415 | -435 | -315 | -530 |               2 | True              |
| 1096 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -170 |        170 | -170 |  -35 | -110 | -155 |  -75 | -170 | -165 |  -10 |               0 | True              |
| 1097 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        125 |       -150 |        275 |  125 |  115 | -150 |  -80 | -110 | -140 | -115 |  -55 |               0 | True              |
| 1098 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         90 |       -180 |        270 |   90 |   25 |   65 | -105 | -120 | -130 | -180 | -125 |               0 | True              |
| 1099 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -340 |        340 |  -85 | -275 | -320 | -280 | -340 | -315 | -315 | -265 |               1 | True              |
| 1100 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        145 |       -120 |        265 |   45 |  105 |   65 |  145 |   30 |  -55 | -120 | -120 |               0 | True              |
| 1101 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          5 |       -320 |        325 |    5 |  -50 | -270 | -245 | -245 | -320 | -225 | -290 |               0 | True              |
| 1102 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -150 | -175 | -130 | -175 | -105 |  -80 | -265 | -100 |               0 | True              |
| 1103 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        185 |        -80 |        265 |  185 |  125 |   85 |  165 |  110 |  -60 |  -25 |  -80 |               0 | True              |
| 1104 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        230 |        -15 |        245 |  185 |  140 |  190 |  100 |  225 |  230 |  185 |  -15 |               0 | True              |
| 1105 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         65 |       -225 |        290 |   65 |   50 |   25 |  -35 |  -40 |   -5 | -225 | -110 |               0 | True              |
| 1106 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        215 |        -60 |        275 |  180 |  115 |  215 |  170 |  150 |  -60 |   -0 |   -0 |               0 | True              |
| 1107 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |         -0 |        245 |  205 |  180 |  220 |  245 |   20 |   -0 |  130 |   60 |               0 | True              |
| 1108 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        170 |       -105 |        275 |  150 |  125 |  170 |   10 |  -80 |  -55 | -105 |  -90 |               0 | True              |
| 1109 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -400 |        400 | -150 | -195 | -230 | -160 | -335 | -370 | -350 | -400 |               2 | True              |
| 1110 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |       -310 |        430 |  120 | -115 | -255 | -310 | -200 | -210 | -280 | -245 |               0 | True              |
| 1111 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -670 |        670 | -510 | -475 | -380 | -380 | -555 | -625 | -670 | -635 |               6 | True              |
| 1112 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -400 |        400 | -190 | -150 | -165 | -140 | -345 | -375 | -400 | -340 |               3 | True              |
| 1113 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         15 |       -200 |        215 |  -35 |  -85 | -200 |   15 | -120 |  -85 |  -45 |  -60 |               0 | True              |
| 1114 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        150 |       -115 |        265 |   95 |  145 |   90 |  150 |  100 | -115 |  -50 |  -45 |               0 | True              |
| 1115 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         35 |       -325 |        360 |   35 |  -85 |  -60 | -265 | -315 | -265 | -175 | -325 |               0 | True              |
| 1116 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -550 |        550 | -235 | -255 | -190 | -245 | -430 | -450 | -550 | -455 |               3 | True              |
| 1117 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -740 |        740 | -495 | -295 | -535 | -375 | -465 | -425 | -740 | -615 |               6 | True              |
| 1118 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -125 |        125 | -115 | -120 |  -85 | -125 | -120 |  -75 | -115 |  -55 |               0 | True              |
| 1119 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        220 |       -145 |        365 |  150 |  155 |  220 |  -95 | -105 |   15 |  -95 | -145 |               0 | True              |
| 1120 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -510 |        510 | -195 | -460 | -415 | -450 | -510 | -465 | -465 | -410 |               3 | True              |
| 1121 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -350 |        350 |  -80 | -115 | -160 | -170 | -170 | -285 | -275 | -350 |               2 | True              |
| 1122 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         35 |       -225 |        260 |  -90 |  -50 | -105 |  -15 |   25 |   35 |   -5 | -225 |               0 | True              |
| 1123 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        195 |        -40 |        235 |  195 |  160 |  -25 |  -20 |   20 |  -40 |   30 |   10 |               0 | True              |
| 1124 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -490 |        490 | -305 | -280 | -320 | -370 | -280 | -300 | -360 | -490 |               1 | True              |
| 1125 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         10 |       -285 |        295 |  -65 |   10 | -260 | -250 | -285 | -180 | -255 | -205 |               1 | True              |
| 1126 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        120 |        -25 |        145 |  -25 |   -0 |  105 |   -5 |  120 |  105 |  -10 |   55 |               0 | True              |
| 1127 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        155 |       -185 |        340 |  155 |   40 | -185 | -105 | -150 | -160 |  -45 |  -45 |               0 | True              |
| 1128 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        110 |       -150 |        260 |   90 |  110 |  110 |   95 | -145 |  -50 | -150 |  -70 |               0 | True              |
| 1129 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -400 |        400 | -195 | -240 | -185 | -185 | -205 |  -95 | -150 | -400 |               0 | True              |
| 1130 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         75 |       -215 |        290 | -100 |  -75 |  -20 |   40 |   75 | -145 | -110 | -215 |               0 | True              |
| 1131 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -390 |        390 |  -75 |  -70 |  -70 | -200 | -335 | -390 | -320 | -215 |               1 | True              |
| 1132 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -470 |        470 | -250 | -350 | -255 | -170 | -235 | -435 | -470 | -440 |               3 | True              |
| 1133 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -385 |        385 | -145 |  -95 | -380 | -335 | -305 | -280 | -385 | -310 |               2 | True              |
| 1134 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        135 |       -135 |        270 |   25 |   50 |  115 |   30 |  135 |  -10 | -135 |  -80 |               0 | True              |
| 1135 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |       -160 |        355 |   65 |  195 |   55 |   60 |  -95 | -100 | -160 | -125 |               0 | True              |
| 1136 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -620 |        620 | -300 | -275 | -360 | -450 | -410 | -520 | -505 | -620 |               4 | True              |
| 1137 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        115 |       -240 |        355 |  115 |   20 |   -0 |   -5 |   75 |  -10 |   25 | -240 |               0 | True              |
| 1138 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |        -35 |        200 |  165 |   20 |   55 |  105 |   65 |    5 |   25 |  -35 |               0 | True              |
| 1139 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        180 |        -50 |        230 |   30 |   60 |    5 |   20 |   20 |    5 |  180 |  -50 |              16 | True              |
| 1140 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        135 |       -105 |        240 |  135 |  135 |  100 |  120 |   35 |  -60 |  -10 | -105 |               0 | True              |
| 1141 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |        -85 |        265 |  165 |  180 |  -20 |  -15 |  -85 |   70 |  -30 |    5 |               0 | True              |
| 1142 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        115 |        -45 |        160 |  -45 |  -35 |   85 |   40 |   40 |   20 |  105 |  115 |               0 | True              |
| 1143 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -135 |        200 |   65 | -135 | -125 | -125 |  -60 |  -35 |  -50 |  -10 |               0 | True              |
| 1144 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        145 |        -70 |        215 |   70 |  125 |   45 |  115 |   50 |   -0 |  145 |  -70 |               0 | True              |
| 1145 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -280 |        280 | -145 | -165 | -190 | -190 | -260 | -165 | -280 | -170 |               0 | True              |
| 1146 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |        -80 |        235 |  155 |   15 |  -10 |  -60 |  -80 |  -70 |   15 |  -40 |               0 | True              |
| 1147 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        135 |       -240 |        375 |   45 |  135 |   25 |   45 | -240 | -175 | -145 | -145 |               0 | True              |
| 1148 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        280 |          0 |        280 |  180 |  235 |  280 |  140 |   40 |  125 |  155 |   60 |               0 | True              |
| 1149 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -305 |        305 | -105 | -145 | -165 |  -65 | -165 | -150 | -205 | -305 |               0 | True              |
| 1150 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -585 |        585 | -365 | -480 | -395 | -360 | -585 | -550 | -520 | -465 |               4 | True              |
| 1151 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -255 |        255 | -130 | -185 |  -90 | -130 | -255 | -160 | -125 | -160 |               0 | True              |
| 1152 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         15 |       -405 |        420 |   15 | -105 | -345 | -205 | -285 | -405 | -345 | -315 |               0 | True              |
| 1153 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        285 |          0 |        285 |  210 |  285 |   40 |  130 |   85 |  125 |  115 |   30 |               0 | True              |
| 1154 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -540 |        540 | -200 | -495 | -480 | -455 | -540 | -415 | -470 | -470 |               2 | True              |
| 1155 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -265 | -245 | -130 | -205 | -245 | -210 | -255 | -190 |               1 | True              |
| 1156 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -385 |        385 |  -50 |  -95 | -205 | -275 | -385 | -320 | -150 | -225 |               1 | True              |
| 1157 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         60 |       -115 |        175 |   20 | -115 |   -0 |  -15 |  -80 |  -55 |   60 |  -60 |              16 | True              |
| 1158 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        340 |          0 |        340 |  175 |  230 |  340 |  220 |  250 |  140 |   50 |   95 |               0 | True              |
| 1159 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        275 |          0 |        275 |  210 |  240 |  135 |  115 |  205 |  165 |  275 |   40 |              14 | True              |
| 1160 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -235 |        335 |  100 |   20 |  -10 |   50 |   35 | -235 |  -95 | -165 |               0 | True              |
| 1161 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         65 |       -100 |        165 |   65 |   30 |   -5 |  -75 |  -35 |  -50 |   35 | -100 |               0 | True              |
| 1162 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         55 |       -315 |        370 |   25 |   55 | -140 | -285 | -240 | -315 | -140 | -200 |               0 | True              |
| 1163 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -140 |        140 |  -75 |  -45 | -140 |  -45 |  -25 | -105 |  -20 |  -55 |               0 | True              |
| 1164 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        245 |        -25 |        270 |  115 |  155 |  245 |  200 |  125 |   85 |   30 |  -25 |               0 | True              |
| 1165 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        145 |       -215 |        360 |  145 |   60 | -215 | -110 | -175 | -185 | -150 | -155 |               0 | True              |
| 1166 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        275 |        -10 |        285 |  120 |  275 |  265 |  265 |  275 |  255 |  -10 |   50 |               0 | True              |
| 1167 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |        -75 |        180 |   35 |   -5 |   55 |  100 |  105 |   25 |   25 |  -75 |               0 | True              |
| 1168 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        120 |       -180 |        300 |   80 |  120 |   80 |   10 |  -30 |   60 |   70 | -180 |               0 | True              |
| 1169 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -405 |        405 | -140 | -170 |  -90 | -405 | -385 | -255 | -295 | -315 |               2 | True              |
| 1170 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         60 |       -265 |        325 |   60 |  -50 |  -10 |   35 |  -40 | -240 | -265 | -145 |               0 | True              |
| 1171 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -360 |        360 | -155 | -190 | -140 | -195 |  -90 | -145 | -215 | -360 |               0 | True              |
| 1172 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        150 |        -40 |        190 |  -40 |  150 |   -0 |   25 |   25 |   35 |   85 |   75 |               0 | True              |
| 1173 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |          0 |        200 |   95 |  200 |  175 |  165 |  140 |   25 |   70 |    5 |               0 | True              |
| 1174 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -180 |        180 | -105 | -165 | -180 | -140 | -170 | -165 | -170 | -130 |               0 | True              |
| 1175 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         95 |       -205 |        300 |  -55 |   25 |   95 |  -60 |   50 | -175 | -205 | -115 |               1 | True              |
| 1176 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -305 |        325 |  -75 |   -0 |  -40 | -115 |    5 | -100 |   20 | -305 |               0 | True              |
| 1177 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        270 |          0 |        270 |  145 |  150 |  160 |  270 |  270 |    5 |   75 |   15 |               0 | True              |
| 1178 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -430 |        430 | -305 | -345 | -390 | -420 | -405 | -420 | -430 | -405 |               4 | True              |
| 1179 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -630 |        630 | -355 | -435 | -455 | -370 | -605 | -550 | -590 | -630 |               4 | True              |
| 1180 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        160 |       -240 |        400 |   35 |  160 | -145 |  -65 | -130 | -240 | -125 | -180 |               0 | True              |
| 1181 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        155 |       -120 |        275 |  155 |  150 |   35 |  -60 |  -85 | -120 | -120 |  -85 |               0 | True              |
| 1182 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -200 |        320 |  120 |   -0 | -195 | -105 | -200 | -135 | -170 |  -90 |               0 | True              |
| 1183 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         25 |       -255 |        280 |   15 |   25 |  -25 |  -10 |  -70 | -255 | -140 | -210 |               0 | True              |
| 1184 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        185 |        -80 |        265 |  185 |  150 |    5 |   10 |  -60 |  -80 |   95 |   -0 |               0 | True              |
| 1185 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        100 |       -165 |        265 |   80 |   50 |  -90 |   -0 |  100 |   60 |   70 | -165 |               0 | True              |
| 1186 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        130 |       -115 |        245 |   60 |   60 |  110 |  110 |  130 |  -95 |  -25 | -115 |               0 | True              |
| 1187 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -495 |        495 | -175 | -385 | -495 | -435 | -390 | -370 | -365 | -390 |               2 | True              |
| 1188 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        230 |        -20 |        250 |   95 |  140 |  230 |  -20 |   15 |   45 |   10 |   10 |               0 | True              |
| 1189 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        245 |        -95 |        340 |  175 |  245 |  235 |   35 |  -80 |  -95 |   -5 |   10 |               0 | True              |
| 1190 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |        -40 |        110 |   25 |  -40 |   70 |  -10 |  -10 |   60 |  -10 |  -10 |               0 | True              |
| 1191 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         90 |       -155 |        245 |   90 |  -75 | -115 | -140 | -150 | -100 | -155 |  -60 |               0 | True              |
| 1192 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -230 |        315 |   85 | -105 | -220 | -115 |  -75 | -230 |  -90 |  -20 |               0 | True              |
| 1193 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         80 |       -295 |        375 |   80 |  -80 | -210 | -165 | -245 | -250 | -295 | -195 |               0 | True              |
| 1194 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         65 |       -220 |        285 |   65 | -160 | -150 | -145 | -155 | -145 | -220 | -140 |               0 | True              |
| 1195 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |       -110 |        260 |  135 |  -10 |  150 |  130 |   95 |  100 |   30 | -110 |               0 | True              |
| 1196 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         10 |       -210 |        220 |  -50 |  -10 |  -45 |   10 | -105 | -135 | -210 |  -85 |               1 | True              |
| 1197 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -350 |        350 | -210 | -165 | -210 | -140 | -110 | -325 | -350 | -310 |               2 | True              |
| 1198 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         40 |       -245 |        285 |   10 |   40 |  -15 |  -45 |  -45 |  -30 | -195 | -245 |               0 | True              |
| 1199 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        105 |       -120 |        225 |  105 |    5 |   45 |   60 | -105 | -115 | -120 |  -90 |               0 | True              |
| 1200 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        275 |        -95 |        370 |  190 |  130 |  275 |    5 |   -5 |   50 |   55 |  -95 |               0 | True              |
| 1201 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        165 |       -170 |        335 |  165 |   50 |  -70 | -170 |  -90 |  -25 | -115 | -130 |               0 | True              |
| 1202 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -315 |        315 |  -25 |   -5 | -315 | -300 | -275 | -300 | -175 | -240 |               0 | True              |
| 1203 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -465 |        465 | -195 | -185 | -195 | -155 | -400 | -395 | -440 | -465 |               2 | True              |
| 1204 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |       -140 |        300 |  115 |  105 |  100 |  160 | -140 |   15 |  -95 | -140 |               0 | True              |
| 1205 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -135 |        135 |   -5 | -135 |  -50 |  -55 |  -10 | -105 |  -35 |  -15 |               0 | True              |
| 1206 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        165 |       -115 |        280 |  120 |   50 |  150 |  165 |  145 |   20 |  -75 | -115 |               0 | True              |
| 1207 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         70 |       -350 |        420 |   70 | -250 | -165 | -250 | -240 | -235 | -350 | -220 |               0 | True              |
| 1208 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |       -120 |        270 |  105 |  150 |   20 |   75 |  150 |   45 |   95 | -120 |               0 | True              |
| 1209 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         70 |       -205 |        275 |   70 |   25 |  -10 |   65 | -180 | -205 | -130 | -140 |               0 | True              |
| 1210 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -405 |        405 | -225 | -270 | -130 | -375 | -405 | -375 | -320 | -330 |               3 | True              |
| 1211 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         -0 |       -245 |        245 |   -0 | -195 | -200 | -245 | -155 | -230 | -240 | -245 |               0 | True              |
| 1212 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -460 |        460 | -250 | -460 | -365 | -445 | -455 | -445 | -365 | -410 |               3 | True              |
| 1213 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        125 |       -125 |        250 |  100 |  125 |  -80 |  -15 |  -90 | -125 | -125 |  -40 |               0 | True              |
| 1214 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        130 |       -165 |        295 |  130 |    5 |   65 |   60 |  -95 | -165 |   -5 | -125 |               0 | True              |
| 1215 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -220 |        220 |  -85 |  -30 | -100 |   -5 | -220 |  -90 | -130 | -220 |               0 | True              |
| 1216 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        185 |        -65 |        250 |  185 |   50 |    5 |  -65 |  -25 |  -15 |   85 |  105 |               0 | True              |
| 1217 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        240 |          0 |        240 |  195 |  235 |  185 |  240 |  230 |   20 |   95 |  120 |               0 | True              |
| 1218 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -285 |        285 | -100 |  -70 |  -30 |  -90 | -115 |  -35 | -135 | -285 |               0 | True              |
| 1219 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -360 |        360 |  -45 | -100 |  -70 |  -75 |  -80 |  -25 |  -95 | -360 |               0 | True              |
| 1220 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         85 |       -165 |        250 |    5 |   20 |   85 |   75 | -145 | -120 |  -90 | -165 |               0 | True              |
| 1221 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        315 |       -155 |        470 |  225 |  270 |  225 |  315 |   25 |  -50 | -155 | -110 |               0 | True              |
| 1222 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        130 |       -155 |        285 |  115 |  130 |  100 |   70 | -155 | -125 | -130 | -120 |               0 | True              |
| 1223 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         40 |       -400 |        440 |   40 | -220 | -225 | -245 | -270 | -240 | -400 | -345 |               0 | True              |
| 1224 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         75 |       -180 |        255 |   75 |  -30 | -180 |  -95 | -125 | -150 |   -5 | -175 |               0 | True              |
| 1225 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -135 |        135 |  -95 |  -60 | -135 |  -85 | -100 | -125 |  -40 |  -20 |               0 | True              |
| 1226 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -515 |        515 | -385 | -380 | -515 | -430 | -440 | -485 | -485 | -455 |               2 | True              |
| 1227 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -200 |        200 | -150 | -170 | -175 | -200 | -145 | -170 | -155 | -155 |               0 | True              |
| 1228 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |       -150 |        345 |  110 |  120 |  195 |  180 | -150 |  -65 |  -75 |  -85 |               0 | True              |
| 1229 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        275 |       -100 |        375 |  115 |  275 |  165 |  235 |  265 |   65 | -100 |   50 |               0 | True              |
| 1230 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -335 |        335 | -160 | -160 | -165 | -155 | -155 | -155 | -180 | -335 |               0 | True              |
| 1231 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        115 |       -210 |        325 |   55 |   75 |  115 |    5 |   20 | -130 | -210 | -170 |               0 | True              |
| 1232 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -545 |        545 | -230 | -240 | -315 | -400 | -480 | -430 | -480 | -545 |               3 | True              |
| 1233 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         90 |       -215 |        305 |   90 |   30 |  -20 |   -0 | -190 | -150 | -215 | -150 |               0 | True              |
| 1234 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |       -140 |        315 |   60 |  175 |  100 |  165 |   70 | -140 | -105 | -135 |               0 | True              |
| 1235 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        125 |       -200 |        325 |   90 |  110 |  125 |  120 |   25 | -200 | -165 |  -65 |               0 | True              |
| 1236 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -605 |        605 | -290 | -295 | -345 | -510 | -440 | -605 | -510 | -535 |               3 | True              |
| 1237 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |       -145 |        305 |  160 |  135 |  115 |  115 |   95 |   70 | -145 | -105 |               0 | True              |
| 1238 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         20 |       -170 |        190 | -115 |  -60 | -135 |  -50 | -165 | -170 |   20 |  -95 |               0 | True              |
| 1239 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        115 |        -45 |        160 |   -0 |  -45 |  -45 |   -5 |  -10 |   60 |  -45 |  115 |               0 | True              |
| 1240 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        210 |        -25 |        235 |  210 |   20 |    5 |   -0 |  -25 |   55 |   30 |   25 |               0 | True              |
| 1241 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        185 |       -155 |        340 |  100 |   35 |  185 |  180 |   65 |   35 | -155 |  -40 |               0 | True              |
| 1242 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        165 |       -115 |        280 |   80 |  115 |  165 |  -25 |  -30 | -115 |   15 |  -25 |               0 | True              |
| 1243 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        215 |        -60 |        275 |  140 |  205 |  200 |  145 |   70 |  215 |  190 |  -60 |               0 | True              |
| 1244 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -240 |        240 | -110 | -140 | -160 | -240 | -165 |  -90 | -200 | -170 |               0 | True              |
| 1245 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        160 |       -110 |        270 |  160 |  135 |    5 |  -90 | -110 |  -30 |  -15 |  -75 |               0 | True              |
| 1246 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         60 |       -245 |        305 |   35 |   60 |   45 | -110 | -150 | -245 | -150 | -120 |               0 | True              |
| 1247 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        140 |       -135 |        275 |   75 |   70 |  120 |  140 |  130 |   70 |  125 | -135 |               0 | True              |
| 1248 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        220 |        -40 |        260 |  205 |  220 |    5 |   75 |   95 |   60 |  -40 |  190 |               0 | True              |
| 1249 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        155 |       -290 |        445 |  125 |  155 | -180 | -180 | -290 | -230 | -220 | -250 |               0 | True              |
| 1250 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        235 |       -145 |        380 |  150 |  235 |  125 | -105 |  -40 | -145 |  -65 | -115 |               0 | True              |
| 1251 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        135 |       -140 |        275 |  135 | -105 |  -40 |  -10 |   45 |   70 | -140 | -125 |               0 | True              |
| 1252 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        120 |       -215 |        335 |   80 |   50 |  120 |  -55 |  -90 |  -10 | -180 | -215 |               0 | True              |
| 1253 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -120 |        145 |  -55 |   25 |    5 |    5 |   25 |  -75 |   25 | -120 |               0 | True              |
| 1254 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -365 |        365 | -105 |  -70 | -315 | -270 | -365 | -305 | -255 | -225 |               2 | True              |
| 1255 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -180 |        180 |  -35 |  -80 | -135 | -110 |  -85 | -115 | -180 |  -10 |               1 | True              |
| 1256 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -530 |        530 | -210 | -260 | -270 | -355 | -395 | -420 | -530 | -470 |               3 | True              |
| 1257 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -365 |        365 | -230 | -235 | -225 | -270 | -260 | -295 | -250 | -365 |               0 | True              |
| 1258 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -370 |        370 |  -80 | -180 |  -45 |  -40 |  -90 |  -35 | -300 | -370 |               1 | True              |
| 1259 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         80 |       -195 |        275 |    5 |   80 | -190 | -140 | -195 | -140 | -195 |  -80 |               0 | True              |
| 1260 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         95 |       -275 |        370 |   35 |   45 |   30 |   95 |   50 |   25 |   80 | -275 |               0 | True              |
| 1261 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        215 |        -75 |        290 |  150 |  160 |   30 |   50 |   30 |  215 |    5 |  -75 |               0 | True              |
| 1262 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        255 |       -120 |        375 |  105 |  135 |  255 | -115 | -120 | -105 |  -95 |  -30 |               0 | True              |
| 1263 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        100 |       -180 |        280 |   15 |   50 |  -15 |  100 |  -35 | -180 |  -95 | -130 |               0 | True              |
| 1264 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -645 |        645 | -450 | -340 | -490 | -310 | -645 | -640 | -575 | -625 |               5 | True              |
| 1265 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        165 |        -25 |        190 |   50 |  125 |  110 |  165 |   40 |  145 |  -25 |   -5 |               0 | True              |
| 1266 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         10 |        -85 |         95 |  -15 |  -35 |  -75 |  -45 |  -85 |  -20 |  -30 |   10 |               0 | True              |
| 1267 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        125 |        -95 |        220 |  125 |   65 |  110 |    5 |  -95 |  -70 |   -0 |   35 |               0 | True              |
| 1268 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |        -80 |        240 |   75 |  125 |  140 |   90 |   95 |  160 |  -65 |  -80 |               0 | True              |
| 1269 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -160 |        235 |   65 |   30 |   75 | -160 | -150 | -160 | -115 | -125 |               0 | True              |
| 1270 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        110 |       -320 |        430 |   40 |   70 |  110 |   40 |   30 | -215 | -320 | -270 |               0 | True              |
| 1271 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -380 |        380 | -155 | -130 | -165 | -205 | -250 | -185 | -210 | -380 |               0 | True              |
| 1272 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        255 |        -75 |        330 |  140 |   80 |  255 |  145 |  225 |  -45 |  -50 |  -75 |               0 | True              |
| 1273 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -300 |        300 |  -20 |  -75 |  -80 |  -35 | -240 | -300 | -200 | -215 |               1 | True              |
| 1274 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         90 |       -145 |        235 |   -0 |   50 |   70 |   80 |   -5 |   90 |   70 | -145 |               0 | True              |
| 1275 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -215 |        215 | -210 | -165 | -140 | -140 | -130 | -195 | -200 | -215 |               0 | True              |
| 1276 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |       -130 |        310 |   75 |  180 |  -45 |  -55 | -130 |  -30 |  -75 |  -35 |               0 | True              |
| 1277 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         95 |        -15 |        110 |  -15 |   50 |   95 |    5 |   40 |   -5 |   70 |   30 |               0 | True              |
| 1278 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -580 |        580 | -275 | -300 | -325 | -545 | -505 | -545 | -510 | -580 |               3 | True              |
| 1279 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -465 |        465 | -125 | -195 | -190 | -195 | -240 | -290 | -240 | -465 |               0 | True              |
| 1280 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        205 |          0 |        205 |  205 |   95 |   85 |  160 |   75 |  145 |  155 |  145 |               0 | True              |
| 1281 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -410 |        410 | -195 | -175 | -135 | -105 | -335 | -410 | -330 | -320 |               2 | True              |
| 1282 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -545 |        545 | -360 | -315 | -305 | -325 | -255 | -450 | -410 | -545 |               4 | True              |
| 1283 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         90 |       -205 |        295 |   90 |  -70 |   10 |   20 |   60 |   80 | -155 | -205 |               0 | True              |
| 1284 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         70 |       -265 |        335 |   50 |   70 |  -20 | -200 | -175 | -265 | -140 | -150 |               0 | True              |
| 1285 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -645 |        645 | -425 | -405 | -315 | -395 | -375 | -580 | -645 | -550 |               5 | True              |
| 1286 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -570 |        570 | -290 | -505 | -530 | -570 | -530 | -470 | -420 | -485 |               3 | True              |
| 1287 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        295 |          0 |        295 |  160 |  295 |  215 |  280 |   30 |  130 |  115 |   75 |               0 | True              |
| 1288 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -540 |        540 | -260 | -340 | -310 | -275 | -435 | -540 | -395 | -450 |               3 | True              |
| 1289 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        260 |        -80 |        340 |  235 |  260 |  220 |  235 |  -80 |   -0 |   50 |   20 |              16 | True              |
| 1290 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -175 |        175 |  -70 | -165 | -100 | -175 | -160 | -135 | -125 | -110 |               0 | True              |
| 1291 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -620 |        620 | -345 | -525 | -485 | -455 | -375 | -400 | -620 | -585 |               4 | True              |
| 1292 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -385 |        385 | -135 | -150 | -185 | -115 | -130 | -155 | -120 | -385 |               0 | True              |
| 1293 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |       -265 |        295 |   30 | -130 | -150 | -265 | -155 | -130 | -160 |  -95 |               0 | True              |
| 1294 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |        -95 |        255 |  145 |  160 |  135 |  -60 |  -55 |  -45 |  -35 |  -95 |               0 | True              |
| 1295 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        100 |       -130 |        230 |  100 |   70 | -130 |  -65 |  -75 | -130 |  -85 | -105 |               0 | True              |
| 1296 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -680 |        680 | -475 | -610 | -680 | -570 | -640 | -515 | -600 | -555 |               5 | True              |
| 1297 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -165 |        185 | -100 | -135 |   20 |  -65 |  -45 |  -50 | -150 | -165 |               0 | True              |
| 1298 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -130 |        270 |  100 |  140 | -115 |  -20 | -130 |   35 | -115 |  -65 |               0 | True              |
| 1299 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -655 |        655 | -560 | -655 | -630 | -470 | -615 | -600 | -655 | -540 |               4 | True              |
| 1300 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        215 |          0 |        215 |  215 |   90 |  105 |  135 |   45 |   70 |   40 |  190 |               0 | True              |
| 1301 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -545 |        545 | -475 | -545 | -480 | -455 | -480 | -540 | -525 | -450 |               3 | True              |
| 1302 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        210 |        -80 |        290 |  100 |  115 |  145 |   80 |  210 |   95 |  125 |  -80 |               0 | True              |
| 1303 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -535 |        535 | -375 | -535 | -435 | -360 | -350 | -390 | -405 | -475 |               2 | True              |
| 1304 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -285 |        285 | -130 | -245 | -150 | -260 | -225 | -210 | -285 | -220 |               0 | True              |
| 1305 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         60 |       -195 |        255 |   60 |   45 |   55 |  -75 | -105 | -170 | -140 | -195 |               0 | True              |
| 1306 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         20 |       -320 |        340 |   20 |   20 | -320 | -285 | -160 | -225 | -180 | -245 |               0 | True              |
| 1307 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -695 |        695 | -365 | -460 | -415 | -635 | -655 | -695 | -660 | -635 |               4 | True              |
| 1308 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |        -60 |        240 |  160 |  130 |  140 |   50 |  180 |  160 |  -45 |  -60 |               0 | True              |
| 1309 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        205 |       -135 |        340 |  205 |  135 |   35 |  115 |   55 | -110 | -135 | -120 |               0 | True              |
| 1310 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -300 |        300 | -155 | -190 | -235 | -190 | -205 | -195 | -185 | -300 |               0 | True              |
| 1311 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        290 |        -30 |        320 |  135 |  185 |  185 |  110 |  290 |  230 |  170 |  -30 |               0 | True              |
| 1312 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        165 |       -110 |        275 |  165 |  -70 |  -55 |  -40 | -110 |  -95 |   -5 |  -70 |               0 | True              |
| 1313 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        175 |        -45 |        220 |  175 |  150 |  105 |  135 |  115 |   95 |  -45 |   -0 |               0 | True              |
| 1314 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          5 |       -250 |        255 |    5 |  -20 | -175 |  -75 | -200 | -225 | -165 | -250 |               0 | True              |
| 1315 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -320 |        320 |  -45 | -130 |  -30 |  -45 | -155 |  -85 | -320 | -295 |               1 | True              |
| 1316 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         90 |       -270 |        360 |   90 | -170 | -250 | -270 | -215 |  -90 | -195 | -135 |               0 | True              |
| 1317 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -240 |        305 |   65 |   25 |  -35 | -240 | -195 | -185 | -190 | -235 |               0 | True              |
| 1318 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -765 |        765 | -365 | -335 | -675 | -765 | -595 | -565 | -625 | -625 |               4 | True              |
| 1319 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -285 |        285 | -185 | -255 | -255 | -235 | -230 | -280 | -285 | -225 |               0 | True              |
| 1320 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         55 |       -200 |        255 |   55 |  -75 | -200 |  -75 | -115 | -155 |  -65 | -195 |               0 | True              |
| 1321 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         55 |       -245 |        300 |   25 |   55 | -100 | -245 | -235 | -180 | -190 | -175 |               0 | True              |
| 1322 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        125 |       -150 |        275 |  125 | -110 | -150 | -145 |  -50 |  -45 | -105 |  -75 |               0 | True              |
| 1323 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |       -135 |        255 |  120 |  -45 |  -95 | -135 |  -45 |  -30 |  -20 |  -25 |               0 | True              |
| 1324 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -245 |        265 |   20 | -200 | -110 | -205 | -245 | -190 | -195 | -165 |               0 | True              |
| 1325 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -505 |        505 | -445 | -385 | -505 | -335 | -350 | -425 | -265 | -405 |               3 | True              |
| 1326 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         30 |       -260 |        290 |  -25 |  -50 |   30 | -155 | -240 | -225 | -180 | -260 |               1 | True              |
| 1327 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        180 |       -120 |        300 |  180 |  -80 | -120 |  -40 |  -10 |   25 |    5 |  -65 |               0 | True              |
| 1328 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        190 |       -155 |        345 |  190 |  110 |   65 |   85 | -140 | -155 |  -80 |  -40 |               0 | True              |
| 1329 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -170 |        170 |  -40 | -135 | -170 | -165 |  -90 | -145 | -110 |  -70 |               0 | True              |
| 1330 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -425 |        425 | -185 | -115 | -170 | -275 | -385 | -390 | -305 | -425 |               2 | True              |
| 1331 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         25 |       -270 |        295 |   15 |  -95 |   25 |  -25 |  -90 |  -25 | -270 | -260 |               0 | True              |
| 1332 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        315 |          0 |        315 |  220 |  315 |  160 |  240 |  140 |    5 |  160 |   15 |               0 | True              |
| 1333 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -635 |        635 | -595 | -635 | -560 | -505 | -520 | -590 | -495 | -515 |               4 | True              |
| 1334 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        185 |        -15 |        200 |  185 |  130 |  160 |  110 |  115 |  125 |  130 |  -15 |               0 | True              |
| 1335 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         50 |        -60 |        110 |   15 |   -0 |   50 |   50 |   20 |   35 |  -60 |   45 |              16 | True              |
| 1336 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -395 |        395 | -155 |  -70 | -170 |  -70 | -145 | -170 |  -80 | -395 |               0 | True              |
| 1337 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        195 |        -55 |        250 |   90 |  -35 |   25 |   75 |  165 |   80 |  195 |  -55 |               0 | True              |
| 1338 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         35 |       -170 |        205 |  -25 |  -20 |   35 |   20 |  -95 |   -0 |  -50 | -170 |               0 | True              |
| 1339 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        145 |       -215 |        360 |   15 |  145 | -175 |  -80 | -215 | -130 | -190 | -190 |               0 | True              |
| 1340 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         20 |       -320 |        340 |   20 |  -60 |   20 |  -80 |  -65 | -205 | -320 | -315 |               0 | True              |
| 1341 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -345 |        345 | -300 | -330 | -300 | -255 | -265 | -210 | -265 | -345 |               1 | True              |
| 1342 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |        -85 |        330 |  200 |  245 |  210 |  215 |  -85 |  -25 |   -5 |   -0 |               0 | True              |
| 1343 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        270 |        -25 |        295 |  215 |  255 |  270 |  110 |  210 |  190 |   -0 |  -25 |               0 | True              |
| 1344 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -785 |        785 | -540 | -495 | -555 | -705 | -725 | -785 | -630 | -735 |               6 | True              |
| 1345 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        185 |       -140 |        325 |  105 |  185 |  155 |    5 |  -90 |  -75 | -140 |  -50 |               0 | True              |
| 1346 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        180 |       -125 |        305 |  180 |   75 |  140 |  175 |    5 | -125 |   10 |  -80 |               0 | True              |
| 1347 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        240 |        -25 |        265 |  200 |  240 |   35 |   35 |  -15 |   35 |   50 |  -25 |               0 | True              |
| 1348 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         70 |       -190 |        260 |   70 |   30 |   50 |  -40 |   60 | -190 | -140 |  -40 |               0 | True              |
| 1349 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        110 |       -160 |        270 |  110 |   40 |  110 |  -60 | -160 | -125 | -145 | -160 |               0 | True              |
| 1350 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |       -165 |        345 |  180 |  150 | -105 |  -95 | -165 |  -70 | -120 | -130 |               0 | True              |
| 1351 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |       -135 |        330 |   30 |  125 |  195 |   90 |  130 |  170 |  -40 | -135 |               0 | True              |
| 1352 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        195 |       -115 |        310 |  110 |  160 |  195 |    5 |    5 | -115 |  -35 |   10 |               0 | True              |
| 1353 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        210 |        -60 |        270 |  140 |  125 |  145 |  210 |  135 |  120 |  115 |  -60 |               0 | True              |
| 1354 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -725 |        725 | -510 | -455 | -485 | -485 | -420 | -575 | -725 | -690 |               6 | True              |
| 1355 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -255 |        280 |  -30 |  -35 |  -75 |  -30 |  -85 |   25 |  -30 | -255 |               0 | True              |
| 1356 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 | -115 | -205 | -180 | -125 | -170 | -150 | -175 | -175 |               0 | True              |
| 1357 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        115 |       -165 |        280 |  -10 |   35 |   20 |    5 |   60 |  115 |   90 | -165 |               0 | True              |
| 1358 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -305 |        305 | -140 |  -95 |  -60 | -175 |  -50 | -110 |  -65 | -305 |               0 | True              |
| 1359 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        175 |        -60 |        235 |   70 |  175 |  170 |  -60 |   85 |  -60 |  -35 |  -30 |               0 | True              |
| 1360 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -705 |        705 | -410 | -520 | -475 | -540 | -515 | -565 | -595 | -705 |               3 | True              |
| 1361 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -385 |        385 | -110 | -130 | -295 | -150 | -225 | -160 | -155 | -385 |               0 | True              |
| 1362 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        120 |        -55 |        175 |  -40 |  -55 |   20 |  -10 |   85 |  120 |   40 |   -0 |               0 | True              |
| 1363 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -370 |        370 | -100 |  -90 | -165 | -210 |  -65 | -320 | -285 | -370 |               1 | True              |
| 1364 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        120 |       -240 |        360 |  100 |  120 |   75 |  100 |   75 | -240 | -145 |  -95 |               0 | True              |
| 1365 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        110 |       -215 |        325 |  110 |   60 |  105 | -120 | -175 |   30 |  -30 | -215 |               0 | True              |
| 1366 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -560 |        560 | -350 | -265 | -310 | -295 | -345 | -325 | -560 | -505 |               4 | True              |
| 1367 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -255 |        255 | -105 |  -80 | -115 | -125 | -205 | -175 | -120 | -255 |               0 | True              |
| 1368 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -440 |        440 | -280 | -245 | -185 | -385 | -385 | -440 | -395 | -350 |               3 | True              |
| 1369 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        240 |       -135 |        375 |  185 |  140 |  160 |  145 |  240 | -135 |  -30 |  -50 |               0 | True              |
| 1370 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         40 |       -175 |        215 |   40 | -125 | -140 | -140 |  -90 | -175 |  -95 |  -95 |               0 | True              |
| 1371 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -270 |        270 | -155 | -140 | -205 | -140 | -130 | -270 | -200 | -205 |               0 | True              |
| 1372 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -370 |        370 |  -95 | -325 | -235 | -240 | -370 | -225 | -240 | -265 |               1 | True              |
| 1373 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        230 |        -35 |        265 |  110 |  110 |  125 |  130 |  230 |  165 |  145 |  -35 |               0 | True              |
| 1374 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -715 |        715 | -570 | -440 | -545 | -445 | -680 | -715 | -680 | -680 |               6 | True              |
| 1375 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -495 |        495 | -225 | -290 | -475 | -470 | -425 | -355 | -440 | -495 |               3 | True              |
| 1376 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |       -210 |        370 |   55 |  140 |   45 |  160 | -210 |  -80 |  -55 |  -55 |               0 | True              |
| 1377 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        125 |        -80 |        205 |  125 |   50 |   75 |    5 |   40 |   -0 |  -80 |  -20 |               0 | True              |
| 1378 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |       -175 |        335 |   55 |   55 |  130 |  160 |   40 |   55 | -175 |   -5 |               0 | True              |
| 1379 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        125 |       -270 |        395 |  125 | -155 |  -95 |  -80 | -170 | -270 |  -10 | -195 |               0 | True              |
| 1380 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        135 |        -50 |        185 |  -20 |  130 |   70 |   95 |   55 |   60 |  135 |  -50 |               0 | True              |
| 1381 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -590 |        590 | -325 | -250 | -355 | -590 | -465 | -545 | -590 | -575 |               4 | True              |
| 1382 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -645 |        645 | -485 | -485 | -520 | -470 | -480 | -645 | -620 | -560 |               5 | True              |
| 1383 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -565 |        565 | -330 | -370 | -430 | -340 | -375 | -285 | -565 | -535 |               4 | True              |
| 1384 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        130 |       -150 |        280 |  130 |  -40 |  -15 | -115 |  -10 | -105 | -150 |  -60 |               0 | True              |
| 1385 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         60 |       -140 |        200 |   60 |   10 |   45 |   -0 | -140 | -115 | -110 | -115 |               0 | True              |
| 1386 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        305 |        -50 |        355 |  190 |  155 |  265 |  205 |  305 |  235 |  210 |  -50 |               0 | True              |
| 1387 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         75 |       -275 |        350 |    5 |  -25 |    5 |   75 |  -55 | -185 | -275 | -205 |               0 | True              |
| 1388 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -490 |        490 | -465 | -410 | -425 | -490 | -365 | -340 | -435 | -395 |               3 | True              |
| 1389 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -485 |        485 | -330 | -270 | -165 | -275 | -335 | -265 | -330 | -485 |               2 | True              |
| 1390 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -485 |        485 | -225 | -485 | -395 | -325 | -365 | -345 | -430 | -360 |               3 | True              |
| 1391 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -290 |        290 | -175 | -145 | -190 | -255 | -235 | -215 | -190 | -290 |               0 | True              |
| 1392 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -590 |        590 | -525 | -450 | -490 | -475 | -505 | -590 | -455 | -555 |               4 | True              |
| 1393 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        275 |        -80 |        355 |  275 |  160 |   -5 |  -50 |  -65 |  -35 |  -80 |    5 |              16 | True              |
| 1394 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        130 |       -130 |        260 |   95 |   70 |  130 | -105 |   45 |  -90 | -130 |  -55 |               0 | True              |
| 1395 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -370 |        370 |  -10 | -220 | -165 |  -45 | -130 | -220 |  -80 | -370 |               0 | True              |
| 1396 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -160 |        305 |   15 |  -55 |   40 |   75 |  145 |   40 | -120 | -160 |               0 | True              |
| 1397 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        290 |        -25 |        315 |  120 |   90 |   75 |  200 |  195 |  290 |  -25 |  -20 |               0 | True              |
| 1398 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        110 |       -110 |        220 |  110 |   65 |   50 |  100 | -100 | -105 |  -65 | -110 |               0 | True              |
| 1399 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -395 |        395 |  -50 | -205 | -325 | -300 | -140 | -215 | -205 | -395 |               0 | True              |
| 1400 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -295 |        295 | -165 |  -90 | -100 | -125 |  -90 |  -40 | -120 | -295 |               0 | True              |
| 1401 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |        -90 |        250 |  155 |  125 |  160 |  130 |  -50 |  -10 |  -90 |  -45 |               0 | True              |
| 1402 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -315 |        315 | -130 | -160 | -185 | -155 | -190 | -170 | -205 | -315 |               0 | True              |
| 1403 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        205 |       -110 |        315 |  185 |  205 | -110 |   -0 |  -85 |  -10 | -100 |  -25 |               0 | True              |
| 1404 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         15 |       -245 |        260 | -135 | -135 | -190 | -155 | -180 | -125 |   15 | -245 |               0 | True              |
| 1405 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -445 |        445 | -165 | -420 | -340 | -295 | -345 | -440 | -445 | -435 |               2 | True              |
| 1406 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        220 |        -90 |        310 |  190 |  205 |  220 |  -20 |  -35 |  -90 |  -15 |   35 |               0 | True              |
| 1407 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        120 |       -175 |        295 |   85 |  120 |   50 |  -45 | -160 | -175 | -115 |  -50 |               0 | True              |
| 1408 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        220 |        -95 |        315 |  135 |  175 |  160 |  120 |  220 |  200 |  -95 |  -45 |               0 | True              |
| 1409 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -245 |        245 | -155 | -200 | -130 | -245 | -115 | -135 | -170 | -220 |               0 | True              |
| 1410 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        235 |        -65 |        300 |  175 |  235 |  175 |  -15 |  -25 |  -45 |  -65 |  -65 |               0 | True              |
| 1411 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        135 |        -95 |        230 |   75 |   75 |  135 |  115 |   25 |   65 |  -15 |  -95 |               0 | True              |
| 1412 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        240 |       -115 |        355 |  165 |  195 |  150 |  240 | -115 |  -35 |   25 |  -65 |               0 | True              |
| 1413 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -260 |        285 |  -70 |  -65 |  -60 |  -85 |   25 |  -60 | -120 | -260 |               0 | True              |
| 1414 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        250 |        -25 |        275 |  215 |  205 |  180 |  235 |  250 |  205 |   -0 |  -25 |               0 | True              |
| 1415 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         75 |       -205 |        280 |   75 | -185 | -180 | -205 | -180 | -165 |  -95 | -105 |               0 | True              |
| 1416 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        125 |       -115 |        240 |  125 |  110 | -110 |  -20 |  -90 |  -45 | -115 |  -85 |               0 | True              |
| 1417 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         80 |       -165 |        245 |   -5 |   25 |  -65 |   80 | -115 |   70 |   15 | -165 |               0 | True              |
| 1418 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        175 |       -135 |        310 |  175 |  115 | -100 |  -30 |  -65 |  -70 | -110 | -135 |               0 | True              |
| 1419 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        170 |       -150 |        320 |  150 |  170 |  -35 | -120 | -120 |  -60 | -150 |  -60 |               0 | True              |
| 1420 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        225 |       -115 |        340 |  100 |  225 |  165 | -110 |  -35 |  -20 |   25 | -115 |               0 | True              |
| 1421 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -155 | -225 | -210 | -130 | -135 | -265 | -190 |  -90 |               0 | True              |
| 1422 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |        -45 |        225 |   90 |  180 |  120 |  105 |  115 |  150 |   15 |  -45 |               0 | True              |
| 1423 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        185 |       -140 |        325 |  160 |  100 |  185 |  185 |  -35 | -140 |  -70 |  -75 |               0 | True              |
| 1424 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -405 |        405 | -100 |  -85 |  -50 | -220 | -405 | -225 | -285 | -270 |               2 | True              |
| 1425 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -765 |        765 | -765 | -670 | -640 | -670 | -635 | -725 | -745 | -700 |               6 | True              |
| 1426 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -540 |        540 | -315 | -530 | -500 | -540 | -460 | -445 | -535 | -445 |               4 | True              |
| 1427 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         -0 |       -345 |        345 |   -5 | -115 |   -0 | -270 | -345 | -210 | -275 | -310 |               0 | True              |
| 1428 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         80 |       -185 |        265 |   -5 |   45 |   30 |   80 |  -55 | -185 | -125 | -160 |               0 | True              |
| 1429 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         75 |       -165 |        240 |   75 |   65 | -165 |  -45 | -105 |   10 | -125 |  -45 |               0 | True              |
| 1430 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -675 |        675 | -510 | -535 | -485 | -485 | -630 | -665 | -675 | -600 |               6 | True              |
| 1431 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         85 |       -240 |        325 |   45 |  -10 |   85 | -170 | -205 | -195 | -240 | -175 |               0 | True              |
| 1432 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -725 |        725 | -460 | -370 | -425 | -525 | -680 | -580 | -725 | -565 |               5 | True              |
| 1433 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -305 |        305 |  -20 | -150 | -270 | -210 | -215 | -305 | -225 | -235 |               0 | True              |
| 1434 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -415 |        415 | -140 | -180 | -240 | -255 | -315 | -415 | -240 | -285 |               2 | True              |
| 1435 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -115 |        260 |  145 |   85 |   60 |  105 |  125 |  110 | -115 |  -60 |               0 | True              |
| 1436 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -100 |        280 |  110 |  160 |  180 |  165 |  175 |  -30 | -100 |  -25 |               0 | True              |
| 1437 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -625 |        625 | -325 | -425 | -320 | -395 | -625 | -575 | -595 | -605 |               4 | True              |
| 1438 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -615 |        615 | -405 | -440 | -340 | -430 | -325 | -615 | -530 | -560 |               5 | True              |
| 1439 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        160 |       -120 |        280 |  160 |   70 | -120 |  -65 |   45 |   40 |  -95 |  -70 |               0 | True              |
| 1440 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        145 |       -105 |        250 |  145 |  130 |  -70 |   30 |  -45 | -105 |  -55 |  -50 |               0 | True              |
| 1441 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -125 |        125 |  -40 |  -50 |  -40 |  -90 |  -65 | -125 |  -65 |  -40 |               0 | True              |
| 1442 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        110 |       -250 |        360 |  110 |   25 |  -55 |   90 |    5 |   -0 | -210 | -250 |               0 | True              |
| 1443 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -180 |        255 |   50 |  -20 |   75 |  -35 | -175 | -180 |  -90 | -125 |               0 | True              |
| 1444 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        175 |       -110 |        285 |  175 |  135 | -100 |  -95 | -110 | -100 |   30 |   -5 |               0 | True              |
| 1445 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        195 |        -45 |        240 |   85 |  195 |  175 |  105 |   85 |   80 |   95 |  -45 |               0 | True              |
| 1446 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        175 |        -15 |        190 |  150 |  115 |  175 |  100 |  115 |  100 |   95 |  -15 |               0 | True              |
| 1447 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         55 |       -165 |        220 |   10 |   55 | -165 | -120 | -135 |  -90 | -125 | -125 |               0 | True              |
| 1448 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        240 |        -40 |        280 |  185 |  240 |  -40 |   70 |   70 |  -20 |   -5 |  145 |               0 | True              |
| 1449 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        160 |       -125 |        285 |  120 |  120 |  160 |  100 |   30 | -125 | -110 |  -60 |               0 | True              |
| 1450 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        225 |        -40 |        265 |  225 |  -40 |  115 |   75 |   55 |   70 |  -15 |   -0 |               0 | True              |
| 1451 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         10 |       -195 |        205 |   -5 |   10 | -160 |  -55 |  -35 |    5 | -175 | -195 |               0 | True              |
| 1452 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |       -215 |        385 |  170 |  150 |   75 |    5 |  145 |  165 | -215 | -155 |               0 | True              |
| 1453 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        265 |        -50 |        315 |  140 |  265 |   10 |  -50 |  -40 |   -5 |  -40 |  -20 |               0 | True              |
| 1454 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -380 |        380 | -160 | -190 | -255 | -175 | -165 | -175 | -150 | -380 |               0 | True              |
| 1455 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        250 |        -25 |        275 |  210 |  250 |  -25 |   75 |   35 |   55 |   70 |   10 |               0 | True              |
| 1456 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        220 |       -130 |        350 |  160 |  220 |   60 |  135 |   -5 |   60 | -130 |   15 |               0 | True              |
| 1457 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -115 |        190 |   75 |   45 |   75 |  -20 | -115 |  -80 |  -40 |  -65 |               0 | True              |
| 1458 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -200 |        220 |   20 | -185 | -130 | -200 |  -70 | -165 |  -95 | -135 |               0 | True              |
| 1459 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        165 |        -80 |        245 |  135 |  110 |  105 |  115 |  165 |  125 |  100 |  -80 |               0 | True              |
| 1460 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         50 |       -230 |        280 |   -5 |   50 | -145 |  -60 | -135 |  -65 |   35 | -230 |               0 | True              |
| 1461 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -720 |        720 | -510 | -695 | -600 | -655 | -720 | -630 | -655 | -675 |               5 | True              |
| 1462 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |        -70 |        230 |   95 |   75 |  160 |  -70 |  -30 |  -65 |  -50 |  -70 |               0 | True              |
| 1463 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        130 |       -180 |        310 |   50 |  110 |  130 |  130 |   15 |  -15 | -180 |  -95 |               0 | True              |
| 1464 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         95 |       -220 |        315 |   95 | -195 | -220 | -165 | -210 | -150 | -210 | -160 |               0 | True              |
| 1465 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        225 |        -95 |        320 |  160 |  225 |   10 |   -5 |    5 |   40 |  -95 |  -50 |               0 | True              |
| 1466 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -330 |        455 |   50 |  -50 |   50 |  125 |  -10 |    5 | -105 | -330 |               0 | True              |
| 1467 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         55 |       -130 |        185 |   40 |   30 |   50 |   45 |   55 |    5 |  -45 | -130 |               0 | True              |
| 1468 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         30 |        -75 |        105 |  -45 |  -25 |   30 |  -55 |   -0 |  -10 |  -75 |   10 |               0 | True              |
| 1469 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        200 |       -160 |        360 |   80 |  200 |  185 |  -80 |  -55 |  -65 |  -70 | -160 |               0 | True              |
| 1470 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -405 |        405 | -280 | -365 | -320 | -405 | -275 | -320 | -350 | -305 |               1 | True              |
| 1471 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          5 |       -320 |        325 |  -75 |  -10 |  -70 |  -55 |    5 |  -90 | -320 | -215 |               1 | True              |
| 1472 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        330 |        -85 |        415 |  185 |  330 |  325 |   55 |    5 |   80 |  -85 |   15 |               0 | True              |
| 1473 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -325 |        325 |  -15 |  -10 |  -70 |  -55 |  -25 | -245 | -280 | -325 |               1 | True              |
| 1474 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        165 |        -75 |        240 |  165 |   50 |  115 |  135 |  120 |  105 |  -75 |   70 |               0 | True              |
| 1475 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -575 |        575 | -485 | -370 | -335 | -345 | -390 | -415 | -420 | -575 |               3 | True              |
| 1476 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        310 |        -40 |        350 |  180 |  310 |  195 |  190 |  270 |  135 |  -40 |  -15 |               0 | True              |
| 1477 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        140 |       -230 |        370 |    5 |   40 |   90 |  140 |  115 | -200 | -185 | -230 |               0 | True              |
| 1478 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         -0 |       -155 |        155 |   -5 |   -0 |  -90 | -150 |  -40 | -155 |  -65 |  -65 |               0 | True              |
| 1479 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -125 |        125 |  -50 |  -75 | -125 |  -80 |  -30 |  -70 | -105 |  -65 |               0 | True              |
| 1480 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -395 |        395 | -155 | -255 | -215 | -190 | -265 | -195 | -160 | -395 |               0 | True              |
| 1481 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        215 |        -45 |        260 |  215 |  210 |  110 |   40 |  -45 |   45 |  105 |   55 |               0 | True              |
| 1482 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -105 |        205 |  100 |   45 |  -20 | -105 |   -0 |   10 |   15 | -100 |               0 | True              |
| 1483 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -705 |        705 | -430 | -510 | -445 | -635 | -675 | -690 | -705 | -655 |               5 | True              |
| 1484 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -425 |        425 | -180 | -230 | -260 | -255 | -240 | -225 | -310 | -425 |               0 | True              |
| 1485 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -200 |        200 |  -40 |  -70 | -150 | -140 |  -75 | -100 |  -75 | -200 |               0 | True              |
| 1486 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        190 |       -120 |        310 |   70 |  190 |  105 |  165 |  -50 |   15 | -120 |   -0 |               0 | True              |
| 1487 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        170 |        -75 |        245 |  170 |  140 |  130 |  -30 |  -20 |  -75 |   -5 |  -15 |               0 | True              |
| 1488 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        240 |        -50 |        290 |  100 |  240 |  180 |  130 |  -40 |   55 |    5 |  -50 |               0 | True              |
| 1489 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         35 |       -160 |        195 |   35 | -160 |  -95 | -160 |  -80 | -120 | -120 |  -15 |               0 | True              |
| 1490 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |       -100 |        260 |    5 |  120 |  160 | -100 |  -50 |  -90 |   -5 |  -20 |               0 | True              |
| 1491 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        340 |          0 |        340 |  225 |  335 |  340 |  170 |  300 |   20 |  135 |  100 |               0 | True              |
| 1492 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        125 |       -100 |        225 |   75 |   65 |   75 |  125 |  105 |   10 |   55 | -100 |               0 | True              |
| 1493 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -390 |        390 | -275 | -235 | -330 | -355 | -390 | -295 | -315 | -330 |               1 | True              |
| 1494 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |        -40 |        140 |  100 |   10 |   55 |   20 |  -40 |   -5 |   -5 |   10 |               0 | True              |
| 1495 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |       -165 |        325 |  155 |  150 |  140 |  160 |  -30 | -130 |  -55 | -165 |               0 | True              |
| 1496 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -380 |        380 | -380 | -335 | -370 | -310 | -365 | -310 | -375 | -270 |               2 | True              |
| 1497 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        105 |       -275 |        380 |  105 | -110 |   50 |  -65 |  -50 | -175 | -180 | -275 |               0 | True              |
| 1498 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -320 |        320 | -170 | -135 | -320 | -160 | -210 | -160 | -165 | -105 |               0 | True              |
| 1499 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         90 |       -185 |        275 |   15 |   90 |   35 |   25 | -110 | -185 | -185 | -185 |               0 | True              |
| 1500 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        160 |        -90 |        250 |  100 |  115 |  -10 |  115 |   60 |   80 |  160 |  -90 |               0 | True              |
| 1501 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -450 |        450 | -210 | -220 | -205 | -385 | -450 | -410 | -420 | -310 |               3 | True              |
| 1502 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -275 |        275 | -100 | -130 | -125 | -275 | -180 | -170 | -200 | -110 |               0 | True              |
| 1503 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        330 |        -35 |        365 |  200 |  290 |  330 |  -35 |   65 |  -15 |   20 |   -0 |               0 | True              |
| 1504 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -540 |        540 | -370 | -330 | -425 | -400 | -325 | -450 | -325 | -540 |               2 | True              |
| 1505 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        130 |       -130 |        260 |  115 |  130 |  100 |   45 |  -65 |  -65 |  -60 | -130 |               0 | True              |
| 1506 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -485 |        485 | -305 | -315 | -290 | -265 | -265 | -475 | -480 | -485 |               3 | True              |
| 1507 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         75 |       -215 |        290 |   -5 |   15 |   25 |  -25 |  -35 |   75 |  -35 | -215 |               0 | True              |
| 1508 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         15 |       -255 |        270 |   15 |  -20 |  -50 | -150 | -115 |  -10 |  -30 | -255 |               0 | True              |
| 1509 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         25 |       -275 |        300 |   25 |   10 | -155 | -270 | -145 | -275 | -250 | -210 |               0 | True              |
| 1510 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        305 |        -10 |        315 |  155 |  175 |  305 |   25 |   70 |   60 |   25 |  -10 |               0 | True              |
| 1511 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        225 |        -25 |        250 |  225 |  210 |  205 |  185 |   80 |  185 |  165 |  -25 |              14 | True              |
| 1512 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        330 |        -50 |        380 |  135 |  140 |  330 |  160 |  190 |  210 |  190 |  -50 |               0 | True              |
| 1513 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        155 |        -50 |        205 |   25 |   50 |  155 |   80 |   30 |  120 |   85 |  -50 |               0 | True              |
| 1514 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -415 |        415 | -185 | -270 | -135 | -195 | -260 | -225 | -235 | -415 |               0 | True              |
| 1515 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        265 |       -110 |        375 |  150 |  155 |  155 |  125 |  265 |   -0 | -110 |  -15 |               0 | True              |
| 1516 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -335 |        335 | -115 |  -40 | -100 |  -70 | -255 | -280 | -290 | -335 |               2 | True              |
| 1517 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -320 |        320 | -295 | -320 | -240 | -305 | -280 | -280 | -270 | -295 |               1 | True              |
| 1518 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -785 |        785 | -645 | -765 | -735 | -645 | -700 | -720 | -675 | -785 |               5 | True              |
| 1519 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -755 |        755 | -415 | -465 | -420 | -550 | -490 | -755 | -735 | -705 |               5 | True              |
| 1520 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         90 |       -125 |        215 |   90 |   35 |  -20 |   25 |  -45 | -125 | -100 |  -70 |               0 | True              |
| 1521 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          5 |       -105 |        110 |  -95 | -105 |    5 |  -40 |  -95 |  -50 |  -75 |  -55 |               0 | True              |
| 1522 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -675 |        675 | -675 | -645 | -645 | -565 | -590 | -660 | -600 | -555 |               5 | True              |
| 1523 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         45 |        -55 |        100 |  -55 |  -30 |  -10 |   20 |   45 |  -40 |  -20 |   20 |               0 | True              |
| 1524 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -105 |        240 |  125 |  135 |  -80 |   45 |  -95 |  -50 |  -35 | -105 |               0 | True              |
| 1525 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -170 |        295 |   90 |   70 |  125 |  125 |   85 |   75 | -170 |  -85 |               0 | True              |
| 1526 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        225 |       -170 |        395 |   95 |  100 |  115 |   85 |  225 | -140 | -170 |  -45 |               0 | True              |
| 1527 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        140 |       -170 |        310 |   35 |  140 |   15 |   55 | -100 |   75 | -170 | -165 |               0 | True              |
| 1528 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        170 |       -120 |        290 |   90 |   65 |  100 |  170 |   55 | -105 |  -20 | -120 |               0 | True              |
| 1529 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        115 |         -5 |        120 |   -5 |   55 |   75 |  115 |   35 |   15 |   80 |   10 |               0 | True              |
| 1530 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        190 |        -75 |        265 |  110 |  170 |  110 |  190 |  125 |  110 |  -50 |  -75 |               0 | True              |
| 1531 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        270 |       -190 |        460 |  230 |  205 |  215 |  235 |  270 |  -30 |  -20 | -190 |               0 | True              |
| 1532 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        120 |       -235 |        355 |  120 |   20 |   60 |  -35 | -140 | -135 | -235 |  -85 |               0 | True              |
| 1533 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        140 |       -170 |        310 |  110 |  140 |   -0 |  -25 |  -55 | -150 |  -70 | -170 |               0 | True              |
| 1534 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        120 |       -145 |        265 |  105 |  120 |  110 |   65 |  110 |   40 | -105 | -145 |               0 | True              |
| 1535 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -540 |        540 | -335 | -295 | -330 | -375 | -505 | -540 | -525 | -460 |               4 | True              |
| 1536 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -215 |        275 |   60 |    5 | -115 |  -85 | -105 |  -80 |  -55 | -215 |               0 | True              |
| 1537 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        130 |       -215 |        345 |   45 |   40 |   20 |   50 |  130 |    5 |  105 | -215 |               0 | True              |
| 1538 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         85 |       -175 |        260 |   25 |   85 |  -90 | -115 | -135 | -175 |   -0 | -150 |               0 | True              |
| 1539 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         70 |       -230 |        300 |   20 |   10 |   40 |   65 |   70 |   65 | -230 | -220 |               0 | True              |
| 1540 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -390 |        390 | -160 |  -45 | -140 | -105 |  -90 | -125 |  -75 | -390 |               0 | True              |
| 1541 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -130 |        130 |   -5 |  -60 | -110 | -100 |  -45 | -130 | -100 | -130 |               0 | True              |
| 1542 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |        -40 |        190 |  115 |   90 |  145 |  150 |  125 |  120 |  -40 |  -10 |               0 | True              |
| 1543 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        110 |       -140 |        250 |  110 |  -65 | -130 |   10 | -135 | -140 |  -95 |  -80 |               0 | True              |
| 1544 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        110 |       -250 |        360 |   55 |   90 |   -5 |  110 |  -50 | -250 | -175 | -160 |               0 | True              |
| 1545 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         -0 |       -240 |        240 |  -75 |  -30 |  -50 |  -25 |  -20 |  -55 |   -0 | -240 |               0 | True              |
| 1546 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        100 |       -140 |        240 |   40 |  100 |   80 |   50 |   40 |   75 | -140 |  -45 |               0 | True              |
| 1547 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        280 |          0 |        280 |   90 |  235 |  280 |  220 |  200 |  270 |  205 |   50 |               0 | True              |
| 1548 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        285 |          0 |        285 |  285 |  185 |  175 |  225 |   25 |  125 |  100 |   10 |              16 | True              |
| 1549 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        215 |        -75 |        290 |  165 |  215 |   35 |  145 |  -75 |    5 |   -5 |  -50 |               0 | True              |
| 1550 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         25 |       -180 |        205 |   25 | -130 | -180 | -170 | -150 | -155 | -175 | -180 |               0 | True              |
| 1551 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -315 |        315 |  -85 | -210 |  -30 | -315 | -280 | -285 | -260 | -290 |               2 | True              |
| 1552 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -160 |        160 |  -55 |  -60 | -160 |  -90 | -105 |  -75 |  -25 | -155 |               0 | True              |
| 1553 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        225 |        -35 |        260 |   90 |  125 |  225 |  145 |  160 |  150 |   10 |  -35 |               0 | True              |
| 1554 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         30 |       -135 |        165 |   -5 |  -15 |  -25 |   30 | -115 | -135 |  -25 |    5 |               0 | True              |
| 1555 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         95 |       -220 |        315 |   30 |   20 |   30 |  -25 |   -0 |  -30 |   95 | -220 |               0 | True              |
| 1556 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -335 |        335 | -100 | -175 |  -80 |  -65 |  -30 | -170 | -140 | -335 |               0 | True              |
| 1557 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         55 |       -205 |        260 |   35 |   55 |  -35 | -130 |   25 | -205 | -145 | -135 |               0 | True              |
| 1558 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        270 |        -40 |        310 |  120 |  270 |   20 |  -20 |  -40 |  -15 |  -10 |   -5 |               0 | True              |
| 1559 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -125 |        125 |  -90 | -120 |  -65 |  -55 |  -25 |  -80 |  -70 | -125 |               0 | True              |
| 1560 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -590 |        590 | -315 | -415 | -340 | -360 | -295 | -340 | -590 | -550 |               4 | True              |
| 1561 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        200 |       -190 |        390 |  110 |  200 |   40 |   40 |  -75 | -170 | -125 | -190 |               0 | True              |
| 1562 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        115 |        -70 |        185 |   40 |    5 |  -55 |  115 |   40 |   95 |   45 |  -70 |               0 | True              |
| 1563 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -310 |        310 | -160 | -185 |  -45 |  -25 |  -70 | -125 |  -20 | -310 |               0 | True              |
| 1564 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -510 |        510 | -210 | -295 | -345 | -290 | -195 | -285 | -405 | -510 |               3 | True              |
| 1565 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -370 |        370 | -150 | -280 | -170 | -195 | -215 | -210 | -190 | -370 |               0 | True              |
| 1566 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        290 |          0 |        290 |   95 |  200 |  290 |  210 |  190 |  220 |   45 |   25 |               0 | True              |
| 1567 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        135 |       -155 |        290 |  135 |  -55 |  -70 | -155 |   20 |  -50 |  -85 |  -55 |               0 | True              |
| 1568 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         10 |       -125 |        135 | -120 |   10 |  -15 | -115 | -100 |  -20 |  -50 | -125 |               0 | True              |
| 1569 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -380 |        380 | -300 | -265 | -240 | -245 | -315 | -305 | -145 | -380 |               2 | True              |
| 1570 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        200 |        -30 |        230 |  200 |  195 |  185 |  -30 |   -0 |   55 |   50 |  110 |               0 | True              |
| 1571 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         45 |        -85 |        130 |   15 |   35 |   10 |   -5 |   45 |  -85 |  -75 |  -45 |               0 | True              |
| 1572 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -260 |        260 | -155 | -205 |  -85 |  -95 |  -70 |  -50 |  -90 | -260 |               0 | True              |
| 1573 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -410 |        410 | -135 | -195 | -200 | -410 | -390 | -340 | -375 | -310 |               2 | True              |
| 1574 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         50 |       -115 |        165 |   50 |  -70 |  -30 |  -55 |   10 |  -25 |  -70 | -115 |               0 | True              |
| 1575 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -390 |        390 | -175 | -115 | -235 | -195 | -160 | -205 | -205 | -390 |               0 | True              |
| 1576 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -770 |        770 | -550 | -555 | -665 | -730 | -745 | -760 | -770 | -695 |               6 | True              |
| 1577 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        210 |       -105 |        315 |  130 |  145 |  210 |   50 |  170 | -105 |  -90 |  -80 |               0 | True              |
| 1578 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -365 |        365 | -125 | -135 | -335 | -335 | -360 | -365 | -320 | -310 |               2 | True              |
| 1579 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        110 |        -85 |        195 |   55 |  -10 |   30 |   25 |   25 |   65 |  110 |  -85 |               0 | True              |
| 1580 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        185 |        -35 |        220 |  185 |   90 |   35 |  -35 |    5 |   25 |   -5 |   65 |               0 | True              |
| 1581 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         25 |       -175 |        200 |   25 |  -25 |   15 |  -70 | -160 | -175 | -165 | -155 |               0 | True              |
| 1582 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         25 |       -245 |        270 |   25 | -245 | -185 | -205 | -235 | -180 | -210 | -220 |               0 | True              |
| 1583 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        110 |       -220 |        330 |   20 |  110 |   95 |   75 |   55 |   45 | -175 | -220 |               0 | True              |
| 1584 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -225 |        285 |   45 |   60 | -115 | -145 | -150 | -140 | -225 |  -90 |               0 | True              |
| 1585 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -275 |        335 |  -25 |   20 |  -75 |   45 |   60 |  -40 |   -5 | -275 |               0 | True              |
| 1586 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        245 |          0 |        245 |  165 |  245 |  175 |  195 |  110 |  180 |  215 |   25 |               0 | True              |
| 1587 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         10 |       -250 |        260 |  -75 | -145 | -135 |  -80 |   10 |  -75 | -150 | -250 |               0 | True              |
| 1588 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         65 |       -145 |        210 |  -20 |  -35 |   25 |   65 |  -10 |   55 |   10 | -145 |               0 | True              |
| 1589 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |        -95 |        270 |  120 |  105 |   60 |   95 |  175 |  -95 |  -30 |  -65 |               0 | True              |
| 1590 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        190 |       -110 |        300 |  190 |  -70 | -110 | -105 |  -15 |  -40 |   30 |   -0 |               0 | True              |
| 1591 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        225 |        -50 |        275 |  200 |  225 |  205 |  165 |  -25 |   25 |  -50 |  -15 |               0 | True              |
| 1592 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         35 |       -220 |        255 |  -50 |  -15 |  -80 |   35 |  -85 |  -35 |  -35 | -220 |               0 | True              |
| 1593 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -260 |        280 |  -55 |   20 | -125 |  -30 |  -70 |  -55 |  -30 | -260 |               0 | True              |
| 1594 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -630 |        630 | -400 | -495 | -520 | -385 | -630 | -610 | -530 | -580 |               4 | True              |
| 1595 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        225 |        -25 |        250 |  225 |  130 |  190 |   80 |  220 |  190 |  -25 |  -10 |              17 | True              |
| 1596 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -460 |        460 | -255 | -330 | -270 | -320 | -300 | -320 | -460 | -400 |               3 | True              |
| 1597 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -230 |        230 | -105 |  -95 | -105 | -185 |   -5 | -110 |  -10 | -230 |               0 | True              |
| 1598 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -210 |        335 |  -25 |  125 |   35 |   10 |    5 |   55 | -180 | -210 |               1 | True              |
| 1599 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        200 |        -70 |        270 |  200 |   25 |  -70 |  -65 |  -70 |  -10 |   30 |   25 |               0 | True              |
| 1600 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        255 |        -70 |        325 |  160 |  160 |  175 |  255 |  185 |   -5 |  -60 |  -70 |               0 | True              |
| 1601 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -340 |        340 |  -90 |  -75 | -125 |  -95 | -140 | -135 | -160 | -340 |               0 | True              |
| 1602 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -130 |        255 |   35 |  125 |   50 |   60 |   90 |  100 | -130 | -130 |               0 | True              |
| 1603 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -405 |        405 | -285 | -365 | -300 | -325 | -405 | -245 | -285 | -365 |               1 | True              |
| 1604 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        145 |       -180 |        325 |   25 |  145 |   75 | -125 | -180 |   80 |  -40 | -180 |               0 | True              |
| 1605 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -300 |        360 |  -15 |   60 |   -0 |  -50 |  -15 |  -10 |  -25 | -300 |               0 | True              |
| 1606 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         45 |       -220 |        265 |   45 |   40 |   10 | -220 | -190 |  -85 | -195 | -140 |               0 | True              |
| 1607 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        135 |        -25 |        160 |  135 |   -5 |    5 |   20 |  -25 |   40 |   15 |   80 |               0 | True              |
| 1608 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        145 |        -95 |        240 |  145 |  -50 |  -90 |  -65 |  -40 |  -95 |  -45 |  -20 |               0 | True              |
| 1609 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        285 |          0 |        285 |  150 |  160 |  270 |  285 |  220 |  275 |   30 |   20 |               0 | True              |
| 1610 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -390 |        390 | -155 | -130 |  -50 | -195 | -390 | -275 | -340 | -380 |               2 | True              |
| 1611 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         15 |        -90 |        105 |  -20 |   10 |  -90 |  -40 |  -20 |  -55 |   15 |  -80 |               0 | True              |
| 1612 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -605 |        605 | -350 | -340 | -420 | -445 | -435 | -395 | -605 | -600 |               4 | True              |
| 1613 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |       -110 |        305 |  105 |  195 |   95 |  115 | -110 |  -30 |  -65 |  -10 |               0 | True              |
| 1614 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        105 |       -155 |        260 |   15 |  105 |  -75 |  -95 | -155 | -125 | -105 |  -45 |               0 | True              |
| 1615 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -445 |        445 | -435 | -370 | -390 | -345 | -440 | -330 | -345 | -445 |               3 | True              |
| 1616 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         55 |       -145 |        200 |  -30 |   20 |   55 |   -5 |  -65 |   -5 |  -30 | -145 |               0 | True              |
| 1617 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -475 |        475 | -305 | -210 | -430 | -475 | -375 | -345 | -445 | -445 |               4 | True              |
| 1618 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         -0 |       -135 |        135 |  -95 |  -25 |  -50 |  -25 | -135 |   -0 |  -15 |  -50 |               0 | True              |
| 1619 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         50 |       -175 |        225 |  -15 |   10 |   45 |  -10 |   45 |   50 |   15 | -175 |               0 | True              |
| 1620 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        135 |        -75 |        210 |  -20 |   40 |  -25 |   60 |   55 |   55 |  -75 |  135 |               0 | True              |
| 1621 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -640 |        640 | -485 | -475 | -440 | -470 | -475 | -520 | -545 | -640 |               3 | True              |
| 1622 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |       -190 |        350 |  105 |   30 |  -50 |   35 |   10 |  160 | -190 | -150 |               0 | True              |
| 1623 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -265 |        265 |   -5 |  -30 |  -65 | -265 | -160 | -185 | -220 | -200 |               1 | True              |
| 1624 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        110 |       -235 |        345 |  110 |   15 |   10 |  105 | -235 | -125 | -150 | -130 |               0 | True              |
| 1625 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         40 |       -130 |        170 |   25 |   15 |  -10 |   40 |  -30 |   25 |  -15 | -130 |               0 | True              |
| 1626 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |       -115 |        315 |   60 |  145 |  200 |   85 |   85 | -115 | -100 |  -20 |               0 | True              |
| 1627 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -160 |        180 |   10 |   15 |    5 |  -75 |   20 |  -15 |   10 | -160 |               0 | True              |
| 1628 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        110 |        -30 |        140 |  -20 |   50 |   35 |   40 |  -30 |   85 |   25 |  110 |               0 | True              |
| 1629 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        210 |       -130 |        340 |  105 |  210 |  180 |  105 | -130 |  -85 |  -25 |  -10 |               0 | True              |
| 1630 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -695 |        695 | -525 | -585 | -655 | -580 | -570 | -695 | -690 | -630 |               4 | True              |
| 1631 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        155 |         -5 |        160 |  155 |   65 |    5 |   -5 |   70 |   75 |   70 |   80 |               0 | True              |
| 1632 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        120 |       -205 |        325 |  120 |  -15 |   50 | -120 |  -50 | -130 |  -95 | -205 |               0 | True              |
| 1633 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -570 |        570 | -420 | -490 | -545 | -515 | -560 | -570 | -515 | -515 |               5 | True              |
| 1634 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         -0 |       -160 |        160 |  -90 |  -20 |  -60 |  -20 |  -65 |  -20 | -160 |   -0 |               0 | True              |
| 1635 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -140 |        240 |  100 | -140 |  -60 | -115 | -115 |  -30 |  -20 | -115 |               0 | True              |
| 1636 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        220 |       -145 |        365 |    5 |   40 |  100 |  100 |  135 |  220 |   -0 | -145 |               0 | True              |
| 1637 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        100 |       -255 |        355 |  -40 |  100 |   -5 | -125 | -245 | -255 | -140 | -215 |               1 | True              |
| 1638 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -375 |        375 | -120 | -170 |  -85 |  -35 | -120 |  -60 | -105 | -375 |               0 | True              |
| 1639 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |        -95 |        155 |  -30 |   10 |   -0 |   60 |  -10 |  -30 |  -95 |  -95 |               0 | True              |
| 1640 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -410 |        410 | -145 | -140 | -220 | -185 | -165 | -360 | -410 | -310 |               2 | True              |
| 1641 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        350 |          0 |        350 |  190 |  135 |  175 |  350 |  250 |  105 |   80 |   30 |               0 | True              |
| 1642 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -420 |        420 | -215 | -265 | -300 | -390 | -355 | -300 | -360 | -420 |               3 | True              |
| 1643 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         20 |       -165 |        185 |  -45 |  -95 |  -90 | -130 |  -65 | -165 |   20 | -125 |               0 | True              |
| 1644 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -615 |        615 | -545 | -410 | -435 | -480 | -615 | -520 | -445 | -465 |               4 | True              |
| 1645 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -135 |        135 |  -45 |  -80 |  -35 | -115 |  -65 | -130 | -135 |   -5 |               0 | True              |
| 1646 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -205 |        340 |   75 |   80 |  135 | -115 | -200 | -130 | -185 | -205 |               0 | True              |
| 1647 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        135 |       -175 |        310 |  125 |   55 |   40 |  135 |  100 |  130 |  -90 | -175 |               0 | True              |
| 1648 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         30 |       -410 |        440 |   30 | -145 |  -90 | -175 |  -70 | -100 | -260 | -410 |               0 | True              |
| 1649 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -200 |        285 |   85 | -200 |  -35 | -125 | -160 |  -70 |  -45 | -110 |               0 | True              |
| 1650 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        210 |       -135 |        345 |   50 |  210 |  170 |  170 |  -20 |  -35 |  -95 | -135 |               0 | True              |
| 1651 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        195 |        -65 |        260 |  195 |  -30 |   45 |  -25 |  -65 |  -35 |  -25 |   -5 |               0 | True              |
| 1652 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         25 |       -255 |        280 |   25 |   25 | -140 |  -60 | -235 | -190 | -255 | -175 |               0 | True              |
| 1653 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -340 |        340 | -295 | -275 | -310 | -245 | -340 | -245 | -290 | -265 |               1 | True              |
| 1654 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        175 |          0 |        175 |  125 |  175 |  105 |  125 |   60 |  175 |  125 |  120 |              15 | True              |
| 1655 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        235 |        -50 |        285 |  145 |  135 |  165 |  205 |  205 |  235 |  -50 |  -40 |               0 | True              |
| 1656 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -445 |        445 | -140 | -405 | -320 | -310 | -445 | -415 | -280 | -370 |               2 | True              |
| 1657 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        175 |        -85 |        260 |  140 |  140 |  145 |   90 |  175 |  -85 |  -55 |  -20 |               0 | True              |
| 1658 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -640 |        640 | -330 | -400 | -310 | -590 | -555 | -640 | -565 | -575 |               4 | True              |
| 1659 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -425 |        425 | -185 | -110 | -180 | -215 | -255 | -120 | -360 | -425 |               2 | True              |
| 1660 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -310 |        310 | -150 | -135 | -120 | -100 |  -40 | -160 |  -50 | -310 |               0 | True              |
| 1661 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -285 |        285 | -160 | -190 | -140 | -170 | -165 |  -90 |  -80 | -285 |               0 | True              |
| 1662 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -465 |        465 | -285 | -320 | -285 | -285 | -310 | -450 | -465 | -445 |               3 | True              |
| 1663 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -545 |        545 | -250 | -425 | -370 | -430 | -395 | -545 | -390 | -445 |               3 | True              |
| 1664 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         15 |       -215 |        230 |  -80 |  -70 |   -0 |   15 |  -45 |  -65 |  -20 | -215 |               0 | True              |
| 1665 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        220 |          0 |        220 |  145 |  170 |  175 |  155 |  135 |  220 |  190 |   40 |               0 | True              |
| 1666 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        315 |          0 |        315 |  150 |  270 |  315 |  280 |  260 |  135 |   65 |   45 |               0 | True              |
| 1667 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        335 |          0 |        335 |  250 |  335 |  240 |  270 |  210 |  255 |  245 |   95 |              14 | True              |
| 1668 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        130 |       -110 |        240 |  130 |   20 |  -80 |   30 |  -80 |   25 |  -35 | -110 |               0 | True              |
| 1669 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        195 |       -175 |        370 |  195 |  155 |  -85 |   -0 |  -95 | -175 | -170 | -130 |               0 | True              |
| 1670 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        305 |        -85 |        390 |  305 |  250 |  220 |  140 |  265 |   35 |   30 |  -85 |              16 | True              |
| 1671 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        130 |       -110 |        240 |  110 |   55 |   60 |  130 |   45 |   -5 | -110 | -105 |               0 | True              |
| 1672 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        345 |          0 |        345 |  205 |  235 |  345 |  270 |  290 |   60 |   45 |   75 |               0 | True              |
| 1673 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        220 |        -45 |        265 |  125 |  170 |  220 |  135 |  220 |  125 |  155 |  -45 |               0 | True              |
| 1674 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -330 |        330 |  -35 |  -55 | -145 |  -70 |  -75 | -135 |  -60 | -330 |               0 | True              |
| 1675 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -120 | -265 | -245 | -140 |  -85 | -175 | -215 | -175 |               0 | True              |
| 1676 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |        -30 |         70 |  -20 |  -10 |  -30 |   40 |  -10 |   15 |   -5 |   25 |               0 | True              |
| 1677 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        220 |        -30 |        250 |  210 |   95 |  170 |  220 |  205 |   35 |   80 |  -30 |               0 | True              |
| 1678 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        175 |       -130 |        305 |   45 |  175 |  100 |   15 |  -50 | -130 | -125 |  -75 |               0 | True              |
| 1679 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -550 |        550 | -220 | -440 | -550 | -390 | -415 | -420 | -480 | -505 |               3 | True              |
| 1680 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        205 |        -15 |        220 |  170 |  205 |  100 |  -10 |  -15 |  105 |   -0 |   -5 |               0 | True              |
| 1681 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        305 |        -65 |        370 |  305 |  190 |  215 |  160 |  280 |   55 |  -15 |  -65 |              15 | True              |
| 1682 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -395 |        395 | -320 | -315 | -335 | -395 | -390 | -335 | -310 | -395 |               2 | True              |
| 1683 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -325 |        325 |  -10 |  -40 | -315 | -225 | -210 | -325 | -245 | -275 |               0 | True              |
| 1684 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        175 |       -140 |        315 |  175 |  160 |   20 |   80 |   60 |    5 |   55 | -140 |               0 | True              |
| 1685 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -530 |        530 | -260 | -270 | -435 | -530 | -435 | -450 | -500 | -425 |               3 | True              |
| 1686 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         65 |       -205 |        270 |   50 |   -0 |   40 |  -25 |   65 | -110 | -205 |  -65 |               0 | True              |
| 1687 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -560 |        560 | -265 | -370 | -280 | -315 | -250 | -560 | -525 | -515 |               3 | True              |
| 1688 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         20 |       -235 |        255 |   10 |   20 |  -35 |   20 | -190 | -155 | -235 | -175 |               0 | True              |
| 1689 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          5 |       -335 |        340 | -105 |  -40 | -105 | -140 | -155 |  -75 |    5 | -335 |               0 | True              |
| 1690 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -695 |        695 | -430 | -465 | -485 | -470 | -435 | -485 | -400 | -695 |               3 | True              |
| 1691 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        100 |       -205 |        305 |  100 |   75 |    5 |   45 |   15 |   80 |   85 | -205 |               0 | True              |
| 1692 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -555 |        555 | -225 | -365 | -295 | -305 | -315 | -320 | -350 | -555 |               1 | True              |
| 1693 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         75 |       -275 |        350 |   40 |   75 | -185 | -275 | -125 | -135 | -195 | -215 |               0 | True              |
| 1694 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          5 |       -295 |        300 |  -10 |    5 | -100 |  -25 |  -80 |  -40 |  -30 | -295 |               0 | True              |
| 1695 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         75 |       -215 |        290 |   25 |   75 | -145 | -180 | -215 |  -55 | -210 |  -65 |               0 | True              |
| 1696 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         90 |       -285 |        375 |   90 |   45 |  -40 | -120 |  -65 | -180 | -135 | -285 |               0 | True              |
| 1697 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -305 |        305 | -115 | -175 | -180 | -225 | -305 | -230 | -275 | -220 |               0 | True              |
| 1698 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        345 |          0 |        345 |  205 |  345 |  200 |  340 |  305 |  335 |    5 |   50 |               0 | True              |
| 1699 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -745 |        745 | -570 | -490 | -690 | -645 | -725 | -745 | -725 | -725 |               6 | True              |
| 1700 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -395 |        395 | -230 | -265 | -355 | -395 | -360 | -370 | -390 | -330 |               1 | True              |
| 1701 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        310 |          0 |        310 |  165 |  245 |  285 |  250 |  175 |  300 |  310 |   45 |               0 | True              |
| 1702 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -295 |        295 | -220 | -250 | -215 | -190 | -190 | -250 | -190 | -295 |               1 | True              |
| 1703 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         80 |       -245 |        325 |   -0 |  -40 |   25 |   80 |   30 | -240 | -245 | -200 |               0 | True              |
| 1704 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |       -260 |        410 |  150 |  -35 |  -15 |  -10 | -260 | -205 | -225 | -215 |               0 | True              |
| 1705 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         70 |       -280 |        350 |   70 |   20 | -110 | -165 | -200 | -170 | -280 | -170 |               0 | True              |
| 1706 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |       -180 |        295 |  115 | -180 | -140 |  -90 | -180 |  -85 |  -70 | -125 |               0 | True              |
| 1707 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -170 |        170 |  -65 | -115 | -110 |  -75 | -145 | -170 | -115 |  -65 |               0 | True              |
| 1708 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |        -10 |        190 |  150 |  180 |  -10 |   35 |   -5 |   20 |    5 |    5 |               0 | True              |
| 1709 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -560 |        560 | -225 | -325 | -260 | -200 | -250 | -275 | -560 | -425 |               3 | True              |
| 1710 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        175 |       -205 |        380 |   35 |  140 |  175 |  125 | -205 |  -95 |  -90 |  -75 |               0 | True              |
| 1711 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -130 |        130 |  -60 |  -40 | -130 | -110 |  -40 | -110 |  -70 |  -30 |               0 | True              |
| 1712 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -260 |        260 | -125 |  -80 | -135 | -240 | -225 | -225 | -170 | -260 |               2 | True              |
| 1713 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        120 |          0 |        120 |    5 |  110 |   55 |   80 |   30 |   85 |  120 |   80 |               0 | True              |
| 1714 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         85 |       -185 |        270 |   85 |  -70 | -145 |  -95 | -185 |  -70 | -155 |  -50 |               0 | True              |
| 1715 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -275 |        275 | -160 | -275 | -200 | -190 | -155 | -150 | -145 | -150 |               0 | True              |
| 1716 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |       -300 |        360 |   60 | -275 | -250 | -210 | -195 | -285 | -200 | -300 |               0 | True              |
| 1717 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        145 |       -110 |        255 |  145 |   90 |  140 |   70 |   40 |  -85 |   35 | -110 |               0 | True              |
| 1718 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -390 |        390 | -100 | -250 | -260 | -300 | -320 | -270 | -335 | -390 |               2 | True              |
| 1719 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |       -155 |        305 |   85 |  150 |   55 |  115 | -125 | -110 | -155 | -115 |               0 | True              |
| 1720 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -265 |        265 | -155 | -130 |  -60 |  -10 |  -80 |  -40 | -140 | -265 |               0 | True              |
| 1721 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -575 |        575 | -265 | -315 | -270 | -290 | -380 | -295 | -555 | -575 |               3 | True              |
| 1722 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        255 |       -120 |        375 |  135 |  235 |   60 |  255 |  210 |  130 | -120 |  -50 |               0 | True              |
| 1723 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        210 |       -105 |        315 |    5 |   45 |  210 |   60 |  140 |  -30 | -105 |  -55 |               0 | True              |
| 1724 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        310 |        -55 |        365 |  200 |  220 |  245 |  220 |  310 |  -55 |  -35 |  -30 |               0 | True              |
| 1725 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |        -75 |        265 |  165 |   95 |   55 |   70 |  190 |  -75 |   30 |  -25 |               0 | True              |
| 1726 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        320 |       -105 |        425 |  240 |  240 |  145 |  160 |  245 |  200 |  320 | -105 |              14 | True              |
| 1727 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         90 |       -220 |        310 |   50 |   45 |   90 |   60 | -110 | -145 | -220 |  -35 |               0 | True              |
| 1728 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -160 |        160 |  -75 |  -70 |  -85 | -140 | -150 | -160 |  -80 | -145 |               0 | True              |
| 1729 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -340 |        340 | -280 | -160 | -120 | -100 | -105 | -160 | -220 | -340 |               1 | True              |
| 1730 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -395 |        395 | -135 | -220 | -140 | -135 | -380 | -380 | -395 | -360 |               2 | True              |
| 1731 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        100 |       -195 |        295 |   90 |  100 |  100 |   65 |   55 | -195 |  -95 | -155 |               0 | True              |
| 1732 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        240 |       -115 |        355 |  235 |  240 |   10 |  -55 |   20 |  -20 |  -60 | -115 |              17 | False             |
| 1733 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        320 |          0 |        320 |  190 |  210 |  320 |  105 |  210 |  145 |   40 |   15 |               0 | True              |
| 1734 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -725 |        725 | -530 | -470 | -470 | -610 | -600 | -455 | -500 | -725 |               4 | True              |
| 1735 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |        -75 |        195 |  120 |  -25 |  -75 |   35 |  -25 |   65 |  -15 |  -50 |               0 | True              |
| 1736 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         50 |       -255 |        305 |   20 |  -20 |   35 |   50 |    5 | -255 | -215 | -110 |               0 | True              |
| 1737 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        155 |       -115 |        270 |   50 |   25 |  105 |   70 |  155 |   60 |   20 | -115 |               0 | True              |
| 1738 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |       -240 |        420 |   50 |   25 |  140 |   30 |  180 |   30 | -195 | -240 |               0 | True              |
| 1739 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        185 |       -195 |        380 |   85 |  185 |  100 |   70 |  -50 |  -85 | -150 | -195 |               0 | True              |
| 1740 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -185 |        250 |   65 |   25 |   15 | -175 | -125 | -105 | -185 | -105 |               0 | True              |
| 1741 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         80 |       -285 |        365 |   80 |   35 |  -40 | -205 | -120 | -155 | -285 | -220 |               0 | True              |
| 1742 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        115 |       -140 |        255 |  115 |  105 |  115 |  -60 | -100 | -140 |  -85 |  -65 |               0 | True              |
| 1743 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |       -105 |        285 |  115 |  140 |  135 |   60 |  180 |  -60 |  -35 | -105 |               0 | True              |
| 1744 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         65 |       -265 |        330 |   65 |   15 |   10 |   30 |   40 |  -10 |   40 | -265 |               0 | True              |
| 1745 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -360 |        360 | -220 | -125 | -245 | -345 | -350 | -325 | -360 | -295 |               3 | True              |
| 1746 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        105 |       -210 |        315 |   15 |   90 |   20 |   50 |  105 |   -0 | -210 | -160 |               0 | True              |
| 1747 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        210 |        -65 |        275 |  115 |  175 |   75 |  210 |   75 |  145 |  -20 |  -65 |               0 | True              |
| 1748 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -450 |        450 | -310 | -350 | -325 | -395 | -450 | -290 | -290 | -250 |               1 | True              |
| 1749 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        385 |          0 |        385 |  195 |  330 |  255 |  385 |  225 |  300 |  300 |  105 |               0 | True              |
| 1750 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        125 |       -110 |        235 |   25 |   95 |   85 |   80 |  125 |   15 | -110 | -105 |               0 | True              |
| 1751 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        200 |        -85 |        285 |  200 |  -85 |  -20 |   -5 |  -50 |  -30 |  110 |  120 |               0 | True              |
| 1752 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -410 |        410 |  -70 | -195 |  -95 | -150 | -115 | -135 | -195 | -410 |               0 | True              |
| 1753 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        210 |        -65 |        275 |  110 |  210 |  190 |   -0 |  -50 |  -65 |   40 |   45 |               0 | True              |
| 1754 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         20 |       -205 |        225 |   -0 |   20 |  -30 |    5 |   15 |  -20 | -130 | -205 |               0 | True              |
| 1755 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -420 |        420 | -165 | -120 | -230 | -120 |  -75 | -180 | -170 | -420 |               0 | True              |
| 1756 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -695 |        695 | -435 | -550 | -520 | -410 | -655 | -665 | -695 | -635 |               5 | True              |
| 1757 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        215 |        -70 |        285 |  160 |  150 |  165 |  115 |  200 |  215 |  150 |  -70 |               0 | True              |
| 1758 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -295 |        295 |  -10 |  -35 |  -20 | -260 | -225 | -265 | -290 | -295 |               0 | True              |
| 1759 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         30 |       -275 |        305 |   30 | -100 |  -80 | -230 | -180 | -275 | -245 | -265 |               0 | True              |
| 1760 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        110 |       -180 |        290 |  110 |   40 |   80 |   -5 | -135 | -120 | -180 | -105 |               0 | True              |
| 1761 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        220 |       -115 |        335 |  180 |  220 | -115 |  -80 |  -25 |   25 |    5 |   20 |               0 | True              |
| 1762 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -265 |        265 | -200 | -265 | -100 | -255 | -250 | -200 | -260 | -245 |               0 | True              |
| 1763 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -415 |        415 | -115 | -195 | -110 | -115 |  -50 | -415 | -300 | -350 |               2 | True              |
| 1764 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         70 |       -145 |        215 |   45 |   50 |   50 |   10 |   70 |   35 | -145 | -130 |               0 | True              |
| 1765 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -220 |        285 |   -5 |  -20 |   65 | -215 | -220 | -170 | -190 |  -95 |               0 | True              |
| 1766 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         90 |        -30 |        120 |  -30 |   45 |   20 |   45 |   15 |  -25 |   75 |   90 |               0 | True              |
| 1767 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        130 |       -200 |        330 |  130 |  110 |  -85 | -200 |  -25 | -165 |  -70 |  -25 |               0 | True              |
| 1768 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          5 |        -90 |         95 |  -80 |  -80 |  -35 |    5 |  -55 |  -70 |  -90 |  -45 |               0 | True              |
| 1769 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -450 |        450 | -345 | -275 | -310 | -325 | -285 | -315 | -450 | -425 |               4 | True              |
| 1770 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -360 |        360 | -325 | -305 | -355 | -360 | -355 | -290 | -325 | -285 |               2 | True              |
| 1771 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        110 |          0 |        110 |  100 |   30 |   55 |   90 |  110 |   60 |  100 |   55 |              16 | True              |
| 1772 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -625 |        625 | -525 | -455 | -575 | -625 | -540 | -500 | -570 | -465 |               4 | True              |
| 1773 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -380 |        380 | -155 | -235 | -150 | -210 | -180 | -170 | -265 | -380 |               0 | True              |
| 1774 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -525 |        525 | -460 | -470 | -480 | -490 | -510 | -525 | -470 | -420 |               3 | True              |
| 1775 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         65 |       -225 |        290 |   40 |   65 |  -10 |  -55 |   30 | -210 | -225 | -165 |               0 | True              |
| 1776 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -305 |        305 | -180 | -185 | -135 | -110 |  -85 | -185 | -305 | -245 |               0 | True              |
| 1777 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         80 |       -220 |        300 |   35 |   20 |   80 |   10 |  -85 | -220 | -145 | -125 |               0 | True              |
| 1778 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |       -120 |        190 |  -25 |  -60 |  -15 |    5 |  -10 |   35 |   70 | -120 |               0 | True              |
| 1779 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         80 |       -245 |        325 |   -0 |   10 |   10 |   80 |   40 | -230 | -245 | -185 |               1 | True              |
| 1780 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        115 |       -110 |        225 |   50 |  100 |   75 |  115 | -110 |  -90 |  -90 | -100 |               0 | True              |
| 1781 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        190 |         -5 |        195 |  130 |  145 |  190 |  160 |   95 |   25 |   -5 |    5 |               0 | True              |
| 1782 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -525 |        525 | -285 | -290 | -405 | -330 | -330 | -355 | -515 | -525 |               3 | True              |
| 1783 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |        -60 |        230 |  105 |   65 |  120 |   35 |   65 |  170 |  -60 |  -20 |               0 | True              |
| 1784 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        195 |        -60 |        255 |  195 |  135 |  165 |   50 |  125 |  125 |  -25 |  -60 |               0 | True              |
| 1785 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        140 |       -115 |        255 |  140 |   25 |  100 |  -65 | -115 |  -65 |  -60 |  -80 |               0 | True              |
| 1786 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        105 |       -220 |        325 |  -10 |  -70 |  -25 | -100 |  -15 |  -95 |  105 | -220 |               0 | True              |
| 1787 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        250 |        -35 |        285 |  150 |  250 |  115 |   35 |  -35 |   85 |   75 |  -20 |               0 | True              |
| 1788 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        270 |        -40 |        310 |  125 |  270 |   -0 |  -35 |  -40 |    5 |  -15 |  -15 |               0 | True              |
| 1789 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        215 |        -55 |        270 |  185 |  215 |   80 |   45 |  -25 |   15 |  -55 |  -20 |               0 | True              |
| 1790 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        160 |       -105 |        265 |  140 |  160 |  -90 |  -60 |  -70 | -105 |  -45 |  -15 |               0 | True              |
| 1791 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        105 |       -175 |        280 |   50 |  105 |   -5 |   95 |  -65 |  -80 | -175 | -160 |               0 | True              |
| 1792 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        380 |         -5 |        385 |  235 |  180 |  210 |  195 |  175 |  380 |  165 |   -5 |              14 | True              |
| 1793 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         -0 |       -240 |        240 |  -45 | -105 |  -90 | -120 |  -70 |  -20 |   -0 | -240 |               0 | True              |
| 1794 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -320 |        320 | -190 |  -95 | -155 | -320 | -240 | -295 | -235 | -280 |               2 | True              |
| 1795 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -135 |        235 |  100 |  -55 |  -50 | -135 |  -75 |  -80 | -115 |  -30 |               0 | True              |
| 1796 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        100 |       -160 |        260 |   50 |   65 |   -0 |  100 | -130 | -160 | -125 | -155 |               0 | True              |
| 1797 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 | -120 | -125 | -205 | -170 | -155 | -155 | -190 | -175 |               0 | True              |
| 1798 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -350 |        350 | -330 | -250 | -220 | -290 | -350 | -275 | -285 | -190 |               2 | True              |
| 1799 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         50 |       -130 |        180 |   50 | -130 | -125 |  -55 | -100 |  -75 | -100 | -100 |               0 | True              |
| 1800 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -180 |        180 | -130 |  -85 |  -90 | -180 | -140 | -115 |  -70 |  -95 |               0 | True              |
| 1801 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -510 |        510 | -345 | -375 | -300 | -390 | -350 | -400 | -415 | -510 |               2 | True              |
| 1802 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        205 |        -10 |        215 |  205 |  190 |  160 |  105 |   15 |   15 |  -10 |   20 |               0 | True              |
| 1803 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        210 |       -170 |        380 |  -25 |  165 |   90 |  210 | -170 |  -60 |  -85 |  -80 |               1 | True              |
| 1804 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        100 |       -100 |        200 |   25 |  100 |   55 |   70 | -100 |   -5 |  -75 | -100 |               0 | True              |
| 1805 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -440 |        440 | -375 | -440 | -330 | -295 | -320 | -200 | -310 | -345 |               2 | True              |
| 1806 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -650 |        650 | -530 | -560 | -500 | -635 | -650 | -620 | -565 | -580 |               4 | True              |
| 1807 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        215 |        -85 |        300 |   65 |  180 |  215 |  140 |  -15 |  -85 |  -75 |  -50 |               0 | True              |
| 1808 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -680 |        680 | -405 | -680 | -595 | -615 | -640 | -565 | -620 | -580 |               4 | True              |
| 1809 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        200 |        -85 |        285 |  -10 |  150 |   25 |  135 |  200 |  160 |  150 |  -85 |               0 | True              |
| 1810 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        160 |        -70 |        230 |  160 |  135 |  -70 |  -45 |   60 |   10 |   -0 |  -55 |               0 | True              |
| 1811 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -615 |        615 | -365 | -615 | -530 | -580 | -440 | -460 | -605 | -480 |               4 | True              |
| 1812 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         85 |       -260 |        345 |   85 | -110 |  -85 | -145 | -130 | -260 | -215 | -225 |               0 | True              |
| 1813 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        190 |          0 |        190 |  190 |  175 |  185 |  145 |   75 |  160 |  190 |   15 |               0 | True              |
| 1814 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -455 |        455 | -235 | -290 | -265 | -295 | -150 | -455 | -420 | -430 |               3 | True              |
| 1815 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        205 |       -120 |        325 |  205 |  -10 |  -10 |   90 | -120 |   15 |   25 |   35 |               0 | True              |
| 1816 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |       -150 |        265 |  115 |  -55 | -120 |  -15 | -150 |  -95 |   -0 |  -10 |               0 | True              |
| 1817 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |       -160 |        300 |   70 |  140 |  -55 |  -30 |   25 |  -40 | -160 | -100 |               0 | True              |
| 1818 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         70 |       -305 |        375 |   -5 |   60 |   70 | -195 | -150 | -305 | -170 | -195 |               1 | True              |
| 1819 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         45 |       -260 |        305 |   45 | -150 | -155 | -145 | -100 | -260 | -175 | -120 |               0 | True              |
| 1820 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |       -175 |        245 | -145 | -110 |  -35 | -175 |  -75 |   -5 |  -25 |   70 |               0 | True              |
| 1821 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |        -95 |        165 |  -40 |   70 |  -10 |   10 |  -30 |  -95 |   15 |  -50 |               0 | True              |
| 1822 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        195 |       -160 |        355 |  195 |    5 |  -15 |  -45 | -100 |   40 |   20 | -160 |               0 | True              |
| 1823 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -315 |        315 | -150 | -245 | -160 | -315 | -225 | -225 |  -80 | -240 |               0 | True              |
| 1824 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        135 |       -145 |        280 |  135 |   95 |  110 |  -55 | -110 | -140 |  -50 | -145 |               0 | True              |
| 1825 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -440 |        440 | -155 | -440 | -370 | -360 | -370 | -365 | -340 | -390 |               2 | True              |
| 1826 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -730 |        730 | -450 | -445 | -465 | -575 | -465 | -485 | -640 | -730 |               5 | True              |
| 1827 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        140 |       -145 |        285 |   65 |  140 |  120 |  125 |   35 | -100 | -125 | -145 |               0 | True              |
| 1828 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        140 |        -70 |        210 |   95 |  140 |  125 |   80 |   15 |   60 |   65 |  -70 |               0 | True              |
| 1829 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        215 |        -30 |        245 |  215 |   10 |    5 |  -10 |   10 |  -30 |    5 |  -20 |              16 | True              |
| 1830 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        195 |        -50 |        245 |  155 |  160 |  195 |  175 |  -25 |  -50 |  -20 |  -45 |               0 | True              |
| 1831 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -435 |        435 | -335 | -355 | -295 | -395 | -415 | -425 | -435 | -425 |               4 | True              |
| 1832 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        175 |       -235 |        410 |  175 |   -0 |  -60 | -235 | -150 | -105 | -225 | -210 |               0 | True              |
| 1833 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         10 |       -285 |        295 |   10 | -255 | -200 | -165 | -110 | -285 | -235 | -150 |               0 | True              |
| 1834 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -365 |        365 | -110 | -120 | -125 |  -70 |  -75 | -175 | -165 | -365 |               0 | True              |
| 1835 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -175 |        175 |  -45 |  -80 |  -55 |  -95 |  -85 |  -35 |  -70 | -175 |               0 | True              |
| 1836 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        280 |        -50 |        330 |  210 |  230 |  280 |  160 |  205 |   65 |  175 |  -50 |               0 | True              |
| 1837 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         -0 |       -135 |        135 |   -0 |   -0 |  -10 |  -45 |  -10 | -100 |  -45 | -135 |               0 | True              |
| 1838 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -580 |        580 | -405 | -370 | -480 | -575 | -525 | -570 | -560 | -580 |               4 | True              |
| 1839 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -610 |        610 | -340 | -315 | -395 | -340 | -230 | -335 | -435 | -610 |               2 | True              |
| 1840 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -690 |        690 | -370 | -450 | -415 | -405 | -500 | -480 | -375 | -690 |               2 | True              |
| 1841 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        135 |       -240 |        375 |   10 |  -40 |   90 |  135 | -115 | -240 | -205 | -225 |               0 | True              |
| 1842 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -690 |        690 | -490 | -500 | -470 | -500 | -500 | -495 | -540 | -690 |               3 | True              |
| 1843 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        185 |       -135 |        320 |   85 |  115 |  185 |  160 |  125 |   75 |  110 | -135 |               0 | True              |
| 1844 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |        -55 |        245 |   65 |  190 |  155 |  165 |  130 |   50 |  -55 |  -25 |               0 | True              |
| 1845 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        255 |        -35 |        290 |  160 |  255 |   30 |   10 |  -35 |  105 |   70 |   -5 |               0 | True              |
| 1846 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -550 |        550 | -280 | -190 | -265 | -240 | -270 | -265 | -420 | -550 |               3 | True              |
| 1847 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |       -170 |        200 |   30 | -170 | -155 | -130 |  -80 | -110 | -145 | -150 |               0 | True              |
| 1848 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |        -90 |        210 |  100 |   30 |   15 |  120 |   40 |   50 |   10 |  -90 |               0 | True              |
| 1849 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        130 |       -220 |        350 |   40 |   20 |  -70 |  130 |   25 | -125 | -110 | -220 |               0 | True              |
| 1850 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -405 |        405 | -190 | -175 | -160 | -225 | -160 | -175 | -365 | -405 |               3 | True              |
| 1851 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -465 |        465 | -280 | -180 | -465 | -415 | -365 | -350 | -405 | -415 |               3 | True              |
| 1852 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        270 |        -15 |        285 |  205 |  160 |  170 |  270 |   45 |   60 |   25 |  -15 |               0 | True              |
| 1853 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        320 |        -50 |        370 |  125 |  320 |  240 |  180 |  165 |  -45 |  -50 |   45 |               0 | True              |
| 1854 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         95 |       -160 |        255 |   35 |    5 |  -60 |   45 |   20 |   95 | -160 | -115 |               0 | True              |
| 1855 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        165 |       -175 |        340 |  165 |   80 |   50 |   -5 | -120 | -160 | -170 | -175 |               0 | True              |
| 1856 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        220 |        -85 |        305 |  130 |  220 |  205 |  190 |   85 |  165 |  195 |  -85 |               0 | True              |
| 1857 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        140 |       -305 |        445 |  140 | -180 | -195 | -250 | -135 | -240 | -305 | -110 |               0 | True              |
| 1858 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        270 |          0 |        270 |  160 |  270 |  240 |  245 |  230 |  230 |   75 |    5 |               0 | True              |
| 1859 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         70 |       -165 |        235 |   70 |   50 |   70 |   70 | -165 | -115 | -110 | -125 |               0 | True              |
| 1860 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        215 |          0 |        215 |   95 |  215 |  105 |  130 |  140 |  205 |  145 |    5 |               0 | True              |
| 1861 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        255 |          0 |        255 |  185 |  255 |  165 |  120 |   40 |    5 |   50 |   50 |               0 | True              |
| 1862 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         75 |       -210 |        285 |   75 |   35 |  -50 | -160 | -210 | -200 | -110 | -210 |               0 | True              |
| 1863 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        135 |       -205 |        340 |  135 |   35 | -110 | -170 | -205 | -125 | -110 | -130 |               0 | True              |
| 1864 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        130 |       -155 |        285 |  130 |  100 |  120 | -100 |   85 |  -40 | -155 |  -80 |               0 | True              |
| 1865 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        135 |        -70 |        205 |  -40 |    5 |  105 |  135 |   35 |   85 |    5 |  -70 |               0 | True              |
| 1866 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -535 |        535 | -390 | -390 | -375 | -535 | -425 | -465 | -490 | -410 |               3 | True              |
| 1867 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        100 |       -205 |        305 |  100 |   95 |   40 |  -95 | -160 | -205 | -150 | -160 |               0 | True              |
| 1868 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        150 |       -100 |        250 |  110 |  150 | -100 |  -45 |  -75 |    5 | -100 |  -25 |               0 | True              |
| 1869 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         10 |       -105 |        115 |   10 |  -60 | -105 |  -85 |  -85 |  -75 |  -55 |  -55 |               0 | True              |
| 1870 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         -0 |       -120 |        120 |  -85 | -120 | -105 |  -55 | -115 |   -0 |  -55 |   -0 |               0 | True              |
| 1871 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         75 |       -120 |        195 |   75 |   55 |   -0 |   -5 |    5 |   15 |  -25 | -120 |               0 | True              |
| 1872 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |        -70 |        240 |  170 |  100 |  105 |   70 |  110 |  165 |    5 |  -70 |               0 | True              |
| 1873 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        145 |        -70 |        215 |   90 |  110 |  145 |  -70 |  -65 |   25 |   -5 |  -30 |               0 | True              |
| 1874 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -325 |        325 |  -20 |  -60 | -165 | -325 | -295 | -215 | -290 | -295 |               1 | True              |
| 1875 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        125 |       -160 |        285 |   45 |  110 |   15 |  125 |   50 |  -90 |  -10 | -160 |               0 | True              |
| 1876 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |       -260 |        310 |  -25 |   50 | -240 | -245 | -260 | -240 | -200 | -205 |               1 | True              |
| 1877 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -235 |        235 | -150 | -175 | -215 | -235 | -195 | -150 |  -90 | -215 |               0 | True              |
| 1878 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        150 |        -85 |        235 |  150 |  -35 |  -85 |   -5 |  -10 |   10 |  -25 |  -45 |               0 | True              |
| 1879 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        340 |          0 |        340 |  210 |  285 |  285 |  340 |  275 |  305 |   20 |   15 |               0 | True              |
| 1880 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        200 |       -145 |        345 |  200 |  195 |  110 |  120 | -140 | -145 |  -55 | -135 |               0 | True              |
| 1881 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -545 |        545 | -325 | -295 | -305 | -305 | -270 | -255 | -520 | -545 |               4 | True              |
| 1882 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -625 |        625 | -310 | -425 | -570 | -580 | -605 | -450 | -475 | -625 |               3 | True              |
| 1883 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         65 |       -245 |        310 |  -30 |   65 |  -50 | -215 | -215 | -220 | -245 | -215 |               1 | True              |
| 1884 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         35 |       -335 |        370 |   35 | -100 | -270 | -315 | -280 | -325 | -335 | -195 |               0 | True              |
| 1885 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        145 |        -80 |        225 |  145 |   95 |  130 |  100 |   70 |  -80 |   10 |  -75 |               0 | True              |
| 1886 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        215 |        -85 |        300 |   95 |   85 |   95 |  215 |  -55 |  -30 |  -85 |  -10 |               0 | True              |
| 1887 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        240 |        -80 |        320 |  100 |  240 |  160 |  100 |  105 |  -20 |   25 |  -80 |               0 | True              |
| 1888 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -590 |        590 | -340 | -305 | -590 | -490 | -590 | -530 | -550 | -505 |               4 | True              |
| 1889 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        125 |       -165 |        290 |  125 | -100 |  -10 |  -60 | -105 | -165 | -130 | -135 |               0 | True              |
| 1890 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         90 |       -175 |        265 |   25 |   15 |   40 |   90 |   20 |   85 | -165 | -175 |               0 | True              |
| 1891 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         75 |       -180 |        255 |   40 |   75 |   75 |  -70 |   20 | -180 | -155 | -165 |               0 | True              |
| 1892 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        245 |        -55 |        300 |  135 |  240 |  170 |  245 |  -25 |  -40 |  -55 |  -35 |               0 | True              |
| 1893 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |       -230 |        260 |   30 | -190 | -175 | -230 | -195 |  -85 | -150 | -195 |               0 | True              |
| 1894 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        125 |       -105 |        230 |   75 |  125 |   65 | -105 |  -10 |  -50 |   80 |  -80 |               0 | True              |
| 1895 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        195 |       -185 |        380 |  105 |  195 |  110 |  -30 |   20 | -185 | -150 |  -85 |               0 | True              |
| 1896 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -495 |        495 | -260 | -225 | -195 | -305 | -300 | -430 | -495 | -480 |               3 | True              |
| 1897 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -770 |        770 | -500 | -680 | -770 | -625 | -715 | -730 | -660 | -715 |               5 | True              |
| 1898 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        250 |       -180 |        430 |  170 |  250 |    5 |   35 |   55 |  -40 |  -20 | -180 |               0 | True              |
| 1899 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -240 |        240 | -150 | -240 |  -40 |  -50 | -150 | -155 |  -75 | -150 |               0 | True              |
| 1900 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        350 |          0 |        350 |  315 |  310 |  250 |  295 |  160 |  200 |  350 |   55 |              13 | True              |
| 1901 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         65 |       -340 |        405 |   55 |  -60 |   20 |   50 |   65 | -340 | -260 | -240 |               0 | True              |
| 1902 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -385 |        385 | -150 | -295 | -215 | -220 | -210 | -210 | -245 | -385 |               0 | True              |
| 1903 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -315 |        315 |  -35 | -160 |  -45 | -180 | -140 | -305 | -315 | -305 |               1 | True              |
| 1904 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -310 |        310 | -245 | -175 | -250 | -175 | -310 | -305 | -310 | -215 |               1 | True              |
| 1905 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        130 |       -190 |        320 |  120 |  120 |  -15 |   75 |  120 |  130 | -190 | -100 |               0 | True              |
| 1906 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         20 |       -395 |        415 |   20 | -305 | -300 | -305 | -395 | -245 | -320 | -325 |               0 | True              |
| 1907 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -750 |        750 | -590 | -750 | -655 | -550 | -655 | -680 | -630 | -650 |               4 | True              |
| 1908 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -175 |        175 | -175 | -160 | -160 | -155 | -175 |  -55 |  -90 | -150 |               0 | True              |
| 1909 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        190 |        -55 |        245 |  160 |  165 |  190 |  -55 |  -20 |  -10 |  -40 |  -30 |               0 | True              |
| 1910 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        115 |       -105 |        220 |  115 |  -80 |  -55 |  -25 |  -15 | -105 |  -60 |  -95 |               0 | True              |
| 1911 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -670 |        670 | -405 | -495 | -420 | -670 | -600 | -590 | -600 | -625 |               5 | True              |
| 1912 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -125 |        225 |  100 |  -55 | -125 |  -85 |  -55 | -125 |  -30 | -100 |               0 | True              |
| 1913 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         60 |       -190 |        250 |   60 |   60 | -175 | -100 | -130 | -180 | -190 |  -60 |               0 | True              |
| 1914 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         25 |       -245 |        270 |   25 | -100 | -100 |  -25 |   10 |  -90 | -110 | -245 |               0 | True              |
| 1915 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        190 |       -110 |        300 |   55 |  120 |  190 |  -45 | -110 |  -80 |  -50 |  -60 |               0 | True              |
| 1916 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -610 |        610 | -400 | -435 | -335 | -420 | -610 | -555 | -575 | -585 |               4 | True              |
| 1917 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -515 |        515 | -405 | -305 | -360 | -435 | -365 | -500 | -515 | -505 |               5 | True              |
| 1918 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        165 |        -20 |        185 |  165 |  160 |  140 |  -10 |  155 |  100 |  150 |  -20 |               0 | True              |
| 1919 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -790 |        790 | -600 | -500 | -780 | -695 | -790 | -770 | -735 | -670 |               6 | True              |
| 1920 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -310 |        310 | -265 | -210 | -235 | -205 | -310 | -220 | -260 | -200 |               1 | True              |
| 1921 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -205 |        205 | -100 |  -50 | -115 |  -60 | -190 | -130 | -205 | -140 |               0 | True              |
| 1922 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         70 |       -225 |        295 |   60 |   70 |   20 |   20 |   15 | -180 | -225 |   15 |               0 | True              |
| 1923 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -330 |        330 | -175 | -190 |  -95 | -120 | -135 | -210 | -175 | -330 |               0 | True              |
| 1924 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         65 |       -220 |        285 |   45 |   65 |  -35 |  -35 |   65 | -170 | -220 | -195 |               0 | True              |
| 1925 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -110 |        110 |  -15 |  -80 | -110 |  -20 |  -45 |  -45 |  -95 |  -45 |               0 | True              |
| 1926 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         90 |       -150 |        240 |   90 |   45 |   70 | -120 | -150 | -135 |  -65 | -100 |               0 | True              |
| 1927 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -430 |        430 | -140 | -210 | -240 | -165 | -200 | -200 | -160 | -430 |               0 | True              |
| 1928 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -775 |        775 | -585 | -595 | -665 | -615 | -710 | -635 | -775 | -775 |               6 | True              |
| 1929 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |       -270 |        450 |   65 |  -35 |  180 |   30 |  -45 |   20 | -150 | -270 |               0 | True              |
| 1930 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         65 |       -260 |        325 |   65 |  -70 | -195 | -140 | -245 | -260 | -130 | -145 |               0 | True              |
| 1931 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |        -55 |        215 |  130 |  160 |  160 |   10 |   50 |   10 |  -55 |  -45 |               0 | True              |
| 1932 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -520 |        520 | -240 | -475 | -520 | -490 | -500 | -390 | -370 | -455 |               3 | True              |
| 1933 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |        -50 |        200 |   65 |   85 |   35 |  150 |   75 |  145 |  110 |  -50 |               0 | True              |
| 1934 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        310 |          0 |        310 |  175 |  310 |  220 |  180 |  235 |  230 |   45 |   15 |               0 | True              |
| 1935 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -195 |        195 | -155 | -140 | -145 | -180 | -195 | -130 | -145 | -175 |               0 | True              |
| 1936 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |       -200 |        240 | -200 |   40 | -110 |  -80 |  -10 | -120 |  -50 |   -5 |               0 | True              |
| 1937 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        145 |       -195 |        340 |  145 | -195 | -130 | -125 | -135 | -185 | -105 |  -95 |               0 | True              |
| 1938 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        225 |       -105 |        330 |  195 |  225 |   -0 |   15 | -105 |   45 |  -80 |  -25 |               0 | True              |
| 1939 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -580 |        580 | -465 | -535 | -360 | -550 | -430 | -520 | -460 | -580 |               3 | True              |
| 1940 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -475 |        475 | -215 | -445 | -475 | -395 | -350 | -375 | -420 | -350 |               3 | True              |
| 1941 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        105 |       -185 |        290 |   75 |  105 |   95 | -130 |  -35 |  -85 | -100 | -185 |               0 | True              |
| 1942 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        195 |       -160 |        355 |  195 |  170 |  155 |  -55 | -130 |  -80 | -160 | -135 |               0 | True              |
| 1943 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        160 |       -170 |        330 |  100 |   65 |  160 |  130 |  110 | -120 |  -40 | -170 |               0 | True              |
| 1944 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        245 |       -160 |        405 |  170 |  175 |  245 | -125 |  -10 | -160 |  -30 |  -65 |               0 | True              |
| 1945 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         60 |       -350 |        410 |  -45 |   50 |  -75 |   60 |  -55 | -125 |   20 | -350 |               0 | True              |
| 1946 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         85 |       -245 |        330 |   85 |  -50 |  -10 |  -50 | -190 | -245 | -180 | -215 |               0 | True              |
| 1947 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |       -155 |        345 |  190 |   40 |   95 |   65 |  115 | -120 | -155 |  -60 |               0 | True              |
| 1948 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -255 |        255 |  -45 |  -10 | -100 | -170 |  -45 |  -70 |  -80 | -255 |               0 | True              |
| 1949 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        210 |        -30 |        240 |   25 |  100 |  200 |  140 |  180 |  100 |  210 |  -30 |               0 | True              |
| 1950 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        190 |        -55 |        245 |  190 |  -35 |  -30 |  -10 |   15 |   20 |   10 |  -55 |               0 | True              |
| 1951 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -225 |        225 | -155 |  -60 | -140 | -140 | -225 | -185 | -115 | -160 |               0 | True              |
| 1952 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -755 |        755 | -455 | -465 | -415 | -440 | -535 | -505 | -455 | -755 |               3 | True              |
| 1953 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        305 |          0 |        305 |  145 |  215 |  270 |  185 |  305 |   70 |  215 |   75 |               0 | True              |
| 1954 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -400 |        400 | -180 | -140 | -155 | -125 | -115 | -215 | -100 | -400 |               0 | True              |
| 1955 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        225 |       -120 |        345 |  160 |  225 | -120 |  -10 |  -90 |   -5 |  -45 |  -85 |               0 | True              |
| 1956 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          5 |       -225 |        230 |  -90 | -170 | -135 |    5 | -170 |  -95 | -140 | -225 |               0 | True              |
| 1957 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        265 |        -40 |        305 |  185 |  235 |  265 |   90 |  135 |  215 |  -40 |   70 |               0 | True              |
| 1958 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        210 |       -135 |        345 |   95 |  205 |  210 |  -45 |  -65 | -135 | -135 | -130 |               0 | True              |
| 1959 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -220 |        220 | -190 |  -30 | -120 |  -60 | -135 | -105 | -135 | -220 |               0 | True              |
| 1960 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -145 |        145 |   -5 |  -20 |  -15 |  -25 |  -25 |  -95 |  -40 | -145 |               0 | True              |
| 1961 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         25 |       -300 |        325 |   25 |  -65 | -185 | -225 | -300 | -275 | -255 | -185 |               0 | True              |
| 1962 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -310 |        310 |  -35 |  -30 |  -20 |  -10 | -310 | -310 | -245 | -305 |               1 | True              |
| 1963 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |       -195 |        295 |  100 |  -90 |  -80 | -195 |  -80 | -125 |  -85 |  -20 |               0 | True              |
| 1964 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -720 |        720 | -530 | -660 | -660 | -675 | -555 | -720 | -660 | -640 |               5 | True              |
| 1965 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        110 |       -215 |        325 |   55 |  105 |   45 |   40 |  110 |   85 |    5 | -215 |               0 | True              |
| 1966 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -515 |        515 | -285 | -275 | -245 | -240 | -240 | -150 | -450 | -515 |               3 | True              |
| 1967 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -730 |        730 | -375 | -385 | -420 | -460 | -345 | -350 | -480 | -730 |               2 | True              |
| 1968 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |          0 |       -535 |        535 | -340 | -515 | -480 | -450 | -440 | -490 | -535 | -460 |               4 | True              |
| 1969 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        110 |       -165 |        275 |   90 |  110 | -100 | -110 | -165 |  -50 |  -75 |  -90 |               0 | True              |
| 1970 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |       -140 |        260 |   70 |  120 | -140 |  -80 | -140 |  -60 | -135 | -120 |               0 | True              |
| 1971 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        290 |       -135 |        425 |  205 |  290 |  135 |  175 |  210 | -135 |  -50 |  -15 |               0 | True              |
| 1972 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         15 |       -290 |        305 |   15 | -215 | -205 | -215 | -290 | -145 | -130 | -245 |               0 | True              |
| 1973 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -625 |        625 | -500 | -550 | -535 | -625 | -485 | -575 | -490 | -545 |               4 | True              |
| 1974 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -120 |        265 |  135 |  125 |  125 |   90 |  145 |  120 | -120 |  -20 |               0 | True              |
| 1975 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -465 |        465 | -135 | -200 | -125 | -255 | -200 | -205 | -115 | -465 |               0 | True              |
| 1976 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -150 |        170 |  -20 |  -75 |  -55 |  -50 |    5 |  -40 |   20 | -150 |               0 | True              |
| 1977 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        200 |        -45 |        245 |  200 |   85 |  160 |  145 |  135 |  -30 |  -30 |  -45 |               0 | True              |
| 1978 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        115 |       -130 |        245 |   35 |  -50 |   20 |  -65 |  115 |  -55 |  -15 | -130 |               0 | True              |
| 1979 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -245 |        275 |   -0 |   30 |  -85 | -150 | -195 | -120 | -225 | -245 |               0 | True              |
| 1980 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        190 |        -30 |        220 |  190 |  -30 |  -30 |   25 |    5 |   -5 |    5 |   35 |               0 | True              |
| 1981 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        270 |          0 |        270 |  270 |  100 |  105 |   85 |   55 |   15 |   25 |   70 |              16 | True              |
| 1982 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        170 |       -195 |        365 |  170 |   30 |  155 |   55 | -125 | -195 | -115 | -170 |               0 | True              |
| 1983 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        190 |       -120 |        310 |  165 |  190 |   70 |  120 |  -20 |  -95 | -120 |   -5 |               0 | True              |
| 1984 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         10 |       -350 |        360 | -135 |   10 |  -20 |  -95 | -310 | -350 | -205 | -335 |               2 | True              |
| 1985 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        145 |       -350 |        495 |  145 |  -70 |  115 | -200 | -190 | -210 | -255 | -350 |               0 | True              |
| 1986 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         35 |       -280 |        315 |   25 |   35 | -280 | -230 | -275 | -220 | -245 | -200 |               0 | True              |
| 1987 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -685 |        685 | -635 | -510 | -660 | -615 | -485 | -605 | -585 | -685 |               5 | True              |
| 1988 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        185 |        -75 |        260 |   65 |  185 |  -50 |  -75 |  -10 |   45 |  -30 |  -30 |               0 | True              |
| 1989 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -445 |        445 | -145 |  -95 | -315 | -135 | -205 | -445 | -435 | -390 |               2 | True              |
| 1990 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -465 |        465 | -275 | -200 | -220 | -225 | -365 | -390 | -435 | -465 |               3 | True              |
| 1991 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         55 |       -200 |        255 |   55 |  -20 |    5 |   -0 | -145 | -200 | -200 | -185 |               0 | True              |
| 1992 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        305 |          0 |        305 |  200 |  165 |  235 |  305 |  115 |   70 |   80 |   10 |               0 | True              |
| 1993 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -400 |        400 | -175 | -180 | -165 | -215 | -275 | -330 | -400 | -370 |               2 | True              |
| 1994 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         40 |       -265 |        305 |   -0 |   40 | -235 | -265 | -230 | -255 | -250 | -150 |               0 | True              |
| 1995 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -435 |        435 | -175 | -265 | -260 | -195 | -215 | -165 | -170 | -435 |               0 | True              |
| 1996 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        205 |       -210 |        415 |   15 |  100 |   90 |  205 | -210 |  -45 |  -40 |  -75 |               0 | True              |
| 1997 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |       -150 |        300 |  140 |   50 |  150 |  115 |  -35 | -150 |  -45 |  -95 |               0 | True              |
| 1998 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |        -55 |        215 |   90 |  135 |  140 |   15 |   80 |  160 |  -55 |   25 |               0 | True              |
| 1999 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        145 |       -150 |        295 |   80 |  125 |  -10 |  105 |  105 |  145 |  -65 | -150 |               0 | True              |
| 2000 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        200 |       -110 |        310 |  200 |  155 |  190 |  175 |   15 |   50 |   10 | -110 |               0 | True              |

# CRR results

|      | rings_are_locked_to      | mrm_crr_equivalent   |   max_cool |   max_heat |   max_diff |     0 |     1 |     2 |     3 |     4 |     5 |     6 |     7 |
|-----:|:-------------------------|:---------------------|-----------:|-----------:|-----------:|------:|------:|------:|------:|------:|------:|------:|------:|
|    0 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        245 |        -10 |        255 |   175 |   135 |   140 |   195 |   240 |   245 |    95 |   -10 |
|    1 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        265 |        -60 |        325 |   190 |   170 |   185 |   265 |   165 |   -60 |   -35 |     5 |
|    2 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -440 |        440 |  -190 |  -270 |  -220 |  -140 |  -440 |  -430 |  -440 |  -425 |
|    3 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -990 |        990 |  -830 |  -890 |  -830 |  -830 |  -780 |  -860 |  -760 |  -990 |
|    4 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -925 |        925 |  -640 |  -835 |  -790 |  -925 |  -890 |  -910 |  -855 |  -810 |
|    5 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        325 |        -35 |        360 |   325 |   225 |   185 |   220 |   200 |    55 |    30 |   -35 |
|    6 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1030 |       1030 |  -810 |  -885 |  -895 |  -825 |  -900 |  -850 |  -925 | -1030 |
|    7 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        215 |       -105 |        320 |   120 |   105 |   210 |   190 |   110 |   215 |  -105 |   -45 |
|    8 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        235 |        -80 |        315 |   225 |   235 |    -5 |    90 |    10 |     5 |    25 |   -80 |
|    9 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        265 |       -125 |        390 |   265 |   130 |   -90 |  -120 |  -125 |   -50 |   -25 |   -50 |
|   10 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -895 |        895 |  -645 |  -735 |  -835 |  -825 |  -895 |  -890 |  -830 |  -770 |
|   11 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -445 |        445 |  -425 |  -235 |  -445 |  -280 |  -265 |  -260 |  -235 |  -440 |
|   12 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1000 |       1000 |  -700 |  -830 |  -845 |  -790 |  -740 | -1000 |  -930 |  -955 |
|   13 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -425 |        425 |  -250 |  -235 |  -185 |  -175 |  -190 |   -80 |  -225 |  -425 |
|   14 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        375 |          0 |        375 |   355 |   375 |   335 |   195 |   110 |   240 |   150 |   225 |
|   15 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1090 |       1090 |  -820 |  -755 |  -765 |  -745 |  -770 |  -840 | -1090 |  -915 |
|   16 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -355 |        355 |  -110 |  -275 |  -285 |  -155 |  -110 |  -205 |  -175 |  -355 |
|   17 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -250 |        250 |  -155 |  -140 |  -135 |  -100 |   -70 |  -155 |   -50 |  -250 |
|   18 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1050 |       1050 |  -770 |  -740 |  -785 |  -825 | -1025 | -1050 | -1020 |  -970 |
|   19 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -250 |        250 |   -15 |   -30 |   -50 |    -5 |   -40 |   -55 |   -85 |  -250 |
|   20 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1180 |       1180 | -1160 | -1145 | -1140 | -1090 | -1065 | -1140 | -1030 | -1180 |
|   21 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        400 |          0 |        400 |   400 |   300 |   280 |   220 |   240 |   285 |   325 |   140 |
|   22 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        110 |       -275 |        385 |    70 |   110 |    45 |    60 |   100 |  -155 |  -275 |  -205 |
|   23 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        375 |          0 |        375 |   330 |   330 |   375 |   285 |   330 |    75 |   195 |   185 |
|   24 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -570 |        570 |  -285 |  -360 |  -375 |  -330 |  -370 |  -420 |  -415 |  -570 |
|   25 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -265 |        265 |   -95 |   -85 |   -70 |   -30 |  -260 |  -265 |  -235 |  -265 |
|   26 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        360 |          0 |        360 |   360 |   195 |   295 |   140 |   170 |   170 |   285 |   160 |
|   27 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -770 |        770 |  -535 |  -615 |  -485 |  -625 |  -695 |  -620 |  -500 |  -770 |
|   28 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        235 |       -115 |        350 |   170 |   235 |   -55 |   -95 |  -115 |  -100 |   -65 |   -60 |
|   29 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        230 |        -30 |        260 |    95 |   145 |   230 |   185 |   200 |   215 |   -30 |    30 |
|   30 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -435 |        435 |  -240 |  -215 |  -195 |  -225 |  -140 |  -130 |  -435 |  -335 |
|   31 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        255 |        -10 |        265 |   255 |    60 |   135 |   110 |   140 |   120 |   140 |   -10 |
|   32 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         35 |       -245 |        280 |   -75 |   -20 |   -85 |    20 |     5 |   -55 |    35 |  -245 |
|   33 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -965 |        965 |  -725 |  -780 |  -755 |  -770 |  -800 |  -730 |  -745 |  -965 |
|   34 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -480 |        480 |  -220 |  -215 |  -335 |  -480 |  -470 |  -470 |  -445 |  -395 |
|   35 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -580 |        580 |  -280 |  -375 |  -310 |  -405 |  -300 |  -315 |  -490 |  -580 |
|   36 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        180 |       -185 |        365 |    35 |   180 |  -120 |  -185 |   -65 |  -140 |  -100 |  -150 |
|   37 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        305 |        -20 |        325 |   295 |   305 |   255 |   210 |   -20 |   100 |   115 |   140 |
|   38 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -260 |        260 |   -10 |   -25 |  -105 |   -25 |  -260 |  -255 |  -230 |  -260 |
|   39 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -720 |        720 |  -605 |  -605 |  -645 |  -585 |  -410 |  -495 |  -575 |  -720 |
|   40 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        135 |        -60 |        195 |    50 |    90 |    35 |   -10 |   135 |   -60 |   -10 |    15 |
|   41 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -790 |        790 |  -525 |  -540 |  -745 |  -750 |  -785 |  -790 |  -725 |  -785 |
|   42 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        250 |        -65 |        315 |   250 |   175 |   215 |   -65 |   -35 |   -30 |   -50 |   -35 |
|   43 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        350 |          0 |        350 |   330 |   230 |   280 |   265 |   350 |   310 |   350 |   295 |
|   44 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1085 |       1085 |  -735 |  -770 |  -780 |  -735 |  -840 |  -885 | -1025 | -1085 |
|   45 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -765 |        765 |  -515 |  -480 |  -575 |  -525 |  -480 |  -480 |  -525 |  -765 |
|   46 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -740 |        740 |  -465 |  -380 |  -590 |  -715 |  -740 |  -580 |  -690 |  -710 |
|   47 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -330 |        330 |  -145 |   -55 |  -105 |   -95 |   -20 |  -205 |  -285 |  -330 |
|   48 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        355 |          0 |        355 |   290 |   335 |   290 |   355 |   290 |   350 |   215 |   185 |
|   49 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        350 |          0 |        350 |   340 |   265 |   255 |   260 |   300 |   240 |   350 |   320 |
|   50 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        175 |        -80 |        255 |   160 |   170 |   130 |   100 |   160 |   150 |   175 |   -80 |
|   51 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -565 |        565 |  -185 |  -430 |  -465 |  -565 |  -435 |  -450 |  -490 |  -450 |
|   52 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        350 |          0 |        350 |   275 |   350 |   325 |   155 |   125 |   190 |    80 |   130 |
|   53 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1060 |       1060 |  -765 |  -720 |  -790 |  -795 |  -775 |  -685 |  -805 | -1060 |
|   54 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -895 |        895 |  -610 |  -600 |  -620 |  -670 |  -845 |  -810 |  -895 |  -810 |
|   55 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -750 |        750 |  -400 |  -675 |  -725 |  -750 |  -710 |  -670 |  -635 |  -685 |
|   56 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        320 |          0 |        320 |   320 |   285 |    55 |   185 |   110 |    75 |   145 |   160 |
|   57 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1030 |       1030 |  -785 |  -825 |  -790 |  -775 |  -620 | -1030 | -1000 |  -935 |
|   58 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        365 |          0 |        365 |   355 |   365 |   340 |   170 |   210 |   150 |   170 |   175 |
|   59 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        165 |       -215 |        380 |   165 |    70 |    95 |    15 |    85 |    25 |  -215 |  -180 |
|   60 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        270 |          0 |        270 |   245 |   270 |   185 |   190 |   245 |    25 |   130 |   180 |
|   61 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -325 |        325 |   -90 |   -25 |  -295 |  -325 |  -325 |  -310 |  -205 |  -215 |
|   62 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        215 |       -100 |        315 |   215 |   190 |   150 |   -60 |   -65 |   -55 |   -80 |  -100 |
|   63 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        375 |      -1195 |       1570 | -1195 | -1180 | -1155 | -1180 | -1140 | -1130 |   375 |   370 |
|   64 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -480 |        480 |  -145 |  -235 |  -450 |  -345 |  -480 |  -355 |  -415 |  -425 |
|   65 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        105 |       -175 |        280 |    45 |    55 |   105 |   100 |  -135 |  -140 |  -120 |  -175 |
|   66 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1115 |       1115 |  -860 |  -845 | -1020 | -1000 | -1115 | -1050 | -1060 | -1075 |
|   67 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -710 |        710 |  -465 |  -510 |  -530 |  -440 |  -655 |  -710 |  -695 |  -690 |
|   68 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1020 |       1020 |  -745 |  -740 |  -995 |  -935 |  -985 |  -990 |  -970 | -1020 |
|   69 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -280 |        280 |   -35 |   -10 |   -50 |  -190 |  -225 |  -280 |  -220 |  -205 |
|   70 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -610 |        610 |  -340 |  -435 |  -505 |  -585 |  -610 |  -485 |  -520 |  -545 |
|   71 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1125 |       1125 |  -940 |  -930 |  -915 |  -905 |  -940 | -1105 | -1125 | -1110 |
|   72 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        215 |        -55 |        270 |   215 |   130 |   110 |    55 |    65 |   150 |   150 |   -55 |
|   73 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -470 |        470 |  -200 |   -70 |  -340 |  -400 |  -335 |  -470 |  -395 |  -350 |
|   74 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -665 |        665 |  -460 |  -565 |  -440 |  -460 |  -395 |  -665 |  -640 |  -560 |
|   75 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -550 |        550 |  -365 |  -350 |  -240 |  -280 |  -475 |  -530 |  -550 |  -445 |
|   76 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -990 |        990 |  -685 |  -755 |  -760 |  -745 |  -770 |  -720 |  -760 |  -990 |
|   77 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        280 |          0 |        280 |   210 |   175 |   280 |   165 |   210 |   215 |    15 |    20 |
|   78 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1025 |       1025 |  -785 |  -730 |  -735 |  -790 |  -960 |  -910 |  -905 | -1025 |
|   79 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        220 |        -90 |        310 |   220 |    60 |   -90 |    -5 |   -70 |    40 |    55 |    -0 |
|   80 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -640 |        640 |  -330 |  -405 |  -335 |  -360 |  -360 |  -345 |  -340 |  -640 |
|   81 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        340 |          0 |        340 |   340 |   240 |   195 |   275 |   225 |   325 |   245 |    55 |
|   82 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1070 |       1070 |  -900 |  -925 |  -915 |  -950 |  -815 |  -960 |  -920 | -1070 |
|   83 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -620 |        620 |  -365 |  -395 |  -270 |  -255 |  -440 |  -520 |  -620 |  -600 |
|   84 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        200 |       -115 |        315 |   165 |   155 |   115 |   105 |   195 |   200 |   115 |  -115 |
|   85 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        365 |          0 |        365 |   265 |   365 |   270 |   265 |   175 |    35 |    45 |   120 |
|   86 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -680 |        680 |  -325 |  -485 |  -680 |  -545 |  -675 |  -650 |  -600 |  -560 |
|   87 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        325 |          0 |        325 |   230 |   295 |   175 |   325 |   230 |    25 |    45 |   105 |
|   88 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        275 |          0 |        275 |   275 |   150 |   125 |   130 |   155 |   260 |   185 |   140 |
|   89 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -415 |        415 |  -345 |  -370 |  -340 |  -345 |  -270 |  -300 |  -415 |  -335 |
|   90 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         -0 |       -330 |        330 |    -5 |    -0 |  -310 |  -330 |  -280 |  -295 |  -265 |  -190 |
|   91 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        235 |        -75 |        310 |   130 |   235 |   185 |    -0 |   -40 |   -75 |   -40 |   -70 |
|   92 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        390 |          0 |        390 |   390 |   200 |   125 |   200 |   220 |   220 |   190 |   115 |
|   93 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1145 |       1145 | -1140 | -1050 | -1025 | -1115 | -1080 | -1145 | -1095 | -1145 |
|   94 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -520 |        520 |  -250 |  -235 |  -230 |  -190 |  -135 |  -210 |  -440 |  -520 |
|   95 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1075 |       1075 |  -835 |  -805 |  -875 |  -850 |  -900 | -1065 | -1055 | -1075 |
|   96 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -955 |        955 |  -705 |  -620 |  -710 |  -890 |  -840 |  -835 |  -955 |  -910 |
|   97 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1115 |       1115 | -1045 | -1115 | -1095 |  -970 | -1115 | -1085 | -1015 | -1035 |
|   98 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -850 |        850 |  -675 |  -720 |  -610 |  -580 |  -660 |  -850 |  -850 |  -725 |
|   99 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        130 |       -150 |        280 |    80 |     5 |    60 |    85 |   130 |    -0 |    10 |  -150 |
|  100 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1125 |       1125 |  -840 |  -850 |  -825 |  -905 |  -975 |  -915 |  -880 | -1125 |
|  101 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -550 |        550 |  -300 |  -275 |  -280 |  -255 |  -305 |  -465 |  -550 |  -420 |
|  102 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -580 |        580 |  -510 |  -430 |  -510 |  -445 |  -460 |  -420 |  -580 |  -575 |
|  103 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -360 |        360 |  -150 |  -230 |  -165 |  -230 |  -290 |  -245 |  -140 |  -360 |
|  104 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -510 |        510 |  -260 |  -220 |  -235 |  -460 |  -430 |  -510 |  -465 |  -400 |
|  105 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        225 |        -15 |        240 |   215 |   155 |   180 |   225 |   160 |   105 |   175 |   -15 |
|  106 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -890 |        890 |  -555 |  -885 |  -835 |  -820 |  -890 |  -875 |  -870 |  -755 |
|  107 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -470 |        470 |  -290 |  -350 |  -145 |  -260 |  -200 |  -260 |  -470 |  -400 |
|  108 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |        -50 |        310 |   225 |   260 |   215 |   -10 |   -50 |    10 |    15 |   -10 |
|  109 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        215 |        -25 |        240 |   190 |   155 |   215 |   160 |   -20 |   -25 |     5 |   -10 |
|  110 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -450 |        450 |  -170 |  -345 |  -420 |  -430 |  -410 |  -370 |  -375 |  -450 |
|  111 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        280 |          0 |        280 |   280 |   210 |   225 |   210 |   215 |   260 |   175 |    95 |
|  112 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1185 |       1185 |  -855 |  -930 |  -985 |  -920 | -1170 | -1185 | -1140 | -1085 |
|  113 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -270 |        270 |   -25 |   -25 |   -75 |   -15 |   -45 |  -200 |  -235 |  -270 |
|  114 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         20 |       -235 |        255 |   -20 |    10 |   -90 |    -0 |   -55 |    20 |  -100 |  -235 |
|  115 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        265 |        -55 |        320 |   215 |   265 |   170 |   -15 |    20 |   -55 |    70 |   -15 |
|  116 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         60 |        -25 |         85 |     5 |    35 |   -25 |    60 |   -10 |   -15 |    -5 |    60 |
|  117 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        390 |          0 |        390 |   360 |   390 |   300 |   350 |   265 |   230 |     5 |   185 |
|  118 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        240 |        -50 |        290 |   240 |   -50 |    70 |   110 |     5 |    10 |    25 |   -10 |
|  119 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        175 |       -150 |        325 |   100 |   175 |    -5 |   -85 |  -115 |  -150 |   -55 |   -95 |
|  120 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        360 |          0 |        360 |   360 |   210 |   235 |   220 |   130 |   215 |   180 |   100 |
|  121 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -655 |        655 |  -450 |  -420 |  -505 |  -655 |  -585 |  -595 |  -495 |  -480 |
|  122 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -915 |        915 |  -690 |  -680 |  -650 |  -740 |  -645 |  -615 |  -785 |  -915 |
|  123 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1150 |       1150 |  -960 | -1085 | -1085 | -1105 | -1090 | -1110 | -1105 | -1150 |
|  124 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -670 |        670 |  -610 |  -540 |  -550 |  -650 |  -535 |  -605 |  -570 |  -670 |
|  125 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        380 |          0 |        380 |   310 |   230 |   225 |   380 |   145 |   260 |   235 |   265 |
|  126 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        390 |          0 |        390 |   390 |   130 |   200 |   160 |    75 |   115 |   135 |    80 |
|  127 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        120 |       -195 |        315 |   120 |   -25 |  -185 |    70 |  -175 |  -125 |  -120 |  -195 |
|  128 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -745 |        745 |  -540 |  -565 |  -475 |  -545 |  -560 |  -435 |  -550 |  -745 |
|  129 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        225 |          0 |        225 |   185 |   200 |   150 |   165 |   125 |   225 |   115 |   175 |
|  130 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -470 |        470 |  -260 |  -180 |  -445 |  -440 |  -415 |  -470 |  -420 |  -320 |
|  131 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -970 |        970 |  -750 |  -790 |  -795 |  -890 |  -940 |  -970 |  -970 |  -880 |
|  132 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        145 |       -200 |        345 |     5 |   145 |   105 |  -175 |  -125 |  -200 |  -150 |  -140 |
|  133 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -650 |        650 |  -540 |  -500 |  -425 |  -385 |  -505 |  -460 |  -395 |  -650 |
|  134 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        225 |         -0 |        225 |   225 |   155 |    85 |   180 |   160 |   215 |    -0 |    75 |
|  135 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         15 |       -195 |        210 |    10 |    15 |   -40 |   -65 |   -50 |   -70 |  -195 |  -155 |
|  136 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        355 |          0 |        355 |   355 |    60 |   120 |   140 |   130 |   140 |    70 |   145 |
|  137 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        355 |          0 |        355 |   330 |   355 |   295 |   255 |   260 |   295 |   195 |   155 |
|  138 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1155 |       1155 |  -920 |  -810 |  -850 |  -870 | -1135 | -1155 | -1085 | -1125 |
|  139 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -565 |        565 |  -375 |  -385 |  -420 |  -540 |  -510 |  -565 |  -550 |  -525 |
|  140 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -575 |        575 |  -305 |  -380 |  -395 |  -255 |  -345 |  -415 |  -215 |  -575 |
|  141 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -925 |        925 |  -855 |  -770 |  -755 |  -925 |  -860 |  -795 |  -815 |  -815 |
|  142 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -440 |        440 |  -140 |  -280 |  -315 |  -365 |  -440 |  -410 |  -275 |  -395 |
|  143 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        305 |        -25 |        330 |   210 |   305 |   235 |    -0 |    45 |    50 |   -25 |    65 |
|  144 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1105 |       1105 |  -875 |  -810 |  -845 |  -810 | -1070 | -1090 | -1025 | -1105 |
|  145 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        205 |          0 |        205 |    85 |    60 |   160 |   195 |   110 |   200 |   205 |    90 |
|  146 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        200 |        -50 |        250 |   200 |   170 |   200 |    -5 |    30 |   -50 |    15 |   -30 |
|  147 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -270 |        270 |   -95 |   -70 |   -60 |   -60 |  -270 |  -235 |  -265 |  -260 |
|  148 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1030 |       1030 |  -770 |  -775 |  -620 |  -920 | -1025 | -1030 |  -885 |  -895 |
|  149 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -330 |        330 |  -115 |  -330 |  -235 |  -270 |  -275 |  -315 |  -275 |  -205 |
|  150 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         65 |       -195 |        260 |    65 |    20 |  -110 |   -60 |   -75 |   -20 |  -195 |  -185 |
|  151 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -840 |        840 |  -840 |  -760 |  -705 |  -750 |  -705 |  -695 |  -750 |  -710 |
|  152 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -390 |        390 |   -50 |  -135 |  -215 |  -160 |  -390 |  -315 |  -310 |  -355 |
|  153 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          5 |       -255 |        260 |     5 |   -65 |  -175 |  -200 |  -250 |  -205 |  -225 |  -255 |
|  154 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        275 |          0 |        275 |   145 |   255 |   220 |   275 |   180 |   220 |   150 |   195 |
|  155 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -950 |        950 |  -710 |  -685 |  -620 |  -865 |  -835 |  -950 |  -910 |  -860 |
|  156 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1115 |       1115 |  -875 | -1015 |  -915 | -1020 | -1000 |  -940 |  -895 | -1115 |
|  157 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        245 |        -25 |        270 |   245 |   120 |   175 |   200 |   225 |   110 |    80 |   -25 |
|  158 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        355 |          0 |        355 |   350 |   265 |   345 |   215 |   320 |   355 |   350 |   255 |
|  159 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -405 |        405 |  -100 |  -335 |  -350 |  -365 |  -360 |  -405 |  -375 |  -310 |
|  160 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        210 |          0 |        210 |   145 |   205 |   170 |   140 |   160 |   145 |   210 |    60 |
|  161 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        180 |          0 |        180 |   165 |   180 |   115 |   175 |   180 |   120 |    75 |   115 |
|  162 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        275 |        -75 |        350 |   275 |    -5 |    30 |    55 |    55 |   135 |   -75 |    50 |
|  163 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -310 |        310 |   -20 |  -100 |  -305 |  -220 |  -285 |  -310 |  -250 |  -270 |
|  164 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1175 |       1175 |  -970 |  -965 |  -875 |  -900 |  -960 | -1015 | -1165 | -1175 |
|  165 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -720 |        720 |  -515 |  -510 |  -610 |  -685 |  -620 |  -670 |  -670 |  -720 |
|  166 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        390 |          0 |        390 |   225 |   390 |   365 |   320 |   225 |   165 |    40 |   185 |
|  167 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -955 |        955 |  -895 |  -905 |  -865 |  -905 |  -955 |  -920 |  -940 |  -840 |
|  168 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -425 |        425 |  -190 |  -185 |  -130 |  -280 |  -325 |  -320 |  -425 |  -415 |
|  169 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -565 |        565 |  -340 |  -300 |  -355 |  -310 |  -230 |  -290 |  -190 |  -565 |
|  170 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -665 |        665 |  -420 |  -370 |  -450 |  -460 |  -440 |  -440 |  -665 |  -605 |
|  171 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        270 |        -90 |        360 |   235 |   270 |   135 |   210 |   265 |   120 |   -60 |   -90 |
|  172 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -860 |        860 |  -655 |  -650 |  -600 |  -610 |  -695 |  -725 |  -660 |  -860 |
|  173 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        120 |          0 |        120 |   120 |    50 |    50 |    90 |    35 |    95 |    60 |    80 |
|  174 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1175 |       1175 |  -960 |  -940 | -1090 |  -925 |  -885 |  -975 | -1115 | -1175 |
|  175 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1075 |       1075 |  -940 |  -850 | -1070 | -1075 | -1010 | -1025 | -1035 | -1015 |
|  176 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -565 |        565 |  -295 |  -380 |  -365 |  -290 |  -230 |  -565 |  -460 |  -535 |
|  177 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -430 |        430 |  -280 |  -220 |  -310 |  -250 |  -205 |  -245 |  -390 |  -430 |
|  178 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        260 |          0 |        260 |   245 |   260 |   170 |   190 |   210 |    10 |    35 |    60 |
|  179 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -305 |        305 |   -65 |   -10 |   -20 |   -40 |  -145 |  -305 |  -290 |  -195 |
|  180 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        350 |        -10 |        360 |   265 |   300 |   230 |   350 |   255 |   300 |   -10 |    35 |
|  181 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -335 |        335 |  -130 |  -120 |  -335 |  -320 |  -305 |  -280 |  -165 |  -300 |
|  182 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1025 |       1025 |  -795 |  -795 | -1025 |  -925 |  -950 |  -980 |  -890 |  -950 |
|  183 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         60 |       -355 |        415 |   -95 |    60 |    -0 |   -35 |  -215 |  -355 |  -225 |  -230 |
|  184 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        260 |        -45 |        305 |   130 |   130 |   210 |   260 |   220 |    65 |   190 |   -45 |
|  185 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        255 |         -5 |        260 |   255 |    20 |    -5 |    60 |    75 |   125 |   115 |    80 |
|  186 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         15 |       -230 |        245 |     5 |    15 |  -160 |  -160 |  -115 |  -230 |  -150 |   -20 |
|  187 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -275 |        275 |    -5 |   -80 |   -40 |    -5 |  -275 |  -215 |  -175 |  -215 |
|  188 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -990 |        990 |  -785 |  -830 |  -850 |  -750 |  -765 |  -990 |  -900 |  -915 |
|  189 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        380 |          0 |        380 |   380 |   255 |   275 |   345 |   345 |   120 |    55 |   110 |
|  190 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -920 |        920 |  -695 |  -655 |  -750 |  -715 |  -685 |  -735 |  -640 |  -920 |
|  191 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        330 |        -10 |        340 |   205 |   190 |   195 |   310 |   330 |   245 |   -10 |    90 |
|  192 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        310 |          0 |        310 |   245 |   310 |   255 |   195 |   105 |   110 |    35 |    50 |
|  193 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -840 |        840 |  -765 |  -790 |  -795 |  -820 |  -740 |  -790 |  -760 |  -840 |
|  194 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        340 |          0 |        340 |   315 |   220 |   235 |   270 |   285 |   275 |   340 |   240 |
|  195 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        320 |          0 |        320 |   165 |   270 |   210 |   235 |   120 |   235 |   320 |    10 |
|  196 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1185 |       1185 | -1020 |  -970 | -1095 |  -975 | -1065 |  -930 | -1185 | -1095 |
|  197 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -935 |        935 |  -640 |  -770 |  -770 |  -635 |  -745 |  -750 |  -920 |  -935 |
|  198 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -405 |        405 |  -190 |  -210 |  -250 |  -390 |  -280 |  -345 |  -405 |  -315 |
|  199 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -590 |        590 |  -375 |  -460 |  -340 |  -385 |  -375 |  -375 |  -430 |  -590 |
|  200 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        155 |       -120 |        275 |   115 |   155 |    65 |    85 |   110 |   105 |  -120 |   -90 |
|  201 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -855 |        855 |  -610 |  -630 |  -665 |  -640 |  -815 |  -775 |  -810 |  -855 |
|  202 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        400 |          0 |        400 |   375 |   320 |   260 |   400 |   370 |   320 |   330 |   105 |
|  203 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -335 |        335 |   -55 |  -165 |  -165 |  -235 |  -165 |  -150 |  -280 |  -335 |
|  204 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        250 |         -5 |        255 |   105 |   240 |   145 |   195 |   250 |   155 |   215 |    -5 |
|  205 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        150 |       -110 |        260 |   120 |    55 |    50 |    75 |   150 |    60 |     5 |  -110 |
|  206 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1155 |       1155 | -1000 |  -885 | -1090 | -1155 | -1135 | -1155 | -1040 | -1090 |
|  207 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -915 |        915 |  -670 |  -770 |  -720 |  -695 |  -915 |  -820 |  -790 |  -865 |
|  208 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1115 |       1115 |  -830 |  -795 |  -765 |  -980 | -1115 | -1055 | -1055 | -1035 |
|  209 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         -0 |       -280 |        280 |    -0 |   -50 |   -60 |   -20 |   -30 |   -25 |  -235 |  -280 |
|  210 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -300 |        300 |   -65 |  -210 |  -230 |  -245 |  -300 |  -250 |   -80 |  -205 |
|  211 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -905 |        905 |  -685 |  -485 |  -905 |  -815 |  -795 |  -730 |  -845 |  -765 |
|  212 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        280 |        -60 |        340 |   145 |    35 |   280 |   115 |    60 |   195 |   130 |   -60 |
|  213 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -380 |        380 |  -120 |  -200 |   -70 |   -15 |  -370 |  -265 |  -380 |  -305 |
|  214 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1070 |       1070 |  -885 |  -855 |  -875 |  -800 |  -830 |  -940 |  -855 | -1070 |
|  215 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -745 |        745 |  -555 |  -565 |  -600 |  -600 |  -595 |  -745 |  -700 |  -745 |
|  216 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -935 |        935 |  -735 |  -720 |  -705 |  -775 |  -790 |  -775 |  -745 |  -935 |
|  217 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -535 |        535 |  -510 |  -415 |  -505 |  -490 |  -535 |  -480 |  -510 |  -515 |
|  218 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        340 |          0 |        340 |   340 |   335 |   340 |    95 |    40 |   100 |    70 |    10 |
|  219 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        380 |          0 |        380 |   345 |   345 |   380 |   320 |   370 |    50 |   140 |    65 |
|  220 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        105 |       -210 |        315 |    45 |   105 |  -155 |  -140 |  -180 |  -180 |   -60 |  -210 |
|  221 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -370 |        370 |  -100 |   -15 |  -130 |  -260 |  -285 |  -270 |  -225 |  -370 |
|  222 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        380 |          0 |        380 |   325 |   350 |   235 |   325 |   310 |   380 |    75 |   230 |
|  223 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -795 |        795 |  -685 |  -735 |  -720 |  -650 |  -795 |  -755 |  -775 |  -640 |
|  224 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -865 |        865 |  -630 |  -805 |  -865 |  -800 |  -780 |  -840 |  -805 |  -810 |
|  225 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -465 |        465 |  -155 |  -415 |  -465 |  -410 |  -405 |  -335 |  -355 |  -355 |
|  226 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        375 |          0 |        375 |   175 |   215 |   375 |    40 |    35 |    60 |    50 |    80 |
|  227 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -800 |        800 |  -575 |  -585 |  -580 |  -585 |  -545 |  -605 |  -660 |  -800 |
|  228 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        115 |       -215 |        330 |   -10 |   115 |    65 |    95 |  -215 |  -125 |  -105 |  -190 |
|  229 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         95 |       -135 |        230 |    90 |    95 |   -80 |  -120 |   -85 |  -100 |  -135 |  -130 |
|  230 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -995 |        995 |  -950 |  -995 |  -950 |  -995 |  -955 |  -980 |  -975 |  -925 |
|  231 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        375 |          0 |        375 |   355 |   375 |   215 |   205 |   245 |   210 |   185 |   185 |
|  232 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        355 |          0 |        355 |   240 |   260 |   310 |   355 |   355 |   355 |   235 |   145 |
|  233 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        230 |        -25 |        255 |   230 |    85 |    85 |   -25 |    55 |   210 |   -10 |   105 |
|  234 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         85 |        -55 |        140 |    85 |     5 |    70 |   -40 |   -55 |    -0 |   -20 |    55 |
|  235 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -215 |        215 |  -215 |  -155 |  -105 |  -105 |  -180 |  -155 |  -210 |  -110 |
|  236 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        210 |       -100 |        310 |   210 |   145 |    -0 |   -75 |    25 |   -65 |  -100 |   -25 |
|  237 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        340 |          0 |        340 |   290 |   295 |   340 |   105 |    55 |   110 |   100 |    35 |
|  238 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         75 |       -185 |        260 |    75 |    70 |   -25 |  -145 |  -135 |  -130 |  -100 |  -185 |
|  239 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        380 |          0 |        380 |   350 |   380 |   380 |   335 |   355 |   100 |   195 |   205 |
|  240 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1180 |       1180 |  -855 |  -815 |  -955 |  -845 |  -955 | -1050 | -1180 | -1145 |
|  241 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -500 |        500 |  -275 |  -325 |  -240 |  -490 |  -500 |  -435 |  -445 |  -460 |
|  242 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1040 |       1040 |  -935 |  -780 |  -815 |  -825 |  -945 |  -855 | -1040 | -1035 |
|  243 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -985 |        985 |  -850 |  -680 |  -670 |  -820 |  -725 |  -840 |  -985 |  -930 |
|  244 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -950 |        950 |  -635 |  -605 |  -675 |  -675 |  -660 |  -680 |  -675 |  -950 |
|  245 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        335 |          0 |        335 |   260 |   335 |   165 |   105 |   195 |   230 |   220 |   135 |
|  246 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -860 |        860 |  -600 |  -550 |  -635 |  -680 |  -625 |  -440 |  -620 |  -860 |
|  247 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -940 |        940 |  -660 |  -690 |  -885 |  -875 |  -875 |  -900 |  -910 |  -940 |
|  248 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -635 |        635 |  -365 |  -335 |  -565 |  -580 |  -515 |  -555 |  -635 |  -435 |
|  249 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -925 |        925 |  -805 |  -720 |  -620 |  -755 |  -815 |  -760 |  -785 |  -925 |
|  250 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        210 |        -95 |        305 |   155 |   200 |    60 |   110 |   105 |    80 |   210 |   -95 |
|  251 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         70 |       -205 |        275 |    35 |    70 |    -5 |   -45 |    65 |    60 |    15 |  -205 |
|  252 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        260 |        -15 |        275 |   260 |    95 |   155 |   145 |   -15 |   105 |    55 |    20 |
|  253 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        375 |          0 |        375 |   375 |   220 |   160 |   120 |   200 |   180 |   130 |   250 |
|  254 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1120 |       1120 | -1005 | -1105 | -1040 | -1120 |  -960 |  -985 | -1080 | -1025 |
|  255 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        195 |       -125 |        320 |   130 |   195 |   -80 |   -40 |     5 |    15 |  -125 |   -20 |
|  256 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -735 |        735 |  -550 |  -445 |  -530 |  -545 |  -720 |  -690 |  -735 |  -625 |
|  257 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -820 |        820 |  -635 |  -620 |  -635 |  -770 |  -645 |  -595 |  -685 |  -820 |
|  258 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -630 |        630 |  -300 |  -300 |  -630 |  -600 |  -545 |  -555 |  -585 |  -510 |
|  259 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        325 |          0 |        325 |   325 |   325 |   235 |    60 |   205 |   185 |   240 |   135 |
|  260 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -730 |        730 |  -400 |  -400 |  -550 |  -610 |  -730 |  -670 |  -675 |  -705 |
|  261 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -855 |        855 |  -805 |  -850 |  -760 |  -780 |  -835 |  -800 |  -855 |  -830 |
|  262 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -430 |        430 |  -175 |  -140 |  -300 |  -380 |  -245 |  -430 |  -255 |  -405 |
|  263 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        285 |          0 |        285 |   200 |   230 |   275 |   255 |   170 |   170 |   280 |   285 |
|  264 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1040 |       1040 |  -865 |  -760 |  -765 | -1025 |  -990 | -1040 |  -980 | -1015 |
|  265 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1000 |       1000 |  -670 |  -765 |  -905 |  -910 |  -860 |  -960 | -1000 |  -920 |
|  266 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        275 |        -45 |        320 |   185 |   130 |   275 |   190 |    65 |   -20 |   -45 |   -20 |
|  267 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        150 |        -60 |        210 |    80 |   140 |   150 |   125 |   120 |   135 |    -5 |   -60 |
|  268 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        140 |       -255 |        395 |   140 |    -5 |   -25 |   -10 |   -10 |  -210 |  -120 |  -255 |
|  269 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -510 |        510 |  -380 |  -355 |  -375 |  -340 |  -300 |  -315 |  -460 |  -510 |
|  270 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        285 |        -10 |        295 |   230 |   270 |   285 |   210 |   215 |   -10 |   130 |    -5 |
|  271 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        320 |          0 |        320 |   220 |   270 |   320 |   320 |   285 |   295 |   290 |    55 |
|  272 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -170 |        170 |  -140 |   -45 |  -140 |  -170 |  -125 |   -55 |  -130 |  -145 |
|  273 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -905 |        905 |  -775 |  -715 |  -710 |  -745 |  -890 |  -840 |  -850 |  -905 |
|  274 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         60 |       -335 |        395 |   -10 |   -45 |    60 |  -100 |  -335 |  -150 |  -170 |  -235 |
|  275 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1000 |       1000 |  -740 |  -925 |  -995 |  -870 |  -935 | -1000 |  -920 |  -960 |
|  276 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -285 |        285 |  -275 |  -240 |  -215 |  -190 |  -285 |  -215 |  -265 |  -245 |
|  277 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         55 |       -215 |        270 |    55 |  -160 |   -85 |  -210 |  -215 |  -185 |  -160 |   -45 |
|  278 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         95 |       -145 |        240 |    95 |    -5 |   -15 |   -25 |    30 |    10 |  -125 |  -145 |
|  279 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -830 |        830 |  -585 |  -515 |  -595 |  -645 |  -635 |  -555 |  -785 |  -830 |
|  280 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        385 |        -45 |        430 |   385 |   340 |   -45 |    15 |    85 |   130 |    40 |    45 |
|  281 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -575 |        575 |  -465 |  -290 |  -420 |  -575 |  -565 |  -565 |  -575 |  -555 |
|  282 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1190 |       1190 | -1015 | -1040 | -1005 |  -955 |  -940 |  -980 |  -965 | -1190 |
|  283 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1195 |       1195 | -1105 | -1110 |  -995 | -1055 |  -980 | -1090 | -1195 | -1190 |
|  284 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         -0 |       -285 |        285 |   -35 |    -0 |  -155 |   -45 |  -115 |   -65 |  -285 |  -255 |
|  285 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        290 |          0 |        290 |   290 |   195 |   145 |    80 |   125 |   140 |    95 |   185 |
|  286 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        330 |          0 |        330 |   250 |   330 |   305 |    60 |    20 |    90 |    20 |   105 |
|  287 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1155 |       1155 |  -900 |  -895 |  -915 |  -985 |  -845 |  -950 |  -980 | -1155 |
|  288 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -920 |        920 |  -570 |  -555 |  -795 |  -855 |  -820 |  -770 |  -920 |  -855 |
|  289 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -755 |        755 |  -585 |  -685 |  -530 |  -495 |  -575 |  -550 |  -660 |  -755 |
|  290 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -960 |        960 |  -700 |  -700 |  -675 |  -685 |  -685 |  -780 |  -650 |  -960 |
|  291 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -430 |        430 |  -115 |  -350 |  -330 |  -430 |  -295 |  -310 |  -245 |  -345 |
|  292 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        235 |        -50 |        285 |   235 |   130 |   120 |   170 |    20 |   -50 |    35 |     5 |
|  293 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1200 |       1200 |  -945 |  -810 | -1000 | -1200 | -1180 | -1135 | -1190 | -1045 |
|  294 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        295 |          0 |        295 |   285 |   210 |   295 |    90 |    40 |    40 |   230 |   135 |
|  295 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -800 |        800 |  -475 |  -505 |  -575 |  -715 |  -700 |  -720 |  -715 |  -800 |
|  296 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -845 |        845 |  -645 |  -580 |  -645 |  -640 |  -590 |  -840 |  -845 |  -825 |
|  297 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1025 |       1025 |  -865 |  -830 |  -775 |  -805 |  -870 |  -755 |  -770 | -1025 |
|  298 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -825 |        825 |  -530 |  -570 |  -785 |  -695 |  -695 |  -825 |  -790 |  -765 |
|  299 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        100 |        -60 |        160 |    60 |   -60 |    80 |   -25 |    30 |    -5 |   100 |   -40 |
|  300 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -545 |        545 |  -305 |  -300 |  -425 |  -465 |  -545 |  -450 |  -435 |  -460 |
|  301 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        315 |        -35 |        350 |   315 |   -15 |    60 |    10 |   -35 |    25 |    50 |    10 |
|  302 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -370 |        370 |  -210 |  -235 |  -340 |  -280 |  -160 |  -270 |  -255 |  -370 |
|  303 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        155 |          0 |        155 |    95 |    30 |    65 |   155 |    15 |   115 |   130 |   125 |
|  304 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -385 |        385 |  -135 |  -155 |  -275 |  -165 |  -165 |  -205 |  -175 |  -385 |
|  305 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        225 |          0 |        225 |   135 |    60 |   160 |   115 |    75 |   135 |    75 |   225 |
|  306 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -415 |        415 |  -265 |  -405 |  -375 |  -335 |  -340 |  -390 |  -415 |  -405 |
|  307 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        205 |        -80 |        285 |   130 |   140 |   175 |   160 |   205 |   160 |   195 |   -80 |
|  308 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1025 |       1025 |  -765 |  -900 |  -955 |  -985 | -1025 |  -915 |  -900 |  -935 |
|  309 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1105 |       1105 |  -825 |  -885 |  -840 |  -865 |  -970 | -1105 |  -950 | -1050 |
|  310 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -785 |        785 |  -625 |  -555 |  -565 |  -575 |  -560 |  -605 |  -550 |  -785 |
|  311 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1165 |       1165 | -1030 |  -990 | -1080 | -1005 | -1055 | -1060 | -1085 | -1165 |
|  312 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        365 |          0 |        365 |   365 |   190 |   210 |   235 |   345 |   355 |   255 |   260 |
|  313 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -680 |        680 |  -515 |  -375 |  -445 |  -405 |  -525 |  -585 |  -655 |  -680 |
|  314 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1025 |       1025 |  -730 |  -785 |  -850 |  -725 |  -770 |  -955 |  -910 | -1025 |
|  315 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -655 |        655 |  -415 |  -505 |  -305 |  -480 |  -655 |  -625 |  -615 |  -650 |
|  316 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         75 |       -130 |        205 |    -0 |  -130 |   -30 |   -30 |   -10 |   -30 |  -105 |    75 |
|  317 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        385 |          0 |        385 |   255 |   385 |   345 |   350 |   365 |   305 |   275 |    90 |
|  318 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -570 |        570 |  -275 |  -260 |  -415 |  -335 |  -380 |  -295 |  -570 |  -480 |
|  319 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -620 |        620 |  -320 |  -275 |  -345 |  -555 |  -485 |  -480 |  -515 |  -620 |
|  320 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        120 |          0 |        120 |    85 |   115 |    40 |   120 |   120 |    80 |    90 |    45 |
|  321 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1020 |       1020 |  -685 |  -760 |  -655 |  -775 |  -720 |  -795 |  -720 | -1020 |
|  322 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        315 |          0 |        315 |   315 |   280 |   305 |   300 |    45 |   130 |   115 |    80 |
|  323 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        265 |          0 |        265 |   250 |   255 |   205 |   265 |   195 |   265 |   250 |   205 |
|  324 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -880 |        880 |  -770 |  -790 |  -880 |  -845 |  -765 |  -730 |  -850 |  -820 |
|  325 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1170 |       1170 |  -935 |  -965 |  -900 |  -885 |  -990 | -1170 | -1110 | -1105 |
|  326 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        280 |        -10 |        290 |   280 |   -10 |   125 |    50 |    -0 |     5 |    50 |   125 |
|  327 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -820 |        820 |  -660 |  -655 |  -620 |  -620 |  -600 |  -675 |  -810 |  -820 |
|  328 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        385 |          0 |        385 |   265 |   370 |   350 |   255 |   325 |   285 |   385 |    95 |
|  329 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1030 |       1030 | -1030 |  -860 | -1010 |  -955 |  -900 |  -870 |  -800 |  -955 |
|  330 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        240 |         -0 |        240 |   240 |   175 |   240 |   175 |   115 |    -0 |    70 |    85 |
|  331 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1080 |       1080 |  -845 |  -775 |  -755 |  -825 |  -820 |  -730 |  -955 | -1080 |
|  332 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -565 |        565 |  -350 |  -395 |  -435 |  -305 |  -440 |  -440 |  -480 |  -565 |
|  333 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -655 |        655 |  -490 |  -320 |  -435 |  -415 |  -415 |  -320 |  -415 |  -655 |
|  334 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        345 |          0 |        345 |   205 |   220 |   345 |   110 |    85 |   120 |    65 |    45 |
|  335 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -505 |        505 |  -180 |  -505 |  -370 |  -435 |  -415 |  -395 |  -420 |  -405 |
|  336 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        375 |          0 |        375 |   375 |   270 |   110 |    80 |   130 |   205 |   145 |   185 |
|  337 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        315 |        -35 |        350 |   185 |   290 |   315 |   245 |   285 |    75 |    20 |   -35 |
|  338 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -410 |        410 |  -315 |  -215 |  -170 |  -285 |  -195 |  -175 |  -170 |  -410 |
|  339 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        235 |        -50 |        285 |   135 |   235 |   115 |    75 |    70 |    30 |   -20 |   -50 |
|  340 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -945 |        945 |  -610 |  -670 |  -500 |  -670 |  -860 |  -800 |  -945 |  -760 |
|  341 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        215 |        -60 |        275 |   215 |   175 |    70 |   -60 |   -25 |   -30 |    55 |    -0 |
|  342 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        265 |        -50 |        315 |   210 |   265 |   175 |   130 |    25 |   -50 |     5 |     5 |
|  343 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1020 |       1020 |  -725 |  -730 |  -635 |  -750 |  -950 |  -870 | -1020 |  -930 |
|  344 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        240 |       -115 |        355 |   125 |   240 |   155 |   155 |   -65 |  -115 |   -65 |   -10 |
|  345 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        270 |        -25 |        295 |   270 |   205 |   200 |   220 |   200 |   205 |   -25 |   -10 |
|  346 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -490 |        490 |  -225 |  -480 |  -410 |  -380 |  -480 |  -490 |  -385 |  -355 |
|  347 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -550 |        550 |  -310 |  -290 |  -300 |  -370 |  -490 |  -550 |  -525 |  -500 |
|  348 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        265 |          0 |        265 |   265 |    95 |   105 |    75 |    50 |    55 |   105 |    20 |
|  349 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1170 |       1170 | -1135 |  -995 | -1170 | -1135 | -1050 | -1070 | -1030 | -1050 |
|  350 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1085 |       1085 |  -845 |  -760 |  -865 |  -890 |  -840 |  -730 |  -985 | -1085 |
|  351 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         50 |       -225 |        275 |    50 |   -50 |   -30 |   -40 |   -95 |    20 |   -15 |  -225 |
|  352 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -645 |        645 |  -370 |  -385 |  -370 |  -620 |  -630 |  -615 |  -645 |  -550 |
|  353 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        345 |          0 |        345 |   295 |   345 |   100 |   135 |   190 |   135 |   160 |    55 |
|  354 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -555 |        555 |  -245 |  -255 |  -475 |  -475 |  -480 |  -470 |  -410 |  -555 |
|  355 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -915 |        915 |  -615 |  -870 |  -830 |  -840 |  -880 |  -770 |  -795 |  -915 |
|  356 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -505 |        505 |  -240 |  -205 |  -250 |  -215 |  -255 |  -255 |  -505 |  -490 |
|  357 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        215 |        -35 |        250 |   215 |   190 |    -0 |    80 |    45 |    35 |   -35 |   -30 |
|  358 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        195 |        -50 |        245 |   195 |     5 |    30 |   115 |   -40 |    80 |    85 |   -50 |
|  359 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        360 |        -15 |        375 |   280 |   210 |   360 |   -15 |    50 |    55 |   100 |   105 |
|  360 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        265 |          0 |        265 |    85 |   170 |   265 |    80 |   205 |   150 |   140 |    70 |
|  361 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        165 |       -105 |        270 |   110 |   165 |    40 |    85 |    45 |   -30 |   -10 |  -105 |
|  362 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        270 |          0 |        270 |   270 |   180 |   265 |   255 |   190 |   205 |   235 |    95 |
|  363 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -470 |        470 |  -265 |  -240 |  -275 |  -280 |  -470 |  -420 |  -430 |  -435 |
|  364 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1105 |       1105 | -1000 |  -980 | -1105 | -1075 | -1085 | -1005 | -1030 | -1040 |
|  365 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        255 |          0 |        255 |   115 |   130 |   125 |   125 |   255 |   210 |   145 |   170 |
|  366 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        365 |          0 |        365 |   140 |   255 |   195 |   125 |   365 |    50 |   145 |    80 |
|  367 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1075 |       1075 |  -850 |  -840 |  -915 |  -925 |  -975 |  -995 |  -965 | -1075 |
|  368 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -530 |        530 |  -240 |  -315 |  -220 |  -350 |  -355 |  -385 |  -455 |  -530 |
|  369 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        285 |        -40 |        325 |   270 |   285 |   200 |   185 |   265 |   -40 |   -20 |    65 |
|  370 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -515 |        515 |  -305 |  -345 |  -200 |  -365 |  -285 |  -295 |  -515 |  -360 |
|  371 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        345 |          0 |        345 |   330 |   320 |   325 |   295 |   345 |    95 |    95 |   170 |
|  372 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1135 |       1135 |  -905 |  -870 |  -920 |  -950 | -1120 |  -990 | -1020 | -1135 |
|  373 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        330 |          0 |        330 |   330 |   205 |   250 |   265 |   150 |   115 |    85 |    75 |
|  374 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -855 |        855 |  -585 |  -695 |  -670 |  -670 |  -635 |  -805 |  -855 |  -810 |
|  375 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        385 |          0 |        385 |   350 |   335 |   375 |   360 |   385 |   160 |   185 |   170 |
|  376 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1045 |       1045 |  -645 |  -970 | -1045 |  -975 |  -935 |  -940 | -1005 |  -900 |
|  377 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         30 |       -335 |        365 |   -20 |    30 |  -130 |  -115 |  -260 |  -245 |  -245 |  -335 |
|  378 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        345 |          0 |        345 |   250 |   285 |   305 |   290 |   345 |   325 |   275 |    50 |
|  379 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        220 |          0 |        220 |   220 |     5 |    45 |    90 |   135 |     5 |    65 |   130 |
|  380 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1005 |       1005 |  -800 |  -765 |  -820 |  -715 |  -855 |  -725 |  -945 | -1005 |
|  381 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         35 |       -395 |        430 |   -65 |  -135 |    35 |   -85 |   -50 |  -160 |  -395 |  -315 |
|  382 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        255 |         -0 |        255 |   255 |   230 |   245 |   235 |    30 |    35 |    40 |    -0 |
|  383 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -905 |        905 |  -605 |  -800 |  -780 |  -795 |  -810 |  -905 |  -760 |  -830 |
|  384 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1160 |       1160 |  -940 |  -945 |  -910 | -1075 | -1045 | -1065 |  -985 | -1160 |
|  385 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -780 |        780 |  -555 |  -560 |  -625 |  -540 |  -630 |  -780 |  -780 |  -745 |
|  386 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        230 |          0 |        230 |   115 |    65 |   195 |   230 |   120 |   125 |    85 |    35 |
|  387 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -910 |        910 |  -840 |  -870 |  -705 |  -910 |  -740 |  -865 |  -865 |  -815 |
|  388 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -960 |        960 |  -830 |  -840 |  -840 |  -945 |  -725 |  -885 |  -925 |  -960 |
|  389 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -420 |        420 |  -170 |  -195 |  -150 |  -205 |  -210 |  -420 |  -350 |  -355 |
|  390 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -825 |        825 |  -390 |  -490 |  -515 |  -520 |  -620 |  -575 |  -505 |  -825 |
|  391 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -460 |        460 |  -355 |  -460 |  -365 |  -390 |  -360 |  -395 |  -340 |  -335 |
|  392 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        390 |          0 |        390 |   320 |   330 |   390 |   265 |   260 |   330 |   310 |   105 |
|  393 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -385 |        385 |   -60 |   -95 |  -325 |  -365 |  -295 |  -275 |  -290 |  -385 |
|  394 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        235 |          0 |        235 |   190 |   170 |   195 |   195 |   235 |   210 |    15 |    60 |
|  395 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         20 |       -310 |        330 |   -15 |    20 |   -75 |   -25 |  -230 |  -300 |  -310 |  -285 |
|  396 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1055 |       1055 |  -735 |  -820 |  -845 | -1015 | -1045 | -1050 | -1030 | -1055 |
|  397 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -815 |        815 |  -550 |  -590 |  -565 |  -580 |  -505 |  -520 |  -545 |  -815 |
|  398 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -645 |        645 |  -320 |  -300 |  -340 |  -515 |  -575 |  -645 |  -535 |  -515 |
|  399 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -465 |        465 |  -275 |  -210 |  -185 |  -455 |  -330 |  -465 |  -410 |  -335 |
|  400 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1150 |       1150 |  -950 |  -900 | -1110 | -1110 | -1115 | -1080 | -1150 | -1045 |
|  401 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1160 |       1160 |  -825 |  -960 |  -950 |  -965 |  -865 |  -875 |  -890 | -1160 |
|  402 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -520 |        520 |  -280 |  -520 |  -415 |  -460 |  -485 |  -445 |  -425 |  -385 |
|  403 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -580 |        580 |  -420 |  -435 |  -385 |  -290 |  -505 |  -505 |  -395 |  -580 |
|  404 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -460 |        460 |  -265 |  -240 |  -240 |  -225 |  -460 |  -430 |  -415 |  -385 |
|  405 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -585 |        585 |  -265 |  -545 |  -510 |  -585 |  -435 |  -470 |  -515 |  -510 |
|  406 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        345 |          0 |        345 |   345 |   155 |    95 |   155 |   165 |   135 |   150 |    85 |
|  407 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -695 |        695 |  -475 |  -370 |  -460 |  -390 |  -450 |  -355 |  -595 |  -695 |
|  408 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        145 |       -170 |        315 |   145 |   110 |    90 |   -40 |  -170 |   -85 |  -135 |  -145 |
|  409 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        210 |       -115 |        325 |   135 |   210 |  -115 |   -10 |    10 |    85 |   -35 |    90 |
|  410 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -925 |        925 |  -605 |  -615 |  -605 |  -645 |  -750 |  -925 |  -900 |  -850 |
|  411 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -500 |        500 |  -345 |  -330 |  -320 |  -270 |  -260 |  -490 |  -480 |  -500 |
|  412 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        215 |        -65 |        280 |   190 |   150 |   170 |   215 |     5 |   215 |   -65 |    -0 |
|  413 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        245 |        -65 |        310 |   180 |   245 |   165 |   230 |   135 |   230 |   -65 |   -30 |
|  414 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        170 |        -50 |        220 |    60 |   -50 |    65 |   100 |   155 |    35 |   170 |    30 |
|  415 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -940 |        940 |  -670 |  -715 |  -710 |  -680 |  -695 |  -755 |  -725 |  -940 |
|  416 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1015 |       1015 |  -955 | -1015 |  -920 |  -910 |  -895 |  -875 |  -965 |  -885 |
|  417 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1025 |       1025 |  -760 |  -715 |  -810 |  -760 |  -950 | -1005 |  -965 | -1025 |
|  418 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        235 |        -20 |        255 |   175 |   155 |    90 |   105 |   235 |   165 |   175 |   -20 |
|  419 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        270 |        -60 |        330 |   225 |   270 |     5 |    95 |    25 |   -60 |   120 |   -20 |
|  420 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -405 |        405 |  -200 |  -220 |   -95 |  -215 |  -210 |  -390 |  -405 |  -380 |
|  421 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -770 |        770 |  -410 |  -420 |  -615 |  -550 |  -770 |  -585 |  -580 |  -575 |
|  422 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1015 |       1015 |  -775 |  -770 |  -715 |  -720 |  -750 |  -715 |  -955 | -1015 |
|  423 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         90 |       -280 |        370 |    30 |    85 |    90 |   -65 |    20 |    60 |    -0 |  -280 |
|  424 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -390 |        390 |  -135 |  -115 |   -80 |  -390 |  -330 |  -340 |  -335 |  -280 |
|  425 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        165 |          0 |        165 |   155 |    60 |    80 |   105 |   120 |   165 |   135 |    85 |
|  426 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -970 |        970 |  -815 |  -770 |  -805 |  -955 |  -945 |  -960 |  -970 |  -925 |
|  427 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        135 |        -75 |        210 |   135 |    85 |   115 |    95 |   -35 |   -75 |    25 |   -40 |
|  428 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        225 |        -50 |        275 |    95 |    80 |   140 |   225 |   145 |   -50 |   -45 |    -5 |
|  429 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        190 |        -30 |        220 |   145 |   115 |   150 |   190 |    80 |   125 |    90 |   -30 |
|  430 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        205 |        -55 |        260 |   205 |   120 |   150 |   135 |   185 |   175 |   -55 |    15 |
|  431 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -975 |        975 |  -970 |  -975 |  -925 |  -920 |  -925 |  -810 |  -825 |  -905 |
|  432 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        395 |          0 |        395 |   345 |   350 |   395 |   320 |   335 |   395 |   275 |   365 |
|  433 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -435 |        435 |  -120 |  -180 |  -115 |  -295 |  -345 |  -435 |  -405 |  -385 |
|  434 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -440 |        440 |  -245 |  -235 |  -375 |  -320 |  -265 |  -355 |  -200 |  -440 |
|  435 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        305 |          0 |        305 |   165 |   175 |   170 |   175 |   305 |   180 |   195 |    20 |
|  436 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        105 |       -150 |        255 |   105 |    45 |    20 |   -15 |    45 |   105 |    35 |  -150 |
|  437 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -605 |        605 |  -505 |  -340 |  -410 |  -330 |  -495 |  -390 |  -370 |  -605 |
|  438 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -895 |        895 |  -780 |  -895 |  -810 |  -790 |  -825 |  -885 |  -800 |  -830 |
|  439 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 |  -865 |  -785 |  -945 |  -830 |  -795 |  -885 |  -865 | -1080 |
|  440 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -510 |        510 |  -380 |  -360 |  -320 |  -270 |  -430 |  -355 |  -355 |  -510 |
|  441 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -470 |        470 |  -230 |  -200 |  -350 |  -365 |  -435 |  -440 |  -430 |  -470 |
|  442 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        250 |        -75 |        325 |    95 |   245 |   190 |   245 |   185 |   235 |   250 |   -75 |
|  443 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -705 |        705 |  -310 |  -400 |  -580 |  -545 |  -505 |  -515 |  -705 |  -585 |
|  444 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         95 |       -220 |        315 |    80 |    55 |    95 |    30 |  -210 |   -85 |  -220 |   -60 |
|  445 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1165 |       1165 |  -980 |  -950 |  -980 |  -990 |  -920 |  -935 | -1165 | -1120 |
|  446 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        310 |          0 |        310 |   310 |   110 |   100 |   130 |   155 |   210 |   135 |   105 |
|  447 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        290 |          0 |        290 |   265 |   290 |   275 |    80 |    70 |    70 |   190 |    90 |
|  448 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        105 |        -85 |        190 |    40 |   -35 |   105 |    10 |   100 |    20 |   -85 |    -0 |
|  449 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1195 |       1195 |  -900 |  -835 |  -865 |  -815 | -1010 | -1095 | -1195 | -1060 |
|  450 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        360 |          0 |        360 |   315 |   255 |   360 |   305 |   260 |   220 |   155 |   100 |
|  451 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        315 |          0 |        315 |   185 |   200 |   225 |   315 |   225 |   225 |   285 |    45 |
|  452 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        250 |          0 |        250 |   230 |   210 |   210 |   250 |   205 |   170 |   145 |    25 |
|  453 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -315 |        315 |  -305 |  -230 |  -270 |  -315 |  -225 |  -245 |  -255 |  -285 |
|  454 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        145 |        -30 |        175 |    15 |   145 |    90 |   -30 |     5 |    10 |    -0 |    50 |
|  455 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1120 |       1120 |  -785 |  -950 |  -995 |  -900 | -1120 | -1080 |  -965 |  -990 |
|  456 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -415 |        415 |  -145 |  -140 |  -240 |  -335 |  -380 |  -415 |  -310 |  -380 |
|  457 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -845 |        845 |  -715 |  -585 |  -630 |  -645 |  -610 |  -635 |  -715 |  -845 |
|  458 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        320 |        -35 |        355 |   295 |   295 |   320 |   270 |   220 |   240 |    10 |   -35 |
|  459 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        315 |          0 |        315 |   275 |   230 |   235 |   275 |   315 |   315 |   240 |   165 |
|  460 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -870 |        870 |  -870 |  -820 |  -770 |  -775 |  -785 |  -725 |  -825 |  -740 |
|  461 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -845 |        845 |  -550 |  -770 |  -810 |  -750 |  -795 |  -825 |  -750 |  -845 |
|  462 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         95 |        -80 |        175 |   -80 |   -55 |    95 |   -75 |     5 |    30 |   -20 |   -35 |
|  463 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -845 |        845 |  -590 |  -660 |  -580 |  -700 |  -590 |  -705 |  -590 |  -845 |
|  464 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -850 |        850 |  -615 |  -750 |  -765 |  -815 |  -850 |  -765 |  -810 |  -760 |
|  465 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -455 |        455 |  -125 |  -185 |  -240 |  -305 |  -190 |  -330 |  -145 |  -455 |
|  466 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -320 |        320 |  -125 |  -195 |  -135 |   -45 |   -85 |  -160 |  -115 |  -320 |
|  467 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        250 |         -0 |        250 |   215 |   250 |   225 |   245 |   200 |    45 |    -0 |    30 |
|  468 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1080 |       1080 |  -840 |  -825 |  -810 | -1080 | -1000 | -1050 | -1000 | -1045 |
|  469 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -585 |        585 |  -290 |  -275 |  -205 |  -265 |  -300 |  -505 |  -585 |  -545 |
|  470 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -710 |        710 |  -455 |  -460 |  -430 |  -485 |  -460 |  -535 |  -480 |  -710 |
|  471 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        310 |        -60 |        370 |   160 |   310 |   215 |   220 |   295 |   -60 |    50 |    15 |
|  472 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -415 |        415 |  -185 |  -165 |  -150 |  -120 |  -365 |  -415 |  -335 |  -320 |
|  473 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -425 |        425 |  -190 |  -165 |  -145 |  -195 |  -240 |  -150 |  -425 |  -335 |
|  474 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        290 |         -5 |        295 |   290 |   250 |   135 |    80 |    75 |   130 |   105 |    -5 |
|  475 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -905 |        905 |  -605 |  -665 |  -660 |  -835 |  -905 |  -855 |  -790 |  -815 |
|  476 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -735 |        735 |  -600 |  -480 |  -490 |  -465 |  -735 |  -655 |  -730 |  -690 |
|  477 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          5 |       -360 |        365 |  -125 |   -75 |  -135 |   -70 |  -120 |     5 |  -360 |  -275 |
|  478 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         20 |       -265 |        285 |    15 |    20 |  -175 |  -180 |  -265 |  -160 |  -205 |  -205 |
|  479 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         45 |        -80 |        125 |     5 |    25 |     5 |     5 |    45 |   -20 |   -30 |   -80 |
|  480 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        270 |        -20 |        290 |   245 |   215 |   265 |   175 |   205 |   270 |   190 |   -20 |
|  481 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -735 |        735 |  -505 |  -520 |  -625 |  -675 |  -715 |  -650 |  -695 |  -735 |
|  482 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -650 |        650 |  -400 |  -325 |  -380 |  -315 |  -310 |  -650 |  -555 |  -575 |
|  483 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        380 |          0 |        380 |   325 |   380 |   215 |   165 |   195 |   165 |   190 |   185 |
|  484 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -635 |        635 |  -300 |  -410 |  -325 |  -490 |  -355 |  -605 |  -635 |  -520 |
|  485 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        350 |          0 |        350 |   295 |   225 |   340 |   350 |   320 |   330 |   220 |    90 |
|  486 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -720 |        720 |  -500 |  -545 |  -515 |  -515 |  -495 |  -515 |  -570 |  -720 |
|  487 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         15 |       -240 |        255 |   -75 |  -170 |   -90 |   -35 |    15 |   -25 |     5 |  -240 |
|  488 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -475 |        475 |  -360 |  -315 |  -375 |  -265 |  -310 |  -290 |  -260 |  -475 |
|  489 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        205 |       -110 |        315 |   170 |   205 |   180 |  -110 |   -25 |    30 |    45 |   -50 |
|  490 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -920 |        920 |  -675 |  -690 |  -700 |  -680 |  -750 |  -705 |  -735 |  -920 |
|  491 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1170 |       1170 |  -970 |  -990 |  -905 |  -870 |  -815 |  -905 |  -850 | -1170 |
|  492 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        235 |        -50 |        285 |   235 |   -20 |    50 |   -50 |    30 |   -35 |    -0 |    -0 |
|  493 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -680 |        680 |  -420 |  -430 |  -485 |  -375 |  -440 |  -605 |  -680 |  -665 |
|  494 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        400 |          0 |        400 |   365 |   355 |   350 |   215 |   400 |   240 |   275 |    15 |
|  495 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         90 |       -110 |        200 |    90 |    35 |    30 |    20 |    55 |    65 |  -100 |  -110 |
|  496 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -490 |        490 |  -155 |  -195 |  -410 |  -490 |  -485 |  -400 |  -455 |  -430 |
|  497 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        385 |          0 |        385 |   275 |   385 |    80 |   150 |    70 |   100 |   100 |   180 |
|  498 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        395 |          0 |        395 |   295 |   395 |   305 |   380 |   180 |   190 |    85 |   130 |
|  499 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        295 |          0 |        295 |   225 |   165 |   295 |   195 |   225 |   225 |   245 |    80 |
|  500 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -380 |        380 |   -75 |   -65 |   -60 |  -215 |  -245 |  -300 |  -205 |  -380 |
|  501 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        395 |          0 |        395 |   390 |   395 |   195 |   185 |   200 |   165 |   195 |   185 |
|  502 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        240 |        -40 |        280 |   240 |    -5 |    30 |   -10 |   -40 |   170 |    60 |    30 |
|  503 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        340 |          0 |        340 |   330 |   240 |   200 |   260 |   340 |   275 |   130 |   285 |
|  504 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        270 |        -75 |        345 |   165 |   230 |   140 |   270 |   185 |   225 |   100 |   -75 |
|  505 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -830 |        830 |  -535 |  -585 |  -570 |  -540 |  -720 |  -765 |  -740 |  -830 |
|  506 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         10 |       -285 |        295 |   -60 |  -135 |    10 |   -50 |  -285 |  -205 |  -160 |  -210 |
|  507 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -525 |        525 |  -330 |  -385 |  -410 |  -320 |  -510 |  -525 |  -520 |  -490 |
|  508 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -870 |        870 |  -745 |  -650 |  -655 |  -870 |  -845 |  -770 |  -785 |  -860 |
|  509 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -860 |        860 |  -595 |  -585 |  -475 |  -550 |  -530 |  -515 |  -630 |  -860 |
|  510 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -765 |        765 |  -615 |  -715 |  -630 |  -600 |  -635 |  -610 |  -745 |  -765 |
|  511 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1015 |       1015 | -1015 |  -935 |  -820 |  -960 |  -940 |  -930 |  -980 |  -990 |
|  512 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -780 |        780 |  -560 |  -490 |  -475 |  -455 |  -500 |  -460 |  -445 |  -780 |
|  513 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        105 |          0 |        105 |    55 |    10 |    35 |    10 |    75 |    30 |    90 |   105 |
|  514 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -460 |        460 |  -255 |  -200 |  -225 |  -185 |  -250 |  -185 |  -460 |  -395 |
|  515 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1065 |       1065 |  -760 |  -950 |  -940 | -1065 |  -880 | -1005 | -1015 |  -950 |
|  516 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1145 |       1145 |  -870 |  -875 |  -900 | -1145 | -1055 | -1090 | -1050 | -1030 |
|  517 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        370 |          0 |        370 |   370 |   100 |   275 |   195 |   160 |   210 |   175 |   180 |
|  518 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -655 |        655 |  -605 |  -645 |  -655 |  -555 |  -645 |  -495 |  -535 |  -555 |
|  519 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        305 |         -0 |        305 |   175 |   305 |   235 |   285 |   225 |    -0 |     5 |    60 |
|  520 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -585 |        585 |  -295 |  -405 |  -545 |  -540 |  -500 |  -570 |  -515 |  -585 |
|  521 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -500 |        500 |  -170 |  -250 |  -115 |  -235 |  -455 |  -350 |  -405 |  -500 |
|  522 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        265 |          0 |        265 |   265 |    55 |   105 |   120 |   195 |    65 |   130 |   100 |
|  523 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -650 |        650 |  -640 |  -550 |  -555 |  -630 |  -600 |  -650 |  -555 |  -525 |
|  524 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         35 |       -205 |        240 |    35 |    30 |   -15 |  -205 |   -90 |  -185 |  -130 |  -195 |
|  525 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -175 |        175 |  -115 |   -75 |  -175 |  -135 |  -115 |  -140 |  -120 |   -75 |
|  526 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -650 |        650 |  -435 |  -335 |  -490 |  -305 |  -365 |  -650 |  -560 |  -640 |
|  527 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         20 |       -220 |        240 |    20 |  -135 |  -220 |  -175 |  -210 |  -150 |  -115 |  -170 |
|  528 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        155 |       -110 |        265 |   155 |   155 |   -55 |   -55 |   -75 |   -65 |    -5 |  -110 |
|  529 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -855 |        855 |  -575 |  -470 |  -510 |  -840 |  -770 |  -715 |  -730 |  -855 |
|  530 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        215 |          0 |        215 |   215 |   120 |    35 |   125 |    80 |   205 |   160 |   145 |
|  531 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        165 |        -35 |        200 |   135 |   165 |   125 |    80 |   100 |   -35 |   100 |    40 |
|  532 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        385 |          0 |        385 |   350 |   385 |   370 |   230 |   335 |   365 |   180 |   125 |
|  533 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -725 |        725 |  -595 |  -550 |  -555 |  -540 |  -500 |  -550 |  -530 |  -725 |
|  534 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -620 |        620 |  -430 |  -435 |  -505 |  -430 |  -415 |  -485 |  -455 |  -620 |
|  535 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -470 |        470 |  -285 |  -290 |  -310 |  -240 |  -325 |  -230 |  -265 |  -470 |
|  536 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1115 |       1115 |  -965 |  -930 |  -960 |  -935 |  -945 |  -895 |  -910 | -1115 |
|  537 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -790 |        790 |  -515 |  -530 |  -395 |  -555 |  -565 |  -740 |  -650 |  -790 |
|  538 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1000 |       1000 |  -935 |  -835 |  -865 |  -920 |  -905 |  -915 | -1000 |  -870 |
|  539 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -770 |        770 |  -570 |  -420 |  -585 |  -545 |  -590 |  -510 |  -770 |  -745 |
|  540 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        325 |          0 |        325 |   270 |   325 |   255 |   120 |   165 |   135 |   120 |   125 |
|  541 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        275 |        -40 |        315 |   185 |   250 |   265 |   275 |   240 |   120 |   -40 |   -25 |
|  542 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        210 |          0 |        210 |   210 |   200 |   200 |   135 |   170 |   185 |   140 |   145 |
|  543 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         35 |       -265 |        300 |   -35 |   -25 |    35 |    30 |    35 |  -205 |  -120 |  -265 |
|  544 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        310 |          0 |        310 |   300 |   220 |   310 |   115 |   260 |   280 |    45 |    30 |
|  545 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        135 |       -115 |        250 |    80 |    25 |    90 |   135 |  -115 |  -105 |   -95 |  -110 |
|  546 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -780 |        780 |  -615 |  -670 |  -620 |  -575 |  -605 |  -765 |  -780 |  -725 |
|  547 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        370 |          0 |        370 |   370 |   220 |   160 |   240 |   275 |   270 |   195 |   235 |
|  548 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        200 |       -110 |        310 |    25 |   200 |    40 |   145 |   130 |   105 |    65 |  -110 |
|  549 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        255 |          0 |        255 |   255 |   255 |   170 |   225 |   145 |    10 |    15 |   100 |
|  550 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -990 |        990 |  -690 |  -730 |  -835 |  -900 |  -905 |  -990 |  -815 |  -960 |
|  551 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -480 |        480 |  -210 |  -220 |  -285 |  -200 |  -480 |  -415 |  -420 |  -470 |
|  552 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -790 |        790 |  -640 |  -525 |  -590 |  -625 |  -790 |  -760 |  -720 |  -675 |
|  553 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        280 |          0 |        280 |   270 |   280 |   270 |   180 |    40 |    60 |   165 |    80 |
|  554 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -995 |        995 |  -695 |  -720 |  -870 |  -995 |  -960 |  -980 |  -850 |  -980 |
|  555 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -720 |        720 |  -655 |  -640 |  -570 |  -705 |  -590 |  -710 |  -720 |  -690 |
|  556 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1190 |       1190 | -1045 | -1010 |  -940 | -1185 | -1190 | -1135 | -1160 | -1150 |
|  557 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        380 |          0 |        380 |   380 |   380 |   350 |    85 |   285 |   215 |   215 |   150 |
|  558 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -410 |        410 |  -205 |  -220 |  -190 |  -280 |  -220 |  -270 |  -355 |  -410 |
|  559 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -865 |        865 |  -590 |  -610 |  -795 |  -865 |  -770 |  -795 |  -790 |  -760 |
|  560 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -975 |        975 |  -685 |  -670 |  -920 |  -940 |  -805 |  -855 |  -975 |  -850 |
|  561 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -555 |        555 |  -465 |  -455 |  -385 |  -345 |  -435 |  -450 |  -380 |  -555 |
|  562 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        395 |          0 |        395 |   315 |   260 |   265 |   320 |   325 |   395 |   240 |   155 |
|  563 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1145 |       1145 | -1045 |  -980 |  -930 |  -920 | -1120 | -1145 | -1135 | -1145 |
|  564 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -850 |        850 |  -520 |  -690 |  -685 |  -530 |  -555 |  -540 |  -840 |  -850 |
|  565 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        400 |          0 |        400 |   390 |   330 |   270 |   285 |   400 |   345 |    35 |    55 |
|  566 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        385 |          0 |        385 |   385 |   245 |   245 |   200 |   235 |   190 |    65 |    40 |
|  567 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -330 |        330 |   -55 |   -55 |   -10 |  -240 |  -330 |  -265 |  -300 |  -220 |
|  568 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        320 |          0 |        320 |   310 |   320 |   215 |    30 |    85 |    50 |   145 |   120 |
|  569 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        350 |        -40 |        390 |   350 |   305 |   305 |   125 |   105 |    90 |    15 |   -40 |
|  570 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        220 |        -80 |        300 |   220 |   -60 |   -35 |   -30 |   -75 |   -80 |   -60 |   -45 |
|  571 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -240 |        240 |   -40 |  -175 |  -235 |  -185 |  -240 |  -165 |  -205 |  -230 |
|  572 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        265 |        -95 |        360 |   230 |   205 |   165 |   265 |   -95 |   -40 |   -25 |   -15 |
|  573 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        205 |          0 |        205 |   120 |    30 |   115 |    75 |    20 |   205 |   100 |   135 |
|  574 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -245 |        245 |   -95 |   -85 |  -125 |   -45 |   -55 |  -145 |  -205 |  -245 |
|  575 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -490 |        490 |  -315 |  -345 |  -405 |  -270 |  -340 |  -365 |  -390 |  -490 |
|  576 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        305 |        -20 |        325 |   260 |   130 |   250 |   305 |   -20 |    30 |    90 |    35 |
|  577 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        125 |       -115 |        240 |   125 |    25 |   -70 |  -115 |   -65 |   -75 |   -75 |   -60 |
|  578 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         20 |       -215 |        235 |   -30 |    20 |   -20 |   -35 |   -20 |   -85 |   -35 |  -215 |
|  579 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        260 |       -140 |        400 |   260 |   150 |    35 |   -35 |    35 |  -140 |   -25 |    60 |
|  580 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        265 |          0 |        265 |   240 |   200 |   265 |   230 |   235 |    20 |    65 |    30 |
|  581 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         10 |       -250 |        260 |   -25 |   -40 |    10 |  -250 |   -95 |  -220 |  -170 |  -235 |
|  582 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1105 |       1105 |  -865 |  -890 |  -735 |  -915 |  -860 | -1010 | -1105 | -1035 |
|  583 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         25 |       -300 |        325 |     5 |   -15 |   -25 |    25 |     5 |  -215 |  -180 |  -300 |
|  584 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          5 |       -265 |        270 |   -90 |    -5 |   -35 |     5 |    -0 |  -265 |  -195 |  -225 |
|  585 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        350 |        -10 |        360 |   350 |   290 |   125 |   110 |    15 |    85 |   110 |   -10 |
|  586 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        245 |        -25 |        270 |   245 |   235 |   230 |   230 |   220 |   165 |   -25 |    95 |
|  587 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        310 |        -20 |        330 |   215 |   185 |   230 |   310 |   165 |   205 |   265 |   -20 |
|  588 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        365 |          0 |        365 |   365 |   330 |   315 |   250 |   270 |   240 |    90 |   145 |
|  589 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         60 |       -260 |        320 |    60 |   -10 |  -100 |   -65 |   -70 |    15 |  -260 |  -250 |
|  590 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         70 |       -225 |        295 |    70 |    45 |   -30 |    30 |    -0 |  -195 |  -175 |  -225 |
|  591 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1025 |       1025 |  -985 |  -890 |  -815 |  -915 |  -865 |  -925 |  -975 | -1025 |
|  592 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        110 |        -95 |        205 |    80 |   110 |   -95 |     5 |   -65 |   -45 |   -55 |   -20 |
|  593 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        385 |          0 |        385 |   320 |   385 |   340 |   285 |   385 |    95 |   165 |   125 |
|  594 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         25 |       -305 |        330 |   -40 |   -70 |    25 |    25 |    10 |  -165 |  -265 |  -305 |
|  595 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -680 |        680 |  -480 |  -680 |  -610 |  -600 |  -635 |  -625 |  -650 |  -610 |
|  596 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -305 |        305 |  -155 |   -25 |   -70 |   -55 |  -125 |  -235 |  -305 |  -305 |
|  597 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         85 |       -180 |        265 |    85 |   -75 |  -180 |  -150 |   -95 |  -150 |   -65 |   -90 |
|  598 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -800 |        800 |  -565 |  -675 |  -675 |  -580 |  -605 |  -680 |  -595 |  -800 |
|  599 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -695 |        695 |  -530 |  -410 |  -410 |  -510 |  -425 |  -685 |  -695 |  -635 |
|  600 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        205 |       -160 |        365 |   205 |   145 |   170 |    25 |  -160 |   -45 |    40 |   -30 |
|  601 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -685 |        685 |  -465 |  -535 |  -600 |  -655 |  -655 |  -600 |  -605 |  -685 |
|  602 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        195 |          0 |        195 |   130 |   155 |   105 |    60 |   135 |   130 |   140 |   195 |
|  603 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        290 |          0 |        290 |   275 |   225 |   205 |   290 |    30 |    30 |    85 |    10 |
|  604 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -205 |        205 |  -135 |  -205 |  -105 |   -45 |  -125 |  -105 |  -175 |  -160 |
|  605 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        375 |          0 |        375 |   375 |   250 |   275 |   185 |   235 |   220 |   205 |   310 |
|  606 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -300 |        300 |   -15 |   -90 |  -300 |  -180 |  -195 |  -215 |  -230 |  -155 |
|  607 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        340 |          0 |        340 |   245 |   290 |   340 |    70 |    10 |    85 |   130 |   150 |
|  608 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -895 |        895 |  -630 |  -620 |  -600 |  -565 |  -665 |  -665 |  -895 |  -805 |
|  609 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -800 |        800 |  -590 |  -575 |  -605 |  -580 |  -535 |  -660 |  -800 |  -750 |
|  610 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        265 |        -90 |        355 |   265 |   225 |   150 |   215 |   -90 |   -50 |    60 |   -10 |
|  611 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -355 |        355 |  -215 |  -220 |  -280 |  -275 |  -240 |  -150 |  -230 |  -355 |
|  612 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        205 |          0 |        205 |   105 |   195 |    70 |   205 |    80 |   175 |   130 |   135 |
|  613 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -595 |        595 |  -400 |  -385 |  -405 |  -320 |  -470 |  -375 |  -400 |  -595 |
|  614 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        155 |          0 |        155 |   155 |    65 |    95 |   125 |    65 |   110 |   100 |    40 |
|  615 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -310 |        310 |  -155 |  -100 |  -135 |  -105 |  -185 |   -60 |  -150 |  -310 |
|  616 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -280 |        280 |   -85 |   -70 |   -35 |  -280 |  -275 |  -240 |  -215 |  -270 |
|  617 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        205 |        -10 |        215 |   205 |    15 |     5 |    50 |    40 |    95 |   -10 |    15 |
|  618 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1085 |       1085 |  -860 |  -760 |  -830 |  -905 |  -870 |  -895 | -1085 |  -960 |
|  619 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1195 |       1195 | -1040 |  -920 |  -905 |  -900 |  -935 |  -960 | -1120 | -1195 |
|  620 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -525 |        525 |  -360 |  -245 |  -265 |  -480 |  -525 |  -500 |  -480 |  -475 |
|  621 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -890 |        890 |  -640 |  -610 |  -640 |  -610 |  -555 |  -560 |  -550 |  -890 |
|  622 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        280 |        -45 |        325 |   280 |    -5 |   -45 |    35 |   -30 |   -40 |    15 |     5 |
|  623 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        255 |        -20 |        275 |   255 |   -20 |   -10 |    60 |    90 |    70 |    35 |    20 |
|  624 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -900 |        900 |  -870 |  -805 |  -860 |  -900 |  -805 |  -860 |  -865 |  -790 |
|  625 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1115 |       1115 |  -855 |  -830 |  -830 |  -845 |  -745 |  -685 | -1020 | -1115 |
|  626 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -630 |        630 |  -405 |  -350 |  -370 |  -360 |  -275 |  -375 |  -630 |  -590 |
|  627 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        355 |          0 |        355 |   280 |   260 |   355 |   285 |   245 |   265 |    90 |   140 |
|  628 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -695 |        695 |  -495 |  -595 |  -450 |  -565 |  -610 |  -695 |  -480 |  -570 |
|  629 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         60 |         -5 |         65 |    60 |    -0 |     5 |    50 |    45 |    30 |    -5 |    55 |
|  630 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        240 |         -5 |        245 |   175 |   240 |   230 |    60 |     5 |    -5 |    45 |    60 |
|  631 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        275 |        -10 |        285 |    35 |   275 |   155 |   140 |   100 |   175 |    25 |   -10 |
|  632 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         70 |       -255 |        325 |    70 |    70 |  -240 |  -185 |  -255 |  -160 |  -145 |  -170 |
|  633 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -860 |        860 |  -670 |  -685 |  -775 |  -680 |  -685 |  -755 |  -715 |  -860 |
|  634 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        215 |        -90 |        305 |   145 |   215 |    20 |   -90 |    30 |   -40 |    10 |    35 |
|  635 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -995 |        995 |  -755 |  -995 |  -945 |  -965 |  -970 |  -835 |  -925 |  -925 |
|  636 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -660 |        660 |  -360 |  -550 |  -400 |  -425 |  -550 |  -415 |  -385 |  -660 |
|  637 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -425 |        425 |  -215 |  -160 |  -235 |  -425 |  -420 |  -420 |  -425 |  -400 |
|  638 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -445 |        445 |  -210 |  -330 |  -245 |  -130 |  -445 |  -435 |  -340 |  -405 |
|  639 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        180 |       -105 |        285 |   140 |   180 |    50 |   130 |  -105 |   -65 |   -45 |   -70 |
|  640 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -740 |        740 |  -520 |  -580 |  -510 |  -565 |  -490 |  -520 |  -545 |  -740 |
|  641 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        180 |       -155 |        335 |    45 |   180 |    50 |  -110 |  -125 |  -145 |  -125 |  -155 |
|  642 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        220 |       -140 |        360 |   115 |   170 |   205 |   215 |   220 |   205 |   165 |  -140 |
|  643 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -780 |        780 |  -500 |  -700 |  -645 |  -660 |  -780 |  -670 |  -690 |  -725 |
|  644 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        330 |          0 |        330 |   330 |   205 |   215 |   210 |   145 |    70 |   160 |    75 |
|  645 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        255 |          0 |        255 |   210 |   210 |   255 |   125 |   195 |   255 |    10 |    55 |
|  646 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -860 |        860 |  -550 |  -595 |  -675 |  -535 |  -555 |  -620 |  -650 |  -860 |
|  647 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -690 |        690 |  -355 |  -430 |  -355 |  -430 |  -400 |  -470 |  -630 |  -690 |
|  648 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        275 |         -0 |        275 |   275 |    10 |    75 |   105 |    80 |    -0 |   155 |    60 |
|  649 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1160 |       1160 |  -960 |  -925 |  -905 |  -895 |  -865 |  -795 |  -855 | -1160 |
|  650 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        365 |        -45 |        410 |   365 |   300 |   260 |   240 |   290 |   275 |    95 |   -45 |
|  651 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -545 |        545 |  -210 |  -470 |  -440 |  -545 |  -495 |  -535 |  -440 |  -465 |
|  652 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        165 |        -35 |        200 |    40 |   165 |    90 |    95 |    30 |   -35 |   -20 |    40 |
|  653 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         70 |       -235 |        305 |    70 |    70 |  -100 |  -235 |  -155 |  -205 |  -175 |  -120 |
|  654 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -845 |        845 |  -605 |  -595 |  -680 |  -650 |  -740 |  -760 |  -755 |  -845 |
|  655 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        300 |          0 |        300 |   300 |   215 |    50 |    30 |    75 |   110 |   115 |    70 |
|  656 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -925 |        925 |  -770 |  -720 |  -775 |  -705 |  -715 |  -925 |  -915 |  -855 |
|  657 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        320 |          0 |        320 |   320 |   295 |   240 |   180 |   250 |   305 |    35 |    55 |
|  658 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        205 |          0 |        205 |   205 |   100 |   190 |   130 |   135 |   105 |   120 |    75 |
|  659 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -745 |        745 |  -560 |  -540 |  -580 |  -565 |  -530 |  -610 |  -545 |  -745 |
|  660 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -445 |        445 |  -270 |  -290 |  -240 |  -290 |  -125 |  -280 |  -255 |  -445 |
|  661 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        100 |        -50 |        150 |    60 |   -15 |   -50 |    35 |    35 |    10 |   100 |    15 |
|  662 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         20 |        -95 |        115 |   -85 |    -5 |    20 |   -15 |   -90 |   -65 |   -95 |   -75 |
|  663 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        360 |          0 |        360 |   265 |   180 |   360 |    25 |    75 |    85 |    70 |   120 |
|  664 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         90 |        -10 |        100 |   -10 |    15 |    40 |    90 |    25 |    90 |    40 |    60 |
|  665 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        375 |          0 |        375 |   355 |   275 |   250 |   285 |   330 |   375 |   150 |    85 |
|  666 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -480 |        480 |  -220 |  -220 |  -370 |  -185 |  -320 |  -275 |  -300 |  -480 |
|  667 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -990 |        990 |  -795 |  -680 |  -725 |  -785 |  -775 |  -720 |  -940 |  -990 |
|  668 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         50 |       -205 |        255 |    15 |    -0 |    50 |  -180 |  -180 |  -125 |  -175 |  -205 |
|  669 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        130 |       -110 |        240 |   120 |   130 |  -110 |   -20 |     5 |   -75 |    25 |    -0 |
|  670 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        180 |        -65 |        245 |   125 |   115 |   175 |   180 |   180 |    -5 |   -55 |   -65 |
|  671 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1195 |       1195 |  -995 | -1005 | -1020 | -1010 |  -970 | -1005 | -1175 | -1195 |
|  672 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        390 |          0 |        390 |   285 |   390 |   150 |   125 |   150 |   280 |   210 |    85 |
|  673 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -950 |        950 |  -885 |  -835 |  -950 |  -755 |  -735 |  -805 |  -820 |  -820 |
|  674 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -845 |        845 |  -730 |  -585 |  -575 |  -795 |  -820 |  -790 |  -845 |  -810 |
|  675 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1195 |       1195 |  -915 |  -900 |  -965 |  -890 |  -925 |  -965 | -1195 | -1160 |
|  676 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -735 |        735 |  -450 |  -460 |  -535 |  -415 |  -330 |  -735 |  -610 |  -680 |
|  677 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        385 |          0 |        385 |   385 |   210 |   195 |   125 |   215 |   220 |   170 |   175 |
|  678 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        295 |        -45 |        340 |   130 |   260 |   275 |   200 |   295 |    70 |   -45 |    -5 |
|  679 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -670 |        670 |  -430 |  -545 |  -420 |  -455 |  -375 |  -415 |  -375 |  -670 |
|  680 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -420 |        420 |   -90 |  -170 |  -270 |  -410 |  -400 |  -420 |  -355 |  -420 |
|  681 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        365 |          0 |        365 |   225 |   365 |    80 |   170 |   130 |   230 |    90 |   125 |
|  682 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -310 |        310 |   -95 |  -290 |  -270 |  -310 |  -300 |  -310 |  -230 |  -220 |
|  683 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         55 |       -205 |        260 |    -0 |    -0 |    30 |   -45 |    15 |    55 |   -65 |  -205 |
|  684 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -840 |        840 |  -655 |  -620 |  -595 |  -815 |  -775 |  -840 |  -820 |  -765 |
|  685 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1135 |       1135 |  -880 |  -920 |  -955 | -1055 | -1075 | -1040 | -1135 | -1125 |
|  686 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        355 |          0 |        355 |   335 |   355 |   300 |   330 |   305 |   255 |   190 |   110 |
|  687 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        370 |          0 |        370 |   340 |   260 |   270 |   165 |   275 |   370 |   305 |   255 |
|  688 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        315 |          0 |        315 |   195 |   315 |   235 |   250 |    95 |   155 |    45 |    75 |
|  689 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1000 |       1000 |  -765 |  -570 |  -695 |  -715 |  -730 |  -890 | -1000 |  -930 |
|  690 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        355 |          0 |        355 |   300 |   250 |   355 |   340 |   135 |    80 |   185 |   105 |
|  691 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -600 |        600 |  -295 |  -350 |  -385 |  -325 |  -405 |  -330 |  -410 |  -600 |
|  692 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        265 |        -85 |        350 |   265 |   135 |   135 |   -45 |    60 |    10 |   -85 |   -70 |
|  693 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         70 |       -270 |        340 |    40 |    45 |     5 |   -35 |    45 |    70 |   -90 |  -270 |
|  694 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -980 |        980 |  -815 |  -855 |  -835 |  -780 |  -860 |  -825 |  -980 |  -940 |
|  695 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        390 |          0 |        390 |   355 |   390 |   380 |   330 |   285 |   165 |   200 |   215 |
|  696 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -585 |        585 |  -390 |  -565 |  -500 |  -480 |  -520 |  -525 |  -470 |  -585 |
|  697 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -525 |        525 |  -180 |  -330 |  -260 |  -335 |  -260 |  -210 |  -505 |  -525 |
|  698 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        350 |          0 |        350 |   330 |   345 |   260 |   350 |   295 |   315 |   315 |   260 |
|  699 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1090 |       1090 |  -795 | -1090 |  -990 |  -925 |  -980 |  -975 |  -940 | -1080 |
|  700 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        130 |       -130 |        260 |    65 |   130 |   -40 |   -55 |   -80 |  -100 |  -130 |  -120 |
|  701 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -675 |        675 |  -450 |  -400 |  -375 |  -330 |  -405 |  -535 |  -585 |  -675 |
|  702 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        290 |          0 |        290 |   290 |   110 |   160 |   215 |   220 |    90 |   260 |   130 |
|  703 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -830 |        830 |  -600 |  -670 |  -830 |  -825 |  -730 |  -730 |  -810 |  -795 |
|  704 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        330 |          0 |        330 |   265 |   330 |   265 |   220 |   115 |   100 |    40 |   120 |
|  705 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        225 |       -220 |        445 |   225 |    55 |    -0 |  -115 |   -85 |   -70 |   -75 |  -220 |
|  706 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         25 |       -180 |        205 |   -35 |  -105 |    25 |  -180 |  -125 |  -160 |   -15 |  -100 |
|  707 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        265 |          0 |        265 |   180 |   175 |   175 |   150 |   180 |   265 |    60 |   110 |
|  708 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1085 |       1085 | -1040 | -1085 | -1045 |  -945 | -1060 |  -975 | -1030 | -1065 |
|  709 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -850 |        850 |  -530 |  -850 |  -815 |  -790 |  -750 |  -810 |  -730 |  -710 |
|  710 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -880 |        880 |  -575 |  -590 |  -570 |  -635 |  -880 |  -830 |  -780 |  -805 |
|  711 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -770 |        770 |  -515 |  -475 |  -655 |  -730 |  -630 |  -770 |  -750 |  -680 |
|  712 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -370 |        370 |  -220 |  -130 |  -290 |  -255 |  -110 |  -135 |  -325 |  -370 |
|  713 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1030 |       1030 |  -725 |  -710 |  -695 |  -895 |  -950 |  -965 | -1030 |  -935 |
|  714 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -305 |        305 |   -70 |   -65 |  -110 |   -75 |   -50 |  -305 |  -210 |  -250 |
|  715 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         75 |       -145 |        220 |    75 |    50 |    60 |   -50 |  -110 |  -135 |   -90 |  -145 |
|  716 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        215 |       -150 |        365 |   215 |   100 |   125 |  -115 |   -20 |   -65 |  -130 |  -150 |
|  717 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        290 |        -35 |        325 |   255 |   250 |   290 |    45 |    40 |   -35 |    85 |    55 |
|  718 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        230 |        -95 |        325 |   160 |   185 |   145 |   230 |    35 |   -35 |   -95 |   -35 |
|  719 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         10 |       -230 |        240 |    10 |   -30 |  -230 |   -65 |  -135 |  -110 |  -145 |  -185 |
|  720 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -440 |        440 |  -300 |  -300 |  -325 |  -320 |  -345 |  -250 |  -335 |  -440 |
|  721 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        195 |       -125 |        320 |    75 |   100 |   155 |   195 |    75 |   125 |  -100 |  -125 |
|  722 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -530 |        530 |  -215 |  -265 |  -195 |  -220 |  -530 |  -375 |  -410 |  -415 |
|  723 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        100 |        -40 |        140 |    70 |   -40 |    55 |   100 |    95 |    30 |    60 |    35 |
|  724 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -645 |        645 |  -495 |  -410 |  -460 |  -415 |  -570 |  -620 |  -640 |  -645 |
|  725 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        150 |       -150 |        300 |   150 |  -110 |   -35 |  -130 |    55 |    15 |   -70 |  -150 |
|  726 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -745 |        745 |  -580 |  -660 |  -650 |  -555 |  -695 |  -705 |  -615 |  -745 |
|  727 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -620 |        620 |  -455 |  -420 |  -490 |  -455 |  -360 |  -445 |  -465 |  -620 |
|  728 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -980 |        980 |  -765 |  -795 |  -720 |  -675 |  -760 |  -750 |  -680 |  -980 |
|  729 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -205 |        205 |   -90 |  -160 |  -170 |  -145 |  -140 |  -140 |  -205 |  -110 |
|  730 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -810 |        810 |  -675 |  -665 |  -710 |  -720 |  -670 |  -810 |  -685 |  -740 |
|  731 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        220 |       -110 |        330 |   220 |   100 |   200 |   140 |    95 |   100 |  -110 |  -105 |
|  732 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         90 |       -125 |        215 |   -80 |    40 |    55 |   -35 |    90 |    -5 |   -95 |  -125 |
|  733 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -825 |        825 |  -540 |  -535 |  -755 |  -705 |  -765 |  -720 |  -825 |  -690 |
|  734 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1165 |       1165 |  -935 |  -885 |  -890 |  -900 |  -900 |  -890 | -1035 | -1165 |
|  735 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 |  -805 |  -900 |  -795 |  -880 |  -930 |  -850 |  -865 | -1080 |
|  736 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        220 |         -0 |        220 |   175 |   220 |    35 |    35 |    -0 |    60 |    75 |    60 |
|  737 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        370 |          0 |        370 |   330 |   370 |   185 |   195 |   225 |   215 |   290 |   180 |
|  738 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -575 |        575 |  -345 |  -385 |  -430 |  -335 |  -490 |  -490 |  -410 |  -575 |
|  739 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        395 |          0 |        395 |   385 |   395 |   250 |   360 |   120 |   200 |   240 |   175 |
|  740 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        330 |        -10 |        340 |   330 |    40 |    25 |   -10 |   125 |    75 |    90 |    90 |
|  741 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        280 |          0 |        280 |    65 |   165 |   125 |   115 |   140 |   280 |   215 |   185 |
|  742 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -865 |        865 |  -505 |  -540 |  -555 |  -495 |  -865 |  -675 |  -740 |  -695 |
|  743 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        215 |       -110 |        325 |    95 |   160 |    95 |   215 |   135 |   -50 |  -110 |   -65 |
|  744 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -790 |        790 |  -550 |  -530 |  -600 |  -560 |  -605 |  -555 |  -760 |  -790 |
|  745 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        155 |          0 |        155 |   140 |   110 |   155 |    35 |   100 |   130 |   150 |   120 |
|  746 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        245 |        -10 |        255 |   160 |   245 |   175 |   240 |   230 |   -10 |    30 |    30 |
|  747 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -755 |        755 |  -410 |  -425 |  -415 |  -515 |  -530 |  -655 |  -755 |  -655 |
|  748 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1160 |       1160 |  -925 |  -820 |  -885 |  -835 |  -815 | -1025 | -1160 | -1125 |
|  749 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -420 |        420 |  -315 |  -350 |  -420 |  -395 |  -295 |  -300 |  -405 |  -360 |
|  750 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         25 |       -195 |        220 |    20 |    25 |   -70 |   -35 |   -10 |   -80 |  -195 |  -145 |
|  751 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        215 |        -45 |        260 |    45 |    75 |   105 |   100 |   155 |   135 |   215 |   -45 |
|  752 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        180 |        -45 |        225 |   110 |    95 |    10 |   115 |   165 |   180 |   175 |   -45 |
|  753 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        370 |          0 |        370 |   270 |   280 |   360 |   285 |   370 |   310 |   100 |    85 |
|  754 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        190 |        -80 |        270 |   190 |   115 |   145 |   165 |   -50 |   -75 |   -80 |   -60 |
|  755 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -755 |        755 |  -550 |  -525 |  -410 |  -740 |  -720 |  -755 |  -755 |  -720 |
|  756 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        325 |          0 |        325 |   325 |   270 |   320 |    30 |   155 |    30 |   125 |   160 |
|  757 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -885 |        885 |  -455 |  -835 |  -810 |  -885 |  -670 |  -760 |  -715 |  -775 |
|  758 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        325 |          0 |        325 |   270 |   240 |   325 |   305 |   200 |   205 |   245 |   125 |
|  759 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         35 |       -110 |        145 |   -10 |   -75 |   -45 |   -30 |    35 |   -80 |    25 |  -110 |
|  760 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        365 |          0 |        365 |   275 |   305 |   285 |   325 |   365 |   210 |   300 |   155 |
|  761 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        165 |       -145 |        310 |   165 |   -40 |  -145 |   -40 |   -95 |   -55 |   -45 |   -25 |
|  762 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1005 |       1005 |  -930 |  -805 |  -775 |  -785 |  -850 |  -860 |  -905 | -1005 |
|  763 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        235 |          0 |        235 |   200 |   130 |   155 |   175 |   145 |   230 |   235 |   175 |
|  764 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        220 |        -20 |        240 |   215 |   220 |   175 |   155 |    45 |    55 |    20 |   -20 |
|  765 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1085 |       1085 |  -810 | -1025 | -1085 |  -965 | -1025 | -1040 |  -955 | -1015 |
|  766 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        330 |          0 |        330 |   300 |   205 |   330 |   270 |   250 |   255 |   315 |   295 |
|  767 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        370 |          0 |        370 |   235 |   335 |   370 |   240 |   300 |   335 |    75 |   170 |
|  768 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        285 |         -0 |        285 |   230 |   245 |   265 |   230 |   285 |   285 |    -0 |    95 |
|  769 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1170 |       1170 |  -885 | -1090 | -1100 | -1010 | -1110 | -1075 | -1025 | -1170 |
|  770 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        110 |       -200 |        310 |    80 |   110 |    65 |    40 |    30 |   -10 |  -200 |   -65 |
|  771 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -645 |        645 |  -625 |  -620 |  -560 |  -585 |  -590 |  -570 |  -585 |  -645 |
|  772 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         55 |       -305 |        360 |    15 |    55 |  -200 |  -305 |  -235 |  -225 |  -200 |  -290 |
|  773 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -640 |        640 |  -405 |  -435 |  -575 |  -640 |  -560 |  -550 |  -465 |  -615 |
|  774 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -925 |        925 |  -780 |  -875 |  -835 |  -860 |  -835 |  -790 |  -820 |  -925 |
|  775 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -355 |        355 |  -225 |  -355 |  -340 |  -350 |  -330 |  -285 |  -320 |  -310 |
|  776 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -610 |        610 |  -395 |  -565 |  -550 |  -475 |  -560 |  -610 |  -430 |  -575 |
|  777 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         45 |       -210 |        255 |    -0 |    20 |    45 |    30 |  -210 |  -120 |  -180 |  -140 |
|  778 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -500 |        500 |  -210 |  -320 |  -285 |  -360 |  -425 |  -500 |  -460 |  -390 |
|  779 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1040 |       1040 | -1040 |  -970 |  -980 |  -940 |  -980 |  -955 |  -960 |  -970 |
|  780 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        205 |       -125 |        330 |   150 |    95 |   145 |   205 |   115 |    80 |   -30 |  -125 |
|  781 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        250 |        -70 |        320 |   250 |   135 |    50 |   -10 |    15 |   -70 |    -5 |   -15 |
|  782 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         15 |       -205 |        220 |    15 |   -90 |   -30 |  -140 |  -195 |  -195 |  -205 |  -145 |
|  783 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -650 |        650 |  -470 |  -430 |  -485 |  -450 |  -485 |  -570 |  -650 |  -570 |
|  784 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -515 |        515 |  -230 |  -265 |  -425 |  -410 |  -275 |  -515 |  -435 |  -510 |
|  785 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        220 |          0 |        220 |   150 |   125 |   165 |   220 |   155 |   220 |   140 |    60 |
|  786 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        340 |          0 |        340 |   210 |   340 |   230 |   325 |   320 |   240 |   170 |   120 |
|  787 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        320 |          0 |        320 |   320 |   265 |   230 |   190 |    60 |    10 |    55 |    50 |
|  788 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -740 |        740 |  -385 |  -710 |  -740 |  -625 |  -565 |  -630 |  -680 |  -700 |
|  789 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        345 |          0 |        345 |   345 |   225 |    50 |   135 |    35 |   110 |    95 |    70 |
|  790 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -760 |        760 |  -425 |  -730 |  -655 |  -655 |  -760 |  -720 |  -705 |  -695 |
|  791 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         15 |       -285 |        300 |    10 |   -20 |    15 |    -5 |    -0 |  -285 |  -240 |  -155 |
|  792 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -730 |        730 |  -555 |  -470 |  -440 |  -400 |  -560 |  -585 |  -685 |  -730 |
|  793 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -735 |        735 |  -495 |  -420 |  -480 |  -375 |  -480 |  -465 |  -735 |  -640 |
|  794 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        165 |       -125 |        290 |   105 |   145 |   165 |   105 |    35 |   130 |  -125 |  -100 |
|  795 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1075 |       1075 |  -700 |  -775 |  -710 | -1075 |  -905 |  -940 |  -970 |  -920 |
|  796 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -520 |        520 |  -380 |  -465 |  -505 |  -495 |  -425 |  -490 |  -520 |  -485 |
|  797 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        280 |          0 |        280 |   250 |   185 |   245 |   195 |   180 |   275 |   280 |   255 |
|  798 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        400 |          0 |        400 |   400 |   265 |   275 |   265 |   260 |   290 |   125 |   215 |
|  799 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -495 |        495 |  -400 |  -325 |  -410 |  -415 |  -220 |  -435 |  -345 |  -495 |
|  800 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        315 |          0 |        315 |   280 |   195 |   250 |   315 |   310 |   255 |   240 |    95 |
|  801 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        255 |        -85 |        340 |   255 |   135 |   165 |   -35 |   -75 |    -5 |    15 |   -85 |
|  802 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        120 |          0 |        120 |    80 |   120 |    65 |    15 |   110 |     5 |    70 |    30 |
|  803 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1185 |       1185 |  -850 |  -975 |  -980 |  -815 |  -885 | -1040 | -1160 | -1185 |
|  804 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        395 |          0 |        395 |   390 |   330 |   395 |   310 |   120 |   205 |   150 |   100 |
|  805 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -770 |        770 |  -575 |  -530 |  -460 |  -545 |  -440 |  -770 |  -685 |  -640 |
|  806 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1180 |       1180 |  -985 | -1100 |  -925 |  -980 | -1010 |  -930 |  -930 | -1180 |
|  807 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -755 |        755 |  -745 |  -650 |  -755 |  -705 |  -705 |  -700 |  -650 |  -680 |
|  808 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -880 |        880 |  -645 |  -705 |  -710 |  -610 |  -835 |  -785 |  -800 |  -880 |
|  809 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        370 |          0 |        370 |   370 |   235 |   185 |   140 |   155 |   190 |   115 |   185 |
|  810 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         20 |       -305 |        325 |   -60 |   -10 |   -55 |    20 |  -100 |   -70 |   -75 |  -305 |
|  811 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -540 |        540 |  -370 |  -265 |  -350 |  -315 |  -335 |  -460 |  -540 |  -490 |
|  812 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        130 |        -10 |        140 |   130 |   -10 |    60 |    -0 |    -5 |    35 |     5 |   -10 |
|  813 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        360 |        -35 |        395 |   285 |   220 |   310 |   260 |   360 |    35 |   -35 |   -35 |
|  814 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        235 |        -55 |        290 |   175 |   225 |   235 |   -15 |   -10 |   -40 |   -20 |   -55 |
|  815 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        290 |          0 |        290 |   245 |   290 |    70 |   135 |   150 |   120 |   180 |   125 |
|  816 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         80 |        -55 |        135 |    30 |    65 |   -30 |    80 |    60 |    20 |   -25 |   -55 |
|  817 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        355 |          0 |        355 |   230 |   210 |   355 |   275 |   350 |   285 |    45 |    60 |
|  818 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1015 |       1015 |  -740 |  -765 |  -725 |  -990 |  -950 |  -960 |  -795 | -1015 |
|  819 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        255 |          0 |        255 |   170 |   135 |   180 |   215 |   165 |   255 |   185 |    65 |
|  820 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        235 |       -115 |        350 |   235 |   120 |    25 |    50 |  -115 |    10 |   -60 |   -20 |
|  821 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         30 |       -310 |        340 |     5 |    30 |   -50 |   -45 |   -40 |  -310 |  -270 |  -190 |
|  822 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        370 |          0 |        370 |   265 |   325 |   315 |   275 |   370 |   180 |   185 |    20 |
|  823 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        360 |          0 |        360 |   360 |   235 |   360 |   125 |   145 |   285 |   145 |   185 |
|  824 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -665 |        665 |  -455 |  -450 |  -645 |  -660 |  -665 |  -650 |  -655 |  -615 |
|  825 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        185 |       -140 |        325 |   185 |   150 |    35 |   130 |   140 |    55 |    90 |  -140 |
|  826 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        315 |          0 |        315 |   315 |   225 |   150 |    70 |    25 |    80 |    10 |     5 |
|  827 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |        -35 |        260 |   200 |   135 |   225 |   195 |    40 |   -15 |   -10 |   -35 |
|  828 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        400 |          0 |        400 |   345 |   400 |   280 |   385 |   335 |    65 |   145 |   135 |
|  829 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -650 |        650 |  -455 |  -490 |  -380 |  -550 |  -485 |  -615 |  -620 |  -650 |
|  830 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        265 |       -105 |        370 |   265 |   170 |   -20 |    50 |   -45 |   -10 |  -105 |    75 |
|  831 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        320 |        -40 |        360 |   165 |   165 |   320 |   140 |   250 |   -40 |    -0 |    10 |
|  832 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        235 |       -105 |        340 |   235 |   195 |   215 |   110 |   150 |   205 |  -105 |   -60 |
|  833 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        190 |       -130 |        320 |   185 |   130 |   190 |   105 |   105 |   135 |   150 |  -130 |
|  834 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        150 |          0 |        150 |    65 |    10 |   150 |    80 |    70 |    85 |    10 |    10 |
|  835 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        225 |        -85 |        310 |   225 |   170 |    60 |    -5 |     5 |   -15 |   -85 |   -45 |
|  836 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1185 |       1185 |  -990 | -1105 | -1105 | -1180 | -1125 | -1170 | -1135 | -1185 |
|  837 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        360 |          0 |        360 |   360 |   355 |   280 |   325 |   340 |   350 |   155 |   100 |
|  838 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        155 |       -185 |        340 |   100 |    40 |    45 |    50 |   -15 |     5 |   155 |  -185 |
|  839 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        390 |          0 |        390 |   390 |   195 |   145 |   295 |   160 |   275 |   145 |   170 |
|  840 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        285 |          0 |        285 |   235 |   205 |   190 |   285 |    10 |   110 |    45 |   185 |
|  841 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1025 |       1025 |  -785 |  -900 |  -900 |  -890 |  -795 | -1025 |  -960 |  -990 |
|  842 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        280 |        -45 |        325 |   175 |   170 |   205 |   215 |    90 |   280 |   240 |   -45 |
|  843 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        360 |          0 |        360 |   215 |   250 |   210 |   360 |   215 |   290 |   235 |   220 |
|  844 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -500 |        500 |  -235 |  -325 |  -305 |  -300 |  -345 |  -330 |  -330 |  -500 |
|  845 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        245 |        -10 |        255 |   175 |   190 |   245 |   225 |   155 |   185 |    15 |   -10 |
|  846 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -970 |        970 |  -765 |  -840 |  -870 |  -925 |  -755 |  -695 |  -970 |  -965 |
|  847 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -725 |        725 |  -600 |  -630 |  -490 |  -620 |  -725 |  -640 |  -635 |  -670 |
|  848 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        115 |       -160 |        275 |   100 |    95 |   110 |   115 |    60 |  -110 |  -130 |  -160 |
|  849 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 |  -905 |  -880 |  -955 |  -935 |  -955 | -1025 | -1000 | -1080 |
|  850 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        315 |          0 |        315 |   180 |   265 |   315 |   120 |   305 |   305 |   215 |   310 |
|  851 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -300 |        300 |   -10 |  -100 |  -145 |   -10 |  -300 |  -255 |  -260 |  -295 |
|  852 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        210 |        -85 |        295 |   190 |   200 |   165 |   180 |   185 |   210 |   -85 |     5 |
|  853 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -840 |        840 |  -810 |  -820 |  -670 |  -800 |  -760 |  -805 |  -840 |  -800 |
|  854 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -865 |        865 |  -590 |  -665 |  -635 |  -750 |  -775 |  -865 |  -840 |  -815 |
|  855 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        275 |        -45 |        320 |   210 |   210 |   180 |   130 |   150 |   170 |   275 |   -45 |
|  856 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -180 |        180 |  -170 |   -80 |   -35 |  -175 |  -180 |   -55 |  -155 |  -175 |
|  857 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1085 |       1085 |  -720 |  -870 | -1085 |  -855 |  -965 |  -915 | -1040 |  -965 |
|  858 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -745 |        745 |  -610 |  -640 |  -575 |  -570 |  -535 |  -545 |  -745 |  -710 |
|  859 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        400 |          0 |        400 |   390 |   355 |   400 |   285 |   200 |    60 |   175 |   150 |
|  860 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        385 |          0 |        385 |   385 |   255 |   255 |   330 |   275 |   355 |   225 |   335 |
|  861 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        350 |          0 |        350 |   345 |   350 |   325 |   155 |   155 |    90 |   200 |   110 |
|  862 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        390 |          0 |        390 |   275 |   390 |   245 |   190 |   180 |   135 |    90 |   135 |
|  863 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1130 |       1130 |  -880 |  -915 | -1115 | -1085 | -1070 | -1130 | -1065 | -1070 |
|  864 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -615 |        615 |  -380 |  -345 |  -485 |  -550 |  -555 |  -615 |  -505 |  -515 |
|  865 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -505 |        505 |  -215 |  -350 |  -295 |  -280 |  -275 |  -260 |  -300 |  -505 |
|  866 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -430 |        430 |  -265 |  -195 |  -180 |  -195 |  -185 |  -215 |  -330 |  -430 |
|  867 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        270 |        -45 |        315 |   255 |   175 |   195 |   220 |   165 |   270 |    60 |   -45 |
|  868 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1165 |       1165 | -1035 | -1070 | -1135 | -1050 | -1085 | -1070 | -1095 | -1165 |
|  869 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -235 |        235 |   -75 |  -140 |  -140 |   -80 |   -75 |   -45 |  -220 |  -235 |
|  870 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        215 |        -80 |        295 |    80 |   105 |   165 |   215 |    35 |    90 |    65 |   -80 |
|  871 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -850 |        850 |  -720 |  -660 |  -735 |  -790 |  -850 |  -820 |  -790 |  -735 |
|  872 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -390 |        390 |  -155 |  -175 |  -125 |  -100 |  -320 |  -390 |  -320 |  -345 |
|  873 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -515 |        515 |  -335 |  -335 |  -265 |  -260 |  -500 |  -420 |  -515 |  -455 |
|  874 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        335 |          0 |        335 |   335 |   115 |   150 |    45 |   135 |   150 |   115 |   120 |
|  875 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        125 |       -160 |        285 |   120 |   125 |  -125 |   -80 |   -65 |  -100 |   -40 |  -160 |
|  876 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         90 |        -55 |        145 |    90 |    85 |    25 |   -40 |    80 |    35 |    40 |   -55 |
|  877 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -425 |        425 |   -75 |   -20 |  -255 |  -250 |  -220 |  -175 |  -425 |  -295 |
|  878 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        265 |        -75 |        340 |   265 |   255 |    25 |    25 |   -75 |    10 |    35 |    55 |
|  879 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -760 |        760 |  -665 |  -635 |  -730 |  -655 |  -670 |  -760 |  -725 |  -715 |
|  880 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        245 |          0 |        245 |   140 |   155 |   150 |    90 |   100 |    75 |   245 |    40 |
|  881 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -350 |        350 |   -90 |  -350 |  -300 |  -205 |  -275 |  -270 |  -260 |  -240 |
|  882 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -780 |        780 |  -575 |  -565 |  -465 |  -525 |  -620 |  -695 |  -680 |  -780 |
|  883 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1155 |       1155 |  -935 |  -970 |  -925 |  -925 | -1040 | -1075 | -1010 | -1155 |
|  884 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -285 |        285 |   -50 |   -30 |  -115 |   -15 |  -145 |   -75 |  -285 |  -285 |
|  885 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        340 |          0 |        340 |   320 |   340 |    90 |   145 |   180 |   210 |   170 |   155 |
|  886 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -415 |        415 |  -205 |  -195 |  -240 |  -190 |  -115 |  -220 |  -185 |  -415 |
|  887 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1055 |       1055 |  -920 |  -745 |  -865 |  -705 |  -815 |  -790 |  -990 | -1055 |
|  888 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -725 |        725 |  -695 |  -720 |  -640 |  -720 |  -640 |  -715 |  -725 |  -665 |
|  889 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |        -20 |        280 |   260 |   215 |   235 |   -20 |   -15 |    50 |    15 |    55 |
|  890 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        195 |        -90 |        285 |   165 |   105 |   190 |   195 |   115 |   -90 |   -40 |    -5 |
|  891 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        350 |        -10 |        360 |   350 |    10 |   -10 |    30 |    60 |    60 |    60 |    50 |
|  892 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -840 |        840 |  -665 |  -720 |  -645 |  -705 |  -650 |  -840 |  -830 |  -765 |
|  893 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        240 |          0 |        240 |   240 |   195 |   160 |   190 |   190 |   205 |   200 |    20 |
|  894 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        310 |          0 |        310 |   290 |   255 |   275 |   310 |   250 |    80 |   160 |    40 |
|  895 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -865 |        865 |  -640 |  -695 |  -835 |  -765 |  -865 |  -815 |  -800 |  -770 |
|  896 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1020 |       1020 |  -935 |  -865 |  -785 |  -850 |  -885 |  -725 | -1020 |  -985 |
|  897 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         90 |       -170 |        260 |    90 |    20 |    -0 |   -40 |  -170 |  -120 |  -160 |  -130 |
|  898 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        100 |       -165 |        265 |   100 |  -105 |  -165 |  -160 |  -130 |   -65 |   -20 |  -120 |
|  899 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -930 |        930 |  -770 |  -815 |  -795 |  -800 |  -685 |  -775 |  -930 |  -845 |
|  900 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        335 |          0 |        335 |   335 |   125 |   225 |   130 |   205 |   230 |   230 |   160 |
|  901 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        240 |        -95 |        335 |   240 |   210 |   130 |   -25 |   -15 |   -95 |   -15 |    -5 |
|  902 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1050 |       1050 |  -870 |  -845 | -1005 | -1050 | -1010 |  -995 | -1000 |  -955 |
|  903 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        320 |          0 |        320 |   320 |   165 |   280 |   165 |   160 |   290 |   215 |   110 |
|  904 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        250 |        -45 |        295 |   185 |   200 |   215 |   200 |   250 |   -15 |    15 |   -45 |
|  905 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1160 |       1160 | -1125 | -1095 | -1090 | -1110 | -1090 | -1035 | -1160 | -1140 |
|  906 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -340 |        340 |  -295 |  -295 |  -260 |  -255 |  -220 |  -340 |  -290 |  -330 |
|  907 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        385 |          0 |        385 |   385 |   230 |   235 |   310 |   345 |   110 |   160 |   115 |
|  908 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -860 |        860 |  -635 |  -515 |  -605 |  -520 |  -600 |  -825 |  -860 |  -760 |
|  909 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -740 |        740 |  -445 |  -580 |  -540 |  -425 |  -740 |  -690 |  -700 |  -650 |
|  910 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        205 |       -115 |        320 |   165 |   205 |    -5 |    40 |  -115 |   -40 |   -95 |   -65 |
|  911 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -645 |        645 |  -565 |  -410 |  -515 |  -520 |  -475 |  -505 |  -555 |  -645 |
|  912 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         60 |       -215 |        275 |   -60 |   -45 |   -65 |    20 |    55 |   -70 |    60 |  -215 |
|  913 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        295 |          0 |        295 |   295 |   200 |    75 |    55 |    60 |    90 |   125 |   105 |
|  914 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -365 |        365 |  -145 |  -150 |  -275 |  -255 |  -235 |  -280 |  -365 |  -330 |
|  915 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        220 |        -60 |        280 |   130 |   220 |   200 |   195 |   195 |   -25 |   -60 |    65 |
|  916 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        220 |       -100 |        320 |    75 |   220 |   165 |   -40 |    45 |    40 |  -100 |   -10 |
|  917 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -395 |        395 |  -230 |  -315 |  -325 |  -275 |  -395 |  -330 |  -320 |  -250 |
|  918 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -560 |        560 |  -370 |  -300 |  -375 |  -300 |  -390 |  -420 |  -370 |  -560 |
|  919 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        285 |        -55 |        340 |    95 |   145 |    80 |   125 |    40 |    90 |   285 |   -55 |
|  920 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -770 |        770 |  -520 |  -525 |  -560 |  -680 |  -770 |  -610 |  -580 |  -730 |
|  921 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          5 |       -215 |        220 |   -15 |     5 |  -180 |  -215 |  -180 |  -175 |  -120 |  -215 |
|  922 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         40 |       -255 |        295 |   -30 |    40 |     5 |  -180 |  -220 |  -185 |  -175 |  -255 |
|  923 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          5 |       -265 |        270 |     5 |  -265 |  -170 |  -170 |  -190 |  -130 |  -190 |  -115 |
|  924 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -555 |        555 |  -255 |  -245 |  -280 |  -295 |  -495 |  -555 |  -460 |  -490 |
|  925 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        270 |          0 |        270 |   270 |   220 |    70 |    65 |    85 |   120 |    70 |    30 |
|  926 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -325 |        325 |   -25 |  -215 |  -180 |  -325 |  -185 |  -315 |  -175 |  -290 |
|  927 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -480 |        480 |  -155 |  -370 |  -310 |  -415 |  -480 |  -345 |  -390 |  -460 |
|  928 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        230 |       -170 |        400 |    90 |   230 |   -10 |  -150 |   -80 |  -170 |  -115 |  -140 |
|  929 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         35 |       -105 |        140 |    35 |   -60 |   -90 |    20 |   -40 |    -0 |  -105 |   -85 |
|  930 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1085 |       1085 |  -745 |  -950 |  -880 |  -895 |  -845 |  -780 |  -815 | -1085 |
|  931 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -395 |        395 |  -190 |  -175 |  -170 |  -200 |  -170 |  -365 |  -395 |  -360 |
|  932 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -940 |        940 |  -720 |  -930 |  -845 |  -825 |  -865 |  -940 |  -885 |  -890 |
|  933 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        165 |       -110 |        275 |   155 |    75 |   165 |   145 |   165 |  -110 |   -50 |   -20 |
|  934 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        180 |        -95 |        275 |   180 |   155 |    -5 |   -95 |   -40 |   -85 |    20 |   -35 |
|  935 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        275 |        -70 |        345 |   220 |   275 |   220 |   230 |    25 |   -20 |   -70 |   -30 |
|  936 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -325 |        325 |  -230 |  -250 |  -185 |  -260 |  -325 |  -180 |  -295 |  -305 |
|  937 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        210 |       -125 |        335 |   160 |   210 |    -5 |   -85 |  -125 |   -65 |   -75 |   -80 |
|  938 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1050 |       1050 |  -855 |  -815 |  -940 |  -945 |  -965 |  -990 |  -995 | -1050 |
|  939 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -565 |        565 |  -340 |  -530 |  -455 |  -565 |  -410 |  -550 |  -520 |  -530 |
|  940 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        260 |          0 |        260 |   180 |   215 |   245 |   215 |   155 |   260 |    30 |    50 |
|  941 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        350 |          0 |        350 |   295 |   275 |   230 |   230 |   350 |   280 |   275 |   350 |
|  942 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -675 |        675 |  -535 |  -455 |  -600 |  -460 |  -460 |  -375 |  -540 |  -675 |
|  943 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        255 |        -70 |        325 |   130 |   255 |   220 |    95 |   140 |   -70 |   -65 |    45 |
|  944 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -500 |        500 |  -190 |  -260 |  -240 |  -175 |  -290 |  -390 |  -500 |  -475 |
|  945 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -655 |        655 |  -380 |  -460 |  -495 |  -400 |  -405 |  -425 |  -450 |  -655 |
|  946 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -710 |        710 |  -450 |  -385 |  -410 |  -390 |  -670 |  -665 |  -710 |  -695 |
|  947 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -805 |        805 |  -510 |  -470 |  -555 |  -570 |  -605 |  -805 |  -675 |  -710 |
|  948 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -520 |        520 |  -295 |  -430 |  -385 |  -295 |  -370 |  -395 |  -435 |  -520 |
|  949 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        235 |       -115 |        350 |   175 |   215 |   235 |   115 |   185 |   225 |    35 |  -115 |
|  950 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        365 |          0 |        365 |   230 |   325 |   365 |   285 |   330 |   320 |    60 |   160 |
|  951 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1060 |       1060 |  -890 |  -820 |  -785 | -1035 | -1060 |  -925 |  -985 | -1050 |
|  952 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -325 |        325 |  -155 |  -135 |   -25 |  -150 |  -180 |   -70 |   -20 |  -325 |
|  953 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -325 |        325 |  -195 |  -255 |  -155 |  -220 |  -190 |  -275 |  -150 |  -325 |
|  954 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -245 |        245 |   -40 |   -45 |   -40 |   -10 |  -245 |   -55 |  -220 |  -150 |
|  955 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1190 |       1190 | -1030 | -1000 | -1015 | -1035 | -1010 |  -965 | -1085 | -1190 |
|  956 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        215 |       -140 |        355 |   215 |   -70 |    20 |    45 |   -50 |   -35 |  -140 |    35 |
|  957 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        240 |        -85 |        325 |   240 |   210 |   215 |   220 |     5 |   -85 |   -25 |   -35 |
|  958 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1135 |       1135 |  -920 | -1010 |  -930 |  -980 |  -975 |  -890 |  -860 | -1135 |
|  959 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        260 |          0 |        260 |   200 |   145 |   185 |   120 |   260 |   205 |   255 |   240 |
|  960 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1195 |       1195 |  -960 | -1160 | -1095 | -1120 | -1155 | -1150 | -1165 | -1195 |
|  961 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -865 |        865 |  -470 |  -560 |  -570 |  -715 |  -690 |  -770 |  -800 |  -865 |
|  962 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -525 |        525 |  -305 |  -335 |  -250 |  -220 |  -240 |  -525 |  -445 |  -375 |
|  963 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        305 |          0 |        305 |   225 |   225 |   125 |   160 |   175 |   280 |   225 |   305 |
|  964 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -885 |        885 |  -595 |  -595 |  -830 |  -885 |  -805 |  -785 |  -780 |  -775 |
|  965 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        235 |        -30 |        265 |   235 |    65 |    50 |   -10 |   105 |   -30 |    30 |    20 |
|  966 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        265 |        -20 |        285 |   230 |   175 |   265 |   165 |   105 |    15 |   -20 |    95 |
|  967 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        250 |        -65 |        315 |    70 |    30 |   155 |   250 |    75 |    65 |   205 |   -65 |
|  968 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        155 |          0 |        155 |    55 |   155 |    20 |   100 |    25 |    60 |    55 |    30 |
|  969 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -560 |        560 |  -385 |  -295 |  -405 |  -510 |  -545 |  -470 |  -530 |  -560 |
|  970 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        345 |          0 |        345 |   345 |   205 |   210 |   175 |   180 |   200 |    95 |   175 |
|  971 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -435 |        435 |  -145 |  -195 |  -170 |   -85 |  -130 |  -420 |  -235 |  -435 |
|  972 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -790 |        790 |  -495 |  -395 |  -475 |  -525 |  -665 |  -690 |  -740 |  -790 |
|  973 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         50 |       -215 |        265 |    50 |  -160 |  -115 |  -150 |   -70 |  -215 |  -120 |  -105 |
|  974 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        255 |        -50 |        305 |   110 |   140 |   235 |   255 |   195 |   200 |   -50 |    15 |
|  975 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        325 |          0 |        325 |   325 |   235 |   245 |    40 |   125 |   100 |   140 |    65 |
|  976 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -335 |        335 |   -75 |  -170 |  -160 |  -170 |   -60 |   -80 |   -70 |  -335 |
|  977 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        330 |          0 |        330 |   255 |   100 |   230 |   205 |   270 |   325 |   330 |    15 |
|  978 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -940 |        940 |  -755 |  -740 |  -710 |  -755 |  -765 |  -775 |  -775 |  -940 |
|  979 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        255 |          0 |        255 |   255 |   240 |    75 |    10 |   110 |    75 |    10 |   100 |
|  980 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          5 |       -130 |        135 |   -90 |   -45 |   -65 |   -30 |   -10 |     5 |     5 |  -130 |
|  981 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        355 |          0 |        355 |   355 |   220 |   155 |   135 |   140 |   160 |    25 |    80 |
|  982 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -235 |        235 |   -55 |  -190 |  -165 |  -165 |  -205 |  -235 |  -210 |  -220 |
|  983 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -770 |        770 |  -435 |  -720 |  -730 |  -690 |  -730 |  -760 |  -770 |  -715 |
|  984 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -640 |        640 |  -350 |  -470 |  -635 |  -640 |  -605 |  -535 |  -520 |  -490 |
|  985 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         90 |        -35 |        125 |    60 |    -5 |    40 |    90 |    35 |    15 |    30 |   -35 |
|  986 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -660 |        660 |  -485 |  -370 |  -460 |  -595 |  -565 |  -660 |  -585 |  -530 |
|  987 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         90 |        -30 |        120 |    90 |   -25 |   -20 |   -30 |    -5 |    20 |   -25 |    15 |
|  988 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1045 |       1045 |  -820 |  -820 |  -780 |  -820 |  -790 |  -760 | -1020 | -1045 |
|  989 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         10 |       -280 |        290 |   -95 |   -60 |  -100 |   -45 |     5 |    -5 |    10 |  -280 |
|  990 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        360 |        -10 |        370 |   255 |   360 |   215 |    85 |   -10 |    10 |    -0 |   135 |
|  991 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -370 |        370 |  -165 |  -170 |  -185 |  -200 |  -130 |  -210 |  -120 |  -370 |
|  992 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -665 |        665 |  -490 |  -425 |  -435 |  -450 |  -515 |  -645 |  -665 |  -650 |
|  993 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -715 |        715 |  -495 |  -455 |  -340 |  -505 |  -715 |  -610 |  -625 |  -640 |
|  994 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         80 |          0 |         80 |    55 |    80 |    40 |    50 |    80 |    35 |    30 |    45 |
|  995 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -880 |        880 |  -595 |  -660 |  -700 |  -695 |  -655 |  -725 |  -870 |  -880 |
|  996 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -970 |        970 |  -920 |  -875 |  -930 |  -945 |  -930 |  -815 |  -850 |  -970 |
|  997 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        275 |          0 |        275 |   105 |   275 |   245 |   190 |   255 |    30 |   120 |    80 |
|  998 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         80 |       -275 |        355 |    25 |    75 |    80 |    20 |  -275 |  -190 |  -105 |   -75 |
|  999 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1180 |       1180 |  -930 | -1145 | -1080 | -1120 | -1170 | -1035 | -1130 | -1180 |
| 1000 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        355 |          0 |        355 |   355 |   185 |   270 |   150 |   240 |   200 |   175 |   260 |
| 1001 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -990 |        990 |  -610 |  -780 |  -990 |  -870 |  -920 |  -870 |  -865 |  -980 |
| 1002 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        335 |          0 |        335 |   195 |   260 |   185 |   245 |   260 |   335 |   300 |    30 |
| 1003 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        295 |          0 |        295 |   265 |   275 |   295 |    35 |   145 |    40 |   170 |   125 |
| 1004 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -265 |        265 |   -80 |  -160 |   -70 |   -80 |   -80 |   -75 |   -75 |  -265 |
| 1005 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -800 |        800 |  -490 |  -540 |  -580 |  -545 |  -515 |  -460 |  -660 |  -800 |
| 1006 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        170 |       -120 |        290 |  -120 |    60 |   -35 |    15 |   170 |    35 |   -15 |   -15 |
| 1007 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -560 |        560 |  -385 |  -435 |  -380 |  -255 |  -330 |  -395 |  -320 |  -560 |
| 1008 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        205 |       -100 |        305 |   205 |   120 |   135 |   -85 |  -100 |   -70 |    20 |    -0 |
| 1009 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -680 |        680 |  -425 |  -620 |  -600 |  -620 |  -645 |  -680 |  -625 |  -610 |
| 1010 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        290 |       -125 |        415 |   255 |   210 |   290 |     5 |  -125 |    75 |    95 |    -5 |
| 1011 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -590 |        590 |  -270 |  -275 |  -500 |  -590 |  -520 |  -585 |  -445 |  -570 |
| 1012 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        260 |          0 |        260 |   260 |   195 |   260 |   235 |   240 |    90 |   235 |   205 |
| 1013 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -350 |        350 |  -100 |   -25 |  -250 |  -315 |  -290 |  -235 |  -350 |  -285 |
| 1014 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -945 |        945 |  -700 |  -765 |  -680 |  -740 |  -880 |  -865 |  -945 |  -900 |
| 1015 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        380 |          0 |        380 |   325 |   215 |   380 |   125 |   105 |   130 |   155 |   155 |
| 1016 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        280 |          0 |        280 |   280 |   260 |   130 |   110 |    65 |    65 |    40 |   135 |
| 1017 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         55 |        -40 |         95 |   -15 |    10 |    55 |   -25 |   -10 |   -40 |     5 |   -30 |
| 1018 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -975 |        975 |  -740 |  -605 |  -725 |  -870 |  -905 |  -975 |  -765 |  -810 |
| 1019 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        350 |          0 |        350 |   350 |   305 |   270 |   320 |   295 |   270 |   135 |   120 |
| 1020 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1040 |       1040 |  -840 |  -960 | -1005 | -1040 |  -935 |  -905 |  -935 |  -975 |
| 1021 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        345 |          0 |        345 |   315 |   325 |   300 |   345 |   300 |   120 |   190 |   130 |
| 1022 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        175 |        -10 |        185 |   110 |    95 |   105 |   175 |    60 |   175 |   -10 |   125 |
| 1023 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -855 |        855 |  -640 |  -605 |  -605 |  -645 |  -825 |  -850 |  -855 |  -845 |
| 1024 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -610 |        610 |  -470 |  -570 |  -475 |  -610 |  -580 |  -585 |  -550 |  -535 |
| 1025 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1050 |       1050 |  -830 |  -800 |  -890 |  -755 |  -765 |  -790 |  -935 | -1050 |
| 1026 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -950 |        950 |  -775 |  -775 |  -705 |  -950 |  -945 |  -905 |  -945 |  -860 |
| 1027 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -955 |        955 |  -720 |  -745 |  -820 |  -875 |  -720 |  -850 |  -730 |  -955 |
| 1028 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        375 |          0 |        375 |   270 |   305 |   370 |   375 |    90 |   190 |   190 |   360 |
| 1029 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -525 |        525 |  -205 |  -285 |  -250 |  -205 |  -215 |  -425 |  -525 |  -420 |
| 1030 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -530 |        530 |  -275 |  -330 |  -290 |  -515 |  -470 |  -530 |  -460 |  -495 |
| 1031 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        240 |        -35 |        275 |   230 |   240 |    25 |   100 |    40 |    80 |   -35 |    95 |
| 1032 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        300 |          0 |        300 |   160 |   115 |   165 |   300 |    80 |    75 |    25 |    15 |
| 1033 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        235 |        -85 |        320 |   195 |   235 |   -15 |   -85 |   -30 |   -35 |   -25 |   -30 |
| 1034 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        215 |        -85 |        300 |   125 |    85 |    50 |   140 |    -0 |   215 |    30 |   -85 |
| 1035 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -240 |        240 |  -215 |  -240 |  -175 |  -135 |  -225 |  -165 |  -195 |  -155 |
| 1036 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -320 |        320 |  -140 |   -95 |  -120 |  -260 |  -230 |  -200 |  -320 |  -260 |
| 1037 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -310 |        310 |  -140 |  -175 |  -125 |  -195 |  -125 |  -135 |  -310 |  -230 |
| 1038 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        350 |          0 |        350 |   350 |   335 |   180 |   165 |   220 |   260 |   120 |   160 |
| 1039 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -670 |        670 |  -490 |  -390 |  -350 |  -410 |  -340 |  -340 |  -570 |  -670 |
| 1040 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -860 |        860 |  -610 |  -700 |  -640 |  -575 |  -700 |  -600 |  -680 |  -860 |
| 1041 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -980 |        980 |  -770 |  -775 |  -765 |  -830 |  -960 |  -865 |  -980 |  -950 |
| 1042 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -895 |        895 |  -465 |  -785 |  -715 |  -765 |  -760 |  -870 |  -750 |  -895 |
| 1043 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -840 |        840 |  -830 |  -690 |  -700 |  -690 |  -685 |  -665 |  -840 |  -690 |
| 1044 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -585 |        585 |  -385 |  -525 |  -430 |  -510 |  -495 |  -585 |  -525 |  -460 |
| 1045 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -370 |        370 |  -175 |  -110 |  -130 |  -135 |   -70 |  -370 |  -280 |  -360 |
| 1046 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        130 |       -210 |        340 |    40 |   -15 |   130 |    20 |   -20 |    25 |  -160 |  -210 |
| 1047 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -985 |        985 |  -665 |  -735 |  -775 |  -850 |  -885 |  -985 |  -955 |  -920 |
| 1048 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -785 |        785 |  -605 |  -685 |  -785 |  -660 |  -720 |  -750 |  -760 |  -710 |
| 1049 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        170 |       -120 |        290 |    35 |   130 |    90 |   120 |   170 |  -120 |  -110 |  -105 |
| 1050 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        135 |        -80 |        215 |   120 |   135 |   -25 |   -70 |     5 |   -80 |    15 |    -0 |
| 1051 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1100 |       1100 |  -805 |  -895 |  -770 | -1000 | -1020 | -1020 |  -870 | -1100 |
| 1052 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         85 |       -125 |        210 |    85 |   -15 |   -55 |  -125 |  -110 |   -15 |   -10 |   -55 |
| 1053 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        335 |          0 |        335 |   335 |   315 |   225 |   150 |    35 |   115 |    25 |   120 |
| 1054 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -410 |        410 |  -200 |  -180 |  -155 |  -165 |  -410 |  -320 |  -335 |  -315 |
| 1055 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1020 |       1020 |  -850 |  -895 |  -805 |  -945 |  -805 |  -815 | -1020 |  -995 |
| 1056 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -375 |        375 |   -10 |  -180 |  -260 |  -375 |  -200 |  -205 |  -320 |  -225 |
| 1057 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         15 |       -270 |        285 |   -35 |   -55 |   -65 |   -80 |  -145 |    15 |   -15 |  -270 |
| 1058 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -485 |        485 |  -245 |  -350 |  -295 |  -235 |  -340 |  -435 |  -485 |  -395 |
| 1059 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -570 |        570 |  -350 |  -530 |  -480 |  -525 |  -395 |  -525 |  -425 |  -570 |
| 1060 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -865 |        865 |  -535 |  -795 |  -855 |  -840 |  -800 |  -855 |  -865 |  -745 |
| 1061 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        315 |          0 |        315 |   280 |   235 |   315 |   255 |   225 |   295 |   290 |   175 |
| 1062 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         25 |       -160 |        185 |    25 |  -160 |  -160 |   -45 |  -160 |  -135 |   -45 |    -0 |
| 1063 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -870 |        870 |  -580 |  -680 |  -500 |  -630 |  -615 |  -645 |  -775 |  -870 |
| 1064 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -345 |        345 |  -145 |  -240 |  -135 |  -220 |  -100 |  -240 |  -325 |  -345 |
| 1065 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         90 |        -80 |        170 |   -55 |   -80 |   -15 |   -65 |    90 |   -35 |   -75 |    45 |
| 1066 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        105 |        -15 |        120 |   -15 |    -5 |    70 |    10 |    60 |    50 |    20 |   105 |
| 1067 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -175 |        175 |   -90 |  -125 |  -160 |  -115 |  -100 |   -50 |   -85 |  -175 |
| 1068 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1005 |       1005 |  -765 |  -625 |  -635 |  -710 |  -595 |  -675 |  -760 | -1005 |
| 1069 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -930 |        930 |  -570 |  -730 |  -690 |  -720 |  -620 |  -930 |  -845 |  -740 |
| 1070 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        250 |       -155 |        405 |   130 |   130 |    80 |   130 |   250 |   -65 |  -155 |   -35 |
| 1071 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        250 |        -25 |        275 |   250 |    10 |    35 |   -25 |    50 |    35 |    25 |    40 |
| 1072 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -835 |        835 |  -645 |  -690 |  -810 |  -575 |  -585 |  -585 |  -765 |  -835 |
| 1073 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -850 |        850 |  -565 |  -520 |  -815 |  -685 |  -830 |  -730 |  -780 |  -850 |
| 1074 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         30 |       -280 |        310 |   -10 |    -5 |   -30 |    -0 |     5 |    30 |    25 |  -280 |
| 1075 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -460 |        460 |  -215 |  -270 |  -220 |  -110 |  -410 |  -460 |  -455 |  -435 |
| 1076 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        325 |          0 |        325 |   325 |    25 |   190 |   135 |   145 |   185 |   155 |    25 |
| 1077 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        345 |          0 |        345 |   255 |   260 |   225 |   345 |    55 |   110 |   135 |    50 |
| 1078 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         85 |        -45 |        130 |    20 |   -45 |    85 |    -0 |   -25 |     5 |    15 |    10 |
| 1079 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -295 |        295 |  -295 |  -245 |  -140 |  -215 |  -200 |  -205 |  -130 |   -70 |
| 1080 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -925 |        925 |  -545 |  -680 |  -625 |  -575 |  -855 |  -925 |  -865 |  -840 |
| 1081 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        300 |          0 |        300 |   300 |   235 |   145 |   170 |   200 |   215 |   180 |   165 |
| 1082 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        230 |        -80 |        310 |   230 |    55 |   -10 |    20 |    60 |   -80 |   -15 |    20 |
| 1083 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        255 |        -60 |        315 |   205 |   245 |   225 |   255 |   235 |    30 |   -60 |    60 |
| 1084 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -875 |        875 |  -555 |  -665 |  -475 |  -575 |  -635 |  -620 |  -535 |  -875 |
| 1085 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        380 |          0 |        380 |   240 |   380 |   265 |   350 |   370 |   255 |   335 |   185 |
| 1086 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        290 |        -50 |        340 |   190 |   290 |   105 |   175 |   185 |   100 |   110 |   -50 |
| 1087 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1070 |       1070 |  -895 |  -835 |  -935 |  -890 |  -765 | -1070 | -1040 | -1045 |
| 1088 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         40 |       -245 |        285 |    40 |  -170 |   -30 |    -0 |   -50 |   -60 |   -55 |  -245 |
| 1089 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        395 |          0 |        395 |   365 |   280 |   320 |   330 |   240 |   395 |   230 |   185 |
| 1090 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1040 |       1040 |  -795 |  -700 |  -750 |  -780 |  -705 |  -710 |  -765 | -1040 |
| 1091 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        330 |          0 |        330 |   310 |   330 |   300 |   185 |   155 |   115 |   220 |   120 |
| 1092 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        155 |        -90 |        245 |   155 |   145 |   -90 |   -65 |    15 |    50 |   -75 |   -65 |
| 1093 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        240 |       -120 |        360 |   240 |   140 |   -65 |   -80 |  -120 |   -40 |  -120 |   -30 |
| 1094 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        220 |        -45 |        265 |   175 |   115 |   220 |   -15 |   -45 |    55 |   -45 |   -20 |
| 1095 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        280 |        -50 |        330 |   165 |   230 |   175 |   245 |   190 |   280 |   245 |   -50 |
| 1096 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -705 |        705 |  -575 |  -520 |  -510 |  -610 |  -590 |  -705 |  -655 |  -640 |
| 1097 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -375 |        375 |  -165 |  -125 |  -350 |  -375 |  -330 |  -360 |  -250 |  -275 |
| 1098 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1130 |       1130 |  -905 |  -905 |  -950 | -1085 |  -975 | -1055 | -1130 | -1050 |
| 1099 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        255 |        -30 |        285 |   255 |    45 |    15 |   -30 |    95 |    80 |    20 |    60 |
| 1100 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -710 |        710 |  -510 |  -620 |  -455 |  -370 |  -485 |  -710 |  -615 |  -640 |
| 1101 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -625 |        625 |  -375 |  -250 |  -485 |  -485 |  -535 |  -510 |  -625 |  -540 |
| 1102 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -445 |        445 |  -360 |  -445 |  -415 |  -260 |  -350 |  -385 |  -395 |  -430 |
| 1103 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -665 |        665 |  -470 |  -435 |  -330 |  -395 |  -415 |  -665 |  -655 |  -640 |
| 1104 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        130 |        -90 |        220 |   115 |   130 |    50 |    20 |   115 |   120 |    10 |   -90 |
| 1105 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        175 |       -170 |        345 |   175 |    80 |    60 |   175 |   175 |   135 |  -170 |   -85 |
| 1106 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -920 |        920 |  -635 |  -740 |  -715 |  -630 |  -580 |  -920 |  -870 |  -875 |
| 1107 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         15 |       -385 |        400 |    15 |   -85 |   -25 |   -45 |  -270 |  -300 |  -385 |  -250 |
| 1108 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -740 |        740 |  -410 |  -450 |  -445 |  -645 |  -665 |  -475 |  -740 |  -680 |
| 1109 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        375 |          0 |        375 |   350 |   320 |   265 |   375 |   175 |    85 |    75 |   105 |
| 1110 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        110 |       -150 |        260 |   110 |   -75 |  -115 |   -80 |  -150 |   -55 |   -10 |  -145 |
| 1111 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        215 |        -85 |        300 |   200 |   185 |   180 |   215 |   130 |   -85 |    -0 |    25 |
| 1112 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        270 |          0 |        270 |   270 |   240 |   265 |   230 |   165 |   185 |    45 |   190 |
| 1113 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -780 |        780 |  -740 |  -730 |  -610 |  -635 |  -780 |  -620 |  -765 |  -695 |
| 1114 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -420 |        420 |  -165 |  -220 |  -215 |  -240 |  -170 |  -385 |  -390 |  -420 |
| 1115 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -745 |        745 |  -475 |  -455 |  -545 |  -675 |  -640 |  -715 |  -745 |  -650 |
| 1116 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        390 |          0 |        390 |   365 |   390 |   375 |   370 |   200 |   185 |   120 |   140 |
| 1117 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        235 |          0 |        235 |   135 |   155 |   220 |   170 |   215 |   235 |     5 |    40 |
| 1118 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -495 |        495 |  -470 |  -470 |  -490 |  -495 |  -375 |  -480 |  -465 |  -450 |
| 1119 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -450 |        450 |  -195 |  -150 |  -220 |  -440 |  -450 |  -400 |  -400 |  -330 |
| 1120 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        215 |       -135 |        350 |   215 |   -25 |   -55 |    -0 |  -135 |   -40 |    25 |    -0 |
| 1121 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        235 |       -145 |        380 |   235 |   145 |    35 |   180 |    90 |  -145 |    -0 |   -45 |
| 1122 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -620 |        620 |  -395 |  -500 |  -375 |  -485 |  -475 |  -435 |  -390 |  -620 |
| 1123 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -535 |        535 |  -145 |  -280 |  -390 |  -405 |  -475 |  -485 |  -400 |  -535 |
| 1124 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        335 |          0 |        335 |   335 |   250 |   285 |   320 |   270 |   305 |   285 |   105 |
| 1125 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        275 |          0 |        275 |   190 |   275 |    55 |    45 |    45 |   150 |   145 |    85 |
| 1126 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -915 |        915 |  -840 |  -855 |  -870 |  -830 |  -830 |  -895 |  -915 |  -790 |
| 1127 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        195 |        -90 |        285 |   195 |   145 |   -85 |   -20 |   -10 |   -10 |   -30 |   -90 |
| 1128 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1080 |       1080 |  -925 |  -890 |  -965 |  -845 | -1055 |  -980 | -1080 | -1050 |
| 1129 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         65 |        -95 |        160 |    20 |    30 |    65 |    65 |    65 |   -80 |    20 |   -95 |
| 1130 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -420 |        420 |  -165 |  -240 |  -155 |  -145 |  -165 |  -240 |  -190 |  -420 |
| 1131 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        320 |          0 |        320 |   320 |   140 |   180 |    35 |    60 |    60 |    50 |    65 |
| 1132 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        400 |          0 |        400 |   260 |   355 |   350 |   180 |   400 |   165 |   125 |   145 |
| 1133 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        275 |        -15 |        290 |   235 |   275 |    10 |    40 |   130 |   -15 |    -0 |    30 |
| 1134 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1195 |       1195 |  -955 |  -865 |  -915 | -1055 |  -870 |  -855 | -1195 | -1135 |
| 1135 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -265 |        265 |   -20 |   -15 |   -75 |   -70 |  -265 |  -245 |  -195 |  -205 |
| 1136 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        370 |          0 |        370 |   370 |   275 |   220 |   115 |    80 |   125 |   135 |   150 |
| 1137 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        145 |        -55 |        200 |    85 |   135 |   130 |   145 |   140 |   120 |   130 |   -55 |
| 1138 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1010 |       1010 |  -740 |  -875 |  -930 |  -915 | -1010 |  -985 |  -960 |  -985 |
| 1139 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -840 |        840 |  -820 |  -715 |  -715 |  -785 |  -725 |  -710 |  -690 |  -840 |
| 1140 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -315 |        315 |   -55 |   -90 |   -45 |  -195 |  -305 |  -315 |  -235 |  -305 |
| 1141 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -910 |        910 |  -655 |  -640 |  -840 |  -855 |  -850 |  -855 |  -895 |  -910 |
| 1142 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -655 |        655 |  -555 |  -620 |  -585 |  -655 |  -610 |  -610 |  -650 |  -625 |
| 1143 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1195 |       1195 |  -885 | -1195 |  -980 | -1055 | -1120 | -1180 | -1145 | -1065 |
| 1144 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -330 |        330 |  -195 |  -145 |  -185 |  -130 |  -145 |  -195 |  -130 |  -330 |
| 1145 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -895 |        895 |  -820 |  -775 |  -845 |  -760 |  -895 |  -845 |  -800 |  -815 |
| 1146 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -700 |        700 |  -505 |  -665 |  -700 |  -655 |  -590 |  -670 |  -565 |  -630 |
| 1147 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -835 |        835 |  -575 |  -640 |  -645 |  -570 |  -730 |  -815 |  -835 |  -790 |
| 1148 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         25 |       -265 |        290 |    25 |   -20 |    20 |  -265 |  -175 |  -195 |  -250 |  -255 |
| 1149 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        185 |        -85 |        270 |   185 |   170 |   160 |   115 |   140 |    80 |   150 |   -85 |
| 1150 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        210 |        -75 |        285 |   180 |   150 |   195 |   210 |   -75 |   -75 |   -20 |    25 |
| 1151 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -285 |        285 |  -200 |  -215 |  -285 |  -235 |  -190 |  -245 |  -230 |  -250 |
| 1152 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         15 |       -185 |        200 |    10 |    15 |  -185 |  -115 |  -140 |  -120 |   -95 |  -125 |
| 1153 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -465 |        465 |   -45 |  -235 |  -360 |  -335 |  -390 |  -310 |  -340 |  -465 |
| 1154 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        400 |          0 |        400 |   400 |   165 |   155 |   270 |   170 |   215 |   185 |   240 |
| 1155 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        320 |          0 |        320 |   255 |   320 |   265 |   210 |   230 |   260 |   305 |   265 |
| 1156 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        275 |          0 |        275 |   275 |   185 |    45 |   140 |    50 |    45 |    70 |    90 |
| 1157 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -995 |        995 |  -995 |  -885 |  -910 |  -995 |  -900 |  -900 |  -890 |  -880 |
| 1158 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        370 |          0 |        370 |   290 |   370 |   345 |   370 |   360 |   145 |    75 |   140 |
| 1159 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1050 |       1050 |  -855 |  -850 |  -870 |  -740 |  -865 |  -885 |  -820 | -1050 |
| 1160 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        275 |          0 |        275 |   235 |   230 |   240 |   275 |   275 |    90 |    40 |    35 |
| 1161 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -660 |        660 |  -425 |  -420 |  -575 |  -570 |  -615 |  -605 |  -660 |  -620 |
| 1162 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        270 |        -10 |        280 |   260 |   270 |    40 |    35 |   -10 |    -5 |   105 |    -5 |
| 1163 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        215 |          0 |        215 |   190 |   145 |   150 |   190 |   215 |   145 |   125 |   175 |
| 1164 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -915 |        915 |  -720 |  -705 |  -580 |  -585 |  -820 |  -715 |  -915 |  -835 |
| 1165 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        260 |        -60 |        320 |   260 |    90 |    20 |   -60 |    20 |   -30 |    25 |    30 |
| 1166 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        340 |        -80 |        420 |   320 |   335 |   255 |   240 |   340 |   200 |   -80 |    20 |
| 1167 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -885 |        885 |  -800 |  -710 |  -710 |  -625 |  -595 |  -685 |  -690 |  -885 |
| 1168 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -365 |        365 |  -190 |  -185 |  -205 |  -210 |  -110 |  -150 |  -235 |  -365 |
| 1169 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        265 |        -70 |        335 |    90 |   265 |   155 |   -60 |   -10 |    70 |   -70 |   -10 |
| 1170 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1035 |       1035 |  -720 |  -650 |  -740 |  -740 |  -700 |  -965 | -1035 |  -980 |
| 1171 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -995 |        995 |  -750 |  -695 |  -755 |  -765 |  -715 |  -760 |  -755 |  -995 |
| 1172 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         20 |       -160 |        180 |    20 |   -50 |  -155 |   -75 |  -105 |  -120 |   -75 |  -160 |
| 1173 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -820 |        820 |  -510 |  -520 |  -570 |  -580 |  -595 |  -820 |  -730 |  -790 |
| 1174 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -220 |        220 |  -100 |   -95 |  -210 |  -220 |  -140 |   -60 |  -100 |  -120 |
| 1175 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        385 |          0 |        385 |   345 |   345 |   385 |   250 |   290 |   125 |    85 |    80 |
| 1176 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        325 |          0 |        325 |   230 |   310 |   325 |   315 |   185 |   290 |   195 |     5 |
| 1177 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        360 |         -0 |        360 |   360 |   315 |   240 |   280 |   260 |    -0 |   120 |    -0 |
| 1178 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        295 |          0 |        295 |   295 |    20 |    50 |   150 |    90 |   115 |   105 |   210 |
| 1179 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        235 |       -125 |        360 |   160 |   115 |   235 |   145 |  -125 |   -45 |   -95 |   -40 |
| 1180 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        175 |       -100 |        275 |   175 |   135 |    15 |   -15 |  -100 |    85 |    65 |   -30 |
| 1181 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -475 |        475 |  -340 |  -240 |  -250 |  -360 |  -475 |  -400 |  -385 |  -335 |
| 1182 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -840 |        840 |  -670 |  -495 |  -775 |  -840 |  -830 |  -735 |  -825 |  -795 |
| 1183 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        280 |        -30 |        310 |   130 |   215 |   255 |   245 |   280 |    50 |   -30 |   -30 |
| 1184 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1120 |       1120 |  -815 |  -900 | -1000 | -1025 | -1060 | -1120 | -1020 |  -950 |
| 1185 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        185 |       -110 |        295 |   145 |   185 |   140 |    20 |   160 |   110 |    40 |  -110 |
| 1186 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -675 |        675 |  -480 |  -485 |  -485 |  -450 |  -440 |  -675 |  -630 |  -560 |
| 1187 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        370 |          0 |        370 |   370 |   175 |   115 |    90 |   130 |    85 |   100 |   170 |
| 1188 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         -0 |       -305 |        305 |    -5 |   -10 |    -0 |  -210 |  -225 |  -290 |  -305 |  -230 |
| 1189 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -500 |        500 |  -235 |  -340 |  -220 |  -470 |  -470 |  -500 |  -450 |  -445 |
| 1190 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        195 |          0 |        195 |   150 |   165 |   195 |   195 |   130 |    75 |   180 |   105 |
| 1191 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -385 |        385 |  -145 |  -290 |  -355 |  -310 |  -375 |  -385 |  -340 |  -355 |
| 1192 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -225 |        225 |   -30 |  -165 |  -225 |  -210 |  -205 |  -210 |  -120 |  -170 |
| 1193 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1015 |       1015 |  -775 |  -820 |  -780 | -1005 | -1015 |  -990 |  -970 | -1015 |
| 1194 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1030 |       1030 |  -785 | -1030 |  -910 |  -900 |  -980 |  -985 |  -945 |  -915 |
| 1195 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        360 |          0 |        360 |   285 |   360 |   335 |   260 |   260 |   290 |   100 |   115 |
| 1196 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        305 |          0 |        305 |   260 |   170 |   245 |   305 |     5 |    45 |    90 |    50 |
| 1197 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        320 |       -100 |        420 |   155 |   320 |   235 |   115 |   145 |   -30 |  -100 |   -60 |
| 1198 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -435 |        435 |  -175 |  -245 |  -245 |  -220 |  -270 |  -205 |  -375 |  -435 |
| 1199 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -470 |        470 |  -340 |  -305 |  -225 |  -145 |  -470 |  -355 |  -410 |  -390 |
| 1200 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1035 |       1035 |  -755 |  -800 |  -815 | -1035 | -1035 |  -995 |  -990 |  -925 |
| 1201 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1090 |       1090 |  -850 |  -830 | -1090 | -1055 |  -955 |  -960 |  -990 |  -915 |
| 1202 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        290 |          0 |        290 |   290 |   280 |    10 |   120 |   100 |   100 |    85 |   100 |
| 1203 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        220 |        -75 |        295 |   100 |   180 |    90 |   220 |   -40 |   -75 |    10 |    30 |
| 1204 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -650 |        650 |  -440 |  -435 |  -385 |  -435 |  -650 |  -620 |  -635 |  -610 |
| 1205 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1150 |       1150 | -1135 | -1095 | -1150 | -1000 | -1100 | -1060 | -1020 | -1135 |
| 1206 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1200 |       1200 |  -930 |  -905 |  -850 |  -915 |  -910 | -1130 | -1200 | -1145 |
| 1207 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -440 |        440 |  -150 |  -390 |  -385 |  -375 |  -435 |  -390 |  -440 |  -330 |
| 1208 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -710 |        710 |  -410 |  -465 |  -495 |  -495 |  -510 |  -480 |  -390 |  -710 |
| 1209 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        340 |          0 |        340 |   320 |   285 |   240 |   340 |    70 |    50 |   125 |   110 |
| 1210 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        215 |        -80 |        295 |   205 |   215 |   165 |    60 |    30 |   -40 |   135 |   -80 |
| 1211 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -460 |        460 |  -160 |  -445 |  -385 |  -420 |  -455 |  -335 |  -460 |  -335 |
| 1212 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        190 |        -15 |        205 |   190 |    50 |    70 |    20 |    20 |    -5 |    70 |   -15 |
| 1213 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -400 |        400 |   -55 |   -85 |  -310 |  -250 |  -400 |  -375 |  -255 |  -290 |
| 1214 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1035 |       1035 |  -900 |  -865 |  -855 |  -820 |  -950 | -1015 | -1035 | -1030 |
| 1215 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -525 |        525 |  -165 |  -210 |  -230 |  -225 |  -205 |  -220 |  -285 |  -525 |
| 1216 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -840 |        840 |  -530 |  -790 |  -790 |  -835 |  -760 |  -695 |  -840 |  -730 |
| 1217 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -510 |        510 |  -175 |  -115 |  -150 |  -200 |  -160 |  -385 |  -510 |  -385 |
| 1218 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        285 |          0 |        285 |   240 |   285 |   210 |   235 |   230 |   165 |   235 |    15 |
| 1219 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1185 |       1185 | -1030 |  -945 | -1045 |  -975 | -1060 |  -990 | -1035 | -1185 |
| 1220 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        210 |        -50 |        260 |   175 |   190 |   170 |   210 |    -5 |   -50 |   -35 |    20 |
| 1221 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -570 |        570 |  -185 |  -270 |  -280 |  -235 |  -520 |  -570 |  -490 |  -385 |
| 1222 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        175 |       -130 |        305 |   175 |    10 |   140 |    75 |   -85 |  -130 |   -95 |  -130 |
| 1223 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -325 |        325 |   -55 |  -190 |  -195 |  -185 |  -300 |  -245 |  -325 |  -270 |
| 1224 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        245 |       -120 |        365 |   245 |    60 |   -30 |   -85 |    25 |   -80 |  -120 |   -45 |
| 1225 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        290 |          0 |        290 |   120 |   160 |   250 |   140 |   120 |   200 |   265 |   290 |
| 1226 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        200 |          0 |        200 |   115 |   200 |    90 |   195 |    85 |    45 |    80 |   100 |
| 1227 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1200 |       1200 | -1080 | -1150 | -1170 | -1150 | -1110 | -1150 | -1200 | -1125 |
| 1228 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -995 |        995 |  -795 |  -800 |  -725 |  -670 |  -960 |  -995 |  -990 |  -930 |
| 1229 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -695 |        695 |  -280 |  -195 |  -250 |  -325 |  -355 |  -695 |  -565 |  -540 |
| 1230 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -285 |        285 |  -165 |   -35 |   -80 |  -125 |  -115 |   -50 |   -40 |  -285 |
| 1231 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         30 |       -315 |        345 |   -80 |    30 |   -70 |   -90 |   -75 |  -270 |  -315 |  -260 |
| 1232 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        385 |          0 |        385 |   365 |   385 |   370 |   245 |   150 |   265 |   175 |   145 |
| 1233 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -940 |        940 |  -610 |  -750 |  -705 |  -715 |  -875 |  -880 |  -940 |  -785 |
| 1234 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -745 |        745 |  -465 |  -495 |  -505 |  -530 |  -505 |  -690 |  -670 |  -745 |
| 1235 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        150 |        -50 |        200 |   105 |   150 |   125 |   150 |    95 |   -45 |   -50 |    -0 |
| 1236 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        360 |          0 |        360 |   360 |   270 |   315 |     5 |   160 |    90 |    65 |    85 |
| 1237 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -765 |        765 |  -490 |  -465 |  -575 |  -590 |  -585 |  -555 |  -655 |  -765 |
| 1238 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -280 |        280 |  -245 |  -275 |  -280 |  -160 |  -265 |  -130 |  -215 |  -220 |
| 1239 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        240 |        -10 |        250 |   240 |   150 |    95 |   125 |    90 |    80 |    80 |   -10 |
| 1240 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        385 |          0 |        385 |   385 |    90 |   155 |   105 |   115 |   130 |     5 |   200 |
| 1241 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -930 |        930 |  -675 |  -760 |  -620 |  -635 |  -910 |  -925 |  -915 |  -930 |
| 1242 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1090 |       1090 |  -825 |  -965 |  -880 | -1070 | -1090 | -1030 | -1065 | -1045 |
| 1243 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1025 |       1025 |  -915 |  -825 |  -855 |  -900 |  -950 |  -905 |  -860 | -1025 |
| 1244 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -395 |        395 |  -340 |  -345 |  -360 |  -360 |  -325 |  -265 |  -395 |  -370 |
| 1245 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        115 |       -160 |        275 |   115 |    15 |  -135 |  -160 |  -160 |  -110 |  -140 |  -155 |
| 1246 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -650 |        650 |  -360 |  -365 |  -385 |  -595 |  -585 |  -650 |  -550 |  -545 |
| 1247 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        390 |          0 |        390 |   385 |   390 |   315 |   290 |   350 |   305 |   330 |   145 |
| 1248 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -445 |        445 |  -180 |  -210 |  -415 |  -445 |  -265 |  -365 |  -330 |  -355 |
| 1249 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        180 |       -155 |        335 |   120 |   180 |   -95 |   -10 |   -50 |  -155 |    25 |   -95 |
| 1250 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -315 |        315 |   -45 |   -70 |  -100 |  -235 |  -270 |  -270 |  -315 |  -255 |
| 1251 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         40 |       -355 |        395 |    40 |    -5 |   -55 |   -20 |    -5 |   -70 |  -355 |  -245 |
| 1252 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -630 |        630 |  -320 |  -285 |  -330 |  -630 |  -610 |  -590 |  -565 |  -580 |
| 1253 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1105 |       1105 |  -875 |  -845 |  -965 |  -915 |  -990 |  -890 |  -860 | -1105 |
| 1254 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        315 |          0 |        315 |   300 |   315 |    60 |    65 |    95 |    75 |    35 |    35 |
| 1255 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        240 |        -45 |        285 |   240 |    90 |   -45 |    65 |    20 |    45 |   -10 |    75 |
| 1256 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        300 |          0 |        300 |   210 |   270 |   300 |    10 |    70 |   145 |   215 |   145 |
| 1257 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -590 |        590 |  -430 |  -445 |  -445 |  -420 |  -465 |  -430 |  -430 |  -590 |
| 1258 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        265 |       -105 |        370 |   175 |   230 |   185 |   265 |   170 |   160 |  -105 |   -75 |
| 1259 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1170 |       1170 |  -975 |  -870 | -1070 | -1135 | -1110 | -1130 | -1160 | -1170 |
| 1260 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1125 |       1125 |  -925 |  -970 |  -905 |  -930 |  -985 | -1010 | -1045 | -1125 |
| 1261 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        285 |          0 |        285 |   220 |   170 |   265 |   125 |   240 |   285 |   160 |    70 |
| 1262 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1045 |       1045 |  -885 |  -825 |  -730 |  -985 |  -990 | -1045 |  -975 | -1015 |
| 1263 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -910 |        910 |  -650 |  -715 |  -655 |  -570 |  -885 |  -895 |  -910 |  -870 |
| 1264 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        270 |       -100 |        370 |   225 |   190 |   270 |   100 |  -100 |   -55 |    30 |   -45 |
| 1265 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1195 |       1195 | -1000 | -1060 |  -960 |  -905 |  -955 | -1005 | -1160 | -1195 |
| 1266 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -725 |        725 |  -610 |  -705 |  -640 |  -725 |  -680 |  -585 |  -690 |  -655 |
| 1267 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -735 |        735 |  -435 |  -620 |  -635 |  -735 |  -715 |  -705 |  -700 |  -695 |
| 1268 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -605 |        605 |  -520 |  -375 |  -420 |  -360 |  -385 |  -405 |  -605 |  -580 |
| 1269 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1055 |       1055 |  -770 |  -730 |  -715 |  -970 |  -830 |  -870 | -1055 |  -950 |
| 1270 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -920 |        920 |  -805 |  -695 |  -665 |  -765 |  -655 |  -905 |  -900 |  -920 |
| 1271 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -865 |        865 |  -700 |  -730 |  -705 |  -690 |  -740 |  -650 |  -670 |  -865 |
| 1272 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -970 |        970 |  -770 |  -750 |  -780 |  -790 |  -630 |  -935 |  -915 |  -970 |
| 1273 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |        -25 |        250 |   225 |   135 |   210 |   190 |    40 |   -25 |    80 |   -20 |
| 1274 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -425 |        425 |  -285 |  -350 |  -295 |  -250 |  -365 |  -310 |  -290 |  -425 |
| 1275 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -980 |        980 |  -980 |  -845 |  -825 |  -870 |  -835 |  -845 |  -850 |  -830 |
| 1276 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -675 |        675 |  -315 |  -305 |  -525 |  -675 |  -500 |  -490 |  -535 |  -555 |
| 1277 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -380 |        380 |  -310 |  -325 |  -310 |  -380 |  -370 |  -325 |  -340 |  -305 |
| 1278 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        340 |          0 |        340 |   285 |   340 |   210 |    90 |    35 |    65 |   105 |    45 |
| 1279 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         20 |       -385 |        405 |     5 |   -75 |   -55 |    20 |   -50 |   -55 |  -150 |  -385 |
| 1280 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -360 |        360 |  -125 |  -255 |  -360 |  -335 |  -310 |  -305 |  -300 |  -330 |
| 1281 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        295 |          0 |        295 |   290 |   280 |   270 |   295 |    85 |    55 |   140 |   250 |
| 1282 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        370 |        -15 |        385 |   305 |   270 |   220 |   300 |   370 |   -15 |    95 |   -15 |
| 1283 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -730 |        730 |  -515 |  -625 |  -560 |  -530 |  -630 |  -630 |  -730 |  -720 |
| 1284 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -980 |        980 |  -820 |  -755 |  -830 |  -935 |  -980 |  -980 |  -955 |  -880 |
| 1285 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        340 |          0 |        340 |   240 |   240 |   340 |   310 |   235 |   190 |    65 |   110 |
| 1286 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        330 |          0 |        330 |   330 |   140 |   110 |   220 |   175 |   100 |   150 |   190 |
| 1287 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         85 |       -150 |        235 |    50 |    65 |   -20 |    85 |  -145 |   -65 |  -150 |   -90 |
| 1288 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        305 |        -25 |        330 |   200 |   145 |   210 |   305 |   -15 |    25 |   -25 |    50 |
| 1289 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1095 |       1095 |  -785 |  -815 |  -775 |  -875 | -1095 |  -935 | -1055 |  -955 |
| 1290 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -780 |        780 |  -780 |  -770 |  -750 |  -740 |  -755 |  -765 |  -775 |  -775 |
| 1291 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        360 |        -35 |        395 |   265 |   235 |   285 |   330 |   205 |   360 |    95 |   -35 |
| 1292 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -280 |        280 |   -85 |  -125 |  -170 |   -40 |  -140 |   -30 |   -50 |  -280 |
| 1293 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        395 |          0 |        395 |   395 |   245 |   185 |    80 |   255 |   235 |   145 |   180 |
| 1294 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1030 |       1030 |  -845 |  -780 |  -800 |  -985 |  -980 |  -915 |  -965 | -1030 |
| 1295 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -835 |        835 |  -585 |  -580 |  -805 |  -675 |  -835 |  -720 |  -675 |  -775 |
| 1296 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        205 |        -55 |        260 |   205 |    80 |    65 |   -55 |    25 |   -30 |    85 |   175 |
| 1297 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -895 |        895 |  -615 |  -645 |  -615 |  -715 |  -590 |  -650 |  -615 |  -895 |
| 1298 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        255 |        -50 |        305 |   255 |   225 |    80 |    80 |   -45 |    35 |   -50 |    25 |
| 1299 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        185 |          0 |        185 |    80 |   145 |   185 |    75 |    95 |    60 |   115 |   180 |
| 1300 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -365 |        365 |   -70 |  -365 |  -245 |  -330 |  -310 |  -340 |  -355 |  -315 |
| 1301 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        230 |          0 |        230 |   115 |    85 |    80 |   165 |   145 |   195 |   130 |   230 |
| 1302 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -275 |        275 |  -140 |  -190 |  -275 |  -215 |  -165 |  -210 |  -115 |  -265 |
| 1303 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         25 |       -130 |        155 |    25 |   -10 |   -65 |   -75 |  -130 |  -115 |   -30 |   -85 |
| 1304 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -495 |        495 |  -495 |  -480 |  -405 |  -450 |  -480 |  -475 |  -295 |  -380 |
| 1305 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -540 |        540 |  -250 |  -255 |  -230 |  -520 |  -535 |  -510 |  -465 |  -540 |
| 1306 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -510 |        510 |  -330 |  -195 |  -370 |  -445 |  -510 |  -475 |  -350 |  -455 |
| 1307 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        385 |          0 |        385 |   385 |   295 |   305 |   185 |   105 |   165 |    85 |   210 |
| 1308 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -550 |        550 |  -330 |  -365 |  -390 |  -365 |  -365 |  -475 |  -550 |  -490 |
| 1309 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -850 |        850 |  -520 |  -635 |  -475 |  -565 |  -575 |  -850 |  -780 |  -770 |
| 1310 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -260 |        260 |  -185 |  -260 |   -60 |  -115 |  -180 |  -155 |  -110 |  -140 |
| 1311 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -755 |        755 |  -445 |  -580 |  -510 |  -540 |  -560 |  -445 |  -470 |  -755 |
| 1312 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -565 |        565 |  -305 |  -535 |  -405 |  -565 |  -350 |  -455 |  -430 |  -515 |
| 1313 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        125 |       -240 |        365 |   -15 |    -0 |    45 |   -30 |    40 |   125 |  -140 |  -240 |
| 1314 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        275 |          0 |        275 |   275 |   210 |   150 |    60 |    10 |    90 |   120 |    15 |
| 1315 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        285 |          0 |        285 |   285 |   245 |   225 |   220 |   260 |   220 |   155 |    65 |
| 1316 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1135 |       1135 |  -925 | -1065 | -1020 |  -990 |  -955 | -1135 | -1025 |  -955 |
| 1317 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        150 |       -165 |        315 |     5 |   150 |    30 |   -95 |   -80 |  -135 |  -165 |  -150 |
| 1318 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        180 |        -75 |        255 |   180 |   170 |    -0 |   -25 |   -70 |   -75 |   -55 |   -15 |
| 1319 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        360 |          0 |        360 |   210 |   290 |   260 |   330 |   275 |   360 |   185 |   325 |
| 1320 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -425 |        425 |  -250 |  -340 |  -315 |  -340 |  -340 |  -325 |  -365 |  -425 |
| 1321 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -515 |        515 |  -410 |  -300 |  -455 |  -445 |  -465 |  -470 |  -395 |  -515 |
| 1322 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1000 |       1000 |  -725 |  -945 |  -915 |  -970 |  -905 |  -895 |  -905 | -1000 |
| 1323 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -310 |        310 |   -55 |  -285 |  -250 |  -195 |  -310 |  -300 |  -150 |  -250 |
| 1324 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        175 |       -105 |        280 |   175 |  -105 |   -15 |    15 |   -40 |  -100 |   -30 |    30 |
| 1325 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        160 |          0 |        160 |    15 |   160 |   110 |    75 |   115 |    45 |    70 |   145 |
| 1326 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        230 |        -75 |        305 |   175 |   230 |    65 |   -20 |   -25 |   -50 |   -40 |   -75 |
| 1327 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -875 |        875 |  -570 |  -875 |  -785 |  -775 |  -815 |  -725 |  -695 |  -875 |
| 1328 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -620 |        620 |  -455 |  -450 |  -330 |  -385 |  -540 |  -605 |  -620 |  -570 |
| 1329 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -820 |        820 |  -615 |  -690 |  -690 |  -690 |  -820 |  -785 |  -760 |  -805 |
| 1330 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        255 |        -10 |        265 |   255 |   195 |   230 |   -10 |    75 |     5 |    20 |    50 |
| 1331 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        145 |       -130 |        275 |    20 |    70 |   145 |    90 |   100 |    60 |  -130 |   -95 |
| 1332 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -650 |        650 |  -355 |  -315 |  -395 |  -350 |  -320 |  -605 |  -525 |  -650 |
| 1333 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         65 |          0 |         65 |    40 |     5 |    20 |    50 |    40 |    65 |    10 |    50 |
| 1334 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -950 |        950 |  -845 |  -745 |  -805 |  -785 |  -715 |  -740 |  -790 |  -950 |
| 1335 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -830 |        830 |  -745 |  -665 |  -660 |  -705 |  -645 |  -740 |  -660 |  -830 |
| 1336 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -720 |        720 |  -560 |  -510 |  -600 |  -570 |  -605 |  -555 |  -540 |  -720 |
| 1337 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        210 |        -15 |        225 |    60 |    80 |   145 |    40 |    55 |    70 |   210 |   -15 |
| 1338 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        160 |        -80 |        240 |   125 |    35 |   150 |   105 |    90 |   160 |   145 |   -80 |
| 1339 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        390 |          0 |        390 |   350 |   390 |   215 |   195 |   250 |   340 |   160 |   310 |
| 1340 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        360 |          0 |        360 |   315 |   360 |   235 |   310 |   305 |    60 |    55 |   135 |
| 1341 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        380 |          0 |        380 |   265 |   270 |   295 |   320 |   380 |   310 |   280 |   335 |
| 1342 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -775 |        775 |  -495 |  -555 |  -565 |  -535 |  -700 |  -775 |  -750 |  -755 |
| 1343 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        265 |        -20 |        285 |   245 |   170 |   265 |   255 |   240 |   235 |    15 |   -20 |
| 1344 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        385 |        -40 |        425 |   385 |   335 |   330 |    50 |    20 |    10 |    -5 |   -40 |
| 1345 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        140 |       -120 |        260 |   140 |    95 |   105 |    65 |  -105 |  -120 |  -120 |   -80 |
| 1346 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -300 |        300 |  -155 |  -155 |   -85 |  -200 |  -300 |  -275 |  -295 |  -295 |
| 1347 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -805 |        805 |  -475 |  -540 |  -805 |  -730 |  -685 |  -705 |  -685 |  -775 |
| 1348 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -560 |        560 |  -230 |  -235 |  -310 |  -250 |  -350 |  -515 |  -560 |  -510 |
| 1349 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -580 |        580 |  -295 |  -400 |  -300 |  -520 |  -450 |  -505 |  -580 |  -485 |
| 1350 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -650 |        650 |  -350 |  -250 |  -600 |  -555 |  -535 |  -570 |  -650 |  -635 |
| 1351 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -615 |        615 |  -360 |  -395 |  -265 |  -395 |  -335 |  -355 |  -615 |  -580 |
| 1352 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -615 |        615 |  -325 |  -390 |  -360 |  -545 |  -575 |  -575 |  -545 |  -615 |
| 1353 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -520 |        520 |  -380 |  -265 |  -355 |  -320 |  -275 |  -345 |  -250 |  -520 |
| 1354 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        350 |        -10 |        360 |   230 |   350 |   275 |   130 |   315 |   265 |   -10 |    45 |
| 1355 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -595 |        595 |  -330 |  -215 |  -355 |  -370 |  -340 |  -375 |  -300 |  -595 |
| 1356 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1080 |       1080 | -1070 | -1070 | -1035 | -1080 | -1060 | -1025 | -1005 | -1020 |
| 1357 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -445 |        445 |  -110 |  -140 |  -200 |  -240 |  -110 |  -220 |  -200 |  -445 |
| 1358 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -640 |        640 |  -480 |  -425 |  -410 |  -475 |  -445 |  -520 |  -380 |  -640 |
| 1359 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -730 |        730 |  -420 |  -375 |  -415 |  -615 |  -685 |  -730 |  -605 |  -660 |
| 1360 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        310 |          0 |        310 |   310 |   235 |   215 |   115 |   200 |   195 |   230 |    10 |
| 1361 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -605 |        605 |  -390 |  -430 |  -480 |  -520 |  -475 |  -305 |  -430 |  -605 |
| 1362 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -915 |        915 |  -885 |  -875 |  -870 |  -880 |  -825 |  -800 |  -860 |  -915 |
| 1363 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        210 |       -125 |        335 |   210 |    65 |   125 |   125 |   150 |  -115 |   -60 |  -125 |
| 1364 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -345 |        345 |  -155 |  -140 |  -155 |  -205 |  -190 |  -345 |  -275 |  -255 |
| 1365 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -900 |        900 |  -690 |  -605 |  -655 |  -855 |  -790 |  -890 |  -900 |  -805 |
| 1366 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        255 |          0 |        255 |   225 |   180 |   210 |   255 |   150 |   150 |    20 |    35 |
| 1367 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -995 |        995 |  -785 |  -755 |  -800 |  -835 |  -780 |  -680 |  -800 |  -995 |
| 1368 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        210 |        -40 |        250 |   110 |   210 |   150 |   -40 |    20 |    35 |    25 |    -5 |
| 1369 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -915 |        915 |  -575 |  -615 |  -615 |  -570 |  -675 |  -915 |  -720 |  -875 |
| 1370 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -675 |        675 |  -410 |  -675 |  -630 |  -525 |  -610 |  -630 |  -625 |  -675 |
| 1371 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         85 |        -10 |         95 |    85 |    50 |    65 |    50 |    40 |   -10 |    50 |    60 |
| 1372 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        285 |          0 |        285 |   285 |   165 |   110 |   155 |   170 |    75 |   175 |    15 |
| 1373 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        255 |        -40 |        295 |   120 |   255 |   220 |   180 |   120 |   185 |   135 |   -40 |
| 1374 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        220 |        -90 |        310 |   220 |   100 |   145 |   145 |   -40 |   -70 |   -90 |   -65 |
| 1375 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        310 |          0 |        310 |   260 |   310 |   130 |    75 |    20 |    65 |   150 |   175 |
| 1376 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -945 |        945 |  -680 |  -760 |  -715 |  -805 |  -945 |  -895 |  -910 |  -920 |
| 1377 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -585 |        585 |  -285 |  -430 |  -585 |  -540 |  -450 |  -535 |  -535 |  -465 |
| 1378 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         50 |       -305 |        355 |   -45 |   -45 |   -50 |    50 |   -15 |    35 |  -190 |  -305 |
| 1379 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        175 |          0 |        175 |   175 |    25 |    15 |    80 |   115 |    80 |   100 |   120 |
| 1380 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -930 |        930 |  -770 |  -715 |  -695 |  -710 |  -770 |  -730 |  -620 |  -930 |
| 1381 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        200 |       -100 |        300 |   160 |   200 |   125 |    -5 |   -75 |   -75 |  -100 |   -20 |
| 1382 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        285 |        -80 |        365 |   220 |   195 |   285 |   245 |   220 |   -10 |   -55 |   -80 |
| 1383 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        375 |          0 |        375 |   340 |   375 |   350 |   225 |   330 |   325 |   190 |   120 |
| 1384 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -330 |        330 |   -80 |  -160 |  -265 |  -310 |  -270 |  -330 |  -260 |  -265 |
| 1385 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1060 |       1060 |  -710 |  -725 |  -860 |  -675 |  -995 | -1005 |  -925 | -1060 |
| 1386 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -840 |        840 |  -570 |  -560 |  -600 |  -610 |  -685 |  -660 |  -605 |  -840 |
| 1387 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -895 |        895 |  -635 |  -700 |  -725 |  -625 |  -895 |  -865 |  -825 |  -810 |
| 1388 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        165 |        -35 |        200 |    30 |   160 |    75 |   165 |    60 |    80 |   120 |   -35 |
| 1389 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        245 |        -15 |        260 |   160 |   130 |   245 |   210 |   230 |   210 |   240 |   -15 |
| 1390 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        235 |        -45 |        280 |   235 |   -15 |   -45 |    80 |    30 |   110 |    65 |   -20 |
| 1391 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        355 |          0 |        355 |   355 |   280 |   235 |   270 |   220 |   290 |   255 |   320 |
| 1392 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        365 |          0 |        365 |   280 |   365 |   290 |   265 |   245 |   160 |   330 |   245 |
| 1393 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1050 |       1050 |  -790 |  -790 | -1050 |  -995 |  -995 | -1045 | -1030 |  -915 |
| 1394 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -815 |        815 |  -575 |  -590 |  -540 |  -760 |  -655 |  -780 |  -770 |  -815 |
| 1395 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1075 |       1075 |  -760 |  -750 |  -835 |  -795 |  -835 |  -790 |  -855 | -1075 |
| 1396 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         15 |       -295 |        310 |  -100 |    -5 |    15 |   -40 |  -135 |   -30 |  -270 |  -295 |
| 1397 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -645 |        645 |  -405 |  -375 |  -530 |  -410 |  -360 |  -475 |  -645 |  -555 |
| 1398 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -680 |        680 |  -330 |  -365 |  -320 |  -400 |  -625 |  -565 |  -570 |  -680 |
| 1399 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        400 |          0 |        400 |   400 |   320 |   400 |   320 |   315 |   335 |   315 |   190 |
| 1400 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         70 |       -160 |        230 |   -55 |   -15 |    60 |    45 |   -40 |    45 |    70 |  -160 |
| 1401 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1120 |       1120 |  -860 |  -850 |  -910 |  -850 | -1025 | -1120 | -1000 | -1085 |
| 1402 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -990 |        990 |  -820 |  -825 |  -845 |  -715 |  -800 |  -865 |  -775 |  -990 |
| 1403 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        335 |          0 |        335 |   335 |   220 |    70 |    15 |    15 |    15 |    45 |    50 |
| 1404 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -365 |        365 |  -185 |  -230 |  -120 |  -265 |  -305 |  -250 |  -215 |  -365 |
| 1405 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        335 |        -20 |        355 |   335 |   100 |    35 |   -20 |    40 |    25 |    90 |    60 |
| 1406 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -725 |        725 |  -480 |  -440 |  -510 |  -695 |  -725 |  -675 |  -630 |  -695 |
| 1407 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -680 |        680 |  -485 |  -430 |  -460 |  -660 |  -635 |  -680 |  -600 |  -590 |
| 1408 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1155 |       1155 |  -885 |  -955 |  -930 |  -940 |  -915 |  -940 | -1090 | -1155 |
| 1409 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -320 |        320 |  -315 |  -210 |  -310 |  -235 |  -210 |  -320 |  -260 |  -315 |
| 1410 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1195 |       1195 |  -985 |  -960 | -1015 | -1155 | -1140 | -1195 | -1130 | -1195 |
| 1411 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1145 |       1145 |  -915 |  -920 | -1005 |  -890 | -1015 |  -930 | -1145 |  -945 |
| 1412 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -790 |        790 |  -520 |  -570 |  -610 |  -485 |  -740 |  -725 |  -750 |  -790 |
| 1413 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1195 |       1195 | -1165 | -1190 | -1005 | -1125 | -1040 | -1130 | -1170 | -1195 |
| 1414 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -490 |        490 |  -260 |  -200 |  -255 |  -240 |  -355 |  -265 |  -490 |  -470 |
| 1415 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         15 |       -165 |        180 |    15 |  -135 |  -120 |  -115 |  -160 |  -165 |  -110 |  -135 |
| 1416 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -380 |        380 |   -50 |  -145 |  -370 |  -380 |  -270 |  -180 |  -275 |  -240 |
| 1417 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -725 |        725 |  -550 |  -430 |  -525 |  -450 |  -540 |  -510 |  -535 |  -725 |
| 1418 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -735 |        735 |  -470 |  -475 |  -650 |  -665 |  -730 |  -735 |  -670 |  -705 |
| 1419 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -790 |        790 |  -530 |  -525 |  -725 |  -680 |  -615 |  -670 |  -720 |  -790 |
| 1420 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1010 |       1010 |  -695 |  -750 |  -695 |  -920 | -1010 |  -900 |  -950 |  -850 |
| 1421 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -275 |        275 |  -260 |  -130 |  -230 |   -95 |  -190 |  -275 |  -230 |  -200 |
| 1422 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        215 |        -40 |        255 |   130 |    70 |   165 |    55 |   155 |   215 |   -40 |    35 |
| 1423 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        310 |        -40 |        350 |   260 |   170 |   230 |   310 |    85 |    80 |    45 |   -40 |
| 1424 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        285 |        -30 |        315 |   165 |   285 |   275 |   265 |   -30 |   -20 |    30 |    10 |
| 1425 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         35 |       -120 |        155 |    20 |    35 |   -45 |   -30 |    20 |  -120 |   -85 |    35 |
| 1426 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        345 |          0 |        345 |   345 |   275 |    90 |    95 |   175 |   100 |   165 |   170 |
| 1427 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -650 |        650 |  -470 |  -490 |  -465 |  -615 |  -650 |  -650 |  -620 |  -615 |
| 1428 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -640 |        640 |  -390 |  -365 |  -355 |  -435 |  -620 |  -590 |  -640 |  -485 |
| 1429 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -620 |        620 |  -310 |  -335 |  -520 |  -465 |  -495 |  -530 |  -620 |  -550 |
| 1430 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        275 |       -100 |        375 |   220 |   275 |   120 |    60 |    -5 |  -100 |    30 |   -20 |
| 1431 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -705 |        705 |  -420 |  -530 |  -445 |  -650 |  -705 |  -685 |  -705 |  -705 |
| 1432 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        325 |        -10 |        335 |   285 |   325 |   210 |    50 |    40 |    75 |   -10 |    40 |
| 1433 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -525 |        525 |  -315 |  -475 |  -525 |  -435 |  -525 |  -495 |  -435 |  -485 |
| 1434 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        330 |         -0 |        330 |   285 |   330 |   290 |   115 |    75 |    -0 |   185 |    10 |
| 1435 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        300 |          0 |        300 |   280 |   260 |   300 |   245 |   270 |   270 |    55 |    85 |
| 1436 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        280 |        -50 |        330 |   250 |   200 |   220 |   140 |   280 |    55 |   -50 |   -15 |
| 1437 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        385 |          0 |        385 |   290 |   295 |   385 |   335 |   215 |   190 |   110 |    40 |
| 1438 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        400 |          0 |        400 |   265 |   400 |   325 |   265 |   340 |   125 |   170 |   125 |
| 1439 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -620 |        620 |  -445 |  -405 |  -620 |  -445 |  -550 |  -565 |  -525 |  -615 |
| 1440 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -565 |        565 |  -395 |  -405 |  -560 |  -555 |  -555 |  -515 |  -545 |  -565 |
| 1441 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        260 |          0 |        260 |   165 |   260 |   220 |    80 |   135 |   225 |   100 |   160 |
| 1442 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         70 |       -245 |        315 |   -85 |    70 |    -5 |    -5 |   -25 |    15 |  -245 |  -210 |
| 1443 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -920 |        920 |  -475 |  -680 |  -630 |  -850 |  -875 |  -825 |  -840 |  -920 |
| 1444 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1005 |       1005 |  -675 |  -780 | -1005 |  -940 |  -980 |  -990 |  -895 |  -955 |
| 1445 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -635 |        635 |  -345 |  -355 |  -280 |  -325 |  -285 |  -350 |  -345 |  -635 |
| 1446 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -335 |        335 |  -105 |  -145 |  -170 |  -120 |   -60 |  -125 |   -30 |  -335 |
| 1447 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        155 |       -150 |        305 |   155 |    60 |  -115 |  -150 |   -50 |   -80 |    -5 |   -65 |
| 1448 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        350 |          0 |        350 |   340 |   350 |   120 |   150 |    90 |    60 |    95 |    90 |
| 1449 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1155 |       1155 |  -870 |  -875 |  -930 |  -885 |  -990 | -1155 | -1080 | -1105 |
| 1450 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -900 |        900 |  -575 |  -760 |  -730 |  -725 |  -835 |  -900 |  -825 |  -835 |
| 1451 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -910 |        910 |  -560 |  -715 |  -700 |  -660 |  -685 |  -625 |  -795 |  -910 |
| 1452 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        345 |          0 |        345 |   310 |   205 |   345 |   225 |   315 |   235 |    95 |   100 |
| 1453 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -600 |        600 |  -340 |  -345 |  -565 |  -600 |  -510 |  -590 |  -515 |  -520 |
| 1454 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        220 |       -115 |        335 |   115 |   120 |   220 |   110 |   150 |    70 |   140 |  -115 |
| 1455 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        180 |       -135 |        315 |    45 |   180 |   -90 |   -65 |  -135 |   -10 |   -55 |   -60 |
| 1456 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -850 |        850 |  -545 |  -565 |  -570 |  -575 |  -695 |  -750 |  -850 |  -715 |
| 1457 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -435 |        435 |  -245 |   -25 |  -130 |  -385 |  -365 |  -435 |  -290 |  -380 |
| 1458 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        305 |          0 |        305 |   305 |   215 |   255 |   180 |   225 |   130 |   145 |   185 |
| 1459 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 |  -955 |  -885 |  -965 |  -960 |  -935 | -1030 |  -835 | -1080 |
| 1460 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -905 |        905 |  -845 |  -815 |  -680 |  -785 |  -795 |  -630 |  -650 |  -905 |
| 1461 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        275 |        -50 |        325 |   275 |    35 |    85 |    20 |   -50 |    75 |    35 |    70 |
| 1462 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        335 |          0 |        335 |   335 |   270 |   290 |    65 |    25 |    50 |     5 |    70 |
| 1463 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -440 |        440 |  -200 |  -265 |  -240 |  -210 |  -205 |  -410 |  -425 |  -440 |
| 1464 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -730 |        730 |  -490 |  -540 |  -675 |  -690 |  -730 |  -635 |  -650 |  -690 |
| 1465 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -790 |        790 |  -540 |  -505 |  -700 |  -780 |  -775 |  -785 |  -725 |  -790 |
| 1466 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        150 |       -140 |        290 |    10 |   -45 |    20 |    30 |   150 |    35 |   -75 |  -140 |
| 1467 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -810 |        810 |  -590 |  -590 |  -605 |  -550 |  -610 |  -560 |  -710 |  -810 |
| 1468 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        350 |          0 |        350 |   350 |   265 |   290 |   170 |   205 |   300 |   315 |   240 |
| 1469 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -605 |        605 |  -300 |  -350 |  -365 |  -555 |  -585 |  -540 |  -595 |  -605 |
| 1470 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        245 |          0 |        245 |   240 |   230 |   185 |   215 |   125 |   150 |   190 |   245 |
| 1471 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        235 |        -55 |        290 |   135 |   235 |   140 |   150 |   170 |   110 |   -25 |   -55 |
| 1472 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -390 |        390 |  -220 |  -180 |   -90 |  -390 |  -310 |  -375 |  -330 |  -335 |
| 1473 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        220 |       -125 |        345 |   175 |   220 |   210 |   135 |   175 |  -125 |    50 |   -20 |
| 1474 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -960 |        960 |  -710 |  -720 |  -685 |  -650 |  -775 |  -740 |  -895 |  -960 |
| 1475 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        240 |       -105 |        345 |   200 |   185 |   235 |   170 |   240 |   230 |   155 |  -105 |
| 1476 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -710 |        710 |  -430 |  -450 |  -350 |  -470 |  -360 |  -455 |  -610 |  -710 |
| 1477 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -445 |        445 |  -150 |  -145 |   -75 |  -105 |  -165 |  -335 |  -445 |  -400 |
| 1478 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -620 |        620 |  -505 |  -620 |  -580 |  -585 |  -505 |  -430 |  -490 |  -485 |
| 1479 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -410 |        410 |  -365 |  -345 |  -410 |  -330 |  -380 |  -325 |  -360 |  -350 |
| 1480 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -390 |        390 |  -160 |  -245 |   -60 |  -200 |  -195 |  -180 |   -95 |  -390 |
| 1481 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         85 |       -135 |        220 |    60 |    85 |  -135 |   -85 |  -125 |  -135 |  -125 |  -125 |
| 1482 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -785 |        785 |  -520 |  -575 |  -700 |  -745 |  -695 |  -680 |  -785 |  -620 |
| 1483 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        225 |        -70 |        295 |   225 |   110 |   165 |    -5 |     5 |    25 |    55 |   -70 |
| 1484 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -615 |        615 |  -495 |  -425 |  -525 |  -400 |  -485 |  -380 |  -470 |  -615 |
| 1485 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -925 |        925 |  -895 |  -835 |  -800 |  -920 |  -845 |  -870 |  -925 |  -915 |
| 1486 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        305 |          0 |        305 |   305 |   200 |   260 |   270 |    90 |   100 |    95 |    15 |
| 1487 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1130 |       1130 |  -980 |  -835 |  -830 | -1090 | -1130 | -1105 | -1080 | -1055 |
| 1488 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -240 |        240 |   -95 |   -20 |   -30 |   -25 |  -190 |  -205 |  -190 |  -240 |
| 1489 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -740 |        740 |  -530 |  -720 |  -690 |  -740 |  -675 |  -665 |  -715 |  -675 |
| 1490 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -425 |        425 |  -250 |  -195 |  -240 |  -335 |  -425 |  -370 |  -400 |  -350 |
| 1491 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -710 |        710 |  -455 |  -385 |  -445 |  -425 |  -395 |  -710 |  -690 |  -635 |
| 1492 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        270 |          0 |        270 |   205 |   260 |   200 |   235 |   270 |   185 |   180 |     5 |
| 1493 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        200 |          0 |        200 |   100 |   155 |    90 |    95 |   100 |   200 |    80 |    55 |
| 1494 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        375 |          0 |        375 |   375 |   145 |   160 |   150 |   195 |    85 |   135 |   180 |
| 1495 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -685 |        685 |  -450 |  -390 |  -450 |  -375 |  -655 |  -550 |  -685 |  -630 |
| 1496 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        240 |          0 |        240 |    70 |    90 |   120 |   145 |   155 |   155 |   240 |    40 |
| 1497 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -395 |        395 |   -80 |  -195 |   -60 |  -150 |  -110 |  -365 |  -210 |  -395 |
| 1498 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        300 |          0 |        300 |   220 |   230 |   200 |   245 |   240 |   300 |   170 |   255 |
| 1499 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -650 |        650 |  -410 |  -350 |  -305 |  -295 |  -615 |  -650 |  -615 |  -620 |
| 1500 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        130 |       -130 |        260 |     5 |    15 |    70 |    95 |    -0 |   130 |    40 |  -130 |
| 1501 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        295 |          0 |        295 |   280 |   285 |   295 |   105 |   115 |    60 |   120 |   210 |
| 1502 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -885 |        885 |  -885 |  -800 |  -835 |  -725 |  -845 |  -845 |  -805 |  -745 |
| 1503 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         -0 |       -270 |        270 |   -60 |   -80 |    -0 |  -180 |  -170 |  -200 |  -270 |  -205 |
| 1504 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        290 |          0 |        290 |   215 |   230 |   270 |   275 |   290 |   275 |   235 |    15 |
| 1505 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -525 |        525 |  -165 |  -200 |  -350 |  -210 |  -525 |  -390 |  -430 |  -415 |
| 1506 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        345 |         -5 |        350 |   270 |   270 |   270 |   195 |   345 |    -5 |    50 |    80 |
| 1507 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -330 |        330 |  -140 |  -185 |  -320 |  -160 |  -140 |  -125 |  -270 |  -330 |
| 1508 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -415 |        415 |  -160 |  -210 |  -225 |  -185 |  -195 |  -195 |  -170 |  -415 |
| 1509 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1110 |       1110 |  -910 |  -950 | -1030 | -1085 | -1060 | -1025 | -1110 | -1080 |
| 1510 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         90 |       -215 |        305 |    90 |    15 |    75 |  -130 |  -165 |  -125 |  -215 |  -165 |
| 1511 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -945 |        945 |  -725 |  -810 |  -775 |  -730 |  -760 |  -785 |  -860 |  -945 |
| 1512 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        215 |       -140 |        355 |    40 |    60 |   215 |    95 |   135 |   155 |   155 |  -140 |
| 1513 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        160 |        -60 |        220 |    85 |   135 |   105 |    30 |   120 |    90 |   160 |   -60 |
| 1514 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        125 |       -115 |        240 |   125 |    20 |    90 |   125 |    35 |   115 |    95 |  -115 |
| 1515 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -340 |        340 |  -165 |  -105 |   -80 |   -25 |  -105 |  -270 |  -340 |  -330 |
| 1516 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        400 |          0 |        400 |   315 |   335 |   295 |   400 |   140 |    75 |   215 |    15 |
| 1517 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        365 |          0 |        365 |   360 |   355 |   305 |   285 |   315 |   300 |   275 |   365 |
| 1518 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        155 |        -40 |        195 |    50 |    15 |    55 |   -40 |    30 |    30 |   155 |    90 |
| 1519 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        325 |       -110 |        435 |   325 |   245 |   270 |   250 |   180 |  -110 |   105 |    65 |
| 1520 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -425 |        425 |  -110 |  -325 |  -415 |  -320 |  -315 |  -425 |  -310 |  -300 |
| 1521 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         80 |        -20 |        100 |    80 |    75 |    35 |   -20 |    80 |   -15 |    65 |    10 |
| 1522 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        125 |        -70 |        195 |    80 |   100 |   125 |   -20 |    30 |    90 |    60 |   -70 |
| 1523 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        205 |        -20 |        225 |    25 |   -20 |    85 |    55 |    80 |    65 |   205 |   160 |
| 1524 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -735 |        735 |  -505 |  -460 |  -600 |  -655 |  -540 |  -735 |  -655 |  -530 |
| 1525 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -920 |        920 |  -635 |  -775 |  -760 |  -780 |  -655 |  -680 |  -920 |  -865 |
| 1526 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1195 |       1195 | -1020 |  -890 |  -975 | -1010 |  -950 | -1125 | -1195 | -1170 |
| 1527 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -955 |        955 |  -845 |  -815 |  -790 |  -820 |  -840 |  -845 |  -955 |  -880 |
| 1528 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         15 |       -185 |        200 |   -70 |   -40 |   -30 |    15 |  -100 |  -155 |  -185 |  -175 |
| 1529 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        190 |         -0 |        190 |   190 |    70 |   135 |   155 |   110 |    -0 |    30 |    95 |
| 1530 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -620 |        620 |  -370 |  -390 |  -325 |  -410 |  -275 |  -345 |  -500 |  -620 |
| 1531 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -845 |        845 |  -535 |  -535 |  -490 |  -440 |  -575 |  -810 |  -730 |  -845 |
| 1532 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        350 |        -55 |        405 |   250 |   300 |   350 |   -55 |    60 |    75 |    -0 |   105 |
| 1533 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        320 |          0 |        320 |   315 |   320 |   200 |    40 |    65 |   130 |    70 |    35 |
| 1534 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -350 |        350 |  -255 |  -200 |  -145 |  -145 |  -195 |  -155 |  -290 |  -350 |
| 1535 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        195 |       -110 |        305 |   150 |    85 |   195 |   145 |   -10 |   -90 |  -110 |   -45 |
| 1536 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -510 |        510 |  -260 |  -370 |  -420 |  -290 |  -295 |  -260 |  -345 |  -510 |
| 1537 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -575 |        575 |  -425 |  -460 |  -410 |  -465 |  -520 |  -365 |  -405 |  -575 |
| 1538 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        245 |        -30 |        275 |   245 |   185 |   -25 |   -30 |    25 |    -5 |   -15 |   -20 |
| 1539 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -780 |        780 |  -655 |  -620 |  -635 |  -595 |  -610 |  -635 |  -780 |  -760 |
| 1540 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -500 |        500 |  -390 |  -425 |  -400 |  -400 |  -390 |  -325 |  -410 |  -500 |
| 1541 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1145 |       1145 | -1050 | -1025 |  -930 | -1145 | -1095 | -1115 | -1050 | -1005 |
| 1542 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -415 |        415 |  -200 |  -195 |  -180 |  -200 |  -245 |  -230 |  -300 |  -415 |
| 1543 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -645 |        645 |  -295 |  -645 |  -590 |  -525 |  -565 |  -560 |  -510 |  -605 |
| 1544 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1125 |       1125 |  -770 |  -825 |  -820 |  -790 |  -995 |  -995 | -1035 | -1125 |
| 1545 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        275 |        -55 |        330 |    95 |   220 |   250 |   220 |   275 |   145 |   140 |   -55 |
| 1546 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1040 |       1040 |  -795 |  -775 |  -870 |  -810 |  -610 |  -685 | -1040 |  -915 |
| 1547 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -475 |        475 |  -380 |  -265 |  -365 |  -285 |  -245 |  -215 |  -250 |  -475 |
| 1548 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1060 |       1060 |  -745 |  -865 |  -930 |  -810 | -1060 | -1025 |  -955 |  -945 |
| 1549 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1000 |       1000 |  -735 |  -795 |  -750 |  -700 |  -855 | -1000 |  -855 |  -935 |
| 1550 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1095 |       1095 |  -865 | -1040 | -1000 | -1095 | -1095 | -1025 | -1040 |  -980 |
| 1551 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        250 |        -90 |        340 |   250 |   140 |   190 |    20 |   -90 |    20 |    -5 |   -65 |
| 1552 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        260 |          0 |        260 |   260 |   235 |   160 |   130 |   165 |   220 |   215 |   170 |
| 1553 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1090 |       1090 |  -685 |  -730 |  -745 |  -695 |  -820 |  -760 |  -995 | -1090 |
| 1554 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -865 |        865 |  -680 |  -865 |  -780 |  -655 |  -750 |  -745 |  -800 |  -725 |
| 1555 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -595 |        595 |  -555 |  -360 |  -505 |  -425 |  -540 |  -540 |  -495 |  -595 |
| 1556 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1100 |       1100 |  -910 |  -875 |  -945 |  -930 |  -885 |  -910 |  -865 | -1100 |
| 1557 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -995 |        995 |  -750 |  -730 |  -815 |  -645 |  -660 |  -875 |  -905 |  -995 |
| 1558 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        175 |        -80 |        255 |   175 |   115 |   -80 |   -80 |   -70 |   -80 |   -70 |     5 |
| 1559 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -535 |        535 |  -475 |  -470 |  -455 |  -495 |  -440 |  -510 |  -400 |  -535 |
| 1560 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        370 |        -30 |        400 |   350 |   370 |   290 |   360 |   230 |   305 |   150 |   -30 |
| 1561 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -795 |        795 |  -585 |  -500 |  -510 |  -550 |  -710 |  -795 |  -710 |  -750 |
| 1562 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         95 |       -260 |        355 |   -70 |   -70 |  -115 |    95 |   -65 |     5 |    20 |  -260 |
| 1563 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         75 |       -235 |        310 |   -60 |   -65 |   -65 |    75 |   -60 |    20 |   -40 |  -235 |
| 1564 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        300 |        -50 |        350 |   260 |   190 |   235 |   230 |   300 |   250 |   -10 |   -50 |
| 1565 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -695 |        695 |  -345 |  -405 |  -405 |  -395 |  -400 |  -360 |  -410 |  -695 |
| 1566 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -325 |        325 |   -65 |  -135 |   -50 |  -140 |   -85 |   -35 |  -325 |  -290 |
| 1567 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        320 |          0 |        320 |   320 |    55 |   165 |   215 |    85 |   190 |    85 |   105 |
| 1568 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -180 |        180 |   -35 |   -95 |  -140 |   -60 |  -180 |  -115 |   -75 |  -130 |
| 1569 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        225 |          0 |        225 |   210 |   100 |   225 |   170 |   165 |   175 |    95 |    80 |
| 1570 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        375 |          0 |        375 |   290 |   375 |   355 |   110 |   200 |   155 |    80 |    45 |
| 1571 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -440 |        440 |  -220 |  -250 |  -175 |  -175 |  -220 |  -325 |  -440 |  -355 |
| 1572 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -570 |        570 |  -375 |  -390 |  -395 |  -415 |  -485 |  -320 |  -405 |  -570 |
| 1573 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        365 |          0 |        365 |   325 |   245 |   365 |   145 |    10 |    65 |   110 |    45 |
| 1574 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -280 |        280 |   -55 |  -280 |  -215 |  -220 |  -245 |  -180 |  -120 |  -250 |
| 1575 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -325 |        325 |  -105 |  -165 |  -165 |  -220 |  -250 |  -245 |  -240 |  -325 |
| 1576 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        245 |       -100 |        345 |   175 |   245 |   160 |   -35 |    15 |   -25 |   100 |  -100 |
| 1577 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        260 |          0 |        260 |   215 |   260 |   220 |   185 |   200 |    80 |    30 |    30 |
| 1578 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        380 |          0 |        380 |   380 |   375 |   140 |    70 |   135 |   275 |   185 |   250 |
| 1579 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        200 |          0 |        200 |   160 |    95 |    10 |   120 |   200 |   130 |   180 |    30 |
| 1580 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -640 |        640 |  -480 |  -325 |  -560 |  -585 |  -640 |  -520 |  -585 |  -610 |
| 1581 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1060 |       1060 |  -790 |  -810 |  -820 |  -890 | -1060 |  -980 |  -955 | -1035 |
| 1582 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         20 |       -215 |        235 |    20 |  -195 |  -215 |  -140 |  -170 |  -140 |  -125 |  -195 |
| 1583 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1200 |       1200 | -1085 |  -995 |  -995 |  -990 | -1025 |  -970 | -1200 | -1080 |
| 1584 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -980 |        980 |  -580 |  -685 |  -860 |  -895 |  -815 |  -850 |  -980 |  -910 |
| 1585 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -815 |        815 |  -650 |  -750 |  -645 |  -645 |  -620 |  -645 |  -625 |  -815 |
| 1586 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         90 |        -75 |        165 |    40 |    85 |    30 |    55 |    70 |    90 |    90 |   -75 |
| 1587 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -640 |        640 |  -520 |  -460 |  -380 |  -410 |  -500 |  -455 |  -475 |  -640 |
| 1588 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        305 |          0 |        305 |   255 |   265 |   255 |   255 |   255 |   230 |   305 |    45 |
| 1589 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -400 |        400 |  -215 |  -195 |  -170 |  -190 |  -135 |  -390 |  -400 |  -365 |
| 1590 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         35 |       -290 |        325 |    35 |  -175 |  -240 |  -205 |  -290 |  -190 |  -290 |  -140 |
| 1591 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -745 |        745 |  -445 |  -460 |  -590 |  -405 |  -690 |  -745 |  -720 |  -630 |
| 1592 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        195 |        -75 |        270 |   195 |   170 |   150 |   100 |   115 |    95 |   110 |   -75 |
| 1593 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        175 |       -115 |        290 |    30 |   140 |   175 |    65 |   165 |    65 |   110 |  -115 |
| 1594 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        355 |          0 |        355 |   280 |   355 |   235 |   200 |   205 |   175 |   145 |   205 |
| 1595 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1010 |       1010 |  -750 |  -675 |  -785 |  -715 |  -785 |  -835 |  -930 | -1010 |
| 1596 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        310 |        -20 |        330 |   220 |   305 |   185 |   310 |   130 |   275 |   -20 |    35 |
| 1597 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         90 |        -50 |        140 |    15 |    70 |    90 |    60 |    50 |    80 |    90 |   -50 |
| 1598 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        265 |        -30 |        295 |   225 |   145 |   170 |   125 |   265 |   180 |    10 |   -30 |
| 1599 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1015 |       1015 |  -800 | -1015 |  -860 |  -955 |  -955 |  -905 |  -870 | -1000 |
| 1600 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1075 |       1075 |  -815 |  -790 |  -780 |  -840 |  -855 | -1005 |  -990 | -1075 |
| 1601 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1105 |       1105 |  -890 |  -970 |  -980 | -1000 |  -925 | -1030 |  -935 | -1105 |
| 1602 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -985 |        985 |  -675 |  -735 |  -700 |  -630 |  -605 |  -655 |  -840 |  -985 |
| 1603 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        350 |          0 |        350 |   270 |   235 |   240 |   235 |   350 |   215 |   340 |   235 |
| 1604 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -860 |        860 |  -590 |  -605 |  -580 |  -765 |  -695 |  -745 |  -840 |  -860 |
| 1605 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1000 |       1000 |  -845 |  -830 |  -765 |  -820 |  -765 |  -780 |  -870 | -1000 |
| 1606 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        230 |       -175 |        405 |   105 |   230 |   165 |  -175 |   -50 |   -30 |  -140 |    15 |
| 1607 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -990 |        990 |  -700 |  -920 |  -990 |  -935 |  -810 |  -860 |  -930 |  -830 |
| 1608 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1190 |       1190 | -1015 | -1075 | -1190 | -1170 | -1145 | -1130 | -1160 | -1175 |
| 1609 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        275 |        -90 |        365 |   200 |   275 |   250 |   135 |   220 |   260 |   -15 |   -90 |
| 1610 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        255 |        -75 |        330 |   185 |   255 |    80 |   240 |    25 |   -40 |   -75 |   -30 |
| 1611 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        105 |       -105 |        210 |    10 |  -105 |   -60 |   -20 |   -70 |   105 |    35 |    75 |
| 1612 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        400 |          0 |        400 |   200 |   365 |   235 |   210 |   400 |   295 |   165 |   160 |
| 1613 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1140 |       1140 |  -900 |  -755 |  -825 |  -885 | -1140 |  -965 |  -995 |  -940 |
| 1614 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -400 |        400 |  -100 |  -120 |  -345 |  -270 |  -400 |  -305 |  -250 |  -300 |
| 1615 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         45 |        -20 |         65 |    40 |    20 |    30 |    45 |   -20 |    -5 |    25 |    10 |
| 1616 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1130 |       1130 |  -890 | -1025 | -1055 |  -915 | -1060 |  -995 |  -990 | -1130 |
| 1617 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        365 |          0 |        365 |   365 |   275 |   100 |    50 |    70 |   160 |    30 |   120 |
| 1618 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -950 |        950 |  -870 |  -780 |  -895 |  -905 |  -805 |  -900 |  -850 |  -950 |
| 1619 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         -0 |       -300 |        300 |   -40 |  -120 |  -185 |   -50 |   -90 |    -0 |  -110 |  -300 |
| 1620 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -855 |        855 |  -780 |  -650 |  -625 |  -750 |  -855 |  -805 |  -780 |  -765 |
| 1621 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        195 |       -110 |        305 |   105 |   125 |   195 |    95 |   120 |    70 |   125 |  -110 |
| 1622 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        345 |          0 |        345 |   345 |   345 |   165 |   235 |   290 |   195 |   120 |    35 |
| 1623 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        210 |          0 |        210 |   205 |   210 |   180 |    65 |    70 |    50 |    10 |    80 |
| 1624 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -695 |        695 |  -460 |  -420 |  -540 |  -225 |  -650 |  -640 |  -695 |  -600 |
| 1625 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        305 |          0 |        305 |   190 |   210 |   275 |   305 |   215 |   210 |   235 |    15 |
| 1626 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -500 |        500 |  -235 |  -380 |  -350 |  -365 |  -290 |  -480 |  -500 |  -500 |
| 1627 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         45 |       -165 |        210 |   -30 |    45 |  -110 |   -60 |  -100 |   -15 |   -65 |  -165 |
| 1628 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        340 |          0 |        340 |   320 |   335 |   315 |   235 |   265 |   340 |   285 |   270 |
| 1629 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        310 |          0 |        310 |   250 |   265 |   185 |   310 |    55 |   160 |    75 |     5 |
| 1630 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        135 |         -0 |        135 |    20 |    70 |   130 |    75 |   130 |   135 |    10 |    -0 |
| 1631 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -700 |        700 |  -385 |  -670 |  -620 |  -640 |  -570 |  -650 |  -665 |  -700 |
| 1632 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -630 |        630 |  -410 |  -380 |  -315 |  -570 |  -445 |  -510 |  -630 |  -565 |
| 1633 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        305 |          0 |        305 |   305 |   155 |   180 |    60 |   145 |    45 |    95 |   115 |
| 1634 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -305 |        305 |  -205 |  -250 |  -235 |  -150 |   -45 |  -120 |  -165 |  -305 |
| 1635 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -950 |        950 |  -695 |  -755 |  -855 |  -880 |  -915 |  -950 |  -870 |  -905 |
| 1636 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -430 |        430 |  -320 |  -270 |  -285 |  -265 |  -195 |  -295 |  -235 |  -430 |
| 1637 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        230 |        -40 |        270 |   230 |   205 |   165 |   200 |   -40 |   -20 |   -25 |     5 |
| 1638 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         40 |       -250 |        290 |   -55 |    40 |    -0 |   -80 |   -40 |   -40 |   -35 |  -250 |
| 1639 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        275 |          0 |        275 |   180 |   275 |   230 |   155 |   270 |   270 |   205 |    10 |
| 1640 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        210 |        -30 |        240 |   200 |   185 |   210 |   150 |   155 |    -5 |   -20 |   -30 |
| 1641 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -435 |        435 |  -215 |  -270 |  -165 |  -260 |  -255 |  -435 |  -425 |  -430 |
| 1642 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        390 |          0 |        390 |   390 |   340 |   125 |   180 |   120 |   220 |   160 |   150 |
| 1643 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -215 |        215 |  -215 |  -100 |  -165 |  -185 |  -205 |  -165 |  -135 |  -145 |
| 1644 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        270 |          0 |        270 |    95 |   270 |   165 |   130 |   185 |   210 |   125 |   185 |
| 1645 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -390 |        390 |  -390 |  -335 |  -380 |  -350 |  -250 |  -310 |  -295 |  -275 |
| 1646 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        210 |        -60 |        270 |    80 |   210 |   175 |   -45 |   -15 |   -60 |    -0 |   -45 |
| 1647 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -880 |        880 |  -730 |  -745 |  -675 |  -545 |  -675 |  -670 |  -880 |  -830 |
| 1648 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -875 |        875 |  -635 |  -700 |  -600 |  -650 |  -725 |  -740 |  -865 |  -875 |
| 1649 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -415 |        415 |  -235 |  -325 |  -320 |  -415 |  -370 |  -370 |  -365 |  -330 |
| 1650 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        335 |          0 |        335 |   305 |   295 |   235 |   335 |    80 |   120 |    85 |   150 |
| 1651 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        265 |        -35 |        300 |   265 |    10 |    70 |   -35 |   -15 |    75 |    55 |   105 |
| 1652 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        290 |          0 |        290 |   240 |   290 |   210 |   245 |   105 |    10 |    85 |    85 |
| 1653 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        385 |          0 |        385 |   385 |   270 |   305 |   345 |   350 |   240 |   315 |   335 |
| 1654 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -830 |        830 |  -775 |  -755 |  -795 |  -675 |  -735 |  -710 |  -775 |  -830 |
| 1655 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1125 |       1125 |  -765 |  -900 | -1020 |  -810 |  -850 |  -895 | -1090 | -1125 |
| 1656 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        220 |        -40 |        260 |   220 |   -40 |     5 |    50 |   -30 |   -20 |    35 |    40 |
| 1657 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        295 |          0 |        295 |   285 |   295 |   275 |   260 |   210 |   100 |    70 |    85 |
| 1658 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        245 |        -70 |        315 |   185 |   140 |   245 |   -20 |   -70 |    30 |   -20 |   -70 |
| 1659 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        300 |          0 |        300 |   165 |   275 |   300 |   160 |   250 |   290 |    55 |    40 |
| 1660 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        370 |          0 |        370 |   335 |   370 |   345 |   285 |   250 |   280 |   280 |   175 |
| 1661 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -760 |        760 |  -510 |  -580 |  -565 |  -555 |  -655 |  -590 |  -595 |  -760 |
| 1662 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        330 |          0 |        330 |   275 |   330 |   310 |   325 |   290 |   100 |   155 |   135 |
| 1663 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        270 |          0 |        270 |   270 |   105 |   125 |   100 |   125 |   130 |    85 |   105 |
| 1664 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        330 |         -5 |        335 |   240 |   330 |   175 |   330 |   190 |   260 |   185 |    -5 |
| 1665 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -525 |        525 |  -235 |  -140 |  -280 |  -195 |  -225 |  -235 |  -205 |  -525 |
| 1666 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -700 |        700 |  -440 |  -360 |  -385 |  -370 |  -470 |  -700 |  -620 |  -595 |
| 1667 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -980 |        980 |  -860 |  -705 |  -785 |  -825 |  -745 |  -700 |  -825 |  -980 |
| 1668 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1105 |       1105 |  -885 | -1055 | -1070 | -1090 | -1060 | -1105 |  -975 | -1025 |
| 1669 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        250 |        -15 |        265 |   250 |   235 |    95 |   -15 |    25 |    70 |    25 |   125 |
| 1670 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1100 |       1100 |  -765 |  -965 |  -875 |  -835 |  -840 | -1035 | -1100 | -1005 |
| 1671 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -960 |        960 |  -760 |  -760 |  -800 |  -805 |  -725 |  -780 |  -960 |  -925 |
| 1672 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1170 |       1170 |  -950 |  -945 |  -945 |  -955 |  -910 | -1070 | -1130 | -1170 |
| 1673 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -710 |        710 |  -525 |  -515 |  -520 |  -520 |  -440 |  -470 |  -465 |  -710 |
| 1674 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -300 |        300 |   -50 |  -165 |   -60 |   -85 |  -175 |  -165 |  -155 |  -300 |
| 1675 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        395 |          0 |        395 |   385 |   380 |   340 |   390 |   395 |   295 |   380 |   350 |
| 1676 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1060 |       1060 |  -920 | -1010 |  -945 | -1040 |  -900 | -1060 | -1025 | -1025 |
| 1677 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -550 |        550 |  -175 |  -350 |  -345 |  -460 |  -300 |  -495 |  -465 |  -550 |
| 1678 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -760 |        760 |  -540 |  -555 |  -450 |  -760 |  -730 |  -720 |  -685 |  -755 |
| 1679 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        375 |          0 |        375 |   375 |   155 |   175 |   110 |   125 |    60 |   115 |   215 |
| 1680 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -675 |        675 |  -435 |  -395 |  -295 |  -595 |  -540 |  -585 |  -660 |  -675 |
| 1681 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1095 |       1095 |  -765 |  -780 |  -835 |  -910 |  -755 | -1095 | -1080 |  -995 |
| 1682 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        315 |          0 |        315 |   300 |   315 |   305 |   165 |   210 |   260 |   255 |   285 |
| 1683 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -960 |        960 |  -680 |  -695 |  -860 |  -825 |  -890 |  -890 |  -870 |  -960 |
| 1684 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          5 |       -285 |        290 |   -15 |  -125 |     5 |  -115 |  -135 |  -110 |  -110 |  -285 |
| 1685 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        395 |          0 |        395 |   345 |   395 |   325 |   225 |   205 |   220 |   175 |    65 |
| 1686 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -820 |        820 |  -700 |  -575 |  -590 |  -515 |  -590 |  -815 |  -820 |  -760 |
| 1687 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        335 |        -40 |        375 |   225 |   315 |   335 |   310 |   310 |    95 |   -40 |   120 |
| 1688 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -440 |        440 |  -325 |  -180 |  -255 |  -310 |  -425 |  -420 |  -440 |  -430 |
| 1689 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        305 |        -50 |        355 |   220 |   220 |   170 |   125 |   175 |   305 |   145 |   -50 |
| 1690 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        300 |          0 |        300 |   270 |   235 |   300 |   250 |   130 |   220 |   285 |   120 |
| 1691 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -585 |        585 |  -445 |  -380 |  -305 |  -385 |  -425 |  -365 |  -450 |  -585 |
| 1692 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        145 |          0 |        145 |   125 |    95 |    50 |    65 |   145 |   110 |    90 |    60 |
| 1693 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -600 |        600 |  -375 |  -360 |  -600 |  -535 |  -550 |  -570 |  -465 |  -495 |
| 1694 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        330 |        -45 |        375 |   165 |   200 |   285 |   250 |   215 |   260 |   330 |   -45 |
| 1695 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -945 |        945 |  -595 |  -535 |  -945 |  -860 |  -825 |  -750 |  -855 |  -775 |
| 1696 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -870 |        870 |  -625 |  -705 |  -635 |  -665 |  -645 |  -795 |  -870 |  -805 |
| 1697 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1120 |       1120 | -1120 | -1080 | -1115 | -1050 |  -990 | -1065 | -1065 | -1100 |
| 1698 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        380 |          0 |        380 |   370 |   285 |   340 |   380 |   295 |   235 |   135 |   100 |
| 1699 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        170 |       -185 |        355 |   170 |    70 |   -40 |  -185 |  -105 |  -150 |   -30 |   -10 |
| 1700 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        215 |          0 |        215 |   190 |   160 |   125 |   145 |   205 |   155 |   150 |   215 |
| 1701 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 |  -755 |  -700 |  -770 |  -805 |  -690 |  -755 |  -685 | -1080 |
| 1702 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        140 |          0 |        140 |    35 |   125 |    60 |    60 |    95 |   105 |   115 |   140 |
| 1703 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         50 |       -295 |        345 |   -55 |   -15 |   -30 |    45 |    50 |  -215 |  -295 |  -195 |
| 1704 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -655 |        655 |  -400 |  -430 |  -465 |  -495 |  -560 |  -530 |  -630 |  -655 |
| 1705 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -260 |        260 |   -20 |   -60 |  -155 |  -260 |  -245 |  -240 |  -185 |  -220 |
| 1706 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        345 |          0 |        345 |   345 |   105 |   105 |   285 |   180 |   185 |   165 |   160 |
| 1707 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -250 |        250 |  -250 |  -220 |  -240 |  -235 |  -230 |  -245 |  -250 |  -200 |
| 1708 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1100 |       1100 |  -790 |  -780 | -1050 |  -870 |  -995 | -1100 | -1070 |  -990 |
| 1709 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        315 |          0 |        315 |   270 |   315 |   270 |   245 |   290 |   275 |    70 |   155 |
| 1710 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -995 |        995 |  -700 |  -660 |  -625 |  -575 |  -995 |  -855 |  -875 |  -770 |
| 1711 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1165 |       1165 | -1150 | -1040 | -1125 | -1165 | -1140 | -1080 | -1060 | -1145 |
| 1712 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        335 |         -0 |        335 |   220 |   335 |   250 |    90 |    95 |    70 |    35 |    -0 |
| 1713 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1085 |       1085 |  -940 | -1015 |  -985 | -1070 | -1020 | -1085 | -1000 | -1055 |
| 1714 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -725 |        725 |  -540 |  -675 |  -710 |  -670 |  -640 |  -720 |  -725 |  -685 |
| 1715 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -290 |        290 |  -240 |  -255 |  -140 |  -270 |  -250 |  -280 |  -290 |  -185 |
| 1716 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        210 |        -85 |        295 |   210 |   -50 |   -45 |   -40 |   -25 |   -85 |    10 |    30 |
| 1717 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -840 |        840 |  -530 |  -515 |  -575 |  -530 |  -575 |  -840 |  -735 |  -795 |
| 1718 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        340 |        -10 |        350 |   340 |   150 |    80 |    30 |   -10 |    50 |   120 |    65 |
| 1719 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -915 |        915 |  -685 |  -645 |  -640 |  -790 |  -915 |  -845 |  -885 |  -870 |
| 1720 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        360 |          0 |        360 |   275 |   360 |   270 |   315 |   320 |   280 |   340 |   110 |
| 1721 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        235 |        -15 |        250 |    90 |   170 |   200 |   235 |   115 |   215 |    15 |   -15 |
| 1722 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1185 |       1185 | -1015 | -1025 |  -970 |  -995 | -1050 |  -985 | -1185 | -1185 |
| 1723 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1015 |       1015 |  -815 |  -750 |  -745 |  -810 |  -745 | -1015 |  -965 |  -960 |
| 1724 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -350 |        350 |  -115 |  -140 |  -130 |  -200 |  -100 |  -295 |  -275 |  -350 |
| 1725 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        385 |          0 |        385 |   330 |   250 |   340 |   365 |   385 |   185 |   185 |    55 |
| 1726 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1050 |       1050 |  -840 |  -840 |  -850 |  -750 |  -940 |  -805 |  -805 | -1050 |
| 1727 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -960 |        960 |  -670 |  -740 |  -700 |  -700 |  -960 |  -920 |  -950 |  -940 |
| 1728 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -375 |        375 |  -290 |  -305 |  -310 |  -245 |  -310 |  -375 |  -320 |  -280 |
| 1729 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        330 |        -25 |        355 |   280 |   180 |   295 |   235 |   330 |   180 |   235 |   -25 |
| 1730 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        280 |        -10 |        290 |   280 |   250 |   195 |   170 |   125 |   -10 |    65 |    95 |
| 1731 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -325 |        325 |   -70 |  -115 |  -140 |  -100 |   -75 |  -325 |  -235 |  -295 |
| 1732 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1255 |       1255 | -1005 |  -870 |  -990 | -1155 | -1135 | -1255 | -1090 | -1055 |
| 1733 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1095 |       1095 |  -935 |  -960 |  -850 |  -830 |  -910 |  -935 | -1095 | -1085 |
| 1734 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        240 |        -55 |        295 |   240 |    75 |   140 |   150 |    80 |   155 |   205 |   -55 |
| 1735 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -695 |        695 |  -440 |  -630 |  -660 |  -680 |  -695 |  -655 |  -645 |  -595 |
| 1736 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -560 |        560 |  -330 |  -310 |  -315 |  -315 |  -295 |  -480 |  -505 |  -560 |
| 1737 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -800 |        800 |  -750 |  -555 |  -660 |  -595 |  -640 |  -655 |  -690 |  -800 |
| 1738 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -920 |        920 |  -725 |  -770 |  -720 |  -695 |  -730 |  -635 |  -865 |  -920 |
| 1739 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -530 |        530 |  -295 |  -290 |  -215 |  -285 |  -460 |  -530 |  -410 |  -360 |
| 1740 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -375 |        375 |   -85 |  -115 |   -85 |  -260 |  -290 |  -290 |  -375 |  -345 |
| 1741 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        365 |          0 |        365 |   280 |   365 |   355 |    30 |   125 |    90 |   135 |    35 |
| 1742 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -495 |        495 |  -280 |  -215 |  -125 |  -425 |  -360 |  -495 |  -300 |  -455 |
| 1743 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -455 |        455 |  -285 |  -105 |  -210 |  -135 |  -250 |  -360 |  -435 |  -455 |
| 1744 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -800 |        800 |  -635 |  -625 |  -535 |  -505 |  -520 |  -535 |  -560 |  -800 |
| 1745 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        270 |       -115 |        385 |   270 |   110 |  -115 |    50 |   -55 |   -50 |   -45 |   -70 |
| 1746 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         75 |       -195 |        270 |    15 |    75 |    60 |   -45 |   -30 |    -5 |  -195 |  -185 |
| 1747 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -870 |        870 |  -670 |  -750 |  -585 |  -660 |  -710 |  -615 |  -845 |  -870 |
| 1748 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        130 |          0 |        130 |   130 |    70 |   100 |    10 |    95 |   100 |   125 |   105 |
| 1749 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -540 |        540 |  -310 |  -295 |  -325 |  -255 |  -270 |  -270 |  -275 |  -540 |
| 1750 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -570 |        570 |  -380 |  -490 |  -350 |  -450 |  -410 |  -430 |  -515 |  -570 |
| 1751 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -540 |        540 |  -360 |  -520 |  -480 |  -475 |  -435 |  -465 |  -445 |  -540 |
| 1752 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1195 |       1195 | -1005 | -1005 | -1070 |  -970 | -1010 | -1065 |  -990 | -1195 |
| 1753 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        200 |       -175 |        375 |   140 |   200 |   110 |  -175 |   -80 |   -85 |   -60 |   -85 |
| 1754 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -845 |        845 |  -525 |  -620 |  -620 |  -525 |  -660 |  -655 |  -845 |  -695 |
| 1755 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -695 |        695 |  -525 |  -415 |  -400 |  -440 |  -460 |  -425 |  -470 |  -695 |
| 1756 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        295 |        -30 |        325 |   295 |   175 |   160 |   145 |   -10 |   -30 |    75 |   -30 |
| 1757 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -990 |        990 |  -730 |  -705 |  -715 |  -785 |  -800 |  -800 |  -765 |  -990 |
| 1758 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -670 |        670 |  -440 |  -495 |  -435 |  -555 |  -620 |  -620 |  -670 |  -635 |
| 1759 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -345 |        345 |  -125 |  -115 |  -105 |  -345 |  -245 |  -315 |  -240 |  -225 |
| 1760 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1050 |       1050 |  -815 |  -815 |  -865 |  -760 |  -985 | -1030 | -1000 | -1050 |
| 1761 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        315 |          0 |        315 |   315 |   265 |    85 |   100 |    80 |   160 |    70 |    95 |
| 1762 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         80 |        -75 |        155 |    80 |   -75 |    -5 |    20 |   -60 |     5 |    45 |     5 |
| 1763 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        285 |          0 |        285 |   280 |   215 |   285 |   245 |   255 |   115 |    20 |   115 |
| 1764 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -220 |        220 |   -60 |   -15 |  -155 |   -80 |   -55 |   -10 |  -160 |  -220 |
| 1765 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1100 |       1100 |  -830 |  -870 |  -900 | -1030 | -1100 | -1065 |  -985 | -1080 |
| 1766 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        215 |          0 |        215 |   160 |   170 |   215 |   150 |    85 |   170 |   140 |   130 |
| 1767 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1095 |       1095 |  -820 |  -685 | -1060 | -1000 | -1095 | -1075 |  -970 | -1000 |
| 1768 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -215 |        215 |  -170 |  -135 |   -55 |  -100 |  -110 |  -215 |  -185 |  -160 |
| 1769 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        210 |        -50 |        260 |   145 |   195 |   165 |    90 |   210 |   110 |   -35 |   -50 |
| 1770 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         95 |       -115 |        210 |    25 |   -20 |   -10 |  -115 |    15 |   -45 |   -70 |    95 |
| 1771 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -880 |        880 |  -810 |  -730 |  -780 |  -880 |  -825 |  -835 |  -805 |  -800 |
| 1772 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        220 |          0 |        220 |   145 |   220 |    80 |   170 |   145 |   155 |   150 |   180 |
| 1773 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1030 |       1030 |  -825 |  -780 |  -735 |  -795 |  -805 |  -855 |  -800 | -1030 |
| 1774 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        300 |          0 |        300 |   260 |   255 |   210 |   300 |   235 |   145 |   290 |   215 |
| 1775 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        130 |       -165 |        295 |    75 |    95 |    70 |   130 |   120 |  -110 |  -100 |  -165 |
| 1776 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1065 |       1065 |  -905 |  -970 |  -995 |  -985 |  -980 |  -985 |  -925 | -1065 |
| 1777 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -775 |        775 |  -440 |  -515 |  -465 |  -495 |  -655 |  -765 |  -775 |  -775 |
| 1778 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -825 |        825 |  -710 |  -745 |  -720 |  -730 |  -825 |  -775 |  -735 |  -795 |
| 1779 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        280 |        -40 |        320 |   180 |    90 |   280 |   140 |   120 |    20 |   -40 |   -25 |
| 1780 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        385 |          0 |        385 |   315 |   385 |   310 |   370 |   225 |   100 |   270 |   125 |
| 1781 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1080 |       1080 |  -840 |  -815 |  -845 |  -760 | -1040 | -1070 | -1055 | -1080 |
| 1782 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        355 |          0 |        355 |   325 |   300 |   230 |   355 |   225 |   255 |   100 |   165 |
| 1783 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1145 |       1145 |  -915 |  -835 |  -835 |  -930 |  -920 |  -985 | -1045 | -1145 |
| 1784 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -660 |        660 |  -395 |  -465 |  -455 |  -400 |  -355 |  -455 |  -630 |  -660 |
| 1785 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        130 |       -160 |        290 |    60 |    65 |   130 |  -160 |    25 |   -40 |  -155 |   -60 |
| 1786 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1145 |       1145 |  -905 |  -865 |  -890 |  -880 |  -900 |  -855 |  -800 | -1145 |
| 1787 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         50 |       -255 |        305 |   -65 |    50 |  -225 |  -205 |  -220 |  -255 |  -230 |  -220 |
| 1788 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         50 |       -265 |        315 |   -65 |    50 |  -235 |  -225 |  -185 |  -210 |  -265 |  -250 |
| 1789 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -555 |        555 |  -300 |  -225 |  -530 |  -555 |  -380 |  -490 |  -395 |  -490 |
| 1790 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1030 |       1030 |  -845 |  -710 | -1000 | -1030 | -1030 | -1015 | -1010 | -1015 |
| 1791 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1175 |       1175 |  -970 |  -915 |  -895 |  -940 | -1095 | -1170 | -1175 | -1115 |
| 1792 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1045 |       1045 |  -955 |  -930 |  -885 |  -745 |  -880 |  -865 |  -820 | -1045 |
| 1793 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -320 |        320 |  -270 |  -150 |  -235 |  -245 |  -225 |   -95 |  -245 |  -320 |
| 1794 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        335 |        -45 |        380 |   335 |   255 |   170 |    -0 |    65 |   -45 |    40 |    -0 |
| 1795 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        380 |          0 |        380 |   380 |   245 |   205 |   250 |   130 |   175 |   275 |   215 |
| 1796 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |       -120 |        345 |   225 |   225 |   145 |   185 |  -120 |   -80 |   -70 |   -50 |
| 1797 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        110 |        -80 |        190 |   110 |   -25 |   -60 |   -70 |    55 |    25 |   -80 |   -10 |
| 1798 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        285 |          0 |        285 |   205 |   120 |   145 |   230 |   170 |   205 |   250 |   285 |
| 1799 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -470 |        470 |  -140 |  -360 |  -365 |  -380 |  -360 |  -435 |  -470 |  -400 |
| 1800 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -715 |        715 |  -500 |  -645 |  -625 |  -715 |  -655 |  -590 |  -650 |  -655 |
| 1801 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        310 |          0 |        310 |   305 |   305 |   310 |   290 |   270 |   305 |   265 |   140 |
| 1802 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1025 |       1025 |  -710 |  -805 |  -775 |  -780 |  -905 | -1025 | -1000 |  -990 |
| 1803 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -770 |        770 |  -565 |  -490 |  -460 |  -550 |  -765 |  -770 |  -745 |  -695 |
| 1804 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         70 |       -165 |        235 |    10 |   -35 |    70 |    50 |  -120 |   -80 |  -165 |  -110 |
| 1805 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        310 |          0 |        310 |   255 |   175 |   235 |   250 |   310 |   265 |   160 |   245 |
| 1806 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        125 |          0 |        125 |   125 |    55 |   110 |   100 |   125 |    35 |   105 |   110 |
| 1807 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -380 |        380 |  -165 |  -140 |   -70 |   -20 |  -380 |  -260 |  -205 |  -190 |
| 1808 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        235 |       -105 |        340 |   235 |   -20 |   -45 |  -105 |   -60 |   -25 |    25 |   -10 |
| 1809 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -435 |        435 |  -195 |  -210 |  -195 |  -195 |  -160 |  -215 |  -250 |  -435 |
| 1810 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        345 |          0 |        345 |   345 |   255 |   135 |   205 |   235 |   125 |   130 |   100 |
| 1811 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        380 |          0 |        380 |   380 |   145 |   210 |   185 |   180 |   205 |   210 |   260 |
| 1812 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -730 |        730 |  -535 |  -500 |  -495 |  -555 |  -640 |  -645 |  -730 |  -685 |
| 1813 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -655 |        655 |  -520 |  -420 |  -525 |  -455 |  -500 |  -475 |  -475 |  -655 |
| 1814 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        215 |        -95 |        310 |    90 |   145 |    55 |    90 |   215 |   -30 |    35 |   -95 |
| 1815 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -265 |        265 |   -40 |   -90 |  -180 |  -175 |  -185 |  -180 |  -180 |  -265 |
| 1816 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -400 |        400 |   -85 |  -305 |  -400 |  -335 |  -315 |  -375 |  -345 |  -305 |
| 1817 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1130 |       1130 |  -835 |  -925 | -1085 | -1055 | -1025 | -1120 | -1130 | -1130 |
| 1818 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        385 |        -15 |        400 |   325 |   355 |   385 |    95 |   150 |   100 |   -15 |   130 |
| 1819 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          5 |       -270 |        275 |     5 |  -235 |  -240 |  -215 |  -270 |  -215 |  -130 |  -200 |
| 1820 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        300 |          0 |        300 |   190 |   150 |   225 |   230 |   245 |   190 |   300 |   185 |
| 1821 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -675 |        675 |  -565 |  -605 |  -545 |  -560 |  -635 |  -565 |  -545 |  -675 |
| 1822 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        360 |          0 |        360 |   360 |   160 |   190 |   155 |   225 |   245 |   215 |   185 |
| 1823 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        135 |         -0 |        135 |    50 |    -0 |    45 |   135 |     5 |    75 |    50 |    50 |
| 1824 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -465 |        465 |  -300 |  -260 |  -225 |  -355 |  -465 |  -455 |  -440 |  -420 |
| 1825 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        340 |          0 |        340 |   340 |   105 |   135 |    40 |   100 |   185 |   130 |   110 |
| 1826 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        310 |          0 |        310 |   250 |   275 |   255 |   310 |   160 |   255 |    70 |    65 |
| 1827 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        255 |       -100 |        355 |   165 |   135 |   165 |   255 |   195 |   -45 |    45 |  -100 |
| 1828 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        160 |       -155 |        315 |   160 |   100 |    85 |    55 |   105 |    45 |   130 |  -155 |
| 1829 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1080 |       1080 |  -755 | -1055 | -1080 |  -895 |  -955 |  -975 |  -960 |  -890 |
| 1830 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |         85 |       -200 |        285 |    75 |   -75 |    40 |    85 |  -100 |  -155 |  -190 |  -200 |
| 1831 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        370 |          0 |        370 |   370 |   340 |   350 |   170 |   155 |    90 |   130 |   100 |
| 1832 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1175 |       1175 |  -875 |  -845 |  -895 | -1175 | -1170 | -1120 | -1035 | -1155 |
| 1833 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         35 |       -270 |        305 |    35 |  -170 |  -175 |  -195 |  -230 |  -270 |  -245 |  -225 |
| 1834 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         65 |       -205 |        270 |    30 |    -5 |   -90 |   -30 |    55 |    -0 |    65 |  -205 |
| 1835 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1135 |       1135 |  -915 |  -840 |  -925 | -1005 |  -855 |  -890 |  -990 | -1135 |
| 1836 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        120 |       -155 |        275 |    80 |    95 |   120 |   -10 |   -25 |    25 |   110 |  -155 |
| 1837 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -395 |        395 |  -250 |  -385 |  -265 |  -320 |  -290 |  -395 |  -325 |  -385 |
| 1838 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        280 |          0 |        280 |   255 |   280 |    45 |    90 |    90 |   105 |    75 |    30 |
| 1839 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        275 |          0 |        275 |   110 |   245 |   175 |   180 |   150 |   190 |   275 |     5 |
| 1840 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        235 |        -40 |        275 |   180 |   125 |   135 |   110 |   235 |   230 |   170 |   -40 |
| 1841 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -665 |        665 |  -430 |  -395 |  -460 |  -410 |  -535 |  -665 |  -555 |  -610 |
| 1842 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        260 |        -55 |        315 |    80 |   115 |   100 |   140 |   230 |   130 |   260 |   -55 |
| 1843 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        325 |        -25 |        350 |   225 |   325 |   160 |   195 |   205 |   180 |   220 |   -25 |
| 1844 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -570 |        570 |  -320 |  -420 |  -280 |  -360 |  -330 |  -545 |  -570 |  -565 |
| 1845 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         25 |       -185 |        210 |    10 |    25 |  -175 |  -185 |  -155 |  -100 |  -175 |  -105 |
| 1846 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        225 |        -15 |        240 |   165 |   190 |   155 |   225 |   195 |   195 |   -15 |   -10 |
| 1847 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -415 |        415 |  -240 |  -340 |  -395 |  -410 |  -340 |  -280 |  -360 |  -415 |
| 1848 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -600 |        600 |  -330 |  -585 |  -600 |  -550 |  -550 |  -535 |  -525 |  -530 |
| 1849 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1150 |       1150 |  -950 |  -865 |  -885 |  -900 |  -865 | -1090 | -1150 | -1090 |
| 1850 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        245 |        -40 |        285 |   245 |   205 |   165 |   220 |   215 |   135 |    45 |   -40 |
| 1851 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        265 |        -65 |        330 |   220 |   265 |    80 |    -0 |   -10 |    35 |   -65 |    65 |
| 1852 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -765 |        765 |  -415 |  -460 |  -400 |  -385 |  -610 |  -600 |  -725 |  -765 |
| 1853 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -530 |        530 |  -265 |  -280 |  -250 |  -235 |  -305 |  -345 |  -510 |  -530 |
| 1854 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        400 |          0 |        400 |   290 |   290 |   245 |   340 |   400 |   260 |    65 |   120 |
| 1855 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -490 |        490 |  -200 |  -200 |  -265 |  -360 |  -490 |  -485 |  -330 |  -465 |
| 1856 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -670 |        670 |  -560 |  -425 |  -460 |  -600 |  -565 |  -535 |  -465 |  -670 |
| 1857 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -915 |        915 |  -700 |  -885 |  -865 |  -915 |  -755 |  -870 |  -815 |  -890 |
| 1858 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -945 |        945 |  -765 |  -670 |  -715 |  -710 |  -775 |  -745 |  -945 |  -945 |
| 1859 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -830 |        830 |  -595 |  -640 |  -565 |  -410 |  -830 |  -755 |  -800 |  -690 |
| 1860 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1080 |       1080 |  -815 |  -725 |  -845 |  -945 |  -855 |  -805 |  -885 | -1080 |
| 1861 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        225 |        -50 |        275 |   225 |   205 |   135 |     5 |    40 |    -0 |   -15 |   -50 |
| 1862 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -275 |        275 |   -10 |   -10 |   -10 |  -275 |  -140 |  -275 |  -260 |  -275 |
| 1863 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -905 |        905 |  -700 |  -625 |  -845 |  -815 |  -905 |  -755 |  -775 |  -845 |
| 1864 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -710 |        710 |  -415 |  -325 |  -515 |  -555 |  -595 |  -655 |  -570 |  -710 |
| 1865 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -430 |        430 |  -270 |  -275 |  -330 |  -350 |  -315 |  -315 |  -375 |  -430 |
| 1866 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        130 |         -0 |        130 |   100 |    90 |    95 |    60 |    45 |    -0 |   130 |    55 |
| 1867 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -575 |        575 |  -220 |  -290 |  -245 |  -575 |  -500 |  -470 |  -485 |  -465 |
| 1868 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        225 |        -25 |        250 |   220 |   225 |    20 |    50 |    90 |    80 |    -0 |   -25 |
| 1869 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -265 |        265 |   -55 |   -75 |  -265 |  -150 |   -90 |  -115 |  -205 |  -135 |
| 1870 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -640 |        640 |  -540 |  -640 |  -470 |  -465 |  -465 |  -495 |  -510 |  -490 |
| 1871 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -610 |        610 |  -465 |  -370 |  -395 |  -275 |  -440 |  -470 |  -455 |  -610 |
| 1872 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -990 |        990 |  -825 |  -650 |  -725 |  -710 |  -720 |  -805 |  -990 |  -885 |
| 1873 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1180 |       1180 | -1010 |  -995 | -1085 | -1170 | -1180 | -1175 | -1145 | -1130 |
| 1874 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        255 |          0 |        255 |   255 |   240 |    90 |    20 |    95 |   150 |    65 |    80 |
| 1875 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        380 |          0 |        380 |   320 |   365 |   345 |   315 |   380 |   255 |   185 |   170 |
| 1876 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        240 |         -0 |        240 |   240 |   240 |    60 |    80 |   115 |    -0 |   120 |    95 |
| 1877 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -395 |        395 |  -300 |  -260 |  -270 |  -360 |  -395 |  -290 |  -375 |  -285 |
| 1878 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -550 |        550 |  -290 |  -415 |  -550 |  -490 |  -435 |  -470 |  -510 |  -450 |
| 1879 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1170 |       1170 |  -860 |  -900 |  -905 |  -965 |  -895 |  -915 | -1170 | -1095 |
| 1880 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -525 |        525 |  -255 |  -270 |  -295 |  -295 |  -515 |  -495 |  -510 |  -525 |
| 1881 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        255 |          0 |        255 |   250 |   195 |   210 |   230 |   255 |   205 |     5 |    95 |
| 1882 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        270 |        -75 |        345 |   270 |   185 |    85 |    15 |    80 |   130 |    65 |   -75 |
| 1883 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        265 |        -50 |        315 |   220 |   170 |   265 |   -50 |    20 |   -45 |   -35 |   -10 |
| 1884 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -645 |        645 |  -365 |  -420 |  -570 |  -645 |  -575 |  -630 |  -630 |  -590 |
| 1885 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -780 |        780 |  -540 |  -495 |  -575 |  -550 |  -525 |  -740 |  -655 |  -780 |
| 1886 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |       -155 |        380 |    70 |   100 |   105 |   225 |  -100 |  -105 |  -155 |   -55 |
| 1887 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1030 |       1030 |  -795 |  -805 |  -780 |  -755 |  -725 |  -990 | -1030 | -1010 |
| 1888 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        310 |          0 |        310 |   285 |   310 |   145 |   165 |   155 |   145 |   185 |    90 |
| 1889 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |         60 |       -295 |        355 |    60 |  -235 |  -115 |   -65 |  -240 |  -295 |  -250 |  -240 |
| 1890 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -570 |        570 |  -360 |  -355 |  -465 |  -480 |  -450 |  -380 |  -510 |  -570 |
| 1891 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |         30 |       -225 |        255 |   -90 |     5 |   -30 |    30 |   -30 |  -185 |  -225 |  -215 |
| 1892 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -845 |        845 |  -620 |  -600 |  -665 |  -615 |  -840 |  -845 |  -830 |  -750 |
| 1893 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1015 |       1015 |  -785 |  -925 |  -975 |  -955 |  -990 |  -960 |  -880 | -1015 |
| 1894 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -785 |        785 |  -535 |  -465 |  -585 |  -760 |  -780 |  -735 |  -785 |  -785 |
| 1895 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         30 |       -210 |        240 |    30 |   -85 |   -25 |  -210 |  -185 |  -170 |  -110 |  -140 |
| 1896 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        395 |          0 |        395 |   380 |   380 |   395 |   305 |   335 |   180 |   120 |    65 |
| 1897 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        300 |        -30 |        330 |   300 |    -0 |    40 |   -30 |   -25 |    60 |    55 |    65 |
| 1898 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -840 |        840 |  -515 |  -485 |  -840 |  -790 |  -765 |  -740 |  -795 |  -655 |
| 1899 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1120 |       1120 | -1000 | -1120 | -1105 | -1085 | -1110 | -1040 |  -995 | -1005 |
| 1900 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -970 |        970 |  -730 |  -705 |  -810 |  -755 |  -745 |  -710 |  -870 |  -970 |
| 1901 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -850 |        850 |  -720 |  -590 |  -695 |  -625 |  -620 |  -850 |  -815 |  -850 |
| 1902 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -235 |        235 |   -75 |   -65 |  -150 |  -150 |   -55 |   -35 |  -100 |  -235 |
| 1903 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        350 |          0 |        350 |   350 |   195 |   240 |   255 |   265 |    65 |   115 |    55 |
| 1904 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        290 |          0 |        290 |   200 |   250 |   245 |   165 |   285 |   255 |   220 |   290 |
| 1905 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -985 |        985 |  -815 |  -730 |  -770 |  -820 |  -720 |  -845 |  -975 |  -985 |
| 1906 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -980 |        980 |  -730 |  -870 |  -965 |  -980 |  -780 |  -940 |  -920 |  -785 |
| 1907 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        140 |        -15 |        155 |    95 |    55 |   -15 |   125 |   120 |   115 |    -0 |   140 |
| 1908 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -275 |        275 |  -170 |  -230 |  -240 |  -115 |  -200 |  -200 |  -185 |  -275 |
| 1909 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -775 |        775 |  -500 |  -550 |  -500 |  -635 |  -635 |  -775 |  -610 |  -685 |
| 1910 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        375 |          0 |        375 |   375 |   240 |   230 |   145 |   145 |   125 |   175 |   195 |
| 1911 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        200 |       -155 |        355 |   185 |   195 |   200 |  -155 |   -15 |   -65 |   -50 |    15 |
| 1912 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1155 |       1155 |  -940 | -1145 | -1150 | -1085 | -1090 | -1090 | -1155 | -1105 |
| 1913 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -755 |        755 |  -510 |  -465 |  -690 |  -715 |  -755 |  -675 |  -620 |  -735 |
| 1914 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -270 |        270 |   -95 |  -125 |  -110 |  -190 |   -90 |  -155 |  -130 |  -270 |
| 1915 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -955 |        955 |  -655 |  -675 |  -670 |  -875 |  -890 |  -855 |  -805 |  -955 |
| 1916 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        260 |        -50 |        310 |   185 |   160 |   260 |   230 |     5 |   -50 |   -10 |    -0 |
| 1917 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        275 |         -0 |        275 |   250 |   185 |   260 |   250 |   275 |    65 |    -0 |    15 |
| 1918 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -445 |        445 |  -215 |  -260 |  -285 |  -300 |  -210 |  -315 |  -255 |  -445 |
| 1919 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        230 |        -35 |        265 |   175 |   230 |    95 |    -5 |    15 |    15 |   -35 |   -25 |
| 1920 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        185 |          0 |        185 |   185 |   145 |    85 |    90 |    80 |    95 |   105 |   175 |
| 1921 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        345 |          0 |        345 |   335 |   280 |   345 |   340 |   345 |   260 |   270 |   335 |
| 1922 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          5 |       -250 |        255 |    -0 |   -80 |   -75 |   -15 |     5 |  -175 |  -155 |  -250 |
| 1923 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -970 |        970 |  -785 |  -830 |  -800 |  -740 |  -670 |  -790 |  -705 |  -970 |
| 1924 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -850 |        850 |  -585 |  -650 |  -660 |  -620 |  -590 |  -850 |  -760 |  -775 |
| 1925 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        230 |          0 |        230 |   105 |   165 |   115 |   230 |   150 |    75 |    75 |   110 |
| 1926 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -825 |        825 |  -590 |  -620 |  -540 |  -825 |  -770 |  -735 |  -660 |  -780 |
| 1927 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -530 |        530 |  -225 |  -225 |  -300 |  -365 |  -360 |  -305 |  -290 |  -530 |
| 1928 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        285 |          0 |        285 |   215 |   155 |   285 |   220 |   255 |   235 |   120 |    65 |
| 1929 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -975 |        975 |  -715 |  -765 |  -900 |  -745 |  -700 |  -775 |  -975 |  -925 |
| 1930 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -755 |        755 |  -555 |  -570 |  -705 |  -740 |  -735 |  -755 |  -645 |  -685 |
| 1931 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -360 |        360 |  -160 |   -85 |  -140 |  -360 |  -295 |  -235 |  -270 |  -345 |
| 1932 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        400 |          0 |        400 |   400 |   160 |   255 |   140 |   195 |   275 |   185 |   140 |
| 1933 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -315 |        315 |   -95 |  -170 |  -110 |   -40 |   -85 |   -55 |  -140 |  -315 |
| 1934 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -395 |        395 |  -145 |  -250 |  -215 |  -140 |  -180 |  -195 |  -395 |  -330 |
| 1935 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -735 |        735 |  -735 |  -725 |  -585 |  -700 |  -665 |  -735 |  -710 |  -685 |
| 1936 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         50 |        -60 |        110 |    50 |    35 |   -10 |   -60 |    -0 |    -0 |    20 |    10 |
| 1937 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -490 |        490 |  -195 |  -485 |  -485 |  -415 |  -385 |  -475 |  -415 |  -490 |
| 1938 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -710 |        710 |  -545 |  -420 |  -685 |  -670 |  -710 |  -580 |  -605 |  -710 |
| 1939 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        220 |          0 |        220 |   220 |    85 |   115 |   160 |   175 |   200 |   200 |   135 |
| 1940 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        310 |        -10 |        320 |   310 |   -10 |   110 |   165 |    70 |    85 |   110 |   190 |
| 1941 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        295 |          0 |        295 |   295 |   280 |   260 |   100 |   110 |    70 |   135 |   115 |
| 1942 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        115 |        -95 |        210 |    95 |   110 |   115 |   -10 |   -75 |   -85 |   -90 |   -95 |
| 1943 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        355 |          0 |        355 |   270 |   355 |   280 |   285 |   275 |   165 |    65 |   100 |
| 1944 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -830 |        830 |  -610 |  -515 |  -605 |  -830 |  -675 |  -785 |  -830 |  -830 |
| 1945 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1155 |       1155 |  -950 |  -945 |  -930 |  -960 |  -950 |  -995 |  -855 | -1155 |
| 1946 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |          0 |        225 |   185 |   210 |   170 |   225 |    15 |    45 |     5 |   115 |
| 1947 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        340 |          0 |        340 |   245 |   235 |   255 |   340 |   320 |    80 |   110 |    60 |
| 1948 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -885 |        885 |  -750 |  -625 |  -560 |  -630 |  -670 |  -650 |  -580 |  -885 |
| 1949 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -290 |        290 |  -200 |   -90 |  -185 |   -55 |   -10 |   -45 |  -130 |  -290 |
| 1950 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        180 |       -130 |        310 |   180 |   -40 |   -50 |   -25 |  -130 |   -70 |   -20 |   -85 |
| 1951 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         55 |       -110 |        165 |   -90 |  -110 |    -5 |    55 |   -50 |   -15 |   -90 |  -110 |
| 1952 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        335 |          0 |        335 |   335 |   205 |   250 |   305 |   205 |   190 |   220 |    70 |
| 1953 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -300 |        300 |  -110 |  -195 |  -110 |  -140 |  -195 |  -185 |  -115 |  -300 |
| 1954 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        320 |          0 |        320 |   225 |   270 |   210 |   320 |   200 |   290 |   145 |    85 |
| 1955 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        135 |       -110 |        245 |   130 |   135 |   -30 |   -55 |   -80 |  -110 |   -75 |   -60 |
| 1956 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -400 |        400 |  -360 |  -400 |  -275 |  -355 |  -320 |  -260 |  -350 |  -365 |
| 1957 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -730 |        730 |  -500 |  -550 |  -525 |  -530 |  -485 |  -555 |  -715 |  -730 |
| 1958 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -510 |        510 |  -330 |  -270 |  -250 |  -470 |  -440 |  -490 |  -510 |  -480 |
| 1959 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -255 |        255 |   -75 |  -110 |   -60 |  -130 |  -120 |  -205 |    -5 |  -255 |
| 1960 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -455 |        455 |  -345 |  -300 |  -375 |  -365 |  -290 |  -430 |  -455 |  -395 |
| 1961 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -435 |        435 |  -205 |  -305 |  -435 |  -410 |  -335 |  -385 |  -330 |  -410 |
| 1962 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        320 |        -10 |        330 |   320 |   220 |   280 |   210 |   -10 |   120 |    30 |    55 |
| 1963 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        365 |          0 |        365 |   365 |   140 |    50 |    80 |   200 |    75 |    40 |   105 |
| 1964 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        120 |       -115 |        235 |   120 |    70 |   -75 |    20 |   -25 |   -20 |    -5 |  -115 |
| 1965 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -815 |        815 |  -575 |  -655 |  -550 |  -600 |  -625 |  -650 |  -670 |  -815 |
| 1966 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        190 |       -100 |        290 |    55 |   190 |   135 |    95 |   135 |   110 |   -90 |  -100 |
| 1967 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        240 |          0 |        240 |   190 |   190 |   240 |   215 |   220 |   140 |   140 |    40 |
| 1968 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        155 |        -65 |        220 |   155 |    85 |   -65 |     5 |    75 |   -40 |    25 |   -30 |
| 1969 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -500 |        500 |  -210 |  -285 |  -500 |  -365 |  -465 |  -485 |  -465 |  -440 |
| 1970 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -335 |        335 |  -120 |   -75 |  -305 |  -335 |  -305 |  -240 |  -320 |  -335 |
| 1971 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -915 |        915 |  -660 |  -585 |  -695 |  -760 |  -625 |  -830 |  -795 |  -915 |
| 1972 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        205 |       -135 |        340 |   205 |   -25 |  -135 |    75 |    25 |    90 |    20 |  -105 |
| 1973 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        350 |          0 |        350 |   230 |   180 |   350 |   325 |   185 |   240 |   265 |   170 |
| 1974 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        240 |       -150 |        390 |   125 |   225 |   220 |   145 |   240 |   130 |   -25 |  -150 |
| 1975 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -885 |        885 |  -705 |  -730 |  -725 |  -735 |  -660 |  -665 |  -680 |  -885 |
| 1976 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          5 |       -330 |        335 |   -80 |   -70 |     5 |  -135 |   -45 |  -105 |   -85 |  -330 |
| 1977 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        250 |        -65 |        315 |   250 |   235 |   190 |   200 |   225 |   -25 |    35 |   -65 |
| 1978 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1125 |       1125 |  -880 |  -935 |  -900 |  -950 |  -960 |  -975 |  -850 | -1125 |
| 1979 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        365 |          0 |        365 |   365 |   355 |    95 |   205 |   120 |   165 |    80 |   140 |
| 1980 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -555 |        555 |  -375 |  -525 |  -540 |  -380 |  -450 |  -555 |  -460 |  -525 |
| 1981 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1030 |       1030 |  -670 |  -840 |  -870 |  -780 | -1030 |  -875 |  -950 |  -905 |
| 1982 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -265 |        265 |  -100 |  -165 |  -110 |   -90 |  -260 |  -265 |  -210 |  -200 |
| 1983 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1200 |       1200 |  -975 |  -930 |  -995 |  -975 | -1010 | -1200 | -1170 | -1160 |
| 1984 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        355 |          0 |        355 |   355 |   320 |   250 |   305 |   115 |   150 |    30 |    70 |
| 1985 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         95 |       -200 |        295 |    95 |   -70 |    35 |  -200 |  -135 |  -180 |  -150 |  -185 |
| 1986 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        150 |       -190 |        340 |   130 |   150 |  -135 |  -175 |   -65 |  -185 |   -85 |  -190 |
| 1987 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         20 |        -85 |        105 |    20 |   -75 |   -85 |   -20 |     5 |   -60 |   -20 |   -65 |
| 1988 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -515 |        515 |  -280 |  -340 |  -470 |  -515 |  -465 |  -485 |  -460 |  -485 |
| 1989 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        225 |        -55 |        280 |   160 |   200 |   225 |   200 |   215 |    85 |   -55 |    20 |
| 1990 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        240 |        -70 |        310 |   145 |   190 |   240 |   165 |    85 |    10 |   -70 |   -50 |
| 1991 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        225 |        -75 |        300 |   190 |   175 |   180 |   225 |   -75 |   -15 |   -25 |   -50 |
| 1992 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -595 |        595 |  -305 |  -320 |  -350 |  -385 |  -575 |  -595 |  -545 |  -425 |
| 1993 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        275 |        -15 |        290 |   140 |   255 |   175 |   230 |   275 |   -15 |   -10 |    65 |
| 1994 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -400 |        400 |   -90 |   -55 |  -340 |  -290 |  -360 |  -325 |  -305 |  -400 |
| 1995 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -230 |        230 |   -75 |  -140 |   -55 |   -25 |   -90 |  -140 |  -150 |  -230 |
| 1996 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -195 |        195 |   -15 |   -15 |  -115 |   -35 |  -195 |  -175 |  -155 |  -195 |
| 1997 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1185 |       1185 |  -975 |  -900 |  -920 |  -865 | -1120 | -1055 | -1040 | -1185 |
| 1998 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -865 |        865 |  -620 |  -570 |  -670 |  -620 |  -655 |  -585 |  -790 |  -865 |
| 1999 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        120 |       -185 |        305 |    40 |     5 |    65 |    75 |    55 |   120 |   -75 |  -185 |
| 2000 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |      -1050 |       1050 |  -805 |  -945 |  -755 |  -800 | -1025 | -1050 | -1000 | -1025 |